A cute little demo showing the simplest usage of minGPT. Configured to run fine on Macbook Air in like a minute.

In [ ]:
function output = LSRegressionGeneric3_original(originalValue,exerciseValue1,exerciseITMCondition,minMaxFlag,decisionVar,nominal)
% LS Regression with one independent decisionVar
%
    modelStatesSize = size(originalValue,2);
    exerciseValueSize = size(exerciseValue1,2);
    exerciseITMConditionSize = size(exerciseITMCondition,2);
    
    if ~(modelStatesSize==exerciseValueSize) || ~(exerciseValueSize == exerciseITMConditionSize)
        error('input vector size do not match!')
    end
    
    numSim = 0;
    aliveVector = zeros(modelStatesSize,1);
    
    % we recalculate exerciseITMCondition
    % simply by checking In the money path
    % for minMaxFlag to be 'min'
    % we choose those paths such that exerciseValue1 >= originalValue
    % for minMaxFlag to be 'max'
    % we choose tose paths such that exerciseValue1 <= originalValue
    
    exerciseITMCondition_orig = zeros(modelStatesSize,1);
    if strcmp(minMaxFlag,'MIN')
        for idx=1:modelStatesSize
            if exerciseValue1(idx) <= originalValue(idx)
                exerciseITMCondition_orig(idx) = 1;
            end
        end
    else
        for idx=1:modelStatesSize
            if exerciseValue1(idx) >= originalValue(idx)
                exerciseITMCondition_orig(idx) = 1;
            end
        end
    end
    
    for idx=1:modelStatesSize
        if exerciseITMCondition_orig(idx) % ITM path
            numSim = numSim +1;
            aliveVector(idx) = 1;
        end
    end
    
    numOfFactors = size(decisionVar,1);
    %zeroth order
    dimensionLS = 1;

    %1st order
    for idxLS =1:numOfFactors
        dimensionLS =  dimensionLS +1; 
    end

    %2nd order
    for idxLS1=1:numOfFactors
        for idxLS2=idxLS1:numOfFactors
            dimensionLS = dimensionLS + 1;
        end
    end

    %3rd order
    for idxLS1=1:numOfFactors
        for idxLS2=idxLS1:numOfFactors
            for idxLS3=idxLS2:numOfFactors
                dimensionLS = dimensionLS + 1;
            end
        end
    end
    
    regMatrix = ones(modelStatesSize,dimensionLS);
    
    for idx_r=1:modelStatesSize
        for idxLS=1:numOfFactors
            regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
        end

        tempIdx = 1;
        for idxLS1=1:numOfFactors
            for idxLS2=idxLS1:numOfFactors
                regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                tempIdx = tempIdx + 1;
            end
        end

        tempIdx2 = 1;
        for idxLS1=1:numOfFactors
            for idxLS2=idxLS1:numOfFactors
                for idxLS3=idxLS2:numOfFactors
                    regMatrix(idx_r,numOfFactors+1+tempIdx-1 + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
                    tempIdx = tempIdx + 1;
                end
            end
        end

    end
    
    %newValue is the LS regression processed values for output
    % initiate with the original value
    newValue = originalValue;
    
    %initialize output in case of no numSim case
    output = newValue;
    
    % if we have ITM paths, then we do the regression
    % if not then we do nothing 
    if numSim > 0
        % target value input for regression, normalized by nominal 10000(fixed)
        % exerciseValue is also normalized
        newValue = newValue/nominal;
        exerciseValue = exerciseValue1/nominal;
   
    %  regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
    %  contiValue2 = regMatrix' * aliveMatrix * contiValue;
        regMatrix2 = zeros(dimensionLS,dimensionLS);
        contiValue2 = zeros(dimensionLS,1);
        for ii=1:dimensionLS
            regVi = regMatrix(:,ii);
            for jj=1:dimensionLS
                regVj = regMatrix(:,jj);
                for idx3=1:modelStatesSize
                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                end
            end
            for idx3=1:modelStatesSize
                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*newValue(idx3);
            end
        end

        beta = regress(contiValue2,regMatrix2);
        regContiValue = regMatrix*beta;
        
        if strcmp(minMaxFlag,'MAX')
            for idx=1:modelStatesSize
                if aliveVector(idx) %live path or ITM path
                    % regContiValue is only used for decision making
                    if regContiValue(idx) <= exerciseValue(idx) 
                        newValue(idx) = exerciseValue(idx);
                    end
                end
            end
        elseif strcmp(minMaxFlag,'MIN')
            for idx=1:modelStatesSize
                if aliveVector(idx) %live path or ITM path
                    % regContiValue is only used for decision making
                    if regContiValue(idx) >= exerciseValue(idx) 
                        newValue(idx) = exerciseValue(idx);
                    end
                end
            end
        else
            error('unsupported minmaxFlag')
        end
        
        output = newValue * nominal;
    end
    
end



In [ ]:
function output = LSRegressionGeneric3(originalValue,exerciseValue,exerciseITMCondition,minMaxFlag,decisionVar1,nOrder,nominal)
% LS Regression with one independent decisionVar
%
    modelStatesSize = size(originalValue,1);
    exerciseValueSize = size(exerciseValue,1);
    exerciseITMConditionSize = size(exerciseITMCondition,1);
    
    if ~(modelStatesSize==exerciseValueSize) || ~(exerciseValueSize == exerciseITMConditionSize)
        error('input vector size do not match!')
    end
    
    % we use upto 3rd order polynomial as regression variables
    
    dimensionLS = nOrder + 1;
    regMatrix = ones(modelStatesSize,dimensionLS);
    for idx_r=1:modelStatesSize
        regMatrix(idx_r,2) = decisionVar1(idx_r);
        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
    end
    
    % we use only ITM path as
    % regression input
    beta =zeros(dimensionLS,1);

    numSim = 0;
    aliveVector = zeros(modelStatesSize,1);
    for idx=1:modelStatesSize
        if exerciseITMCondition(idx) % ITM path
            numSim = numSim +1;
            aliveVector(idx) = 1;
        end
    end
    
    
    %newValue is the LS regression processed values for output
    % initiate with the original value
    newValue = originalValue;
    
    %initialize output in case of no numSim case
    output = newValue;
    
    % if we have ITM paths, then we do the regression
    % if not then we do nothing 
    if numSim > 0
        % target value input for regression, normalized by nominal 10000(fixed)
        % exerciseValue is also normalized
        newValue = newValue/nominal;
        exerciseValue = exerciseValue/nominal;
   
    %  regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
    %  contiValue2 = regMatrix' * aliveMatrix * contiValue;
        regMatrix2 = zeros(dimensionLS,dimensionLS);
        contiValue2 = zeros(dimensionLS,1);
        for ii=1:dimensionLS
            regVi = regMatrix(:,ii);
            for jj=1:dimensionLS
                regVj = regMatrix(:,jj);
                for idx3=1:modelStatesSize
                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                end
            end
            for idx3=1:modelStatesSize
                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*newValue(idx3);
            end
        end

        beta = regress(contiValue2,regMatrix2);
        regContiValue = regMatrix*beta;
        
        if strcmp(minMaxFlag,'MAX')
            for idx=1:modelStatesSize
                if aliveVector(idx) %live path or ITM path
                    % regContiValue is only used for decision making
                    if regContiValue(idx) <= exerciseValue(idx) 
                        newValue(idx) = exerciseValue(idx);
                    end
                end
            end
        elseif strcmp(minMaxFlag,'MIN')
            for idx=1:modelStatesSize
                if aliveVector(idx) %live path or ITM path
                    % regContiValue is only used for decision making
                    if regContiValue(idx) >= exerciseValue(idx) 
                        newValue(idx) = exerciseValue(idx);
                    end
                end
            end
        else
            error('unsupported minmaxFlag')
        end
        
        output = newValue* nominal;
    end
    
end



In [ ]:
function output = LSRegressionGeneric2_original(originalValue,exerciseValue1,exerciseITMCondition,minMaxFlag,decisionVar,nominal)
% LS Regression with one independent decisionVar
%
    modelStatesSize = size(originalValue,2);
    exerciseValueSize = size(exerciseValue1,2);
    exerciseITMConditionSize = size(exerciseITMCondition,2);
    
    if ~(modelStatesSize==exerciseValueSize) || ~(exerciseValueSize == exerciseITMConditionSize)
        error('input vector size do not match!')
    end
    
    numSim = 0;
    aliveVector = zeros(modelStatesSize,1);
    
    % we recalculate exerciseITMCondition
    % simply by checking In the money path
    % for minMaxFlag to be 'min'
    % we choose those paths such that exerciseValue1 >= originalValue
    % for minMaxFlag to be 'max'
    % we choose tose paths such that exerciseValue1 <= originalValue
    
    exerciseITMCondition_orig = zeros(modelStatesSize,1);
    if strcmp(minMaxFlag,'MIN')
        for idx=1:modelStatesSize
            if exerciseValue1(idx) <= originalValue(idx)
                exerciseITMCondition_orig(idx) = 1;
            end
        end
    else
        for idx=1:modelStatesSize
            if exerciseValue1(idx) >= originalValue(idx)
                exerciseITMCondition_orig(idx) = 1;
            end
        end
    end
    
    for idx=1:modelStatesSize
        if exerciseITMCondition_orig(idx) % ITM path
            numSim = numSim +1;
            aliveVector(idx) = 1;
        end
    end
    
    numOfFactors = size(decisionVar,1);
    %zeroth order
    dimensionLS = 1;

    %1st order
    for idxLS =1:numOfFactors
        dimensionLS =  dimensionLS +1; 
    end

    %2nd order
    for idxLS1=1:numOfFactors
        for idxLS2=idxLS1:numOfFactors
            dimensionLS = dimensionLS + 1;
        end
    end

%     %3rd order
%     for idxLS1=1:numOfFactors
%         for idxLS2=idxLS1:numOfFactors
%             for idxLS3=idxLS2:numOfFactors
%                 dimensionLS = dimensionLS + 1;
%             end
%         end
%     end
    
    regMatrix = ones(modelStatesSize,dimensionLS);
    
    for idx_r=1:modelStatesSize
        for idxLS=1:numOfFactors
            regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
        end

        tempIdx = 1;
        for idxLS1=1:numOfFactors
            for idxLS2=idxLS1:numOfFactors
                regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                tempIdx = tempIdx + 1;
            end
        end

%         tempIdx2 = 1;
%         for idxLS1=1:numOfFactors
%             for idxLS2=idxLS1:numOfFactors
%                 for idxLS3=idxLS2:numOfFactors
%                     regMatrix(idx_r,numOfFactors+1+tempIdx-1 + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
%                     tempIdx = tempIdx + 1;
%                 end
%             end
%         end

    end
    
    %newValue is the LS regression processed values for output
    % initiate with the original value
    newValue = originalValue;
    
    %initialize output in case of no numSim case
    output = newValue;
    
    % if we have ITM paths, then we do the regression
    % if not then we do nothing 
    if numSim > 0
        % target value input for regression, normalized by nominal 10000(fixed)
        % exerciseValue is also normalized
        newValue = newValue/nominal;
        exerciseValue = exerciseValue1/nominal;
   
    %  regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
    %  contiValue2 = regMatrix' * aliveMatrix * contiValue;
        regMatrix2 = zeros(dimensionLS,dimensionLS);
        contiValue2 = zeros(dimensionLS,1);
        for ii=1:dimensionLS
            regVi = regMatrix(:,ii);
            for jj=1:dimensionLS
                regVj = regMatrix(:,jj);
                for idx3=1:modelStatesSize
                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                end
            end
            for idx3=1:modelStatesSize
                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*newValue(idx3);
            end
        end

        beta = regress(contiValue2,regMatrix2);
        regContiValue = regMatrix*beta;
        
        if strcmp(minMaxFlag,'MAX')
            for idx=1:modelStatesSize
                if aliveVector(idx) %live path or ITM path
                    % regContiValue is only used for decision making
                    if regContiValue(idx) <= exerciseValue(idx) 
                        newValue(idx) = exerciseValue(idx);
                    end
                end
            end
        elseif strcmp(minMaxFlag,'MIN')
            for idx=1:modelStatesSize
                if aliveVector(idx) %live path or ITM path
                    % regContiValue is only used for decision making
                    if regContiValue(idx) >= exerciseValue(idx) 
                        newValue(idx) = exerciseValue(idx);
                    end
                end
            end
        else
            error('unsupported minmaxFlag')
        end
        
        output = newValue * nominal;
    end
    
end



In [ ]:
function output = LSRegressionGeneric2(originalValue,exerciseValue1,exerciseITMCondition,minMaxFlag,decisionVar,nominal)
% LS Regression with one independent decisionVar
%
    modelStatesSize = size(originalValue,2);
    exerciseValueSize = size(exerciseValue1,2);
    exerciseITMConditionSize = size(exerciseITMCondition,2);
    
    if ~(modelStatesSize==exerciseValueSize) || ~(exerciseValueSize == exerciseITMConditionSize)
        error('input vector size do not match!')
    end
    
    numSim = 0;
    aliveVector = zeros(modelStatesSize,1);
    for idx=1:modelStatesSize
        if exerciseITMCondition(idx) % ITM path
            numSim = numSim +1;
            aliveVector(idx) = 1;
        end
    end
    
    numOfFactors = size(decisionVar,1);
    %zeroth order
    dimensionLS = 1;

    %1st order
    for idxLS =1:numOfFactors
        dimensionLS =  dimensionLS +1; 
    end

    %2nd order
    for idxLS1=1:numOfFactors
        for idxLS2=idxLS1:numOfFactors
            dimensionLS = dimensionLS + 1;
        end
    end

%     %3rd order
%     for idxLS1=1:numOfFactors
%         for idxLS2=idxLS1:numOfFactors
%             for idxLS3=idxLS2:numOfFactors
%                 dimensionLS = dimensionLS + 1;
%             end
%         end
%     end
    
    regMatrix = ones(modelStatesSize,dimensionLS);
    
    for idx_r=1:modelStatesSize
        for idxLS=1:numOfFactors
            regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
        end

        tempIdx = 1;
        for idxLS1=1:numOfFactors
            for idxLS2=idxLS1:numOfFactors
                regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                tempIdx = tempIdx + 1;
            end
        end

%         tempIdx2 = 1;
%         for idxLS1=1:numOfFactors
%             for idxLS2=idxLS1:numOfFactors
%                 for idxLS3=idxLS2:numOfFactors
%                     regMatrix(idx_r,numOfFactors+1+tempIdx-1 + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
%                     tempIdx = tempIdx + 1;
%                 end
%             end
%         end

    end
    
    %newValue is the LS regression processed values for output
    % initiate with the original value
    newValue = originalValue;
    
    %initialize output in case of no numSim case
    output = newValue;
    
    % if we have ITM paths, then we do the regression
    % if not then we do nothing 
    if numSim > 0
        % target value input for regression, normalized by nominal 10000(fixed)
        % exerciseValue is also normalized
        newValue = newValue/nominal;
        exerciseValue = exerciseValue1/nominal;
   
    %  regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
    %  contiValue2 = regMatrix' * aliveMatrix * contiValue;
        regMatrix2 = zeros(dimensionLS,dimensionLS);
        contiValue2 = zeros(dimensionLS,1);
        for ii=1:dimensionLS
            regVi = regMatrix(:,ii);
            for jj=1:dimensionLS
                regVj = regMatrix(:,jj);
                for idx3=1:modelStatesSize
                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                end
            end
            for idx3=1:modelStatesSize
                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*newValue(idx3);
            end
        end

        beta = regress(contiValue2,regMatrix2);
        regContiValue = regMatrix*beta;
        
        if strcmp(minMaxFlag,'MAX')
            for idx=1:modelStatesSize
                if aliveVector(idx) %live path or ITM path
                    % regContiValue is only used for decision making
                    if regContiValue(idx) <= exerciseValue(idx) 
                        newValue(idx) = exerciseValue(idx);
                    end
                end
            end
        elseif strcmp(minMaxFlag,'MIN')
            for idx=1:modelStatesSize
                if aliveVector(idx) %live path or ITM path
                    % regContiValue is only used for decision making
                    if regContiValue(idx) >= exerciseValue(idx) 
                        newValue(idx) = exerciseValue(idx);
                    end
                end
            end
        else
            error('unsupported minmaxFlag')
        end
        
        output = newValue * nominal;
    end
    
end



In [ ]:
function output = LSRegressionGeneric(originalValue,exerciseValue1,exerciseITMCondition,minMaxFlag,decisionVar,nominal)
% LS Regression with one independent decisionVar
%
    modelStatesSize = size(originalValue,2);
    exerciseValueSize = size(exerciseValue1,2);
    exerciseITMConditionSize = size(exerciseITMCondition,2);
    
    if ~(modelStatesSize==exerciseValueSize) || ~(exerciseValueSize == exerciseITMConditionSize)
        error('input vector size do not match!')
    end
    
    numSim = 0;
    aliveVector = zeros(modelStatesSize,1);
    for idx=1:modelStatesSize
        if exerciseITMCondition(idx) % ITM path
            numSim = numSim +1;
            aliveVector(idx) = 1;
        end
    end
    
    numOfFactors = size(decisionVar,1);
    %zeroth order
    dimensionLS = 1;

    %1st order
    for idxLS =1:numOfFactors
        dimensionLS =  dimensionLS +1; 
    end

    %2nd order
    for idxLS1=1:numOfFactors
        for idxLS2=idxLS1:numOfFactors
            dimensionLS = dimensionLS + 1;
        end
    end

    %3rd order
    for idxLS1=1:numOfFactors
        for idxLS2=idxLS1:numOfFactors
            for idxLS3=idxLS2:numOfFactors
                dimensionLS = dimensionLS + 1;
            end
        end
    end
    
    regMatrix = ones(modelStatesSize,dimensionLS);
    
    for idx_r=1:modelStatesSize
        for idxLS=1:numOfFactors
            regMatrix(idx_r,idxLS+1) = decisionVar(idxLS,idx_r);
        end

        tempIdx = 1;
        for idxLS1=1:numOfFactors
            for idxLS2=idxLS1:numOfFactors
                regMatrix(idx_r,numOfFactors+1+tempIdx) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r);
                tempIdx = tempIdx + 1;
            end
        end

        tempIdx2 = 1;
        for idxLS1=1:numOfFactors
            for idxLS2=idxLS1:numOfFactors
                for idxLS3=idxLS2:numOfFactors
                    regMatrix(idx_r,numOfFactors+1+tempIdx-1 + tempIdx2) = decisionVar(idxLS1,idx_r)*decisionVar(idxLS2,idx_r)*decisionVar(idxLS3,idx_r);
                    tempIdx = tempIdx + 1;
                end
            end
        end

    end
    
    %newValue is the LS regression processed values for output
    % initiate with the original value
    newValue = originalValue;
    
    %initialize output in case of no numSim case
    output = newValue;
    
    % if we have ITM paths, then we do the regression
    % if not then we do nothing 
    if numSim > 0
        % target value input for regression, normalized by nominal 10000(fixed)
        % exerciseValue is also normalized
        newValue = newValue/nominal;
        exerciseValue = exerciseValue1/nominal;
   
    %  regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
    %  contiValue2 = regMatrix' * aliveMatrix * contiValue;
        regMatrix2 = zeros(dimensionLS,dimensionLS);
        contiValue2 = zeros(dimensionLS,1);
        for ii=1:dimensionLS
            regVi = regMatrix(:,ii);
            for jj=1:dimensionLS
                regVj = regMatrix(:,jj);
                for idx3=1:modelStatesSize
                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                end
            end
            for idx3=1:modelStatesSize
                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*newValue(idx3);
            end
        end

        beta = regress(contiValue2,regMatrix2);
        regContiValue = regMatrix*beta;
        
        if strcmp(minMaxFlag,'MAX')
            for idx=1:modelStatesSize
                if aliveVector(idx) %live path or ITM path
                    % regContiValue is only used for decision making
                    if regContiValue(idx) <= exerciseValue(idx) 
                        newValue(idx) = exerciseValue(idx);
                    end
                end
            end
        elseif strcmp(minMaxFlag,'MIN')
            for idx=1:modelStatesSize
                if aliveVector(idx) %live path or ITM path
                    % regContiValue is only used for decision making
                    if regContiValue(idx) >= exerciseValue(idx) 
                        newValue(idx) = exerciseValue(idx);
                    end
                end
            end
        else
            error('unsupported minmaxFlag')
        end
        
        output = newValue * nominal;
    end
    
end



In [ ]:
function output = LSRegression1(originalValue,exerciseValue,exerciseITMCondition,minMaxFlag,decisionVar1,nOrder,nominal)
% LS Regression with one independent decisionVar
%
    modelStatesSize = size(originalValue,1);
    exerciseValueSize = size(exerciseValue,1);
    exerciseITMConditionSize = size(exerciseITMCondition,1);
    
    if ~(modelStatesSize==exerciseValueSize) || ~(exerciseValueSize == exerciseITMConditionSize)
        error('input vector size do not match!')
    end
    
    % we use upto 3rd order polynomial as regression variables
    
    dimensionLS = nOrder + 1;
    regMatrix = ones(modelStatesSize,dimensionLS);
    for idx_r=1:modelStatesSize
        regMatrix(idx_r,2) = decisionVar1(idx_r);
        regMatrix(idx_r,3) = decisionVar1(idx_r)*decisionVar1(idx_r);
        regMatrix(idx_r,4) = decisionVar1(idx_r)*decisionVar1(idx_r)*decisionVar1(idx_r);
    end
    
    % we use only ITM path as
    % regression input
    beta =zeros(dimensionLS,1);

    numSim = 0;
    aliveVector = zeros(modelStatesSize,1);
    for idx=1:modelStatesSize
        if exerciseITMCondition(idx) % ITM path
            numSim = numSim +1;
            aliveVector(idx) = 1;
        end
    end
    
    
    %newValue is the LS regression processed values for output
    % initiate with the original value
    newValue = originalValue;
    
    %initialize output in case of no numSim case
    output = newValue;
    
    % if we have ITM paths, then we do the regression
    % if not then we do nothing 
    if numSim > 0
        % target value input for regression, normalized by nominal 10000(fixed)
        % exerciseValue is also normalized
        newValue = newValue/nominal;
        exerciseValue = exerciseValue/nominal;
   
    %  regMatrix2 =  regMatrix' * aliveMatrix * regMatrix;
    %  contiValue2 = regMatrix' * aliveMatrix * contiValue;
        regMatrix2 = zeros(dimensionLS,dimensionLS);
        contiValue2 = zeros(dimensionLS,1);
        for ii=1:dimensionLS
            regVi = regMatrix(:,ii);
            for jj=1:dimensionLS
                regVj = regMatrix(:,jj);
                for idx3=1:modelStatesSize
                    regMatrix2(ii,jj) = regMatrix2(ii,jj) + regVi(idx3)*aliveVector(idx3)*regVj(idx3);
                end
            end
            for idx3=1:modelStatesSize
                contiValue2(ii) = contiValue2(ii) + regVi(idx3)*aliveVector(idx3)*newValue(idx3);
            end
        end

        beta = regress(contiValue2,regMatrix2);
        regContiValue = regMatrix*beta;
        
        if strcmp(minMaxFlag,'MAX')
            for idx=1:modelStatesSize
                if aliveVector(idx) %live path or ITM path
                    % regContiValue is only used for decision making
                    if regContiValue(idx) <= exerciseValue(idx) 
                        newValue(idx) = exerciseValue(idx);
                    end
                end
            end
        elseif strcmp(minMaxFlag,'MIN')
            for idx=1:modelStatesSize
                if aliveVector(idx) %live path or ITM path
                    % regContiValue is only used for decision making
                    if regContiValue(idx) >= exerciseValue(idx) 
                        newValue(idx) = exerciseValue(idx);
                    end
                end
            end
        else
            error('unsupported minmaxFlag')
        end
        
        output = newValue* nominal;
    end
    
end



In [ ]:
function [ output ] = LoadingM( correlation, numFactors)
a= correlation;

% eig returns eigevalues as diagonal matrix with the ascending order by
% default

[v,d]= eig(a);  
oldieg = diag(d);
[neweig,ix] = sort(oldieg,'descend');

[m,n] = size(v);  % m is equal to n

% eigenvector shuffling 
w = zeros(m,n);
for i=1:m
    w(:,i)= v(:,ix(i));
end

out = zeros(m,n);
for i=1:m
    out=out + neweig(i)*w(:,i)*w(:,i)';
end

% factor reduction correlation matrice

factor_size = numFactors;
% or following andersen piterbarg chapter 14 poor man's correlation
lambda_m = diag(neweig(1:factor_size));
E_m =  w(:,1:factor_size);
R_m =  E_m*lambda_m*E_m';

%normalize the correlation matrix and calculate the loading matrix D
rInv_m = zeros(m,m);

for i=1:m
    rInv_m(i,i) = 1.0/sqrt(R_m(i,i));
end

% loading matrix
D = zeros(m,factor_size);
D = rInv_m * E_m * sqrt(lambda_m);
output = D;
end



In [ ]:
function out = LNVolFromNVol(forward,strike,nVol,expiry)
% forward,strike,nVol,expiry -> lnVol

if forward >= strike
    otmPrice = NormalVanillaFwd(expiry*365.0,forward,strike,nVol,'P');
    lnVol = blackVolLBR(otmPrice,forward,strike,expiry,-1);
else
    otmPrice = NormalVanillaFwd(expiry*365.0,forward,strike,nVol,'C');
    lnVol = blackVolLBR(otmPrice,forward,strike,expiry,1);
end

out = lnVol;
end



In [ ]:
classdef LGM3FRisky < IRModelRisky
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        Alpha1
        Alpha2
        Alpha3
        dH1
        dH2
        dH3
        correl12
        correl13
        correl23
        dH1Orig
        dH2Orig
        dH3Orig
        smoothdH
        smoothAlpha
        smoothTermCorr
        targetTermCorr
    end
    
    methods
        function lgm3F = LGM3FRisky(params)
            if nargin > 0
                lgm3F.zeroCurve = params.zeroCurve;
                lgm3F.discountCurve = params.discountCurve;
                lgm3F.mktData = params.mktData;
                lgm3F.modelParams = params.modelParams;
                
                lgm3F.Alpha1 = params.modelParams('Alpha1');
                lgm3F.Alpha2 = params.modelParams('Alpha2');
                lgm3F.Alpha3 = params.modelParams('Alpha3');
                lgm3F.dH1 =  params.modelParams('dH1');
                lgm3F.dH2 =  params.modelParams('dH2');
                lgm3F.dH3 =  params.modelParams('dH3');
                
                lgm3F.dH1Orig =  params.modelParams('dH1Orig');
                lgm3F.dH2Orig =  params.modelParams('dH2Orig');
                lgm3F.dH3Orig =  params.modelParams('dH3Orig');
                
                lgm3F.correl12 =  params.modelParams('correl12');
                lgm3F.correl13 =  params.modelParams('correl13');
                lgm3F.correl23 =  params.modelParams('correl23');
                
                lgm3F.smoothdH =  params.modelParams('smoothdH');
                lgm3F.smoothAlpha =  params.modelParams('smoothAlpha');
%                 lgm3F.smoothTermCorr = params.modelParams('smoothTermCorr');
%                 lgm3F.targetTermCorr = params.modelParams('targetTermCorr');
            end
        end
        
        function out = SimpleSRPrOverHedge(lgm3F,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(lgm3F,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(lgm3F,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(lgm3F,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = lgm3F.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = lgm3F.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = lgm3F.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out =  CMS_PSA_SimpleDate(lgm3F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           tau = 0.25;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = lgm3F.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = lgm3F.DF(to);
           df_TN = lgm3F.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = lgm3F.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = lgm3F.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = lgm3F.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = lgm3F.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:3
               for j=1:3
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + c_nN(1)*modelStatesT(1,i) + c_nN(2) * modelStatesT(2,i) + c_nN(3) * modelStatesT(3,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
        end
        
        function out = HtT_r(lgm3F,t,T)
            out =  lgm3F.H(T,lgm3F.dH1) - lgm3F.H(t,lgm3F.dH1); 
        end
        
        function out = HtT_s(lgm3F,t,T)
            out =  lgm3F.H(T,lgm3F.dH2) - lgm3F.H(t,lgm3F.dH2); 
        end
        
        function out = HtT_t(lgm3F,t,T)
            out =  lgm3F.H(T,lgm3F.dH3) - lgm3F.H(t,lgm3F.dH3); 
        end
        
        function out = fwdZcVariance(lgm3F, a, b, T1, T2)
            varMatrix = lgm3F.LocalVariance(a,b);
            
            localVariance = zeros(9,1);
            measureChange = zeros(9,1);
            
            localVariance(1) = (lgm3F.HtT_r(a,T1)*lgm3F.HtT_r(a,T1)-lgm3F.HtT_r(a,b)*lgm3F.HtT_r(a,b))*varMatrix(1,1);
            localVariance(2) = (lgm3F.HtT_r(a,T1)*lgm3F.HtT_s(a,T1)-lgm3F.HtT_r(a,b)*lgm3F.HtT_s(a,b))*varMatrix(1,2);
            localVariance(3) = (lgm3F.HtT_r(a,T1)*lgm3F.HtT_t(a,T1)-lgm3F.HtT_r(a,b)*lgm3F.HtT_t(a,b))*varMatrix(1,3);
            
            localVariance(4) = (lgm3F.HtT_s(a,T1)*lgm3F.HtT_r(a,T1)-lgm3F.HtT_s(a,b)*lgm3F.HtT_r(a,b))*varMatrix(2,1);
            localVariance(5) = (lgm3F.HtT_s(a,T1)*lgm3F.HtT_s(a,T1)-lgm3F.HtT_s(a,b)*lgm3F.HtT_s(a,b))*varMatrix(2,2);
            localVariance(6) = (lgm3F.HtT_s(a,T1)*lgm3F.HtT_t(a,T1)-lgm3F.HtT_s(a,b)*lgm3F.HtT_t(a,b))*varMatrix(2,3);
            
            localVariance(7) = (lgm3F.HtT_t(a,T1)*lgm3F.HtT_r(a,T1)-lgm3F.HtT_t(a,b)*lgm3F.HtT_r(a,b))*varMatrix(3,1);
            localVariance(8) = (lgm3F.HtT_t(a,T1)*lgm3F.HtT_s(a,T1)-lgm3F.HtT_t(a,b)*lgm3F.HtT_s(a,b))*varMatrix(3,2);
            localVariance(9) = (lgm3F.HtT_t(a,T1)*lgm3F.HtT_t(a,T1)-lgm3F.HtT_t(a,b)*lgm3F.HtT_t(a,b))*varMatrix(3,3);
            
            
            
            measureChange(1) = (lgm3F.HtT_r(a,T1)-lgm3F.HtT_r(a,b))*lgm3F.HtT_r(a,T2)*varMatrix(1,1);
            measureChange(2) = (lgm3F.HtT_r(a,T1)-lgm3F.HtT_r(a,b))*lgm3F.HtT_s(a,T2)*varMatrix(1,2);
            measureChange(3) = (lgm3F.HtT_r(a,T1)-lgm3F.HtT_r(a,b))*lgm3F.HtT_t(a,T2)*varMatrix(1,3);
            
            measureChange(4) = (lgm3F.HtT_s(a,T1)-lgm3F.HtT_s(a,b))*lgm3F.HtT_r(a,T2)*varMatrix(2,1);
            measureChange(5) = (lgm3F.HtT_s(a,T1)-lgm3F.HtT_s(a,b))*lgm3F.HtT_s(a,T2)*varMatrix(2,2);
            measureChange(6) = (lgm3F.HtT_s(a,T1)-lgm3F.HtT_s(a,b))*lgm3F.HtT_t(a,T2)*varMatrix(2,3);
            
            measureChange(7) = (lgm3F.HtT_t(a,T1)-lgm3F.HtT_t(a,b))*lgm3F.HtT_r(a,T2)*varMatrix(3,1);
            measureChange(8) = (lgm3F.HtT_t(a,T1)-lgm3F.HtT_t(a,b))*lgm3F.HtT_s(a,T2)*varMatrix(3,2);
            measureChange(9) = (lgm3F.HtT_t(a,T1)-lgm3F.HtT_t(a,b))*lgm3F.HtT_t(a,T2)*varMatrix(3,3);
            
            value = localVariance(1) + localVariance(2) + localVariance(3);
            value = value + localVariance(4) + localVariance(5) + localVariance(6);
            value = value + localVariance(7) + localVariance(8) + localVariance(9);
            
            measureChangeTerm = measureChange(1) + measureChange(2) + measureChange(3);
            measureChangeTerm = measureChangeTerm + measureChange(4) + measureChange(5) + measureChange(6);
            measureChangeTerm = measureChangeTerm + measureChange(7) + measureChange(8) + measureChange(9);
            
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
        end
        
        function out = discountFactorRisky(lgm3F, startTime, endTime,numMatTime,modelStatesT)
           zcT = lgm3F.DFRisky(endTime/365.0);
           zct = lgm3F.DFRisky(startTime/365.0);
           beta = lgm3F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = lgm3F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i) ...
                   - beta(2) * modelStatesT(2,i) - beta(3) * modelStatesT(3,i)); 
           end
           
           out = discountFactorOut;
        end
        
        function out = discountFactor(lgm3F, startTime, endTime,numMatTime,modelStatesT)
           zcT = lgm3F.DF(endTime/365.0);
           zct = lgm3F.DF(startTime/365.0);
           beta = lgm3F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = lgm3F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i) ...
                   - beta(2) * modelStatesT(2,i) - beta(3) * modelStatesT(3,i)); 
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoffRisky(lgm3F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = lgm3F.DFRisky(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = lgm3F.discountFactorRisky(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = discountPayoff(lgm3F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = lgm3F.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = lgm3F.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(lgm3F,fromTime, toTime)
            
            out = eye(3,3);
        end
        
        function out = stateLocalVariance(lgm3F,fromTime, toTime)
            out = lgm3F.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(lgm3F,from,to)

            sig(1) = lgm3F.Alpha1;
            sig(2) = lgm3F.Alpha2;
            sig(3) = lgm3F.Alpha3;
            corr_rs = lgm3F.correl12;
            corr_rt = lgm3F.correl13;
            corr_st = lgm3F.correl23;
            
            corr(1,1) = 1.0;
            corr(1,2) = corr_rs.quote(1);
            corr(1,3) = corr_rt.quote(1);
            
            corr(2,1) = corr(1,2);
            corr(2,2) = 1.0;
            corr(2,3) = corr_st.quote(1);
            
            corr(3,1) = corr(1,3);
            corr(3,2) = corr(2,3);
            corr(3,3) = 1.0;
            
            alphaSize = length(lgm3F.Alpha1.quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           variances = zeros(3,3);
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end;
                if (i< alphaSize) vol(3) = sig(3).quote(i);else vol(3) = sig(3).quote(alphaSize);end;
                correl(1,1) =1.0;
                correl(2,2) =1.0;
                correl(3,3) =1.0;
                
                if (i< alphaSize) correl(1,2) = corr_rs.quote(i);else correl(1,2) = corr_rs.quote(alphaSize);end;
                if (i< alphaSize) correl(1,3) = corr_rt.quote(i);else correl(1,3) = corr_rt.quote(alphaSize);end;
                if (i< alphaSize) correl(2,3) = corr_st.quote(i);else correl(2,3) = corr_st.quote(alphaSize);end;
                
                correl(2,1) = correl(1,2);
                correl(3,1) = correl(1,3);
                correl(3,2) = correl(2,3);
                
                for j=1:3
                    for k=1:3
                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end
        
        function out = H(lgm3F,t,dH)
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    break;
                end
            end
            tidx = i;
            sum = 0.0;
            prev_t = 0.0;
            curr_t = 0.0;
            for i=1:tidx
                if i< tidx
                    curr_t = dH.tenor(i);
                else
                    curr_t = t;
                end
                sum = sum + dH.quote(i)*(curr_t-prev_t);

                prev_t = dH.tenor(i);
            end
            out = sum;
        end
        
        function out = Alpha_Test(lgm3F,t,Alpha)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(Alpha.tenor)
                alpha = Alpha.quote(i);
                if t<= Alpha.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end
        
        function out = dH_Test(lgm3F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = dH.quote(i);
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = dH.quote(i);
            end
            out = sum;
        end

        function out = H_Test(lgm3F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = sum + dH.quote(i)*(t-prev_t);
                    inRangeYN = true;
                    break;
                else
                    sum = sum + dH.quote(i)*(dH.tenor(i)-prev_t);
                end
                prev_t = dH.tenor(i);
            end
            
            if ~inRangeYN 
                sum = sum + dH.quote(i) * (t - dH.tenor(i));
            end
            out = sum;
        end
        
        function c_Test = C_Test(lgm3F,tx, tnr,tau)
            pv01_1 = lgm3F.PV01(tx,tx+tnr,tau);
            fsr_1  = lgm3F.Fsr(tx,tx+tnr,tau);
            c_nN_1 = lgm3F.C_nN(tx,tx+tnr,tau,pv01_1,fsr_1);
            c_Test = c_nN_1;
        end

        function beta = Beta(lgm3F,t,T)
            beta =[];
            
            beta(1) = lgm3F.H(T,lgm3F.dH1)-lgm3F.H(t,lgm3F.dH1);
            beta(2) = lgm3F.H(T,lgm3F.dH2)-lgm3F.H(t,lgm3F.dH2);
            beta(3) = lgm3F.H(T,lgm3F.dH3)-lgm3F.H(t,lgm3F.dH3);
            
        end
        
        function beta = BetaDate(lgm3F,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            beta =[];
            
            beta(1) = lgm3F.H(T,lgm3F.dH1)-lgm3F.H(t,lgm3F.dH1);
            beta(2) = lgm3F.H(T,lgm3F.dH2)-lgm3F.H(t,lgm3F.dH2);
            beta(3) = lgm3F.H(T,lgm3F.dH3)-lgm3F.H(t,lgm3F.dH3);
            
        end
        
        function c_nN = C_nN(lgm3F,tn,tN,tau,pnN,ynN)
            p = @lgm3F.DF;
            c_nN=[0.0 0.0 0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm3F.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
                c_nN(3) = c_nN(3) +tau * beta(3) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            c_nN(3) = c_nN(3)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            c_nN(3) = c_nN(3) + beta(3)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
            c_nN(3) = c_nN(3)/pnN;
            
        end
        
        function c_nN = C_nNDate(lgm3F,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @lgm3F.DF;
            c_nN=[0.0 0.0 0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = lgm3F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
                c_nN(3) = c_nN(3) +tau * beta(3) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            c_nN(3) = c_nN(3)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            c_nN(3) = c_nN(3) + beta(3)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
            c_nN(3) = c_nN(3)/pnN;
        end
        
        function g_nN = G_nN(lgm3F,tn,tN,tau,pnN)
            p = @lgm3F.DF;
            g_nN=[0.0 0.0 0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm3F.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
                g_nN(3) = g_nN(3) +tau * beta(3) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
            g_nN(3) = g_nN(3)/pnN;
        end
        
        function g_nN = G_nNDate(lgm3F,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @lgm3F.DF;
            g_nN=[0.0 0.0 0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = lgm3F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
                g_nN(3) = g_nN(3) +tau * beta(3) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
            g_nN(3) = g_nN(3)/pnN;
        end
        
        function h_nTerm = H_nTerm(lgm3F,tn,tTerm)
           h_nTerm =[];
           beta = lgm3F.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
           h_nTerm(3) = -1.0*beta(3);
        end
        
        function h_nTerm = H_nTermDate(lgm3F,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = lgm3F.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
           h_nTerm(3) = -1.0*beta(3);
        end
        
        function out = CMSDigitalArg(lgm3F,to,tp,tenor,tau)
           % probability that CMS rate > strike
           p_nN = lgm3F.PV01(to,to+tenor,tau);
           df_Tn = lgm3F.DF(to);
           df_TN = lgm3F.DF(to + tenor);
           y_nN = (df_Tn - df_TN)/p_nN;
           c_nN = lgm3F.C_nN(to,to + tenor,tau,p_nN,y_nN);
           g_nN = lgm3F.G_nN(to,to + tenor,tau,p_nN);
           h_nTerm = lgm3F.H_nTerm(to,tp);
           varCovar = lgm3F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:3
               for j=1:3
                    driftTerm = driftTerm + c_nN(i) * (h_nTerm(j) + g_nN(j)) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:3
               for j=1:3
                    variance = variance + c_nN(i) * c_nN(j) * varCovar(i,j);
               end
           end
           
           out.mu = y_nN + driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN;
           out.varCovar = varCovar;
        end
        
        function out = CMSDigital(lgm3F,to,tp,tenor,tau,strike)
           cmsDigitalArg= lgm3F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(lgm3F,to,tp,tenor,tau,lower,upper)
            lower_value = lgm3F.CMSDigital(to,tp,tenor,tau,lower);
            upper_value = lgm3F.CMSDigital(to,tp,tenor,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalArg(lgm3F,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           p_nN1 = lgm3F.PV01(to,to+tenor1,tau);
           df_Tn1 = lgm3F.DF(to);
           df_TN1 = lgm3F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = lgm3F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = lgm3F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = lgm3F.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = lgm3F.PV01(to,to+tenor2,tau);
           df_Tn2 = lgm3F.DF(to);
           df_TN2 = lgm3F.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = lgm3F.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = lgm3F.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = lgm3F.H_nTerm(to,tp);
           
           varCovar = lgm3F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:3
               for j=1:3
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:3
               for j=1:3
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1);
           out.sig2 = sqrt(variance2);
           
           out.pmu = driftTerm1 - driftTerm2;
           out.pmu1 = driftTerm1;
           out.pmu2 = driftTerm2;
           
           out.y1 = y_nN1;
           out.y2 = y_nN2;
           
           
        end
        
        function out = CMSSpreadDigitalArgDate(lgm3F,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = lgm3F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = lgm3F.DF(to);
           df_TN1 = lgm3F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = lgm3F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = lgm3F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = lgm3F.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = lgm3F.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = lgm3F.DF(to);
           df_TN2 = lgm3F.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = lgm3F.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = lgm3F.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = lgm3F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = lgm3F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:3
               for j=1:3
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:3
               for j=1:3
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigital(lgm3F,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm3F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(lgm3F,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm3F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalRange(lgm3F,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm3F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = lgm3F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalRangeDate(lgm3F,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm3F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = lgm3F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function atmSwaption = ATMSwaption(lgm3F,tx,tnr)
            tau = 0.25;
            pv01 = lgm3F.PV01(tx,tx+tnr,tau);
            fsr  = lgm3F.Fsr(tx,tx+tnr,tau);
            c_nN = lgm3F.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = lgm3F.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:3
                for j=1:3
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        
        function spreadInfo = SpreadInfo(lgm3F,tx,tnr1,tnr2)
            tau = 0.25;
            cmsSpreadDigitalArg =  lgm3F.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function impVariance = ImpVariance(lgm3F,tx,tnr1)
            tau = 0.25;
            pv01_1 = lgm3F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm3F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm3F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            variances = lgm3F.LocalVariance(0,tx);
            var1 = 0;
            for j=1:3
                for k=1:3
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end
        
        function impCorrel = ImpCorrel(lgm3F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = lgm3F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm3F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm3F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm3F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm3F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm3F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm3F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:3
                for k=1:3
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impCovar = ImpCovar(lgm3F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = lgm3F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm3F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm3F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm3F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm3F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm3F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm3F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:3
                for k=1:3
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCovar = coVar;
        end
        
        function instFwdCorr = InsFwdCorr(lgm3F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(3,1);
            dH_tT1(1) = (lgm3F.H_Test(tx + tnr1 +0.001,lgm3F.dH1)-lgm3F.H_Test(tx + tnr1,lgm3F.dH1))/0.001;
            dH_tT1(2) = (lgm3F.H_Test(tx + tnr1 +0.001,lgm3F.dH2)-lgm3F.H_Test(tx + tnr1,lgm3F.dH2))/0.001;
            dH_tT1(3) = (lgm3F.H_Test(tx + tnr1 +0.001,lgm3F.dH3)-lgm3F.H_Test(tx + tnr1,lgm3F.dH3))/0.001;
            
            dH_tT2 = zeros(3,1);
            dH_tT2(1) = (lgm3F.H_Test(tx + tnr2 +0.001,lgm3F.dH1)-lgm3F.H_Test(tx + tnr2,lgm3F.dH1))/0.001;
            dH_tT2(2) = (lgm3F.H_Test(tx + tnr2 +0.001,lgm3F.dH2)-lgm3F.H_Test(tx + tnr2,lgm3F.dH2))/0.001;
            dH_tT2(3) = (lgm3F.H_Test(tx + tnr2 +0.001,lgm3F.dH3)-lgm3F.H_Test(tx + tnr2,lgm3F.dH3))/0.001;
            
            instVariances = (lgm3F.LocalVariance(0,tx+0.001)-lgm3F.LocalVariance(0,tx))/0.001;
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:3
                for k=1:3
                    var1 = var1 + dH_tT1(j)*dH_tT1(k)*instVariances(j,k);
                    var2 = var2 + dH_tT2(j)*dH_tT2(k)*instVariances(j,k);
                    covar = covar + dH_tT1(j)*dH_tT2(k)*instVariances(j,k);
                end
            end
            instFwdCorr = covar/sqrt(var1*var2);
        end
        
        function instFwdCorrTest = InsFwdCorrTest(lgm3F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(3,1);
            dH_tT1(1) = lgm3F.dH_Test(tx + tnr1,lgm3F.dH1);
            dH_tT1(2) = lgm3F.dH_Test(tx + tnr1,lgm3F.dH2);
            dH_tT1(3) = lgm3F.dH_Test(tx + tnr1,lgm3F.dH3);
            
            dH_tT2 = zeros(3,1);
            dH_tT2(1) = lgm3F.dH_Test(tx + tnr2,lgm3F.dH1);
            dH_tT2(2) = lgm3F.dH_Test(tx + tnr2,lgm3F.dH2);
            dH_tT2(3) = lgm3F.dH_Test(tx + tnr2,lgm3F.dH3);
            
            Alpha_t = zeros(3,1);
            Alpha_t(1) = lgm3F.Alpha_Test(tx,lgm3F.Alpha1);
            Alpha_t(2) = lgm3F.Alpha_Test(tx,lgm3F.Alpha2);
            Alpha_t(3) = lgm3F.Alpha_Test(tx,lgm3F.Alpha3);
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:3
                var1 = var1 + dH_tT1(j)*dH_tT1(j)*Alpha_t(j)*Alpha_t(j);
                var2 = var2 + dH_tT2(j)*dH_tT2(j)*Alpha_t(j)*Alpha_t(j);
                covar = covar + dH_tT1(j)*dH_tT2(j)*Alpha_t(j)*Alpha_t(j);
            end
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function instFwdSwapCorrTest = InsSwapFwdCorrTest(lgm3F,tx,tnr1,tnr2)
            tau = 0.25;
            C_tT1 = zeros(3,1);
            pv01_1 = lgm3F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm3F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm3F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
            C_tT1(2) = c_nN_1(2);
            C_tT1(3) = c_nN_1(3);
            
            C_tT2 = zeros(3,1);
            pv01_2 = lgm3F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm3F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm3F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
            C_tT2(2) = c_nN_2(2);
            C_tT2(3) = c_nN_2(3);
            
            Alpha_t = zeros(3,1);
            Alpha_t(1) = lgm3F.Alpha_Test(tx,lgm3F.Alpha1);
            Alpha_t(2) = lgm3F.Alpha_Test(tx,lgm3F.Alpha2);
            Alpha_t(3) = lgm3F.Alpha_Test(tx,lgm3F.Alpha3);
            
            covar = 0.0;
            var1 = 0.0;
            var2 = 0.0;
            for j=1:3
                var1 = var1 + C_tT1(j)*C_tT1(j)*Alpha_t(j)*Alpha_t(j);
                var2 = var2 + C_tT2(j)*C_tT2(j)*Alpha_t(j)*Alpha_t(j);
                covar = covar + C_tT1(j)*C_tT2(j)*Alpha_t(j)*Alpha_t(j);
            end
            instFwdSwapCorrTest = covar/sqrt(var1*var2);
        end
        
        function instFwdCovarTest = InsFwdCovarTest(lgm3F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(3,1);
            dH_tT1(1) = lgm3F.dH_Test(tx + tnr1,lgm3F.dH1);
            dH_tT1(2) = lgm3F.dH_Test(tx + tnr1,lgm3F.dH2);
            dH_tT1(3) = lgm3F.dH_Test(tx + tnr1,lgm3F.dH3);
            
            dH_tT2 = zeros(3,1);
            dH_tT2(1) = lgm3F.dH_Test(tx + tnr2,lgm3F.dH1);
            dH_tT2(2) = lgm3F.dH_Test(tx + tnr2,lgm3F.dH2);
            dH_tT2(3) = lgm3F.dH_Test(tx + tnr2,lgm3F.dH3);
            
            Alpha_t = zeros(3,1);
            Alpha_t(1) = lgm3F.Alpha_Test(tx,lgm3F.Alpha1);
            Alpha_t(2) = lgm3F.Alpha_Test(tx,lgm3F.Alpha2);
            Alpha_t(3) = lgm3F.Alpha_Test(tx,lgm3F.Alpha3);
            
            covar = 0.0;
            for j=1:3
                covar = covar + dH_tT1(j)*dH_tT2(j)*Alpha_t(j)*Alpha_t(j);
            end
            instFwdCovarTest = covar;
        end
        
        function instFwdCovarTestMatrix = InsFwdCovarTest2(lgm3F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCovarTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),3);
                eta =  zeros(3,3);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH1) - lgm3F.H_Test(newtnr(i),lgm3F.dH1))/tau;
                    dH_tT(i,2) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH2) - lgm3F.H_Test(newtnr(i),lgm3F.dH2))/tau;
                    dH_tT(i,3) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH3) - lgm3F.H_Test(newtnr(i),lgm3F.dH3))/tau;
                end

                eta(1,1) = lgm3F.Alpha_Test(tx,lgm3F.Alpha1);
                eta(2,2) = lgm3F.Alpha_Test(tx,lgm3F.Alpha2);
                eta(3,3) = lgm3F.Alpha_Test(tx,lgm3F.Alpha3);
                
                eta(1,2) = 0;
                eta(1,3) = 0;
                
                eta(2,1) = 0;
                eta(2,3) = 0;
                
                eta(3,1) = 0;
                eta(3,2) = 0;
                
                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
            end
            
        end
        
        function instFwdCorrTestMatrix = InsFwdCorrTest2(lgm3F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCorrTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),3);
                eta =  zeros(3,3);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH1) - lgm3F.H_Test(newtnr(i),lgm3F.dH1))/tau;
                    dH_tT(i,2) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH2) - lgm3F.H_Test(newtnr(i),lgm3F.dH2))/tau;
                    dH_tT(i,3) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH3) - lgm3F.H_Test(newtnr(i),lgm3F.dH3))/tau;
                end

                eta(1,1) = lgm3F.Alpha_Test(tx,lgm3F.Alpha1);
                eta(2,2) = lgm3F.Alpha_Test(tx,lgm3F.Alpha2);
                eta(3,3) = lgm3F.Alpha_Test(tx,lgm3F.Alpha3);
                
                eta(1,2) = 0;
                eta(1,3) = 0;
                
                eta(2,1) = 0;
                eta(2,3) = 0;
                
                eta(3,1) = 0;
                eta(3,2) = 0;
                
                testLoadingD = dH_tT*eta;
                
                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
                %normalize the correlation matrix and calculate the loading matrix D
                rInv_m = zeros(length(newtnr),length(newtnr));
                for i=1:length(newtnr)
                    rInv_m(i,i) = 1.0/sqrt(coVar(i,i));
                end
                instFwdCorrTestMatrix = zeros(length(newtnr),length(newtnr));
                instFwdCorrTestMatrix = rInv_m*instFwdCovarTestMatrix*rInv_m';
                
            end
            
        end
        
        function deltaHMatrix = DeltaHTest2(lgm3F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                deltaHMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                
                dH_tT = zeros(length(newtnr),3);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH1) - lgm3F.H_Test(newtnr(i),lgm3F.dH1))/tau;
                    dH_tT(i,2) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH2) - lgm3F.H_Test(newtnr(i),lgm3F.dH2))/tau;
                    dH_tT(i,3) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH3) - lgm3F.H_Test(newtnr(i),lgm3F.dH3))/tau;
                
                end

                deltaHMatrix = dH_tT;
            end
            
        end
        
        function etaMatrix = EtaTest2(lgm3F,tx)
            eta =  zeros(3,3);
            eta(1,1) = lgm3F.Alpha_Test(tx,lgm3F.Alpha1);
            eta(2,2) = lgm3F.Alpha_Test(tx,lgm3F.Alpha2);
            eta(3,3) = lgm3F.Alpha_Test(tx,lgm3F.Alpha3);
            
            eta(1,2) = 0;
            eta(1,3) = 0;
            
            eta(2,1) = 0;
            eta(2,3) = 0;
            
            eta(3,1) = 0;
            eta(3,2) = 0;
            
            etaMatrix = eta;
            
        end
        
        function instFwdSwapCovarTest = InsSwapFwdCovarTest(lgm3F,tx,tnr1,tnr2)
            tau = 0.25;
            C_tT1 = zeros(3,1);
            pv01_1 = lgm3F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm3F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm3F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
            C_tT1(2) = c_nN_1(2);
            C_tT1(3) = c_nN_1(3);
            
            C_tT2 = zeros(3,1);
            pv01_2 = lgm3F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm3F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm3F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
            C_tT2(2) = c_nN_2(2);
            C_tT2(3) = c_nN_2(3);
            
            Alpha_t = zeros(3,1);
            Alpha_t(1) = lgm3F.Alpha_Test(tx,lgm3F.Alpha1);
            Alpha_t(2) = lgm3F.Alpha_Test(tx,lgm3F.Alpha2);
            Alpha_t(2) = lgm3F.Alpha_Test(tx,lgm3F.Alpha3);
            
            covar = 0.0;
            for j=1:3
                covar = covar + C_tT1(j)*C_tT2(j)*Alpha_t(j)*Alpha_t(j);
            end
            instFwdSwapCovarTest = covar;
        end
        
        function out = RebonatoFwdCorr(lgm3F,tx,tnr1,tnr2)
           rho_inf = 0.4;
           rho_decay = 0.15;
           
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
       
        function out = CalibrateToATMSwaptionSurface(lgm3F,black,calibrationFlag)
            % Market Data Object , Vanilla Object
            % targetSwaption
            % targetCap
            
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% target instruments set up
            params.swaption_expiry_size = expirySize;
            params.swaption_tenor_size = tenorSize;
            
            params.swaption_calibration_yn = ones(expirySize,tenorSize);
            
            params.swaptionSurface.expiry = swaptionVolSurface.expiry;
            params.swaptionSurface.tenor  = swaptionVolSurface.tenor;
            params.swaptionSurface.quote  = zeros(expirySize,tenorSize);
            params.swaptionSurface.strike = zeros(expirySize,tenorSize);
            params.swaptionSurface.blackPrice = zeros(expirySize,tenorSize);
            
            % setting target diagonal swaption
            for i=1:expirySize
                for j=1:tenorSize
                    if params.swaption_calibration_yn(i,j) == 1
                        Ti = params.swaptionSurface.expiry(i);
                        Tj = params.swaptionSurface.tenor(j);
                        blackVolQuote = swaptionVolSurface.swaptionVol(expiry(i),tenor(j));
                        params.swaptionSurface.quote(i,j) = blackVolQuote;
                        params.swaptionSurface.strike(i,j) = black.Fsr(Ti,Ti + Tj,0.25);
                        params.swaptionSurface.blackPrice(i,j) = black.BlackSwaptionWithVol(Ti,Tj,params.swaptionSurface.strike(i,j),blackVolQuote);
                    end
                end

            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            params.dH1 = lgm3F.dH1;
            params.dH2 = lgm3F.dH2;
            params.dH3 = lgm3F.dH3;
            
            dH1_size = length(lgm3F.dH1.quote);
            Alpha1_size = length(lgm3F.Alpha1.quote);
            
            params.dH1_size = dH1_size;
            params.Alpha1_size = Alpha1_size;
            
            params.Alpha1 = lgm3F.Alpha1;
            params.Alpha2 = lgm3F.Alpha2;
            params.Alpha3 = lgm3F.Alpha3;
            
            lgm3F.dH1    = params.dH1;
            lgm3F.dH2    = params.dH2;
            lgm3F.dH3    = params.dH3;
            
            lgm3F.Alpha1 = params.Alpha1;
            lgm3F.Alpha2 = params.Alpha2;
            lgm3F.Alpha3 = params.Alpha3;
            
            params.dH1Orig = lgm3F.dH1Orig;
            params.dH2Orig = lgm3F.dH2Orig;
            params.dH3Orig = lgm3F.dH3Orig;
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            out.dH1 = params.dH1;
            out.dH2 = params.dH2;
            out.dH3 = params.dH3;
            
            out.Alpha1 = params.Alpha1;
            out.Alpha2 = params.Alpha2;
            out.Alpha3 = params.Alpha3;
            
            out.blackSwaptionPrice = params.swaptionSurface.blackPrice;
            out.lgm3FSwaptionPrice = zeros(expirySize,tenorSize);
            out.swaptionPriceRelDiff = zeros(expirySize,tenorSize);
            out.swaptionPriceRmseTotal = 0.0;
           
            out.market_swaption = zeros(expirySize,tenorSize);
            out.model_swaption = zeros(expirySize,tenorSize);
            
            tvar = [];
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
                
            case 'global'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 3*dH1_size + 3*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              3*dH1_size + ...;     % constraint on dH    
                                              3*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm3F.dH1.quote)
                        tvar(j) = lgm3F.dH1.quote(j);
                        tvar(length(lgm3F.dH1.quote)+j) = lgm3F.dH2.quote(j);
                        tvar(2*length(lgm3F.dH1.quote)+j) = lgm3F.dH3.quote(j);
                    end
                    
                    for k=1:length(lgm3F.Alpha1.quote)
                       tvar(3*length(lgm3F.dH1.quote)+k) =lgm3F.Alpha1.quote(k);
                       tvar(3*length(lgm3F.dH1.quote)+length(lgm3F.Alpha1.quote)+k) = lgm3F.Alpha2.quote(k);
                       tvar(3*length(lgm3F.dH1.quote)+2*length(lgm3F.Alpha1.quote)+k) = lgm3F.Alpha3.quote(k);
                    end
                    % initial parameters costval
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
                    
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM3FATMSwaptionSurfaceGlobalAlphaAbs',tvar,x, 200, options,'unc',lgm3F,optParams);
                    
                    toc
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        out.dH3.quote(j) = popt(j + 2*dH1_size);
                        lgm3F.dH1.quote(j) = popt(j);
                        lgm3F.dH2.quote(j) = popt(j + dH1_size);
                        lgm3F.dH3.quote(j) = popt(j + 2*dH1_size);
                    end
                    
%                     for j=1:Alpha1_size
%                         out.Alpha1.quote(j) = popt(j + 2*dH1_size);
%                         out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
%                         lgm3F.Alpha1.quote(j) = popt(j + 2*dH1_size);
%                         lgm3F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
%                     end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = abs(popt(j + 3*dH1_size));
                        out.Alpha2.quote(j) = abs(popt(j + 3*dH1_size + Alpha1_size));
                        out.Alpha3.quote(j) = abs(popt(j + 3*dH1_size + 2*Alpha1_size));
                        
                        lgm3F.Alpha1.quote(j) = abs(popt(j + 3*dH1_size));
                        lgm3F.Alpha2.quote(j) = abs(popt(j + 3*dH1_size + Alpha1_size));
                        lgm3F.Alpha3.quote(j) = abs(popt(j + 3*dH1_size + 2*Alpha1_size));
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm3F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
            
            
                    
            otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToFwdCorr(lgm3F,black,calibrationFlag)
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
%             expiry = [0,expiry];
            expiry = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];
            expirySize = size(expiry,1);
            tenorSize = size(tenor,2);
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            params.dH1Orig = lgm3F.dH1Orig;
            params.dH2Orig = lgm3F.dH2Orig;
            params.dH3Orig = lgm3F.dH3Orig;
            
            params.dH1 = lgm3F.dH1;
            params.dH2 = lgm3F.dH2;
            params.dH3 = lgm3F.dH3;
            
            params.Alpha1 = lgm3F.Alpha1;
            params.Alpha2 = lgm3F.Alpha2;
            params.Alpha3 = lgm3F.Alpha3;
            
            out.dH1 = params.dH1;
            out.dH2 = params.dH2;
            out.dH3 = params.dH3;
            
            out.InsFwdCorr = zeros(expirySize, expirySize);
            out.Diff = zeros(expirySize,expirySize);
            out.DiffTotal=0.0;
            
            targetTenor = [2;5;10];
            targetMaturity = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15];
                    
            out.TermCorr = zeros(length(targetMaturity),length(targetTenor),length(targetTenor));
            out.TermCorrDiff = zeros(length(targetMaturity),length(targetTenor),length(targetTenor));
            out.TermCorrDiffTotal = 0;
            
            targetTermCorr = 0.92;
            
            tvar = [];
            
            calibrationType = 'globalFwdLiborLev';
            if nargin < 3
                calibrationType = 'globalFwdLiborLev'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
            
            case 'globalFwdLiborLev'
                    for j=1:length(lgm3F.dH1.quote)
                        tvar(j) = lgm3F.dH1.quote(j);
                        tvar(length(lgm3F.dH1.quote)+j) = lgm3F.dH2.quote(j);
                        tvar(2*length(lgm3F.dH1.quote)+j) = lgm3F.dH3.quote(j);
                        lb(j)= 0.0001;
                        lb(length(lgm3F.dH1.quote)+j)= 0.0001;
                        lb(length(2*lgm3F.dH1.quote)+j)= 0.0001;
                        ub(j)= 10;
                        ub(length(lgm3F.dH1.quote)+j)= 10;
                        ub(length(2*lgm3F.dH1.quote)+j)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    
                    numOfEquation = 3*length(lgm3F.dH1.quote) + expirySize*(expirySize-1)/2;
                    x=  zeros(numOfEquation,1);
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM3FFwdLiborCorrLevGlobal',tvar,x, 200, options,'unc',lgm3F,params);

                    toc
                    
                    for j=1:length(lgm3F.dH1.quote)
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j+length(lgm3F.dH1.quote));
                        out.dH3.quote(j) = popt(j+2*length(lgm3F.dH1.quote));
                        
                        lgm3F.dH1.quote(j) = popt(j);
                        lgm3F.dH2.quote(j) = popt(j+length(lgm3F.dH1.quote));
                        lgm3F.dH3.quote(j) = popt(j+2*length(lgm3F.dH1.quote));
                    end
                    
                    tx = 0;
                    tnr1 = expiry;
                    tau = 0.25;    
                    lgm3FFwdLiborCorr = lgm3F.InsFwdCorrTest2(tx,tnr1,tau);
                    rebonatoCorr = zeros(length(expiry),length(expiry));
                    N=0;
                    for j=1:length(expiry)-1
                        for k=j+1:length(expiry)
                            out.lgm3FFwdCorr(j,k) = lgm3FFwdLiborCorr(j,k);
                            out.rebonatoCorr(j,k) = lgm3F.RebonatoFwdCorr(0,expiry(j),expiry(k));
                            out.Diff(j,k) = out.lgm3FFwdCorr(j,k)-out.rebonatoCorr(j,k);
                            out.DiffTotal = out.DiffTotal + out.Diff(j,k)*out.Diff(j,k);
                            N = N+1;
                        end
                    end
                   
                    out.DiffTotal = sqrt(out.DiffTotal/N);
            
            otherwise
                    disp('unImplemented')
            end
        end
        
    end
    
end



In [ ]:
classdef LGM3F < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        Alpha1
        Alpha2
        Alpha3
        dH1
        dH2
        dH3
        correl12
        correl13
        correl23
        dH1Orig
        dH2Orig
        dH3Orig
        smoothdH
        smoothAlpha
        smoothTermCorr
        targetTermCorr
    end
    
    methods
        function lgm3F = LGM3F(params)
            if nargin > 0
                lgm3F.zeroCurve = params.zeroCurve;
                lgm3F.Alpha1 = params.modelParams('Alpha1');
                lgm3F.Alpha2 = params.modelParams('Alpha2');
                lgm3F.Alpha3 = params.modelParams('Alpha3');
                lgm3F.dH1 =  params.modelParams('dH1');
                lgm3F.dH2 =  params.modelParams('dH2');
                lgm3F.dH3 =  params.modelParams('dH3');
                
                lgm3F.dH1Orig =  params.modelParams('dH1Orig');
                lgm3F.dH2Orig =  params.modelParams('dH2Orig');
                lgm3F.dH3Orig =  params.modelParams('dH3Orig');
                
                lgm3F.correl12 =  params.modelParams('correl12');
                lgm3F.correl13 =  params.modelParams('correl13');
                lgm3F.correl23 =  params.modelParams('correl23');
                
                lgm3F.smoothdH =  params.modelParams('smoothdH');
                lgm3F.smoothAlpha =  params.modelParams('smoothAlpha');
                lgm3F.smoothTermCorr = params.modelParams('smoothTermCorr');
                lgm3F.targetTermCorr = params.modelParams('targetTermCorr');
            end
        end
        
        function out = SimpleSRPrOverHedge(lgm3F,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer))
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer))
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(lgm3F,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(lgm3F,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(lgm3F,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN
                for i =1:pathSize
                    outVec(i) = lgm3F.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = lgm3F.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out =  CMS_PSA_SimpleDate(lgm3F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           tau = 0.25;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = lgm3F.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = lgm3F.DF(to);
           df_TN = lgm3F.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = lgm3F.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = lgm3F.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = lgm3F.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = lgm3F.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,3);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:3
               for j=1:3
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + c_nN(1)*modelStatesT(1,i) + c_nN(2) * modelStatesT(2,i) + c_nN(3) * modelStatesT(3,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
        end
        
        function out = HtT_r(lgm3F,t,T)
            out =  lgm3F.H(T,lgm3F.dH1) - lgm3F.H(t,lgm3F.dH1); 
        end
        
        function out = HtT_s(lgm3F,t,T)
            out =  lgm3F.H(T,lgm3F.dH2) - lgm3F.H(t,lgm3F.dH2); 
        end
        
        function out = HtT_t(lgm3F,t,T)
            out =  lgm3F.H(T,lgm3F.dH3) - lgm3F.H(t,lgm3F.dH3); 
        end
        
        function out = fwdZcVariance(lgm3F, a, b, T1, T2)
            varMatrix = lgm3F.LocalVariance(a,b);
            
            localVariance = zeros(9,1);
            measureChange = zeros(9,1);
            
            localVariance(1) = (lgm3F.HtT_r(a,T1)*lgm3F.HtT_r(a,T1)-lgm3F.HtT_r(a,b)*lgm3F.HtT_r(a,b))*varMatrix(1,1);
            localVariance(2) = (lgm3F.HtT_r(a,T1)*lgm3F.HtT_s(a,T1)-lgm3F.HtT_r(a,b)*lgm3F.HtT_s(a,b))*varMatrix(1,2);
            localVariance(3) = (lgm3F.HtT_r(a,T1)*lgm3F.HtT_t(a,T1)-lgm3F.HtT_r(a,b)*lgm3F.HtT_t(a,b))*varMatrix(1,3);
            
            localVariance(4) = (lgm3F.HtT_s(a,T1)*lgm3F.HtT_r(a,T1)-lgm3F.HtT_s(a,b)*lgm3F.HtT_r(a,b))*varMatrix(2,1);
            localVariance(5) = (lgm3F.HtT_s(a,T1)*lgm3F.HtT_s(a,T1)-lgm3F.HtT_s(a,b)*lgm3F.HtT_s(a,b))*varMatrix(2,2);
            localVariance(6) = (lgm3F.HtT_s(a,T1)*lgm3F.HtT_t(a,T1)-lgm3F.HtT_s(a,b)*lgm3F.HtT_t(a,b))*varMatrix(2,3);
            
            localVariance(7) = (lgm3F.HtT_t(a,T1)*lgm3F.HtT_r(a,T1)-lgm3F.HtT_t(a,b)*lgm3F.HtT_r(a,b))*varMatrix(3,1);
            localVariance(8) = (lgm3F.HtT_t(a,T1)*lgm3F.HtT_s(a,T1)-lgm3F.HtT_t(a,b)*lgm3F.HtT_s(a,b))*varMatrix(3,2);
            localVariance(9) = (lgm3F.HtT_t(a,T1)*lgm3F.HtT_t(a,T1)-lgm3F.HtT_t(a,b)*lgm3F.HtT_t(a,b))*varMatrix(3,3);
            
            
            
            measureChange(1) = (lgm3F.HtT_r(a,T1)-lgm3F.HtT_r(a,b))*lgm3F.HtT_r(a,T2)*varMatrix(1,1);
            measureChange(2) = (lgm3F.HtT_r(a,T1)-lgm3F.HtT_r(a,b))*lgm3F.HtT_s(a,T2)*varMatrix(1,2);
            measureChange(3) = (lgm3F.HtT_r(a,T1)-lgm3F.HtT_r(a,b))*lgm3F.HtT_t(a,T2)*varMatrix(1,3);
            
            measureChange(4) = (lgm3F.HtT_s(a,T1)-lgm3F.HtT_s(a,b))*lgm3F.HtT_r(a,T2)*varMatrix(2,1);
            measureChange(5) = (lgm3F.HtT_s(a,T1)-lgm3F.HtT_s(a,b))*lgm3F.HtT_s(a,T2)*varMatrix(2,2);
            measureChange(6) = (lgm3F.HtT_s(a,T1)-lgm3F.HtT_s(a,b))*lgm3F.HtT_t(a,T2)*varMatrix(2,3);
            
            measureChange(7) = (lgm3F.HtT_t(a,T1)-lgm3F.HtT_t(a,b))*lgm3F.HtT_r(a,T2)*varMatrix(3,1);
            measureChange(8) = (lgm3F.HtT_t(a,T1)-lgm3F.HtT_t(a,b))*lgm3F.HtT_s(a,T2)*varMatrix(3,2);
            measureChange(9) = (lgm3F.HtT_t(a,T1)-lgm3F.HtT_t(a,b))*lgm3F.HtT_t(a,T2)*varMatrix(3,3);
            
            value = localVariance(1) + localVariance(2) + localVariance(3);
            value = value + localVariance(4) + localVariance(5) + localVariance(6);
            value = value + localVariance(7) + localVariance(8) + localVariance(9);
            
            measureChangeTerm = measureChange(1) + measureChange(2) + measureChange(3);
            measureChangeTerm = measureChangeTerm + measureChange(4) + measureChange(5) + measureChange(6);
            measureChangeTerm = measureChangeTerm + measureChange(7) + measureChange(8) + measureChange(9);
            
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
        end
        
        function out = discountFactor(lgm3F, startTime, endTime,numMatTime,modelStatesT)
           zcT = lgm3F.DF(endTime/365.0);
           zct = lgm3F.DF(startTime/365.0);
           beta = lgm3F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = lgm3F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,3);
           discountFactorOut = zeros(1,stateSize);
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i) ...
                   - beta(2) * modelStatesT(2,i) - beta(3) * modelStatesT(3,i)); 
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoff(lgm3F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = lgm3F.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = lgm3F.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,3);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(lgm3F,fromTime, toTime)
            
            out = eye(3,3);
        end
        
        function out = stateLocalVariance(lgm3F,fromTime, toTime)
            out = lgm3F.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(lgm3F,from,to)

            sig(1) = lgm3F.Alpha1;
            sig(2) = lgm3F.Alpha2;
            sig(3) = lgm3F.Alpha3;
            corr_rs = lgm3F.correl12;
            corr_rt = lgm3F.correl13;
            corr_st = lgm3F.correl23;
            
            corr(1,1) = 1.0;
            corr(1,2) = corr_rs.quote(1);
            corr(1,3) = corr_rt.quote(1);
            
            corr(2,1) = corr(1,2);
            corr(2,2) = 1.0;
            corr(2,3) = corr_st.quote(1);
            
            corr(3,1) = corr(1,3);
            corr(3,2) = corr(2,3);
            corr(3,3) = 1.0;
            
            alphaSize = length(lgm3F.Alpha1.quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           variances = zeros(3,3);
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end;
                if (i< alphaSize) vol(3) = sig(3).quote(i);else vol(3) = sig(3).quote(alphaSize);end;
                correl(1,1) =1.0;
                correl(2,2) =1.0;
                correl(3,3) =1.0;
                
                if (i< alphaSize) correl(1,2) = corr_rs.quote(i);else correl(1,2) = corr_rs.quote(alphaSize);end;
                if (i< alphaSize) correl(1,3) = corr_rt.quote(i);else correl(1,3) = corr_rt.quote(alphaSize);end;
                if (i< alphaSize) correl(2,3) = corr_st.quote(i);else correl(2,3) = corr_st.quote(alphaSize);end;
                
                correl(2,1) = correl(1,2);
                correl(3,1) = correl(1,3);
                correl(3,2) = correl(2,3);
                
                for j=1:3
                    for k=1:3
                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end
        
        function out = H(lgm3F,t,dH)
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    break;
                end
            end
            tidx = i;
            sum = 0.0;
            prev_t = 0.0;
            curr_t = 0.0;
            for i=1:tidx
                if i< tidx
                    curr_t = dH.tenor(i);
                else
                    curr_t = t;
                end
                sum = sum + dH.quote(i)*(curr_t-prev_t);

                prev_t = dH.tenor(i);
            end
            out = sum;
        end
        
        function out = Alpha_Test(lgm3F,t,Alpha)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(Alpha.tenor)
                alpha = Alpha.quote(i);
                if t<= Alpha.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end
        
        function out = dH_Test(lgm3F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = dH.quote(i);
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = dH.quote(i);
            end
            out = sum;
        end

        function out = H_Test(lgm3F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = sum + dH.quote(i)*(t-prev_t);
                    inRangeYN = true;
                    break;
                else
                    sum = sum + dH.quote(i)*(dH.tenor(i)-prev_t);
                end
                prev_t = dH.tenor(i);
            end
            
            if ~inRangeYN 
                sum = sum + dH.quote(i) * (t - dH.tenor(i));
            end
            out = sum;
        end
        
        function c_Test = C_Test(lgm3F,tx, tnr,tau)
            pv01_1 = lgm3F.PV01(tx,tx+tnr,tau);
            fsr_1  = lgm3F.Fsr(tx,tx+tnr,tau);
            c_nN_1 = lgm3F.C_nN(tx,tx+tnr,tau,pv01_1,fsr_1);
            c_Test = c_nN_1;
        end

        function beta = Beta(lgm3F,t,T)
            beta =[];
            
            beta(1) = lgm3F.H(T,lgm3F.dH1)-lgm3F.H(t,lgm3F.dH1);
            beta(2) = lgm3F.H(T,lgm3F.dH2)-lgm3F.H(t,lgm3F.dH2);
            beta(3) = lgm3F.H(T,lgm3F.dH3)-lgm3F.H(t,lgm3F.dH3);
            
        end
        
        function beta = BetaDate(lgm3F,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            beta =[];
            
            beta(1) = lgm3F.H(T,lgm3F.dH1)-lgm3F.H(t,lgm3F.dH1);
            beta(2) = lgm3F.H(T,lgm3F.dH2)-lgm3F.H(t,lgm3F.dH2);
            beta(3) = lgm3F.H(T,lgm3F.dH3)-lgm3F.H(t,lgm3F.dH3);
            
        end
        
        function c_nN = C_nN(lgm3F,tn,tN,tau,pnN,ynN)
            p = @lgm3F.DF;
            c_nN=[0.0 0.0 0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm3F.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
                c_nN(3) = c_nN(3) +tau * beta(3) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            c_nN(3) = c_nN(3)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            c_nN(3) = c_nN(3) + beta(3)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
            c_nN(3) = c_nN(3)/pnN;
            
        end
        
        function c_nN = C_nNDate(lgm3F,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @lgm3F.DF;
            c_nN=[0.0 0.0 0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = lgm3F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
                c_nN(3) = c_nN(3) +tau * beta(3) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            c_nN(3) = c_nN(3)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            c_nN(3) = c_nN(3) + beta(3)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
            c_nN(3) = c_nN(3)/pnN;
        end
        
        function g_nN = G_nN(lgm3F,tn,tN,tau,pnN)
            p = @lgm3F.DF;
            g_nN=[0.0 0.0 0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm3F.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
                g_nN(3) = g_nN(3) +tau * beta(3) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
            g_nN(3) = g_nN(3)/pnN;
        end
        
        function g_nN = G_nNDate(lgm3F,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @lgm3F.DF;
            g_nN=[0.0 0.0 0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = lgm3F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
                g_nN(3) = g_nN(3) +tau * beta(3) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
            g_nN(3) = g_nN(3)/pnN;
        end
        
        function h_nTerm = H_nTerm(lgm3F,tn,tTerm)
           h_nTerm =[];
           beta = lgm3F.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
           h_nTerm(3) = -1.0*beta(3);
        end
        
        function h_nTerm = H_nTermDate(lgm3F,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = lgm3F.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
           h_nTerm(3) = -1.0*beta(3);
        end
        
        function out = CMSDigitalArg(lgm3F,to,tp,tenor,tau)
           % probability that CMS rate > strike
           p_nN = lgm3F.PV01(to,to+tenor,tau);
           df_Tn = lgm3F.DF(to);
           df_TN = lgm3F.DF(to + tenor);
           y_nN = (df_Tn - df_TN)/p_nN;
           c_nN = lgm3F.C_nN(to,to + tenor,tau,p_nN,y_nN);
           g_nN = lgm3F.G_nN(to,to + tenor,tau,p_nN);
           h_nTerm = lgm3F.H_nTerm(to,tp);
           varCovar = lgm3F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:3
               for j=1:3
                    driftTerm = driftTerm + c_nN(i) * (h_nTerm(j) + g_nN(j)) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:3
               for j=1:3
                    variance = variance + c_nN(i) * c_nN(j) * varCovar(i,j);
               end
           end
           
           out.mu = y_nN + driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN;
           out.varCovar = varCovar;
        end
        
        function out = CMSDigital(lgm3F,to,tp,tenor,tau,strike)
           cmsDigitalArg= lgm3F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(lgm3F,to,tp,tenor,tau,lower,upper)
            lower_value = lgm3F.CMSDigital(to,tp,tenor,tau,lower);
            upper_value = lgm3F.CMSDigital(to,tp,tenor,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalArg(lgm3F,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           p_nN1 = lgm3F.PV01(to,to+tenor1,tau);
           df_Tn1 = lgm3F.DF(to);
           df_TN1 = lgm3F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = lgm3F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = lgm3F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = lgm3F.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = lgm3F.PV01(to,to+tenor2,tau);
           df_Tn2 = lgm3F.DF(to);
           df_TN2 = lgm3F.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = lgm3F.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = lgm3F.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = lgm3F.H_nTerm(to,tp);
           
           varCovar = lgm3F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:3
               for j=1:3
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:3
               for j=1:3
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1);
           out.sig2 = sqrt(variance2);
           
           out.pmu = driftTerm1 - driftTerm2;
           out.pmu1 = driftTerm1;
           out.pmu2 = driftTerm2;
           
           out.y1 = y_nN1;
           out.y2 = y_nN2;
           
           
        end
        
        function out = CMSSpreadDigitalArgDate(lgm3F,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = lgm3F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = lgm3F.DF(to);
           df_TN1 = lgm3F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = lgm3F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = lgm3F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = lgm3F.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = lgm3F.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = lgm3F.DF(to);
           df_TN2 = lgm3F.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = lgm3F.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = lgm3F.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = lgm3F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = lgm3F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:3
               for j=1:3
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:3
               for j=1:3
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigital(lgm3F,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm3F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(lgm3F,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm3F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalRange(lgm3F,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm3F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = lgm3F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalRangeDate(lgm3F,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm3F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = lgm3F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function atmSwaption = ATMSwaption(lgm3F,tx,tnr)
            tau = 0.25;
            pv01 = lgm3F.PV01(tx,tx+tnr,tau);
            fsr  = lgm3F.Fsr(tx,tx+tnr,tau);
            c_nN = lgm3F.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = lgm3F.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:3
                for j=1:3
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        
        function spreadInfo = SpreadInfo(lgm3F,tx,tnr1,tnr2)
            tau = 0.25;
            cmsSpreadDigitalArg =  lgm3F.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function impVariance = ImpVariance(lgm3F,tx,tnr1)
            tau = 0.25;
            pv01_1 = lgm3F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm3F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm3F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            variances = lgm3F.LocalVariance(0,tx);
            var1 = 0;
            for j=1:3
                for k=1:3
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end
        
        function impCorrel = ImpCorrel(lgm3F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = lgm3F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm3F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm3F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm3F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm3F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm3F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm3F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:3
                for k=1:3
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impCovar = ImpCovar(lgm3F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = lgm3F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm3F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm3F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm3F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm3F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm3F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm3F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:3
                for k=1:3
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCovar = coVar;
        end
        
        function instFwdCorr = InsFwdCorr(lgm3F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(3,1);
            dH_tT1(1) = (lgm3F.H_Test(tx + tnr1 +0.001,lgm3F.dH1)-lgm3F.H_Test(tx + tnr1,lgm3F.dH1))/0.001;
            dH_tT1(2) = (lgm3F.H_Test(tx + tnr1 +0.001,lgm3F.dH2)-lgm3F.H_Test(tx + tnr1,lgm3F.dH2))/0.001;
            dH_tT1(3) = (lgm3F.H_Test(tx + tnr1 +0.001,lgm3F.dH3)-lgm3F.H_Test(tx + tnr1,lgm3F.dH3))/0.001;
            
            dH_tT2 = zeros(3,1);
            dH_tT2(1) = (lgm3F.H_Test(tx + tnr2 +0.001,lgm3F.dH1)-lgm3F.H_Test(tx + tnr2,lgm3F.dH1))/0.001;
            dH_tT2(2) = (lgm3F.H_Test(tx + tnr2 +0.001,lgm3F.dH2)-lgm3F.H_Test(tx + tnr2,lgm3F.dH2))/0.001;
            dH_tT2(3) = (lgm3F.H_Test(tx + tnr2 +0.001,lgm3F.dH3)-lgm3F.H_Test(tx + tnr2,lgm3F.dH3))/0.001;
            
            instVariances = (lgm3F.LocalVariance(0,tx+0.001)-lgm3F.LocalVariance(0,tx))/0.001;
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:3
                for k=1:3
                    var1 = var1 + dH_tT1(j)*dH_tT1(k)*instVariances(j,k);
                    var2 = var2 + dH_tT2(j)*dH_tT2(k)*instVariances(j,k);
                    covar = covar + dH_tT1(j)*dH_tT2(k)*instVariances(j,k);
                end
            end
            instFwdCorr = covar/sqrt(var1*var2);
        end
        
        function instFwdCorrTest = InsFwdCorrTest(lgm3F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(3,1);
            dH_tT1(1) = lgm3F.dH_Test(tx + tnr1,lgm3F.dH1);
            dH_tT1(2) = lgm3F.dH_Test(tx + tnr1,lgm3F.dH2);
            dH_tT1(3) = lgm3F.dH_Test(tx + tnr1,lgm3F.dH3);
            
            dH_tT2 = zeros(3,1);
            dH_tT2(1) = lgm3F.dH_Test(tx + tnr2,lgm3F.dH1);
            dH_tT2(2) = lgm3F.dH_Test(tx + tnr2,lgm3F.dH2);
            dH_tT2(3) = lgm3F.dH_Test(tx + tnr2,lgm3F.dH3);
            
            Alpha_t = zeros(3,1);
            Alpha_t(1) = lgm3F.Alpha_Test(tx,lgm3F.Alpha1);
            Alpha_t(2) = lgm3F.Alpha_Test(tx,lgm3F.Alpha2);
            Alpha_t(3) = lgm3F.Alpha_Test(tx,lgm3F.Alpha3);
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:3
                var1 = var1 + dH_tT1(j)*dH_tT1(j)*Alpha_t(j)*Alpha_t(j);
                var2 = var2 + dH_tT2(j)*dH_tT2(j)*Alpha_t(j)*Alpha_t(j);
                covar = covar + dH_tT1(j)*dH_tT2(j)*Alpha_t(j)*Alpha_t(j);
            end
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function instFwdSwapCorrTest = InsSwapFwdCorrTest(lgm3F,tx,tnr1,tnr2)
            tau = 0.25;
            C_tT1 = zeros(3,1);
            pv01_1 = lgm3F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm3F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm3F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
            C_tT1(2) = c_nN_1(2);
            C_tT1(3) = c_nN_1(3);
            
            C_tT2 = zeros(3,1);
            pv01_2 = lgm3F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm3F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm3F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
            C_tT2(2) = c_nN_2(2);
            C_tT2(3) = c_nN_2(3);
            
            Alpha_t = zeros(3,1);
            Alpha_t(1) = lgm3F.Alpha_Test(tx,lgm3F.Alpha1);
            Alpha_t(2) = lgm3F.Alpha_Test(tx,lgm3F.Alpha2);
            Alpha_t(3) = lgm3F.Alpha_Test(tx,lgm3F.Alpha3);
            
            covar = 0.0;
            var1 = 0.0;
            var2 = 0.0;
            for j=1:3
                var1 = var1 + C_tT1(j)*C_tT1(j)*Alpha_t(j)*Alpha_t(j);
                var2 = var2 + C_tT2(j)*C_tT2(j)*Alpha_t(j)*Alpha_t(j);
                covar = covar + C_tT1(j)*C_tT2(j)*Alpha_t(j)*Alpha_t(j);
            end
            instFwdSwapCorrTest = covar/sqrt(var1*var2);
        end
        
        function instFwdCovarTest = InsFwdCovarTest(lgm3F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(3,1);
            dH_tT1(1) = lgm3F.dH_Test(tx + tnr1,lgm3F.dH1);
            dH_tT1(2) = lgm3F.dH_Test(tx + tnr1,lgm3F.dH2);
            dH_tT1(3) = lgm3F.dH_Test(tx + tnr1,lgm3F.dH3);
            
            dH_tT2 = zeros(3,1);
            dH_tT2(1) = lgm3F.dH_Test(tx + tnr2,lgm3F.dH1);
            dH_tT2(2) = lgm3F.dH_Test(tx + tnr2,lgm3F.dH2);
            dH_tT2(3) = lgm3F.dH_Test(tx + tnr2,lgm3F.dH3);
            
            Alpha_t = zeros(3,1);
            Alpha_t(1) = lgm3F.Alpha_Test(tx,lgm3F.Alpha1);
            Alpha_t(2) = lgm3F.Alpha_Test(tx,lgm3F.Alpha2);
            Alpha_t(3) = lgm3F.Alpha_Test(tx,lgm3F.Alpha3);
            
            covar = 0.0;
            for j=1:3
                covar = covar + dH_tT1(j)*dH_tT2(j)*Alpha_t(j)*Alpha_t(j);
            end
            instFwdCovarTest = covar;
        end
        
        function instFwdCovarTestMatrix = InsFwdCovarTest2(lgm3F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCovarTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),3);
                eta =  zeros(3,3);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH1) - lgm3F.H_Test(newtnr(i),lgm3F.dH1))/tau;
                    dH_tT(i,2) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH2) - lgm3F.H_Test(newtnr(i),lgm3F.dH2))/tau;
                    dH_tT(i,3) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH3) - lgm3F.H_Test(newtnr(i),lgm3F.dH3))/tau;
                end

                eta(1,1) = lgm3F.Alpha_Test(tx,lgm3F.Alpha1);
                eta(2,2) = lgm3F.Alpha_Test(tx,lgm3F.Alpha2);
                eta(3,3) = lgm3F.Alpha_Test(tx,lgm3F.Alpha3);
                
                eta(1,2) = 0;
                eta(1,3) = 0;
                
                eta(2,1) = 0;
                eta(2,3) = 0;
                
                eta(3,1) = 0;
                eta(3,2) = 0;
                
                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
            end
            
        end
        
        function instFwdCorrTestMatrix = InsFwdCorrTest2(lgm3F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCorrTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),3);
                eta =  zeros(3,3);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH1) - lgm3F.H_Test(newtnr(i),lgm3F.dH1))/tau;
                    dH_tT(i,2) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH2) - lgm3F.H_Test(newtnr(i),lgm3F.dH2))/tau;
                    dH_tT(i,3) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH3) - lgm3F.H_Test(newtnr(i),lgm3F.dH3))/tau;
                end

                eta(1,1) = lgm3F.Alpha_Test(tx,lgm3F.Alpha1);
                eta(2,2) = lgm3F.Alpha_Test(tx,lgm3F.Alpha2);
                eta(3,3) = lgm3F.Alpha_Test(tx,lgm3F.Alpha3);
                
                eta(1,2) = 0;
                eta(1,3) = 0;
                
                eta(2,1) = 0;
                eta(2,3) = 0;
                
                eta(3,1) = 0;
                eta(3,2) = 0;
                
                testLoadingD = dH_tT*eta;
                
                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
                %normalize the correlation matrix and calculate the loading matrix D
                rInv_m = zeros(length(newtnr),length(newtnr));
                for i=1:length(newtnr)
                    rInv_m(i,i) = 1.0/sqrt(coVar(i,i));
                end
                instFwdCorrTestMatrix = zeros(length(newtnr),length(newtnr));
                instFwdCorrTestMatrix = rInv_m*instFwdCovarTestMatrix*rInv_m';
                
            end
            
        end
        
        function deltaHMatrix = DeltaHTest2(lgm3F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                deltaHMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                
                dH_tT = zeros(length(newtnr),3);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH1) - lgm3F.H_Test(newtnr(i),lgm3F.dH1))/tau;
                    dH_tT(i,2) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH2) - lgm3F.H_Test(newtnr(i),lgm3F.dH2))/tau;
                    dH_tT(i,3) = (lgm3F.H_Test(newtnr(i) + tau,lgm3F.dH3) - lgm3F.H_Test(newtnr(i),lgm3F.dH3))/tau;
                
                end

                deltaHMatrix = dH_tT;
            end
            
        end
        
        function etaMatrix = EtaTest2(lgm3F,tx)
            eta =  zeros(3,3);
            eta(1,1) = lgm3F.Alpha_Test(tx,lgm3F.Alpha1);
            eta(2,2) = lgm3F.Alpha_Test(tx,lgm3F.Alpha2);
            eta(3,3) = lgm3F.Alpha_Test(tx,lgm3F.Alpha3);
            
            eta(1,2) = 0;
            eta(1,3) = 0;
            
            eta(2,1) = 0;
            eta(2,3) = 0;
            
            eta(3,1) = 0;
            eta(3,2) = 0;
            
            etaMatrix = eta;
            
        end
        
        function instFwdSwapCovarTest = InsSwapFwdCovarTest(lgm3F,tx,tnr1,tnr2)
            tau = 0.25;
            C_tT1 = zeros(3,1);
            pv01_1 = lgm3F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm3F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm3F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
            C_tT1(2) = c_nN_1(2);
            C_tT1(3) = c_nN_1(3);
            
            C_tT2 = zeros(3,1);
            pv01_2 = lgm3F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm3F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm3F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
            C_tT2(2) = c_nN_2(2);
            C_tT2(3) = c_nN_2(3);
            
            Alpha_t = zeros(3,1);
            Alpha_t(1) = lgm3F.Alpha_Test(tx,lgm3F.Alpha1);
            Alpha_t(2) = lgm3F.Alpha_Test(tx,lgm3F.Alpha2);
            Alpha_t(2) = lgm3F.Alpha_Test(tx,lgm3F.Alpha3);
            
            covar = 0.0;
            for j=1:3
                covar = covar + C_tT1(j)*C_tT2(j)*Alpha_t(j)*Alpha_t(j);
            end
            instFwdSwapCovarTest = covar;
        end
        
        function out = RebonatoFwdCorr(lgm3F,tx,tnr1,tnr2)
           rho_inf = 0.4;
           rho_decay = 0.15;
           
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
       
        function out = CalibrateToATMSwaptionSurface(lgm3F,black,calibrationFlag)
            % Market Data Object , Vanilla Object
            % targetSwaption
            % targetCap
            
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% target instruments set up
            params.swaption_expiry_size = expirySize;
            params.swaption_tenor_size = tenorSize;
            
            params.swaption_calibration_yn = ones(expirySize,tenorSize);
            
            params.swaptionSurface.expiry = swaptionVolSurface.expiry;
            params.swaptionSurface.tenor  = swaptionVolSurface.tenor;
            params.swaptionSurface.quote  = zeros(expirySize,tenorSize);
            params.swaptionSurface.strike = zeros(expirySize,tenorSize);
            params.swaptionSurface.blackPrice = zeros(expirySize,tenorSize);
            
            params.swaptionSurface.blackSwaptionVega = zeros(expirySize,tenorSize);
            params.swaptionSurface.blackSwaptionFsr = zeros(expirySize,tenorSize);
            
            % setting target diagonal swaption
            for i=1:expirySize
                for j=1:tenorSize
                    if params.swaption_calibration_yn(i,j) == 1
                        Ti = params.swaptionSurface.expiry(i);
                        Tj = params.swaptionSurface.tenor(j);
                        blackVolQuote = swaptionVolSurface.swaptionVol(expiry(i),tenor(j));
                        params.swaptionSurface.quote(i,j) = blackVolQuote;
                        params.swaptionSurface.strike(i,j) = black.Fsr(Ti,Ti + Tj,0.25);
                        params.swaptionSurface.blackPrice(i,j) = black.BlackSwaptionWithVol(Ti,Tj,params.swaptionSurface.strike(i,j),blackVolQuote);
                        params.swaptionSurface.blackSwaptionVega(i,j) = black.BlackSwaptionVegaWithVol(Ti,Tj,params.swaptionSurface.strike(i,j),blackVolQuote);
                    end
                end

            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            params.dH1 = lgm3F.dH1;
            params.dH2 = lgm3F.dH2;
            params.dH3 = lgm3F.dH3;
            
            dH1_size = length(lgm3F.dH1.quote);
            Alpha1_size = length(lgm3F.Alpha1.quote);
            
            params.dH1_size = dH1_size;
            params.Alpha1_size = Alpha1_size;
            
            params.Alpha1 = lgm3F.Alpha1;
            params.Alpha2 = lgm3F.Alpha2;
            params.Alpha3 = lgm3F.Alpha3;
            
            lgm3F.dH1    = params.dH1;
            lgm3F.dH2    = params.dH2;
            lgm3F.dH3    = params.dH3;
            
            lgm3F.Alpha1 = params.Alpha1;
            lgm3F.Alpha2 = params.Alpha2;
            lgm3F.Alpha3 = params.Alpha3;
            
            params.dH1Orig = lgm3F.dH1Orig;
            params.dH2Orig = lgm3F.dH2Orig;
            params.dH3Orig = lgm3F.dH3Orig;
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            out.dH1 = params.dH1;
            out.dH2 = params.dH2;
            out.dH3 = params.dH3;
            
            out.Alpha1 = params.Alpha1;
            out.Alpha2 = params.Alpha2;
            out.Alpha3 = params.Alpha3;
            
            out.blackSwaptionPrice = params.swaptionSurface.blackPrice;
            out.lgm3FSwaptionPrice = zeros(expirySize,tenorSize);
            out.swaptionPriceRelDiff = zeros(expirySize,tenorSize);
            out.swaptionVolRelDiff = zeros(expirySize,tenorSize);
            
            out.swaptionPriceRmseTotal = 0.0;
            out.swaptionVolRmseTotal = 0.0;
            
            out.market_swaption = zeros(expirySize,tenorSize);
            out.market_vega = zeros(expirySize,tenorSize);
            out.model_swaption = zeros(expirySize,tenorSize);
            
            tvar = [];
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
                
            case 'global'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 3*dH1_size + 3*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              3*dH1_size + ...;     % constraint on dH    
                                              3*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm3F.dH1.quote)
                        tvar(j) = lgm3F.dH1.quote(j);
                        tvar(length(lgm3F.dH1.quote)+j) = lgm3F.dH2.quote(j);
                        tvar(2*length(lgm3F.dH1.quote)+j) = lgm3F.dH3.quote(j);
                    end
                    
                    for k=1:length(lgm3F.Alpha1.quote)
                       tvar(3*length(lgm3F.dH1.quote)+k) =lgm3F.Alpha1.quote(k);
                       tvar(3*length(lgm3F.dH1.quote)+length(lgm3F.Alpha1.quote)+k) = lgm3F.Alpha2.quote(k);
                       tvar(3*length(lgm3F.dH1.quote)+2*length(lgm3F.Alpha1.quote)+k) = lgm3F.Alpha3.quote(k);
                    end
                    % initial parameters costval
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
                    
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM3FATMSwaptionSurfaceGlobalAlphaAbs',tvar,x, 200, options,'unc',lgm3F,optParams);
                    
                    toc
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        out.dH3.quote(j) = popt(j + 2*dH1_size);
                        lgm3F.dH1.quote(j) = popt(j);
                        lgm3F.dH2.quote(j) = popt(j + dH1_size);
                        lgm3F.dH3.quote(j) = popt(j + 2*dH1_size);
                    end
                    
%                     for j=1:Alpha1_size
%                         out.Alpha1.quote(j) = popt(j + 2*dH1_size);
%                         out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
%                         lgm3F.Alpha1.quote(j) = popt(j + 2*dH1_size);
%                         lgm3F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
%                     end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = abs(popt(j + 3*dH1_size));
                        out.Alpha2.quote(j) = abs(popt(j + 3*dH1_size + Alpha1_size));
                        out.Alpha3.quote(j) = abs(popt(j + 3*dH1_size + 2*Alpha1_size));
                        
                        lgm3F.Alpha1.quote(j) = abs(popt(j + 3*dH1_size));
                        lgm3F.Alpha2.quote(j) = abs(popt(j + 3*dH1_size + Alpha1_size));
                        lgm3F.Alpha3.quote(j) = abs(popt(j + 3*dH1_size + 2*Alpha1_size));
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm3F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
           
            case 'globalVegaWeight'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 3*dH1_size + 3*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              3*dH1_size + ...;     % constraint on dH    
                                              3*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm3F.dH1.quote)
                        tvar(j) = lgm3F.dH1.quote(j);
                        tvar(length(lgm3F.dH1.quote)+j) = lgm3F.dH2.quote(j);
                        tvar(2*length(lgm3F.dH1.quote)+j) = lgm3F.dH3.quote(j);
                    end
                    
                    for k=1:length(lgm3F.Alpha1.quote)
                       tvar(3*length(lgm3F.dH1.quote)+k) =lgm3F.Alpha1.quote(k);
                       tvar(3*length(lgm3F.dH1.quote)+length(lgm3F.Alpha1.quote)+k) = lgm3F.Alpha2.quote(k);
                       tvar(3*length(lgm3F.dH1.quote)+2*length(lgm3F.Alpha1.quote)+k) = lgm3F.Alpha3.quote(k);
                    end
                    % initial parameters costval
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
                    
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM3FATMSwaptionSurfaceGlobalVegaWeight',tvar,x, 200, options,'unc',lgm3F,optParams);
                    
                    toc
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        out.dH3.quote(j) = popt(j + 2*dH1_size);
                        lgm3F.dH1.quote(j) = popt(j);
                        lgm3F.dH2.quote(j) = popt(j + dH1_size);
                        lgm3F.dH3.quote(j) = popt(j + 2*dH1_size);
                    end
                    
%                     for j=1:Alpha1_size
%                         out.Alpha1.quote(j) = popt(j + 2*dH1_size);
%                         out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
%                         lgm3F.Alpha1.quote(j) = popt(j + 2*dH1_size);
%                         lgm3F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
%                     end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = abs(popt(j + 3*dH1_size));
                        out.Alpha2.quote(j) = abs(popt(j + 3*dH1_size + Alpha1_size));
                        out.Alpha3.quote(j) = abs(popt(j + 3*dH1_size + 2*Alpha1_size));
                        
                        lgm3F.Alpha1.quote(j) = abs(popt(j + 3*dH1_size));
                        lgm3F.Alpha2.quote(j) = abs(popt(j + 3*dH1_size + Alpha1_size));
                        lgm3F.Alpha3.quote(j) = abs(popt(j + 3*dH1_size + 2*Alpha1_size));
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.market_vega(i,j) = params.swaptionSurface.blackSwaptionVega(i,j);
                            out.model_swaption(i,j)  = lgm3F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionVolRelDiff(i,j) = (out.model_swaption(i,j) - out.market_swaption(i,j))/out.market_vega(i,j);
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            out.swaptionVolRmseTotal = out.swaptionVolRmseTotal + out.swaptionVolRelDiff(i,j)*out.swaptionVolRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
                    out.swaptionVolRmseTotal = sqrt(out.swaptionVolRmseTotal/N);
                    
            otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToFwdCorr(lgm3F,black,calibrationFlag)
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
%             expiry = [0,expiry];
            expiry = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];
            expirySize = size(expiry,1);
            tenorSize = size(tenor,2);
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            params.dH1Orig = lgm3F.dH1Orig;
            params.dH2Orig = lgm3F.dH2Orig;
            params.dH3Orig = lgm3F.dH3Orig;
            
            params.dH1 = lgm3F.dH1;
            params.dH2 = lgm3F.dH2;
            params.dH3 = lgm3F.dH3;
            
            params.Alpha1 = lgm3F.Alpha1;
            params.Alpha2 = lgm3F.Alpha2;
            params.Alpha3 = lgm3F.Alpha3;
            
            out.dH1 = params.dH1;
            out.dH2 = params.dH2;
            out.dH3 = params.dH3;
            
            out.InsFwdCorr = zeros(expirySize, expirySize);
            out.Diff = zeros(expirySize,expirySize);
            out.DiffTotal=0.0;
            
            targetTenor = [2;5;10];
            targetMaturity = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15];
                    
            out.TermCorr = zeros(length(targetMaturity),length(targetTenor),length(targetTenor));
            out.TermCorrDiff = zeros(length(targetMaturity),length(targetTenor),length(targetTenor));
            out.TermCorrDiffTotal = 0;
            
            targetTermCorr = 0.92;
            
            tvar = [];
            
            calibrationType = 'globalFwdLiborLev';
            if nargin < 3
                calibrationType = 'globalFwdLiborLev'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
            
            case 'globalFwdLiborLev'
                    for j=1:length(lgm3F.dH1.quote)
                        tvar(j) = lgm3F.dH1.quote(j);
                        tvar(length(lgm3F.dH1.quote)+j) = lgm3F.dH2.quote(j);
                        tvar(2*length(lgm3F.dH1.quote)+j) = lgm3F.dH3.quote(j);
                        lb(j)= 0.0001;
                        lb(length(lgm3F.dH1.quote)+j)= 0.0001;
                        lb(length(2*lgm3F.dH1.quote)+j)= 0.0001;
                        ub(j)= 10;
                        ub(length(lgm3F.dH1.quote)+j)= 10;
                        ub(length(2*lgm3F.dH1.quote)+j)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    
                    numOfEquation = 3*length(lgm3F.dH1.quote) + expirySize*(expirySize-1)/2;
                    x=  zeros(numOfEquation,1);
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM3FFwdLiborCorrLevGlobal',tvar,x, 200, options,'unc',lgm3F,params);

                    toc
                    
                    for j=1:length(lgm3F.dH1.quote)
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j+length(lgm3F.dH1.quote));
                        out.dH3.quote(j) = popt(j+2*length(lgm3F.dH1.quote));
                        
                        lgm3F.dH1.quote(j) = popt(j);
                        lgm3F.dH2.quote(j) = popt(j+length(lgm3F.dH1.quote));
                        lgm3F.dH3.quote(j) = popt(j+2*length(lgm3F.dH1.quote));
                    end
                    
                    tx = 0;
                    tnr1 = expiry;
                    tau = 0.25;    
                    lgm3FFwdLiborCorr = lgm3F.InsFwdCorrTest2(tx,tnr1,tau);
                    rebonatoCorr = zeros(length(expiry),length(expiry));
                    N=0;
                    for j=1:length(expiry)-1
                        for k=j+1:length(expiry)
                            out.lgm3FFwdCorr(j,k) = lgm3FFwdLiborCorr(j,k);
                            out.rebonatoCorr(j,k) = lgm3F.RebonatoFwdCorr(0,expiry(j),expiry(k));
                            out.Diff(j,k) = out.lgm3FFwdCorr(j,k)-out.rebonatoCorr(j,k);
                            out.DiffTotal = out.DiffTotal + out.Diff(j,k)*out.Diff(j,k);
                            N = N+1;
                        end
                    end
                   
                    out.DiffTotal = sqrt(out.DiffTotal/N);
            
            otherwise
                    disp('unImplemented')
            end
        end
        
    end
    
end



In [ ]:
classdef LGM2FRisky_Freq < IRModelRisky
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        Alpha1
        Alpha2
        dH1
        dH2
        correl
        dH1Orig
        dH2Orig
        smoothdH
        smoothAlpha
        numOfFactor
        tau
    end
    
    methods
        function lgm2F = LGM2FRisky_Freq(params)
            if nargin > 0
                if isKey(params.modelParams,'freq')
                    lgm2F.tau = 1.0/params.modelParams('freq');
                else
                    lgm2F.tau = 0.25; % default
                end
                % initialize it with the paraent class attribute
                lgm2F.zeroCurve = params.zeroCurve;
                lgm2F.discountCurve = params.discountCurve;
                lgm2F.mktData = params.mktData;
                lgm2F.modelParams = params.modelParams;
                
                lgm2F.Alpha1 = params.modelParams('Alpha1');
                lgm2F.Alpha2 = params.modelParams('Alpha2');
                lgm2F.dH1 =  params.modelParams('dH1');
                lgm2F.dH2 =  params.modelParams('dH2');
                
                lgm2F.dH1Orig =  params.modelParams('dH1Orig');
                lgm2F.dH2Orig =  params.modelParams('dH2Orig');
                lgm2F.correl =  params.modelParams('correl');
                lgm2F.smoothdH =  params.modelParams('smoothdH');
                lgm2F.smoothAlpha =  params.modelParams('smoothAlpha');
                
                lgm2F.numOfFactor = 2;
            end
        end
        
        function out = SimpleSRPrOverHedge(lgm2F,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(lgm2F,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(lgm2F,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(lgm2F,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = lgm2F.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out = SimpleDRPr(lgm2F,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS)
            outVec = 0.0;
            if cmsC >= ralower && cmsC <= raupper && spreadC >= ralowerS && spreadC <= raupperS
                outVec = 1.0;
            end
            out = outVec;
        end
        
        function out = DRPr(lgm2F,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
%                     outVec(i) = lgm2F.SimpleDRPrOverHedge(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            else
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            end
            
            out = outVec;
        end
        
        function out =  Libor_SimpleDate(lgm2F,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           %CMS
           fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
           fwdEndDate = fwdStartDate.AddDate(tenor,'year');
           fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
           deltaT = (fwdEndTime - fwdStartTime)/365.0;
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           
           stateSize = size(modelStatesT,2);
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           df_Tn = lgm2F.discountFactor(evalTime,fwdStartTime,numMatTime,modelStatesT);
           df_TN = lgm2F.discountFactor(evalTime,fwdEndTime,numMatTime,modelStatesT);
           
           liborOut = zeros(1,stateSize);
           
           for i=1:stateSize
                liborOut(i) = (df_Tn(i)/df_TN(i) - 1.0)/deltaT;
           end
           
           out = liborOut;
            
        end
        
        function out =  CMS_PSA_SimpleDate(lgm2F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
%            tau = 0.25;
           tau = lgm2F.tau;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           if tenor < 1
               tenorMonth = round(tenor*12);
               tNDate = toDate.AddDate(tenorMonth,'month');
           else
               tNDate = toDate.AddDate(tenor,'year');
           end
           
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = lgm2F.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = lgm2F.DF(to);
           df_TN = lgm2F.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = lgm2F.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = lgm2F.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = lgm2F.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = lgm2F.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:2
               for j=1:2
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + c_nN(1)*modelStatesT(1,i) + c_nN(2) * modelStatesT(2,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
        end
        
        function out = HtT_r(lgm2F,t,T)
            out =  lgm2F.H(T,lgm2F.dH1) - lgm2F.H(t,lgm2F.dH1); 
        end
        
        function out = HtT_s(lgm2F,t,T)
            out =  lgm2F.H(T,lgm2F.dH2) - lgm2F.H(t,lgm2F.dH2); 
        end
        
        function out = fwdZcVariance(lgm2F, a, b, T1, T2)
            varMatrix = lgm2F.LocalVariance(a,b);
            
            localVariance = zeros(4,1);
            measureChange = zeros(4,1);
            
            localVariance(1) = (lgm2F.HtT_r(a,T1)*lgm2F.HtT_r(a,T1)-lgm2F.HtT_r(a,b)*lgm2F.HtT_r(a,b))*varMatrix(1,1);
            localVariance(2) = (lgm2F.HtT_r(a,T1)*lgm2F.HtT_s(a,T1)-lgm2F.HtT_r(a,b)*lgm2F.HtT_s(a,b))*varMatrix(1,2);
            localVariance(3) = (lgm2F.HtT_s(a,T1)*lgm2F.HtT_r(a,T1)-lgm2F.HtT_s(a,b)*lgm2F.HtT_r(a,b))*varMatrix(2,1);
            localVariance(4) = (lgm2F.HtT_s(a,T1)*lgm2F.HtT_s(a,T1)-lgm2F.HtT_s(a,b)*lgm2F.HtT_s(a,b))*varMatrix(2,2);
            
            measureChange(1) = (lgm2F.HtT_r(a,T1)-lgm2F.HtT_r(a,b))*lgm2F.HtT_r(a,T2)*varMatrix(1,1);
            measureChange(2) = (lgm2F.HtT_r(a,T1)-lgm2F.HtT_r(a,b))*lgm2F.HtT_s(a,T2)*varMatrix(1,2);
            measureChange(3) = (lgm2F.HtT_s(a,T1)-lgm2F.HtT_s(a,b))*lgm2F.HtT_r(a,T2)*varMatrix(2,1);
            measureChange(4) = (lgm2F.HtT_s(a,T1)-lgm2F.HtT_s(a,b))*lgm2F.HtT_s(a,T2)*varMatrix(2,2);
            
            value = localVariance(1) + localVariance(2);
            value = value + localVariance(3) + localVariance(4);
            
            measureChangeTerm = measureChange(1) + measureChange(2);
            measureChangeTerm = measureChangeTerm + measureChange(3) + measureChange(4);
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
        end
        
        function out = discountFactorRisky(lgm2F, startTime, endTime,numMatTime,modelStatesT)
           %exception handling when startTime <= 0 -> deterministic discout 
           if startTime <= 0
               zcT = lgm2F.DFRisky(endTime/365.0);
               stateSize = size(modelStatesT,2);
               discountFactorOut = zeros(1,stateSize);
               for i=1:stateSize
                   discountFactorOut(i) = zcT; 
               end
               out = discountFactorOut;
           else
               zcT = lgm2F.DFRisky(endTime/365.0);
               zct = lgm2F.DFRisky(startTime/365.0);
               beta = lgm2F.Beta(startTime/365.0,endTime/365.0);
               zcDriftTerm = lgm2F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
               %numOfPath
               stateSize = size(modelStatesT,2);
               discountFactorOut = zeros(1,stateSize);

               for i=1:stateSize
                   discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i) ...
                       - beta(2) * modelStatesT(2,i)); 
               end

               out = discountFactorOut;
           end
        end
        
        function out = discountFactor(lgm2F, startTime, endTime,numMatTime,modelStatesT)
           zcT = lgm2F.DF(endTime/365.0);
           
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           % deterministic discount factor
           if startTime <= 1e-8
                for i=1:stateSize
                    discountFactorOut(i) = zcT;
                end
                out = discountFactorOut;
                return;
           end
           
           % deterministic discount factor or maturity in the past
           if startTime >= endTime
                out = ones(1,stateSize);
                return;
           end
           
           zct = lgm2F.DF(startTime/365.0);
           beta = lgm2F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = lgm2F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i) ...
                   - beta(2) * modelStatesT(2,i)); 
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoffRisky(lgm2F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = lgm2F.DFRisky(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = lgm2F.discountFactorRisky(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = discountPayoff(lgm2F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = lgm2F.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = lgm2F.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(lgm2F,fromTime, toTime)
            
            out = eye(2,2);
        end
        
        function out = stateLocalVariance(lgm2F,fromTime, toTime)
            out = lgm2F.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(lgm2F,from,to)

            sig(1) = lgm2F.Alpha1;
            sig(2) = lgm2F.Alpha2;
            corr_rs = lgm2F.correl;
            corr(1,1) = 1.0;
            corr(2,1) = corr_rs.quote(1);
            corr(1,2) = corr(2,1);
            corr(2,2) = 1.0;
            
            alphaSize = length(lgm2F.Alpha1.quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           variances = zeros(2,2);
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end;
                correl(1,1) =1.0;
                correl(2,2) =1.0;
                if (i< alphaSize) correl(2,1) = corr_rs.quote(i);else correl(2,1) = corr_rs.quote(alphaSize);end;
                correl(1,2) = correl(2,1);
                for j=1:2
                    for k=1:2
                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end
        
        function out = H(lgm2F,t,dH)
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    break;
                end
            end
            tidx = i;
            sum = 0.0;
            prev_t = 0.0;
            curr_t = 0.0;
            for i=1:tidx
                if i< tidx
                    curr_t = dH.tenor(i);
                else
                    curr_t = t;
                end
                sum = sum + dH.quote(i)*(curr_t-prev_t);

                prev_t = dH.tenor(i);
            end
            out = sum;
        end


        function beta = Beta(lgm2F,t,T)
            beta =[];
            
            beta(1) = lgm2F.H(T,lgm2F.dH1)-lgm2F.H(t,lgm2F.dH1);
            beta(2) = lgm2F.H(T,lgm2F.dH2)-lgm2F.H(t,lgm2F.dH2);
            
        end
        
        function beta = BetaDate(lgm2F,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            beta =[];
            
            beta(1) = lgm2F.H(T,lgm2F.dH1)-lgm2F.H(t,lgm2F.dH1);
            beta(2) = lgm2F.H(T,lgm2F.dH2)-lgm2F.H(t,lgm2F.dH2);
            
        end
        
        function c_nN = C_nN(lgm2F,tn,tN,tau,pnN,ynN)
            p = @lgm2F.DF;
            c_nN=[0.0 0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm2F.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
        end
        
        function c_nN = C_nNDate(lgm2F,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @lgm2F.DF;
            c_nN=[0.0 0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = lgm2F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
        end
        
        function g_nN = G_nN(lgm2F,tn,tN,tau,pnN)
            p = @lgm2F.DF;
            g_nN=[0.0 0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm2F.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
        end
        
        function g_nN = G_nNDate(lgm2F,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @lgm2F.DF;
            g_nN=[0.0 0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = lgm2F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
        end
        
        function h_nTerm = H_nTerm(lgm2F,tn,tTerm)
           h_nTerm =[];
           beta = lgm2F.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
        end
        
        function h_nTerm = H_nTermDate(lgm2F,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = lgm2F.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
        end
        
        function out = CDDigitalArgDate(lgm2F,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           toTime =  toDate.DateDiff(valueDate);
           to      = toTime/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1Time = tN1Date.DateDiff(valueDate);
           tN1   =  tN1Time/365.0;
           
           deltaT = (tN1Time - toTime)/365.0;
           
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(tN1);
           
           c_nN1 = lgm2F.BetaDate(valueDate,toDate,tN1Date);
           c_nTerm = lgm2F.BetaDate(valueDate,toDate,tpDate);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:2
               for j=1:2
                    driftTerm = driftTerm + c_nN1(i) * c_nTerm(j) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:2
               for j=1:2
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
%            out.mu = 0.5*variance  -1.0*log(df_TN1/df_Tn1) - driftTerm;
           out.mu = -0.5*variance  +1.0*log(df_TN1/df_Tn1) + driftTerm;
           
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
           out.deltaT = deltaT;
        end
        
        function out = CMSDigitalArgDate(lgm2F,valueDate,toDate,tpDate,tenor1,tau)
           % probability that strikeU > CMS tenor1 > strikeL
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = lgm2F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = lgm2F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = lgm2F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = lgm2F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           variance1 = 0.0;
           for i=1:2
               for j=1:2
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
               end
           end
           
           out.fwdCMS = y_nN1;
           out.mu = (y_nN1 + driftTerm1);
           out.sigma = sqrt(variance1);
           out.c_nN1 = c_nN1;
           out.mu1 = y_nN1 + driftTerm1;
           out.sig1 = sqrt(variance1/to);
           out.varCovar = varCovar;
           
        end
        
        function out = CMSDigitalArg(lgm2F,to,tp,tenor,tau)
           % probability that CMS rate > strike
           p_nN = lgm2F.PV01(to,to+tenor,tau);
           df_Tn = lgm2F.DF(to);
           df_TN = lgm2F.DF(to + tenor);
           y_nN = (df_Tn - df_TN)/p_nN;
           c_nN = lgm2F.C_nN(to,to + tenor,tau,p_nN,y_nN);
           g_nN = lgm2F.G_nN(to,to + tenor,tau,p_nN);
           h_nTerm = lgm2F.H_nTerm(to,tp);
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:2
               for j=1:2
                    driftTerm = driftTerm + c_nN(i) * (h_nTerm(j) + g_nN(j)) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:2
               for j=1:2
                    variance = variance + c_nN(i) * c_nN(j) * varCovar(i,j);
               end
           end
           
           out.mu = y_nN + driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN;
           out.varCovar = varCovar;
        end
        
        function out = CDDigitalDate(lgm2F,valueDate,toDate,tpDate,tenor,tau,strike)
           
           % probability that CD > strikeCD 
           
           % CD
           cdDigitalArg = lgm2F.CDDigitalArgDate(valueDate,toDate,tpDate,tenor,tau);
           mu = cdDigitalArg.mu;
           sigma = cdDigitalArg.sigma;
           c_nN = cdDigitalArg.c_nN;
           varCovar = cdDigitalArg.varCovar;
           deltaT = cdDigitalArg.deltaT;
           
           
           z1 = (mu + log(1.0 + deltaT*strike))/sigma;
           if ~isreal(z1) || ~isfinite(z1)
               out = 0.0;
           else
               out = H_ncdf(z1);
           end
        end
        
        function out = CMSDigital(lgm2F,to,tp,tenor,tau,strike)
           cmsDigitalArg= lgm2F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalDate(lgm2F,valueDate,toDate,tpDate,tenor1,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = lgm2F.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor1,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(lgm2F,to,tp,tenor,tau,lower,upper)
            lower_value = lgm2F.CMSDigital(to,tp,tenor,tau,lower);
            upper_value = lgm2F.CMSDigital(to,tp,tenor,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSDigitalRangeDate(lgm2F,valueDate,toDate,tpDate,tenor1,tau,lower,upper)
            lower_value = lgm2F.CMSDigitalDate(valueDate,toDate,tpDate,tenor1,tau,lower);
            upper_value = lgm2F.CMSDigitalDate(valueDate,toDate,tpDate,tenor1,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalArg(lgm2F,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           p_nN1 = lgm2F.PV01(to,to+tenor1,tau);
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = lgm2F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = lgm2F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = lgm2F.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = lgm2F.PV01(to,to+tenor2,tau);
           df_Tn2 = lgm2F.DF(to);
           df_TN2 = lgm2F.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = lgm2F.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = lgm2F.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = lgm2F.H_nTerm(to,tp);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:2
               for j=1:2
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigitalArgDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = lgm2F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = lgm2F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = lgm2F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = lgm2F.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = lgm2F.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = lgm2F.DF(to);
           df_TN2 = lgm2F.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = lgm2F.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = lgm2F.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = lgm2F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:2
               for j=1:2
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigital(lgm2F,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSCMSSpreadDigitalDate(lgm2F,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,strike,strikeS)
           
           % probability that CMS tenor1 - CMS tenor 2 > strike
           % CMS 1
           cmsDigitalArg = lgm2F.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           c_nN = cmsDigitalArg.c_nN1;
           varCovar = cmsDigitalArg.varCovar;
           
           %CMS Spread
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           
           c_nN1 = cmsSpreadDigitalArg.c_nN1;
           c_nN2 = cmsSpreadDigitalArg.c_nN2;
           
           muS = cmsSpreadDigitalArg.mu;
           sigmaS = cmsSpreadDigitalArg.sigma;
           
           % covariance between CMS Rate & CMS Spread
           covar = 0.0;
           for i=1:2
               for j=1:2
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           corr = covar/(sigma*sigmaS);
           
           z1 = (mu - strike)/sigma;
           z2 = (muS - strikeS)/sigmaS;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CDCMSSpreadDigitalDate(lgm2F,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,strike,strikeS)
           
           % probability that CD > strikeCD 
           % probability that CMS tenor1 - CMS tenor 2 > strike
           
           % CD
           cdDigitalArg = lgm2F.CDDigitalArgDate(valueDate,toDate,tpDate,tenor,tau);
           mu = cdDigitalArg.mu;
           sigma = cdDigitalArg.sigma;
           c_nN = cdDigitalArg.c_nN;
           varCovar = cdDigitalArg.varCovar;
           deltaT = cdDigitalArg.deltaT;
           
           %CMS Spread
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           
           c_nN1 = cmsSpreadDigitalArg.c_nN1;
           c_nN2 = cmsSpreadDigitalArg.c_nN2;
           
           muS = cmsSpreadDigitalArg.mu;
           sigmaS = cmsSpreadDigitalArg.sigma;
           
           % covariance between CMS Rate & CMS Spread
           covar = 0.0;
           for i=1:2
               for j=1:2
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           corr = covar/(sigma*sigmaS);
           
           
%            z1 = (mu - log(1.0 + deltaT*strike))/sigma;

%            z1 = -1.0*(mu + log(1.0 + deltaT*strike))/sigma;
%            z2 = (muS - strikeS)/sigmaS;
%            
           z1 = (mu + log(1.0 + deltaT*strike))/sigma;
           z2 = -1.0*(muS - strikeS)/sigmaS;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           
           if ~isreal(z1) || ~isfinite(z1) || ~isreal(z2) || ~isfinite(z2)
               out = 0.0;
           else
               out = mvncdf(xbar, mbar, corrBar);
           end
%            out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CMSSpreadDigitalRange(lgm2F,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm2F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = lgm2F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalRangeDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm2F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = lgm2F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSCMSSpreadDigitalRangeDate(lgm2F,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = lgm2F.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = lgm2F.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = lgm2F.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = lgm2F.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,upperS);
            
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function out = CDCMSSpreadDigitalRangeDate(lgm2F,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = lgm2F.CDCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = lgm2F.CDCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = lgm2F.CDCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = lgm2F.CDCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,upperS);
            
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function out = CDDigitalRangeDate(lgm2F,valueDate,toDate,tpDate,tenor,tau,lower,upper)
            upper_value = lgm2F.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,upper);
            lower_value = lgm2F.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,lower);
            out = upper_value - lower_value;
            
        end
        
        function atmSwaption = ATMSwaption(lgm2F,tx,tnr)
%             tau = 0.25;
            tau = lgm2F.tau;
            pv01 = lgm2F.PV01(tx,tx+tnr,tau);
            fsr  = lgm2F.Fsr(tx,tx+tnr,tau);
            c_nN = lgm2F.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = lgm2F.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:2
                for j=1:2
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        
        function spreadInfo = SpreadInfo(lgm2F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = lgm2F.tau;
            cmsSpreadDigitalArg =  ck2F.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
       
        function out = CalibrateToATMSwaptionSurface(lgm2F,black,calibrationFlag,numOfIter)
            % Market Data Object , Vanilla Object
            % targetSwaption
            % targetCap
            
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% target instruments set up
            params.swaption_expiry_size = expirySize;
            params.swaption_tenor_size = tenorSize;
            
            params.swaption_calibration_yn = ones(expirySize,tenorSize);
            
            params.swaptionSurface.expiry = swaptionVolSurface.expiry;
            params.swaptionSurface.tenor  = swaptionVolSurface.tenor;
            params.swaptionSurface.quote  = zeros(expirySize,tenorSize);
            params.swaptionSurface.strike = zeros(expirySize,tenorSize);
            params.swaptionSurface.blackPrice = zeros(expirySize,tenorSize);
            tau1 = lgm2F.tau;
            % setting target diagonal swaption
            for i=1:expirySize
                for j=1:tenorSize
                    if params.swaption_calibration_yn(i,j) == 1
                        Ti = params.swaptionSurface.expiry(i);
                        Tj = params.swaptionSurface.tenor(j);
                        blackVolQuote = swaptionVolSurface.swaptionVol(expiry(i),tenor(j));
                        params.swaptionSurface.quote(i,j) = blackVolQuote;
%                         params.swaptionSurface.strike(i,j) = black.Fsr(Ti,Ti + Tj,0.25);
                        params.swaptionSurface.strike(i,j) = black.Fsr(Ti,Ti + Tj,tau1);
                        params.swaptionSurface.blackPrice(i,j) = black.BlackSwaptionWithVol(Ti,Tj,params.swaptionSurface.strike(i,j),blackVolQuote);
                    end
                end

            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            params.dH1 = lgm2F.dH1;
            params.dH2 = lgm2F.dH2;
            
            dH1_size = length(lgm2F.dH1.quote);
            Alpha1_size = length(lgm2F.Alpha1.quote);
            
            params.dH1_size = dH1_size;
            params.Alpha1_size = Alpha1_size;
            
            params.Alpha1 = lgm2F.Alpha1;
            params.Alpha2 = lgm2F.Alpha2;
            
            lgm2F.dH1    = params.dH1;
            lgm2F.dH2    = params.dH2;
            
            lgm2F.Alpha1 = params.Alpha1;
            lgm2F.Alpha2 = params.Alpha2;
            
            params.dH1Orig = lgm2F.dH1Orig;
            params.dH2Orig = lgm2F.dH2Orig;
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            out.dH1 = params.dH1;
            out.dH2 = params.dH2;
            
            out.Alpha1 = params.Alpha1;
            out.Alpha2 = params.Alpha2;
            
            out.blackSwaptionPrice = params.swaptionSurface.blackPrice;
            out.lgm2FSwaptionPrice = zeros(expirySize,tenorSize);
            out.swaptionPriceRelDiff = zeros(expirySize,tenorSize);
            out.swaptionPriceRmseTotal = 0.0;
           
            out.market_swaption = zeros(expirySize,tenorSize);
            out.model_swaption = zeros(expirySize,tenorSize);
            
            tvar = [];
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
                
            case 'global'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH    
                                              2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    x=  zeros(optParams.numOfEquation,1);
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobal',tvar,x, numOfIter, options,'unc',lgm2F,optParams);
                    toc
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                        lgm2F.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        lgm2F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
                    
            otherwise
                    disp('unImplemented')
            end
        end
        
        
    end
    
end



In [ ]:
classdef LGM2FRisky < IRModelRisky
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        Alpha1
        Alpha2
        dH1
        dH2
        correl
        dH1Orig
        dH2Orig
        smoothdH
        smoothAlpha
        numOfFactor
        numOfFactors
    end
    
    methods
        function lgm2F = LGM2FRisky(params)
            if nargin > 0
                % initialize it with the paraent class attribute
                lgm2F.zeroCurve = params.zeroCurve;
                lgm2F.discountCurve = params.discountCurve;
                lgm2F.mktData = params.mktData;
                lgm2F.modelParams = params.modelParams;
                
                lgm2F.Alpha1 = params.modelParams('Alpha1');
                lgm2F.Alpha2 = params.modelParams('Alpha2');
                lgm2F.dH1 =  params.modelParams('dH1');
                lgm2F.dH2 =  params.modelParams('dH2');
                
                lgm2F.dH1Orig =  params.modelParams('dH1Orig');
                lgm2F.dH2Orig =  params.modelParams('dH2Orig');
                lgm2F.correl =  params.modelParams('correl');
                lgm2F.smoothdH =  params.modelParams('smoothdH');
                lgm2F.smoothAlpha =  params.modelParams('smoothAlpha');
                
                lgm2F.numOfFactor = 2;
                lgm2F.numOfFactors = 2;
            end
        end
        
        function out = SimpleSRPrOverHedge(lgm2F,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(lgm2F,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(lgm2F,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(lgm2F,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = lgm2F.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out = SimpleDRPrOverHedge(lgm2F,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS)
            outVec = 0.0;
            out1 = SimpleSRPrOverHedge(lgm2F,cmsC,cmsN,ralower,raupper);
            out2 = SimpleSRPrOverHedge(lgm2F,spreadC,spreadN,ralowerS,raupperS);
%             out2 = SimpleSRPrOverHedge(lgm2F,spreadC,spreadN,ralowerS,raupperS);
            outVec = out1*out2;
            out = outVec;
        end
        
        function out = SimpleDRPr(lgm2F,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS)
            outVec = 0.0;
            if cmsC >= ralower && cmsC <= raupper && spreadC >= ralowerS && spreadC <= raupperS
                outVec = 1.0;
            end
            out = outVec;
        end
        
        function out = DRPr(lgm2F,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            elseif simpleYN == 2
                for i =1:pathSize
%                     outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                    outVec(i) = lgm2F.SimpleDRPrOverHedge(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            else
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            end
            
            out = outVec;
        end
        
        function out =  Libor_SimpleDate(lgm2F,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           %CMS
           fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
           fwdEndDate = fwdStartDate.AddDate(tenor,'year');
           fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
           deltaT = (fwdEndTime - fwdStartTime)/365.0;
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           
           stateSize = size(modelStatesT,2);
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           df_Tn = lgm2F.discountFactor(evalTime,fwdStartTime,numMatTime,modelStatesT);
           df_TN = lgm2F.discountFactor(evalTime,fwdEndTime,numMatTime,modelStatesT);
           
           liborOut = zeros(1,stateSize);
           
           for i=1:stateSize
                liborOut(i) = (df_Tn(i)/df_TN(i) - 1.0)/deltaT;
           end
           
           out = liborOut;
            
        end
        
        function out =  CMS_SimpleDate(lgm2F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           tau = 0.25;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           if tenor < 1
               tenorMonth = round(tenor*12);
               tNDate = toDate.AddDate(tenorMonth,'month');
           else
               tNDate = toDate.AddDate(tenor,'year');
           end
           
           tNTime = tNDate.DateDiff(valueDate); 
           tN     = tNTime/365.0;
            
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           addMonthUnit = round(12*tau);
           cnt = round((tN-to)/tau);
           
%            p_nN = lgm2F.PV01Date(valueDate,toDate,tNDate,tau);
           stateSize = size(modelStatesT,2);
           
           annuity = zeros(1,stateSize);
           df_Tp = zeros(1,stateSize);
           
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           for i=1:cnt
                tpDate = toDate.AddDate(i*addMonthUnit,'month');
                tpTime = tpDate.DateDiff(valueDate);
                df_Tp = lgm2F.discountFactor(observeTime,tpTime,numMatTime,modelStatesT);
                annuity = annuity + tau * df_Tp;
            end
           
           df_Tn = lgm2F.discountFactor(observeTime,observeTime,numMatTime,modelStatesT);
           df_TN = lgm2F.discountFactor(observeTime,tNTime,numMatTime,modelStatesT);
           
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           for i=1:stateSize
                cms_PCA_SimpleOut(i) = (df_Tn(i) - df_TN(i))/annuity(i);
           end
           
           out = cms_PCA_SimpleOut;
            
        end
        
        function out = Rate_IndexGeneric(lgm2F,rateType,valueDate,observeTime,numMatTime,tenor,modelStatesT)
            if strcmp(rateType,'CMS')
                out = CMS_PSA_SimpleDateGeneric(lgm2F,valueDate,observeTime,numMatTime, tenor,modelStatesT);
            elseif strcmp(rateType,'LIBOR')
                out = Libor_SimpleDateGeneric(lgm2F,valueDate,observeTime,numMatTime, tenor,modelStatesT);
            else
                error('unsupported rateType!')
            end
        end
        
        function out = Libor_SimpleDateGeneric(lgm2F,valueDate,observeTime,numMatTime,tenor,modelStatesT)
            if observeTime < 0
                allFixings = lgm2F.modelParams('allFixings');
                fixingDate = AddDate(valueDate,observeTime,'day');
                tenorMonth = round(tenor*12);
                fixingTicker = ['LIBOR' num2str(tenorMonth,'%d') 'M'];
                irIndexFixings = allFixings(fixingTicker);
                try
                    fixing = irIndexFixings(StrDate(fixingDate,'str'));
                    stateSize = size(modelStatesT,2);
                    out = ones(1,stateSize)*fixing;
                    
                catch exception
                    throw(exception)
                end
                
            else
                out = Libor_SimpleDate(lgm2F,valueDate,observeTime,observeTime,numMatTime, tenor,modelStatesT);
            end
        end
        
        function out =  CMS_PSA_SimpleDateGeneric(lgm2F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
            if observeTime < 0
                allFixings = lgm2F.modelParams('allFixings');
                fixingDate = AddDate(valueDate,observeTime,'day');
                fixingTicker = ['CMS' num2str(tenor,'%d') 'Y'];
                irIndexFixings = allFixings(fixingTicker);
                try
                    fixing = irIndexFixings(StrDate(fixingDate,'str'));
                    stateSize = size(modelStatesT,2);
                    out = ones(1,stateSize)*fixing;
                    
                catch exception
                    throw(exception)
                end
                
            else
                out = CMS_PSA_SimpleDate(lgm2F,valueDate,observeTime,numMatTime, tenor,modelStatesT);
            end
        
        end
        
        function out =  CMS_PSA_SimpleDate(lgm2F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           tau = 0.25;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           if tenor < 1
               tenorMonth = round(tenor*12);
               tNDate = toDate.AddDate(tenorMonth,'month');
           else
               tNDate = toDate.AddDate(tenor,'year');
           end
           
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = lgm2F.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = lgm2F.DF(to);
           df_TN = lgm2F.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = lgm2F.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = lgm2F.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = lgm2F.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = lgm2F.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:2
               for j=1:2
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + c_nN(1)*modelStatesT(1,i) + c_nN(2) * modelStatesT(2,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
        end
        
        function out = HtT_r(lgm2F,t,T)
            out =  lgm2F.H(T,lgm2F.dH1) - lgm2F.H(t,lgm2F.dH1); 
        end
        
        function out = HtT_s(lgm2F,t,T)
            out =  lgm2F.H(T,lgm2F.dH2) - lgm2F.H(t,lgm2F.dH2); 
        end
        
        function out = fwdZcVariance(lgm2F, a, b, T1, T2)
            varMatrix = lgm2F.LocalVariance(a,b);
            
            localVariance = zeros(4,1);
            measureChange = zeros(4,1);
            
            localVariance(1) = (lgm2F.HtT_r(a,T1)*lgm2F.HtT_r(a,T1)-lgm2F.HtT_r(a,b)*lgm2F.HtT_r(a,b))*varMatrix(1,1);
            localVariance(2) = (lgm2F.HtT_r(a,T1)*lgm2F.HtT_s(a,T1)-lgm2F.HtT_r(a,b)*lgm2F.HtT_s(a,b))*varMatrix(1,2);
            localVariance(3) = (lgm2F.HtT_s(a,T1)*lgm2F.HtT_r(a,T1)-lgm2F.HtT_s(a,b)*lgm2F.HtT_r(a,b))*varMatrix(2,1);
            localVariance(4) = (lgm2F.HtT_s(a,T1)*lgm2F.HtT_s(a,T1)-lgm2F.HtT_s(a,b)*lgm2F.HtT_s(a,b))*varMatrix(2,2);
            
            measureChange(1) = (lgm2F.HtT_r(a,T1)-lgm2F.HtT_r(a,b))*lgm2F.HtT_r(a,T2)*varMatrix(1,1);
            measureChange(2) = (lgm2F.HtT_r(a,T1)-lgm2F.HtT_r(a,b))*lgm2F.HtT_s(a,T2)*varMatrix(1,2);
            measureChange(3) = (lgm2F.HtT_s(a,T1)-lgm2F.HtT_s(a,b))*lgm2F.HtT_r(a,T2)*varMatrix(2,1);
            measureChange(4) = (lgm2F.HtT_s(a,T1)-lgm2F.HtT_s(a,b))*lgm2F.HtT_s(a,T2)*varMatrix(2,2);
            
            value = localVariance(1) + localVariance(2);
            value = value + localVariance(3) + localVariance(4);
            
            measureChangeTerm = measureChange(1) + measureChange(2);
            measureChangeTerm = measureChangeTerm + measureChange(3) + measureChange(4);
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
        end
        
        function out = discountFactorRisky(lgm2F, startTime, endTime,numMatTime,modelStatesT)
           %exception handling when startTime <= 0 -> deterministic discout 
           if startTime <= 0
               zcT = lgm2F.DFRisky(endTime/365.0);
               stateSize = size(modelStatesT,2);
               discountFactorOut = zeros(1,stateSize);
               for i=1:stateSize
                   discountFactorOut(i) = zcT; 
               end
               out = discountFactorOut;
           else
               zcT = lgm2F.DFRisky(endTime/365.0);
               zct = lgm2F.DFRisky(startTime/365.0);
               beta = lgm2F.Beta(startTime/365.0,endTime/365.0);
               zcDriftTerm = lgm2F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
               %numOfPath
               stateSize = size(modelStatesT,2);
               discountFactorOut = zeros(1,stateSize);

               for i=1:stateSize
                   discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i) ...
                       - beta(2) * modelStatesT(2,i)); 
               end

               out = discountFactorOut;
           end
        end
        
        function out = discountFactor(lgm2F, startTime, endTime,numMatTime,modelStatesT)
           zcT = lgm2F.DF(endTime/365.0);
           
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           % deterministic discount factor
           if startTime <= 1e-8
                for i=1:stateSize
                    discountFactorOut(i) = zcT;
                end
                out = discountFactorOut;
                return;
           end
           
           % deterministic discount factor or maturity in the past
           if startTime >= endTime
                out = ones(1,stateSize);
                return;
           end
           
           zct = lgm2F.DF(startTime/365.0);
           beta = lgm2F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = lgm2F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i) ...
                   - beta(2) * modelStatesT(2,i)); 
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoffRisky(lgm2F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = lgm2F.DFRisky(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = lgm2F.discountFactorRisky(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = discountPayoff(lgm2F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = lgm2F.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = lgm2F.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(lgm2F,fromTime, toTime)
            
            out = eye(2,2);
        end
        
        function out = stateLocalVariance(lgm2F,fromTime, toTime)
            out = lgm2F.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(lgm2F,from,to)

            sig(1) = lgm2F.Alpha1;
            sig(2) = lgm2F.Alpha2;
            corr_rs = lgm2F.correl;
            corr(1,1) = 1.0;
            corr(2,1) = corr_rs.quote(1);
            corr(1,2) = corr(2,1);
            corr(2,2) = 1.0;
            
            alphaSize = length(lgm2F.Alpha1.quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           variances = zeros(2,2);
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end;
                correl(1,1) =1.0;
                correl(2,2) =1.0;
                if (i< alphaSize) correl(2,1) = corr_rs.quote(i);else correl(2,1) = corr_rs.quote(alphaSize);end;
                correl(1,2) = correl(2,1);
                for j=1:2
                    for k=1:2
                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end
        
        function out = H(lgm2F,t,dH)
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    break;
                end
            end
            tidx = i;
            sum = 0.0;
            prev_t = 0.0;
            curr_t = 0.0;
            for i=1:tidx
                if i< tidx
                    curr_t = dH.tenor(i);
                else
                    curr_t = t;
                end
                sum = sum + dH.quote(i)*(curr_t-prev_t);

                prev_t = dH.tenor(i);
            end
            out = sum;
        end


        function beta = Beta(lgm2F,t,T)
            beta =[];
            
            beta(1) = lgm2F.H(T,lgm2F.dH1)-lgm2F.H(t,lgm2F.dH1);
            beta(2) = lgm2F.H(T,lgm2F.dH2)-lgm2F.H(t,lgm2F.dH2);
            
        end
        
        function beta = BetaDate(lgm2F,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            beta =[];
            
            beta(1) = lgm2F.H(T,lgm2F.dH1)-lgm2F.H(t,lgm2F.dH1);
            beta(2) = lgm2F.H(T,lgm2F.dH2)-lgm2F.H(t,lgm2F.dH2);
            
        end
        
        function c_nN = C_nN(lgm2F,tn,tN,tau,pnN,ynN)
            p = @lgm2F.DF;
            c_nN=[0.0 0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm2F.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
        end
        
        function c_nN = C_nNDate(lgm2F,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @lgm2F.DF;
            c_nN=[0.0 0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = lgm2F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
        end
        
        function g_nN = G_nN(lgm2F,tn,tN,tau,pnN)
            p = @lgm2F.DF;
            g_nN=[0.0 0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm2F.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
        end
        
        function g_nN = G_nNDate(lgm2F,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @lgm2F.DF;
            g_nN=[0.0 0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = lgm2F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
        end
        
        function h_nTerm = H_nTerm(lgm2F,tn,tTerm)
           h_nTerm =[];
           beta = lgm2F.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
        end
        
        function h_nTerm = H_nTermDate(lgm2F,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = lgm2F.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
        end
        
        function out = CDDigitalArgDate(lgm2F,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           toTime =  toDate.DateDiff(valueDate);
           to      = toTime/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1Time = tN1Date.DateDiff(valueDate);
           tN1   =  tN1Time/365.0;
           
           deltaT = (tN1Time - toTime)/365.0;
           
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(tN1);
           
           c_nN1 = lgm2F.BetaDate(valueDate,toDate,tN1Date);
           c_nTerm = lgm2F.BetaDate(valueDate,toDate,tpDate);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:2
               for j=1:2
                    driftTerm = driftTerm + c_nN1(i) * c_nTerm(j) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:2
               for j=1:2
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
%            out.mu = 0.5*variance  -1.0*log(df_TN1/df_Tn1) - driftTerm;
           out.mu = -0.5*variance  +1.0*log(df_TN1/df_Tn1) + driftTerm;
           
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
           out.deltaT = deltaT;
        end
        
        function out = CMSDigitalArgDate(lgm2F,valueDate,toDate,tpDate,tenor1,tau)
           % probability that strikeU > CMS tenor1 > strikeL
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = lgm2F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = lgm2F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = lgm2F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = lgm2F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           variance1 = 0.0;
           for i=1:2
               for j=1:2
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
               end
           end
           
           out.fwdCMS = y_nN1;
           out.mu = (y_nN1 + driftTerm1);
           out.sigma = sqrt(variance1);
           out.c_nN1 = c_nN1;
           % additional output
           out.c_nN = c_nN1;
           out.mu1 = y_nN1 + driftTerm1;
           out.sig1 = sqrt(variance1/to);
           out.varCovar = varCovar;
           
        end
        
        function out = CMSDigitalArg(lgm2F,to,tp,tenor,tau)
           % probability that CMS rate > strike
           p_nN = lgm2F.PV01(to,to+tenor,tau);
           df_Tn = lgm2F.DF(to);
           df_TN = lgm2F.DF(to + tenor);
           y_nN = (df_Tn - df_TN)/p_nN;
           c_nN = lgm2F.C_nN(to,to + tenor,tau,p_nN,y_nN);
           g_nN = lgm2F.G_nN(to,to + tenor,tau,p_nN);
           h_nTerm = lgm2F.H_nTerm(to,tp);
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:2
               for j=1:2
                    driftTerm = driftTerm + c_nN(i) * (h_nTerm(j) + g_nN(j)) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:2
               for j=1:2
                    variance = variance + c_nN(i) * c_nN(j) * varCovar(i,j);
               end
           end
           
           out.mu = y_nN + driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN;
           out.varCovar = varCovar;
        end
        
        function out = CDDigitalDate(lgm2F,valueDate,toDate,tpDate,tenor,tau,strike)
           
           % probability that CD > strikeCD 
           
           % CD
           cdDigitalArg = lgm2F.CDDigitalArgDate(valueDate,toDate,tpDate,tenor,tau);
           mu = cdDigitalArg.mu;
           sigma = cdDigitalArg.sigma;
           c_nN = cdDigitalArg.c_nN;
           varCovar = cdDigitalArg.varCovar;
           deltaT = cdDigitalArg.deltaT;
           
           
           z1 = (mu + log(1.0 + deltaT*strike))/sigma;
           if ~isreal(z1) || ~isfinite(z1)
               out = 0.0;
           else
               out = H_ncdf(z1);
           end
        end
        
        function out = CMSDigital(lgm2F,to,tp,tenor,tau,strike)
           cmsDigitalArg= lgm2F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalDate(lgm2F,valueDate,toDate,tpDate,tenor1,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = lgm2F.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor1,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(lgm2F,to,tp,tenor,tau,lower,upper)
            lower_value = lgm2F.CMSDigital(to,tp,tenor,tau,lower);
            upper_value = lgm2F.CMSDigital(to,tp,tenor,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSDigitalRangeDate(lgm2F,valueDate,toDate,tpDate,tenor1,tau,lower,upper)
            lower_value = lgm2F.CMSDigitalDate(valueDate,toDate,tpDate,tenor1,tau,lower);
            upper_value = lgm2F.CMSDigitalDate(valueDate,toDate,tpDate,tenor1,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalArg(lgm2F,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           p_nN1 = lgm2F.PV01(to,to+tenor1,tau);
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = lgm2F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = lgm2F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = lgm2F.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = lgm2F.PV01(to,to+tenor2,tau);
           df_Tn2 = lgm2F.DF(to);
           df_TN2 = lgm2F.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = lgm2F.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = lgm2F.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = lgm2F.H_nTerm(to,tp);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:2
               for j=1:2
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigitalArgDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = lgm2F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = lgm2F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = lgm2F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = lgm2F.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = lgm2F.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = lgm2F.DF(to);
           df_TN2 = lgm2F.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = lgm2F.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = lgm2F.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = lgm2F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:2
               for j=1:2
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigitalGCArgDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,termCorr)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = lgm2F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = lgm2F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = lgm2F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = lgm2F.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = lgm2F.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = lgm2F.DF(to);
           df_TN2 = lgm2F.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = lgm2F.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = lgm2F.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = lgm2F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           coVarianceTerm = 0.0;
           for i=1:2
               for j=1:2
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           coVarianceTerm = variance1 + variance2 - 2.0*termCorr*sqrt(variance1)*sqrt(variance2);
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(coVarianceTerm);
           out.sigmaOld = sqrt(spreadVariance);
           
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVarianceTerm/sqrt(variance1*variance2);
           out.corrOld = coVariance/sqrt(variance1*variance2);
           
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigital(lgm2F,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalGCDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike,termCorr)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalGCArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,termCorr);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSCMSSpreadDigitalDate(lgm2F,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,strike,strikeS)
           
           % probability that CMS tenor1 - CMS tenor 2 > strike
           % CMS 1
           cmsDigitalArg = lgm2F.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           c_nN = cmsDigitalArg.c_nN1;
           varCovar = cmsDigitalArg.varCovar;
           
           %CMS Spread
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           
           c_nN1 = cmsSpreadDigitalArg.c_nN1;
           c_nN2 = cmsSpreadDigitalArg.c_nN2;
           
           muS = cmsSpreadDigitalArg.mu;
           sigmaS = cmsSpreadDigitalArg.sigma;
           
           % covariance between CMS Rate & CMS Spread
           covar = 0.0;
           for i=1:2
               for j=1:2
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           corr = covar/(sigma*sigmaS);
           
           z1 = (mu - strike)/sigma;
           z2 = (muS - strikeS)/sigmaS;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CDCMSSpreadDigitalDate(lgm2F,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,strike,strikeS)
           
           % probability that CD > strikeCD 
           % probability that CMS tenor1 - CMS tenor 2 > strike
           
           % CD
           cdDigitalArg = lgm2F.CDDigitalArgDate(valueDate,toDate,tpDate,tenor,tau);
           mu = cdDigitalArg.mu;
           sigma = cdDigitalArg.sigma;
           c_nN = cdDigitalArg.c_nN;
           varCovar = cdDigitalArg.varCovar;
           deltaT = cdDigitalArg.deltaT;
           
           %CMS Spread
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           
           c_nN1 = cmsSpreadDigitalArg.c_nN1;
           c_nN2 = cmsSpreadDigitalArg.c_nN2;
           
           muS = cmsSpreadDigitalArg.mu;
           sigmaS = cmsSpreadDigitalArg.sigma;
           
           % covariance between CMS Rate & CMS Spread
           covar = 0.0;
           for i=1:2
               for j=1:2
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           corr = covar/(sigma*sigmaS);
           
           
%            z1 = (mu - log(1.0 + deltaT*strike))/sigma;

%            z1 = -1.0*(mu + log(1.0 + deltaT*strike))/sigma;
%            z2 = (muS - strikeS)/sigmaS;
%            
           z1 = (mu + log(1.0 + deltaT*strike))/sigma;
           z2 = -1.0*(muS - strikeS)/sigmaS;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           
           if ~isreal(z1) || ~isfinite(z1) || ~isreal(z2) || ~isfinite(z2)
               out = 0.0;
           else
               out = mvncdf(xbar, mbar, corrBar);
           end
%            out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CMSSpreadDigitalRange(lgm2F,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm2F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = lgm2F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalRangeDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm2F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = lgm2F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        
        function out = CMSSpreadDigitalRangeGCDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper,termCorr)
            lower_value = lgm2F.CMSSpreadDigitalGCDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,termCorr);
            upper_value = lgm2F.CMSSpreadDigitalGCDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper,termCorr);
            out = lower_value - upper_value;
        end
        
        function out = CMSCMSSpreadDigitalRangeDate(lgm2F,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = lgm2F.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = lgm2F.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = lgm2F.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = lgm2F.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,upperS);
            
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function out = CDCMSSpreadDigitalRangeDate(lgm2F,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = lgm2F.CDCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = lgm2F.CDCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = lgm2F.CDCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = lgm2F.CDCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,upperS);
            
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function out = CDDigitalRangeDate(lgm2F,valueDate,toDate,tpDate,tenor,tau,lower,upper)
            upper_value = lgm2F.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,upper);
            lower_value = lgm2F.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,lower);
            out = upper_value - lower_value;
            
        end
        
        function out = inductMCForwardNew(lgm2F,fromTime,toTime,lastX,dZ)
        
            stateLocalVariance = lgm2F.stateLocalVariance(fromTime,toTime);
            stateLocalDrift = lgm2F.stateLocalDrift(fromTime,toTime);
            L = chol(stateLocalVariance,'lower');
            nextX = stateLocalDrift * lastX + L*dZ;
            out.nextX = nextX;
            

        end
        
        function out = AssignExercise(lgm2F,currentTimeStep,PricerColumn,exerciseValueIn,exerciseConditionIn,...
                                      minMaxFlagIn,decisionVar1In,nominal,numeraire,cachedModelStatesV)
                                  
           numMatTime = numeraire.numMatTime;
           
%            df_e = eqCOMDupireSpotGF.discountCurve.DF(currentTimeStep/365.0);
%            df_e = model.discountCurve.DF(currentTimeStep/365.0);
           onePayoff = ones(1,size(cachedModelStatesV,2));
           onePayoffD = lgm2F.discountPayoffRisky(currentTimeStep,numMatTime ...
                            ,cachedModelStatesV,onePayoff);
                        
           originalValue = PricerColumn.payoff./onePayoffD;
           exerciseValue = exerciseValueIn;
           newValue = zeros(size(originalValue));
           exerciseITMCondition = exerciseConditionIn;
           

           minMaxFlag = minMaxFlagIn;
           decisionVar1 = decisionVar1In;
           %regression using polynomial of decisionVar1
           %upto nOrder 
%            nOrder = nOrderIn;

%            newValue = LSRegression1(originalValue,exerciseValue,exerciseITMCondition,minMaxFlag,decisionVar1,nOrder,nominal);
           
%            newValue = LSRegressionGeneric(originalValue,exerciseValue,exerciseITMCondition,minMaxFlag,decisionVar1,nominal);
           
           newValue = LSRegressionGeneric2(originalValue,exerciseValue,exerciseITMCondition,minMaxFlag,decisionVar1,nominal);
 
           %discount newValue and assign to the
           %pricerColumn payoff
           PricerColumn.payoff = newValue .* onePayoffD;
           out = PricerColumn.payoff; 
            
        end
        
        function out = computeAVGSteepener(lgm2F,valueDate,steepenerParams,numMethod)
%           
            % model schedule generation
            dealStartDate = steepenerParams.startDate;
            dealStartEventTime = DateDiff(dealStartDate,valueDate);
            expiryDate = steepenerParams.endDate;
            dealEndEventTime = DateDiff(expiryDate,valueDate);
            %% couponSchedule start
            
            couponScheduleInt = steepenerParams.params.couponInfo.couponScheduleInt;
            
            % schedule is couponSchedule
            % main backbone schedule
            scheduleSize = size(couponScheduleInt,1);
            couponSchedule= struct;
            couponSchedule.scheduleSize = scheduleSize;
            % coupon payment event happens at endDate
            % eventTime is endDate
            % payment date can be different from endDate
            
            % startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:couponSchedule.scheduleSize
%                 couponSchedule.payDate(i) = H_Date(couponScheduleInt(i,4));
                couponStartDate = H_Date(couponScheduleInt(i,2));
                couponEndDate = H_Date(couponScheduleInt(i,3));
                couponPayDate = H_Date(couponScheduleInt(i,4));
                
                couponSchedule.dayCount(i) = couponEndDate.DateDiff(couponStartDate);
                couponSchedule.dayCountFraction(i) = couponSchedule.dayCount(i)/365.0; 
                
                couponEndDateP = couponEndDate.AddDate( -1 ,'day');
                
                couponSchedule.startTime(i) = DateDiff(couponStartDate,valueDate);
                couponSchedule.endTimeP(i)  = DateDiff(couponEndDateP,valueDate);
                couponSchedule.endTime(i) = DateDiff(couponEndDate,valueDate);
                
                couponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                couponSchedule.payTime(i) = DateDiff(couponPayDate,valueDate);
                
            end
            
            
            tenorL = steepenerParams.params.couponInfo.tenorL;
            tenorS = steepenerParams.params.couponInfo.tenorS;
            localFloor = steepenerParams.params.couponInfo.localFloor;
            localCap = steepenerParams.params.couponInfo.localCap;
            multiplier = steepenerParams.params.couponInfo.multiplier;
            spread     = steepenerParams.params.couponInfo.spread;
%             expiryDate = couponSchedule.payDate(scheduleSize);
           
            %% coupon schedule end
           
            %% call exercise event happens the same date of payment
            %% callable schedule start
            callableScheduleInt = steepenerParams.params.callableInfo.callableScheduleInt;
            callableScheduleSize = size(callableScheduleInt,1);
            callableSchedule= struct;
            for i=1:callableScheduleSize
                callExerciseDate = H_Date(callableScheduleInt(i,1));
                callPayDate = H_Date(callableScheduleInt(i,2));
                callableSchedule.eventTime(i) = DateDiff(callExerciseDate,valueDate);
                callableSchedule.payTime(i) = DateDiff(callPayDate,valueDate);
                
                 callableSchedule.callPayment(i) = callableScheduleInt(i,3);
            end
            callableSchedule.scheduleSize = callableScheduleSize;
           
            
           %% callable schedule end
            
          %% pricing time Step Generation start
            % if valueDate is passed expiryDate then price is 0
            maturity = DateDiff(expiryDate,valueDate);
            if maturity <=0
                out = 0;
                return;
            end
            
            % eventTimeStep Generate
            mcOneTimeStep = numMethod.mcOneTimeStep;
            
           %% eventTime from dealStartTime and dealEndTime
           eventTimeStep = dealStartEventTime;
           eventTimeStep = union(eventTimeStep,dealEndEventTime);
           
           %% eventTime From CouponSchedule:eventTime(endTime) +
           %% startTime,endTimeP
           %% daily weekly timeStep for couponSchedule
            
            eventTimeStep = union(eventTimeStep,couponSchedule.eventTime);
            eventTimeStep = union(eventTimeStep, couponSchedule.startTime);
            eventTimeStep = union(eventTimeStep, couponSchedule.endTimeP);
           %% eventTime From CallSchedule:eventTime(endTime) +
            eventTimeStep = union(eventTimeStep,callableSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
            % alive Coupon PayTime
            % from dealStartEventTime to 2nd AliveCoupon EventTime
            % dailyTimeStep should be added to eventTimeStep
            % for stable theta calculation
            
            aliveCouponNums = max(find(couponSchedule.eventTime >=0));
            aliveCouponNear = min(aliveCouponNums,2);
            if aliveCouponNear > 0
              %% dailyTimeSteps for near couponEventTime, till dealStartEventTime and valueTime
                % for fixing handling and theta issue
                dailyTimeSteps = [couponSchedule.eventTime(aliveCouponNear):-1:dealStartEventTime];
                dailyTimeSteps = union(dailyTimeSteps,0);
                eventTimeStep = union(eventTimeStep,dailyTimeSteps);
              %% weeklyTimeSteps for far couponEventTime
                weeklyTimeSteps = [eventTimeStep(end):-1*mcOneTimeStep:couponSchedule.eventTime(aliveCouponNear)];
                eventTimeStep = union(eventTimeStep,weeklyTimeSteps);
            else
              %% if no aliveCouponEventTime( if last CouponEventTime is passed but not paid yet)
                % if dealStartEventTime is passed then we add dailyTimeStep
                % from valueTime till dealStartEventTime 
                % for fixing handling
                
                if dealStartEventTime < 0 
                    dailyTimeSteps = [0:-1:dealStartEventTime];
                end
                eventTimeStep = union(eventTimeStep,dailyTimeSteps);
            end
           
            eventTimeStep = sort(eventTimeStep,'ascend');
            
%           %% debugging purpose 20201013 start
%             passedTimeStep = eventTimeStep(eventTimeStep<0);
%             load('cmsSteepnerTimeStep20201013.mat','cmsSteepnerTimeStep20201013');
%             aliveTimeStep = sort(cmsSteepnerTimeStep20201013,'ascend');
%             eventTimeStep = [];
%             eventTimeStep = union(passedTimeStep,cmsSteepnerTimeStep20201013);
%             eventTimeStep = sort(eventTimeStep,'ascend');
%           
%           %% debugging purpose 20201013 end
            
          %% aliveTimeStep for mc path generation
            aliveTimeStep = eventTimeStep(eventTimeStep>=0);
            aliveTimeStep = sort(aliveTimeStep,'ascend');
            
            totalAliveTimeStepSize = length(aliveTimeStep);
            
           %% eventTimeStep Generation End
           
           %% numeraire info
            numMatTime = eventTimeStep(end);
            dfNumMat = lgm2F.DFRisky(numMatTime/365.0);

            numeraire.numMatTime = numMatTime;
            numeraire.dfNumMat = dfNumMat;
              
            % should we use eventTimeIdx as future only time stes?
            % In this case we better use more generic approach
            % such time all times steps are included 
            % and when payoff description is decoded, time should be
            % checked later
            

          %% pricing time Step Generation end
           
          %% MC init
            rng('default');
            rng(0);
            
            numOfFactors = lgm2F.numOfFactor;
            NMC = numMethod.pathSize; 
            MCTimeStep = totalAliveTimeStepSize -1; % nowDate (0) should be skipped
%             Z= randn(NMC/2,MCTimeStep);
            Z= randn(NMC/2,numOfFactors*MCTimeStep);
            Z=[Z;-Z];
            Z=Z';

            %TimeInversion
            sizeZ = size(Z,1);
            A = zeros(sizeZ,sizeZ);
            for i=1:sizeZ
                A(i,sizeZ + 1 -i) = 1;
            end
            Z = A*Z;

%                 U = Z';
            U = Z;

           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = zeros(numOfFactors,NMC);
           
           nextX = initX;
            
          %% payoff init
           nominal = steepenerParams.params.nominal;
           basePrice = steepenerParams.params.basePrice;
           coupon = steepenerParams.params.couponInfo.coupon;
           
           payoffInfo.nominal = nominal;

           payoffInfo.couponSchedule = couponSchedule;
           payoffInfo.callableSchedule = callableSchedule;
%            payoffInfo.refixingSchedule = refixingSchedule;
           
           % EODFlag : if YES then pricing as EndOfDate(cashflow on
           % valueDate wil be not included(already paid)
           % if NO then pricing as Intraday ( cashflow on valueDate will 
           % be included)
           
           %EODFlag_Cashflow: Cashflow EOD Flag
           %EODFlag_Exercise: Exercise(Option) EOD Flag
           EODFlag_Cashflow = steepenerParams.params.callableInfo.EODFlag_Cashflow;
           EODFlag_Exercise = steepenerParams.params.callableInfo.EODFlag_Exercise;
           
           
           modelStatesSize = NMC;
           
          %% Define Pricing Columns
           % pricingColumns : nonCall,Callable  
           % we need floatingCoupon as state variables for storing floating
           % CMS Steepener Coupon
           
           % for the cashflow generated pricing columns as pricerInfo
           % we do use pricerInfo Structure
           
         %% state variable initialize
           
          % avg steeper coupon
           floatingCoupon = zeros(couponSchedule.scheduleSize,modelStatesSize);
           cachedModelStatesCoupon = zeros(couponSchedule.scheduleSize*2,modelStatesSize);
  
          % for regression (2 modelState variables)
%            savedModelState = zeros(callableSchedule.scheduleSize*2,modelStatesSize);
           cachedModelStatesCall = zeros(callableSchedule.scheduleSize*2,modelStatesSize);
           
          %% PricingColumns that has cashflows
           % we have nonCall, callable
           
            %nonCall  
            nonCall = PricingColumnInfoGeneric(scheduleSize,modelStatesSize);
            
%             %callable
%             callable = PricingColumnInfoGeneric(scheduleSize,modelStatesSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(1,modelStatesSize);
           payoffStateA.cashflow_npv = zeros(1,modelStatesSize);
           
           payoffStateB.cashflow = zeros(1,modelStatesSize);
           payoffStateB.cashflow_npv = zeros(1,modelStatesSize);
           
           payoffStateC.cashflow = zeros(1,modelStatesSize);
           payoffStateC.cashflow_npv = zeros(1,modelStatesSize);
           
           % variable to use accruedSumOfStockPrice for moving average calculation 
           
           cD = 0; % cumulated day count
           cP = 0; 
           dI = 0; % dummy idx for loop
           dcfC = 0;
           dcfN = 0;
           dD = 0;
      
           idxL = zeros(1,modelStatesSize);
           idxS = zeros(1,modelStatesSize);

           idxC = zeros(1,modelStatesSize);
           idxN = zeros(1,modelStatesSize);

           spreadC = zeros(1,modelStatesSize);
           spreadN = zeros(1,modelStatesSize);
%            probSpread = zeros(1,pathSize);
           
           currentPeriodCoupon = zeros(1,modelStatesSize);
           currentPeriodCouponNPV = zeros(1,modelStatesSize);
            % we use all path for regression
            % conversion ITM , puttable ITM ,  callable ITM
            % what's the use of Alive Path only Regression Sampling ?
           
          %% Path Generation Forward
          %% Store path dependent variables for coupon calculation 
          %% Store state variables to be used for AMC Regression
           currentCouponPeriod = 1;
           
           %payoff descrption update index
           currentTimeIdx = 1;
           %MC numMethodState update index
           aliveTimeIdx = 1;
           % currentTimeStep is update in for statement
%            for currentTimeStep = eventTimeStep(1):1:eventTimeStep(end)
           for idxT = 1:1:length(eventTimeStep)
                currentTimeStep = eventTimeStep(idxT);
                
                if idxT < length(eventTimeStep)
                    nextTimeStep = eventTimeStep(idxT+1);
                end
                % evaluate scripts 
                % evaluate stateVariables(stores)
                % evaluate expression per each pricing columns
                % at the end of the loop we induct forward
                % from left to right
                % fictional payoff script columns are
                % cumCoupon,nonCall,Callable
                
                        
                % find the current
                % if find return errors then?
                try
                    
                    % cachedModelStateCoupon for modelState
                    idxCouponEvent = find(currentTimeStep == couponSchedule.eventTime);
                    if ~isempty(idxCouponEvent)
                        cachedModelStatesCoupon(numOfFactors*(idxCouponEvent-1)+1:numOfFactors*idxCouponEvent,:) = nextX;
                    end
                    
                    % cachedModelState for modelState
                    idxCallEvent = find(currentTimeStep == callableSchedule.eventTime);
                    if ~isempty(idxCallEvent)
                        cachedModelStatesCall(numOfFactors*(idxCallEvent-1)+1:numOfFactors*idxCallEvent,:) = nextX;
                    end
                    
                    
                    
                    % find the corresponding nearest future coupon event
                    idx = min(find(currentTimeStep <= couponSchedule.eventTime));
                    if ~isempty(idx)
                      %% stateVariable floatingCoupon
                        idxL = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorL,nextX);
                        idxS = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorS,nextX);
                        
                        dcfN = nextTimeStep;
                        dcfC = currentTimeStep;
                        % if coupon accrual condition is met
                        % period startDate is included but 
                        % endDate is not included
                        
                        if currentTimeStep >= couponSchedule.startTime(idx) ...
                           && currentTimeStep < couponSchedule.eventTime(idx)
                            
                            dD = dcfN - dcfC;
                            cD = cD + dD;
                            cP = cP + (idxL-idxS)*dD;
                            
                        end
                         
                        % now if we are at couponSchedule eventTime
                        % update floatingCoupon
                        if currentTimeStep == couponSchedule.eventTime(idx)

                            floatingCoupon(idx,:) =  max(min(multiplier * cP/cD + spread,localCap),localFloor) * couponSchedule.dayCountFraction(idx) * nominal;
                            
                            % resetting local variables for averaging
                            cD = 0; % cumulated day count
                            cP = 0; % cumulated steepener sum
                            
                            if idx == couponSchedule.scheduleSize
                                payoffStateA.cashflow = nominal + floatingCoupon(idx,:);
                             else
                                payoffStateA.cashflow = floatingCoupon(idx,:);
                             end

                             payTime = couponSchedule.payTime(idx);
                             
                             %process payoff cashflow 
                             nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                     payoffStateA.cashflow,idx);
     
                        end
                    end

                    % if currentTimeSteps is today or future then inductMCForward
                    if currentTimeStep >= 0 && currentTimeStep < eventTimeStep(end)
                        mcmcOut   = inductMCForwardNew(lgm2F,currentTimeStep,nextTimeStep,...
                                    nextX,U(numOfFactors*(aliveTimeIdx-1)+1:numOfFactors*aliveTimeIdx,:));
                        nextX = mcmcOut.nextX;
                        %MC numMethodState update
                        aliveTimeIdx = aliveTimeIdx + 1;
                    end
                    
                    currentTimeIdx = currentTimeIdx +1;
                catch exception
                    throw(exception)
                end
                
                
           end
           
          %% Forward Induction End
          
           % backward pricingColumn
           % nonCallB,
           % callable
           
           %nonCallB  
           nonCallB = PricingColumnInfoGeneric(scheduleSize,modelStatesSize);
           callable = PricingColumnInfoGeneric(scheduleSize,modelStatesSize);
           
           % variables initialize to be used in backward induction
           cachedModelStatesCouponV= zeros(numOfFactors,modelStatesSize);
           cachedModelStatesCallV = zeros(numOfFactors,modelStatesSize);
           %% Backward induction start
           for idxT=length(eventTimeStep):-1:1
               
               currentTimeStep = eventTimeStep(idxT);
               try 
                 %% coupon event  
                   idxCoupon = find(currentTimeStep == couponSchedule.eventTime);
                   % if currentTime  is couponSchedule.eventTime
                   if ~isempty(idxCoupon)
                       
                       if currentTimeStep == couponSchedule.eventTime(idxCoupon)
                             % we use cached(saved) floating Coupon that
                             % was generated from Forward Induction
                             payTime = couponSchedule.payTime(idxCoupon);
                             
                             if idxCoupon == couponSchedule.scheduleSize
                                payoffStateA.cashflow = nominal + floatingCoupon(idxCoupon,:);
                             else
                                payoffStateA.cashflow = floatingCoupon(idxCoupon,:);
                             end
                             cachedModelStatesCouponV = cachedModelStatesCoupon(numOfFactors*(idxCoupon-1)+1:numOfFactors*idxCoupon,:);
                             
                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,idxCoupon);

                             callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,idxCoupon);

%                              if idxCoupon == couponSchedule.scheduleSize
%                                 %process payoff cashflow
%                                 nonCallB.ProcessAssignPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
%                                                      payoffStateA.cashflow,idxCoupon);
%                                                  
%                                 callable.ProcessAssignPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
%                                                      payoffStateA.cashflow,idxCoupon);
%                                                      
%                              else
%                                %process payoff cashflow 
%                                nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
%                                                      payoffStateA.cashflow,idxCoupon);
%                                
%                                callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
%                                                      payoffStateA.cashflow,idxCoupon);
%                                          
%                              end
                            
                       end
                   end
                 
                 %% issuer callable event
                 %% AssignExercise Functor : applied only when eventTime >=0
                 %% if EODAdjust is YES : endOfDate conversion option has expired
                 %% if EODAdjust is NO : intraDay, and option is still alive
                   idxCall = find(currentTimeStep == callableSchedule.eventTime);
                   if ~isempty(idxCall)
                       if currentTimeStep == callableSchedule.eventTime(idxCall) && currentTimeStep >=0
                           if EODAdjust(EODFlag_Exercise,currentTimeStep)
                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);
                        
                               exerciseValue = zeros(1,modelStatesSize);
                               exerciseITMCondition = zeros(1,modelStatesSize);
                               
                               idxCouponE = find(currentTimeStep <= couponSchedule.eventTime);
                               % find the nearest floating coupon
                               if ~isempty(idxCouponE)
                                   idxCouponE = min(idxCouponE);
                               else
                                   error('wrong schedule')
                               end

                               for idxEx=1:modelStatesSize
                                   exerciseValue(idxEx) = callPayment*nominal + floatingCoupon(idxCouponE,idxEx);

                                   exerciseITMCondition(idxEx) = 1;
                                   
%                                %% callable condition activated 
%                                    if savedStockPriceV(idxEx) >= callableConditionalStrike
%                                         exerciseITMCondition(idxEx) = 1;
%                                    end
                               end

                               % callable is minimizing the payoff
                               minMaxFlag   = 'MIN';
                               decisionVar1 =  cachedModelStatesCallV;
                               outCallablePayoff = AssignExercise(lgm2F,currentTimeStep,callable,exerciseValue,exerciseITMCondition,...
                                          minMaxFlag,decisionVar1,nominal,numeraire,cachedModelStatesCallV);

                           end
                       end
                       
                   end
                   
               catch exception
                    throw(exception)
               end    
           end
           
           
          %% Finalize output
           nonCall.npv = mean(nonCall.payoff);
           nonCallB.npv = mean(nonCallB.payoff);
           callable.npv = mean(callable.payoff);
       
           %forward
           out.nonCall = nonCall; 
           %backward
           out.nonCallB = nonCallB; 
           out.callable = callable;
           
        end
        
        function out = computeAVGSteepenerGeneric(lgm2F,valueDate,steepenerParams,numMethod)
%           
            % model schedule generation
            dealStartDate = steepenerParams.startDate;
            dealStartEventTime = DateDiff(dealStartDate,valueDate);
            expiryDate = steepenerParams.endDate;
            dealEndEventTime = DateDiff(expiryDate,valueDate);
            %% structuredCouponSchedule start
            
            structuredCouponScheduleInt = steepenerParams.params.structuredCouponInfo.structuredCouponScheduleInt;
            
            % structuredCouponSchedule is a schedule of Structured Coupon
            % events
            
            % main backbone schedule
            structuredCouponScheduleSize = size(structuredCouponScheduleInt,1);
            structuredCouponSchedule= struct;
            structuredCouponSchedule.scheduleSize = structuredCouponScheduleSize;
            structuredCouponSchedule.eventType = steepenerParams.params.structuredCouponInfo.eventType;
            % coupon payment event happens at endDate
            % eventTime is endDate
            % payment date can be different from endDate
            
            % startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:structuredCouponSchedule.scheduleSize
%                 structuredCouponSchedule.payDate(i) = H_Date(structuredCouponScheduleInt(i,4));
                couponStartDate = H_Date(structuredCouponScheduleInt(i,2));
                couponEndDate = H_Date(structuredCouponScheduleInt(i,3));
                couponPayDate = H_Date(structuredCouponScheduleInt(i,4));
                
                structuredCouponSchedule.dayCount(i) = couponEndDate.DateDiff(couponStartDate);
                structuredCouponSchedule.dayCountFraction(i) = structuredCouponSchedule.dayCount(i)/365.0; 
                
                couponEndDateP = couponEndDate.AddDate( -1 ,'day');
                
                structuredCouponSchedule.startTime(i) = DateDiff(couponStartDate,valueDate);
                structuredCouponSchedule.endTimeP(i)  = DateDiff(couponEndDateP,valueDate);
                structuredCouponSchedule.endTime(i) = DateDiff(couponEndDate,valueDate);
                
                if strcmp(structuredCouponSchedule.eventType,'END')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                elseif strcmp(structuredCouponSchedule.eventType,'START')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponStartDate,valueDate);
                else % default is END : event happens at EndDate for StructuredCoupon
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                end
                structuredCouponSchedule.payTime(i) = DateDiff(couponPayDate,valueDate);
                
            end
            
            
            tenorL     = steepenerParams.params.structuredCouponInfo.tenorL;
            tenorS     = steepenerParams.params.structuredCouponInfo.tenorS;
            localFloor = steepenerParams.params.structuredCouponInfo.localFloor;
            localCap   = steepenerParams.params.structuredCouponInfo.localCap;
            multiplier = steepenerParams.params.structuredCouponInfo.multiplier;
            spread     = steepenerParams.params.structuredCouponInfo.spread;
            coupon     = steepenerParams.params.structuredCouponInfo.coupon;
           %% structuredCoupon schedule end
           
           %% call exercise event happens at the callExerciseDate
           %% callable schedule start
            callableScheduleInt = steepenerParams.params.callableInfo.callableScheduleInt;
            callableScheduleSize = size(callableScheduleInt,1);
            callableSchedule= struct;
            for i=1:callableScheduleSize
                callExerciseDate = H_Date(callableScheduleInt(i,1));
                callPayDate = H_Date(callableScheduleInt(i,2));
                callableSchedule.eventTime(i) = DateDiff(callExerciseDate,valueDate);
                callableSchedule.payTime(i) = DateDiff(callPayDate,valueDate);
                
                 callableSchedule.callPayment(i) = callableScheduleInt(i,3);
            end
            callableSchedule.scheduleSize = callableScheduleSize;
           
            isExercised = steepenerParams.params.callableInfo.isExercised;
           %% callable schedule end
            
          %% pricing time Step Generation start
            
            % eventTimeStep Generate
            mcOneTimeStep = numMethod.mcOneTimeStep;
            
          %% eventTime from dealStartTime and dealEndTime
           eventTimeStep = dealStartEventTime;
           eventTimeStep = union(eventTimeStep,dealEndEventTime);
           
          %% eventTime From structuredCouponSchedule:eventTime(endTime) +
          %% startTime,endTimeP
          %% daily weekly timeStep for couponSchedule
          
          %% eventTime From structuredCouponSchedule:eventTime
            eventTimeStep = union(eventTimeStep,structuredCouponSchedule.eventTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.startTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.endTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.endTimeP);
           %% eventTime From CallSchedule:eventTime
            eventTimeStep = union(eventTimeStep,callableSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
            % alive Coupon : event has not happend Yet
            % from dealStartEventTime to 2nd AliveCoupon EventTime
            % dailyTimeStep should be added to eventTimeStep
            % for stable theta calculation
             
            aliveCouponIdxMax = max(find(structuredCouponSchedule.eventTime > 0));
            aliveCouponIdxMin = min(find(structuredCouponSchedule.eventTime > 0));
            
            aliveCouponNums = aliveCouponIdxMax - aliveCouponIdxMin +1;
            numOfAliveCouponDaily = min(aliveCouponNums,2);
            if numOfAliveCouponDaily > 0
              %% dailyTimeSteps for near couponEventTime, till dealStartEventTime and valueTime
                % for fixing handling and theta issue
                dailyTimeSteps = [structuredCouponSchedule.eventTime(aliveCouponIdxMin + numOfAliveCouponDaily - 1):-1:dealStartEventTime];
                dailyTimeSteps = union(dailyTimeSteps,0);
                eventTimeStep = union(eventTimeStep,dailyTimeSteps);
              %% weeklyTimeSteps for far couponEventTime
                weeklyTimeSteps = [eventTimeStep(end):-1*mcOneTimeStep:structuredCouponSchedule.eventTime(aliveCouponIdxMin + numOfAliveCouponDaily - 1)];
                eventTimeStep = union(eventTimeStep,weeklyTimeSteps);
            else
              %% if no aliveCouponEventTime( if last StructuredCouponEventTime is passed but not paid yet)
                % if dealStartEventTime is passed then we add dailyTimeStep
                % from valueTime till dealStartEventTime 
                % for fixing handling
                
                if dealStartEventTime < 0 
                    dailyTimeSteps = [0:-1:dealStartEventTime];
                    eventTimeStep = union(eventTimeStep,dailyTimeSteps);
                end
                
            end
           
            eventTimeStep = sort(eventTimeStep,'ascend');
            
%           %% debugging purpose 20201013 start
%             passedTimeStep = eventTimeStep(eventTimeStep<0);
%             load('cmsSteepnerTimeStep20201013.mat','cmsSteepnerTimeStep20201013');
%             aliveTimeStep = sort(cmsSteepnerTimeStep20201013,'ascend');
%             eventTimeStep = [];
%             eventTimeStep = union(passedTimeStep,cmsSteepnerTimeStep20201013);
%             eventTimeStep = sort(eventTimeStep,'ascend');
%           
%           %% debugging purpose 20201013 end
            
          %% aliveTimeStep for mc path generation
            aliveTimeStep = eventTimeStep(eventTimeStep>=0);
            aliveTimeStep = sort(aliveTimeStep,'ascend');
            
            totalAliveTimeStepSize = length(aliveTimeStep);
            
           %% eventTimeStep Generation End
           
           %% numeraire info
            numMatTime = eventTimeStep(end);
            dfNumMat = lgm2F.DFRisky(numMatTime/365.0);

            numeraire.numMatTime = numMatTime;
            numeraire.dfNumMat = dfNumMat;
              
            % should we use eventTimeIdx as future only time stes?
            % In this case we better use more generic approach
            % so that all times steps are included 
            
          %% pricing time Step Generation end
           
          %% MC init
            rng('default');
            rng(0);
            
            numOfFactors = lgm2F.numOfFactor;
            NMC = numMethod.pathSize; 
            MCTimeStep = totalAliveTimeStepSize -1; % nowDate (0) should be skipped
%             Z= randn(NMC/2,MCTimeStep);
            Z= randn(NMC/2,numOfFactors*MCTimeStep);
            Z=[Z;-Z];
            Z=Z';

            %TimeInversion
            sizeZ = size(Z,1);
            A = zeros(sizeZ,sizeZ);
            for i=1:sizeZ
                A(i,sizeZ + 1 -i) = 1;
            end
            Z = A*Z;

%                 U = Z';
            U = Z;

           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = zeros(numOfFactors,NMC);
           
           nextX = initX;
            
          %% payoff init
           nominal = steepenerParams.params.nominal;
                      
%            payoffInfo.nominal = nominal;
% 
%            payoffInfo.couponSchedule = couponSchedule;
%            payoffInfo.callableSchedule = callableSchedule;
% %            payoffInfo.refixingSchedule = refixingSchedule;
           
           % EODFlag : if true then pricing as EndOfDate(cashflow on
           % valueDate wil be not included(already paid)
           % if false then pricing as Intraday ( cashflow on valueDate will 
           % be included)
           
           %EODFlag_Cashflow: Cashflow EOD Flag
           %EODFlag_Exercise: Exercise(Option) EOD Flag
           EODFlag_Cashflow = steepenerParams.params.EODFlag_Cashflow;
           EODFlag_Exercise = steepenerParams.params.EODFlag_Exercise;
           
           
           modelStatesSize = NMC;
           
          %% Define Pricing Columns
           % pricingColumns : nonCall,Callable  
           % we need steepenerCoupon as state variables for storing 
           % CMS Steepener Coupon
           
           % for the cashflow generated pricing columns as pricerInfo
           % we do use pricerInfo Structure
           
         %% state variable initialize
           
          % avg steeper coupon
           steepenerCoupon = zeros(structuredCouponSchedule.scheduleSize,modelStatesSize);
           cachedModelStatesCoupon = zeros(structuredCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
  
          % for regression (2 modelState variables)
           cachedModelStatesCall = zeros(callableSchedule.scheduleSize*numOfFactors,modelStatesSize);
           
          %% PricingColumnGenericCashflow
          %% a structure that has cashflows with cashflowNames
          %% for CMS AVG structured we have nonCall, nonCallB, callable
           
            %nonCall
            cashflowNames = {'steepenerCoupon'};
            cashflowSizeMap = containers.Map({'steepenerCoupon'},{structuredCouponSchedule.scheduleSize});
            
            cashflowInfo.cashflowNames = cashflowNames;
            cashflowInfo.cashflowSizeMap = cashflowSizeMap;
            cashflowInfo.pathSize = modelStatesSize;
            
           nonCall = PricingColumnInfoGenericCashflow(cashflowInfo);
           nonCallB = PricingColumnInfoGenericCashflow(cashflowInfo);
           callable = PricingColumnInfoGenericCashflow(cashflowInfo);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(1,modelStatesSize);
           payoffStateA.cashflow_npv = zeros(1,modelStatesSize);
           
           % variable to use accruedSumOfStockPrice for moving average calculation 
           
           cD = 0; % cumulated day count
           cP = 0; 
           dI = 0; % dummy idx for loop
           dcfC = 0;
           dcfN = 0;
           dD = 0;
      
           idxL = zeros(1,modelStatesSize);
           idxS = zeros(1,modelStatesSize);

           idxC = zeros(1,modelStatesSize);
           idxN = zeros(1,modelStatesSize);

           spreadC = zeros(1,modelStatesSize);
           spreadN = zeros(1,modelStatesSize);
%            probSpread = zeros(1,pathSize);
           
            % we use all path for regression
            % conversion ITM , puttable ITM ,  callable ITM
            % what's the use of Alive Path only Regression Sampling ?
           
          %% Path Generation Forward
          %% Store path dependent variables for coupon calculation 
          %% Store state variables to be used for AMC Regression
           
           %payoff descrption update index
           currentTimeIdx = 1;
           %MC numMethodState update index
           aliveTimeIdx = 1;
           % currentTimeStep is update in for statement
%            for currentTimeStep = eventTimeStep(1):1:eventTimeStep(end)
           for idxT = 1:1:length(eventTimeStep)
                currentTimeStep = eventTimeStep(idxT);
                
                if idxT < length(eventTimeStep)
                    nextTimeStep = eventTimeStep(idxT+1);
                end
                % evaluate scripts 
                % evaluate stateVariables(stores)
                % evaluate expression per each pricing columns
                % at the end of the loop we induct forward
                % from left to right
                % fictional payoff script columns are
                % cumCoupon,nonCall,Callable
                
                        
                % find the current
                % if find return errors then?
                try
                    
                    % cachedModelStateCoupon for modelState
                    idxCouponEvent = find(currentTimeStep == structuredCouponSchedule.eventTime);
                    if ~isempty(idxCouponEvent)
                        cachedModelStatesCoupon(numOfFactors*(idxCouponEvent-1)+1:numOfFactors*idxCouponEvent,:) = nextX;
                    end
                    
                    % cachedModelState for modelState
                    idxCallEvent = find(currentTimeStep == callableSchedule.eventTime);
                    if ~isempty(idxCallEvent)
                        cachedModelStatesCall(numOfFactors*(idxCallEvent-1)+1:numOfFactors*idxCallEvent,:) = nextX;
                    end
                    
                    
                    
                    % find the corresponding nearest future coupon event
                    idx = min(find(currentTimeStep <= structuredCouponSchedule.eventTime));
                    if ~isempty(idx)
                      %% stateVariable floatingCoupon
                        idxL = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorL,nextX);
                        idxS = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorS,nextX);
                        
                        dcfN = nextTimeStep;
                        dcfC = currentTimeStep;
                        % if coupon accrual condition is met
                        % period startDate is included but 
                        % endDate is not included
                        
                        if currentTimeStep >= structuredCouponSchedule.startTime(idx) ...
                           && currentTimeStep < structuredCouponSchedule.eventTime(idx)
                            
                            dD = dcfN - dcfC;
                            cD = cD + dD;
                            cP = cP + (idxL-idxS)*dD;
                            
                        end
                         
                        % now if we are at couponSchedule eventTime
                        % update floatingCoupon
                        if currentTimeStep == structuredCouponSchedule.eventTime(idx)

                            steepenerCoupon(idx,:) =  max(min(multiplier * cP/cD + spread,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idx) * nominal;
                            
                            % resetting local variables for averaging
                            cD = 0; % cumulated day count
                            cP = 0; % cumulated steepener sum
                            
                            if idx == structuredCouponSchedule.scheduleSize
                                payoffStateA.cashflow = nominal + steepenerCoupon(idx,:);
                             else
                                payoffStateA.cashflow = steepenerCoupon(idx,:);
                            end
                            
                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             payTime = structuredCouponSchedule.payTime(idx);
                             
                             %process payoff cashflow 
                             nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                     payoffStateA.cashflow,'steepenerCoupon',idx);
     
                        end
                    end

                    % if currentTimeSteps is today or future then inductMCForward
                    if currentTimeStep >= 0 && currentTimeStep < eventTimeStep(end)
                        mcmcOut   = inductMCForwardNew(lgm2F,currentTimeStep,nextTimeStep,...
                                    nextX,U(numOfFactors*(aliveTimeIdx-1)+1:numOfFactors*aliveTimeIdx,:));
                        nextX = mcmcOut.nextX;
                        %MC numMethodState update
                        aliveTimeIdx = aliveTimeIdx + 1;
                    end
                    
                    currentTimeIdx = currentTimeIdx +1;
                catch exception
                    throw(exception)
                end
                
                
           end
           
          %% Forward Induction End
          
           % backward pricingColumn
           % nonCallB,
           % callable
           
           
          %% Backward induction start
           for idxT=length(eventTimeStep):-1:1
               
               currentTimeStep = eventTimeStep(idxT);
               try
                   
                 %% issuer callable event
                 %% AssignExercise Functor : applied only when eventTime >=0
                 %% if EODAdjust is YES : endOfDate conversion option has expired
                 %% if EODAdjust is NO : intraDay, and option is still alive
                   idxCall = find(currentTimeStep == callableSchedule.eventTime);
                   if ~isempty(idxCall)
                       if isExercised 
                           
                            payTime = callableSchedule.payTime(idxCall);
                            
                            % future payment <- is this necessary ?
                            if payTime >= 0
                                
                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);

                               % exercised by notional amount
                               exerciseValue = callPayment*nominal*ones(1,modelStatesSize);
                               
                               % discount exerciseValues from payTime to eventTime
                               df_ep = lgm2F.discountFactorRisky(currentTimeStep, payTime,numMatTime,cachedModelStatesCallV);
                               for k=1:modelStatesSize
                                   exerciseValue(k) = exerciseValue(k) * df_ep(k);
                               end
                               
                               outCallablePayoff = callable.ProcessAssignPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCallV, ...
                                     exerciseValue);
                                
                            end
                       else
                           
                           if currentTimeStep >=0 && EODAdjust(EODFlag_Exercise,currentTimeStep)

                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);

                               exerciseValue = zeros(1,modelStatesSize);
                               exerciseITMCondition = zeros(1,modelStatesSize);

                               for idxEx=1:modelStatesSize
                                   exerciseValue(idxEx) = callPayment*nominal; 
                                   exerciseITMCondition(idxEx) = 1;
                               end

                               payTime = callableSchedule.payTime(idxCall);

                               % discount exerciseValues from payTime to eventTime
                               df_ep = lgm2F.discountFactorRisky(currentTimeStep, payTime,numMatTime,cachedModelStatesCallV);
                               for k=1:modelStatesSize
                                   exerciseValue(k) = exerciseValue(k) * df_ep(k);
                               end

                               % callable is minimizing the payoff
                               minMaxFlag   = 'MIN';
                               decisionVar1 =  cachedModelStatesCallV;
                               outCallablePayoff = AssignExercise(lgm2F,currentTimeStep,callable,exerciseValue,exerciseITMCondition,...
                                          minMaxFlag,decisionVar1,nominal,numeraire,cachedModelStatesCallV);

                           end
                       
                       end
                   end
                   
                 %% coupon event  
                   idxCoupon = find(currentTimeStep == structuredCouponSchedule.eventTime);
                   % if currentTime  is couponSchedule.eventTime
                   if ~isempty(idxCoupon)
                       
                       if currentTimeStep == structuredCouponSchedule.eventTime(idxCoupon)
                             % we use cached(saved) floating Coupon that
                             % was generated from Forward Induction
                             payTime = structuredCouponSchedule.payTime(idxCoupon);
                             
                             if idxCoupon == structuredCouponSchedule.scheduleSize
                                payoffStateA.cashflow = nominal + steepenerCoupon(idxCoupon,:);
                             else
                                payoffStateA.cashflow = steepenerCoupon(idxCoupon,:);
                             end
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
                            
                             cachedModelStatesCouponV = cachedModelStatesCoupon(numOfFactors*(idxCoupon-1)+1:numOfFactors*idxCoupon,:);
                             
                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,'steepenerCoupon',idxCoupon);

                             callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,'steepenerCoupon',idxCoupon);
                            
                       end
                   end
                 
                 
                   
               catch exception
                    throw(exception)
               end    
           end
           
           
          %% Finalize output
           nonCall.npv = mean(nonCall.payoff);
           nonCallB.npv = mean(nonCallB.payoff);
           callable.npv = mean(callable.payoff);
       
           %forward
           out.nonCall = nonCall; 
           %backward
           out.nonCallB = nonCallB; 
           out.callable = callable;
           
           out.eventTimeStep = eventTimeStep';
           
        end
        
        function out = computeAVGSteepenerSwapGeneric(lgm2F,valueDate,steepenerParams,numMethod)
%           
            % model schedule generation
            dealStartDate = steepenerParams.startDate;
            dealStartEventTime = DateDiff(dealStartDate,valueDate);
            expiryDate = steepenerParams.endDate;
            dealEndEventTime = DateDiff(expiryDate,valueDate);
            %% structuredCouponSchedule start
            
            structuredCouponScheduleInt = steepenerParams.params.structuredCouponInfo.structuredCouponScheduleInt;
            
            % structuredCouponSchedule is a schedule of Structured Coupon
            % events
            
            % main backbone schedule
            structuredCouponScheduleSize = size(structuredCouponScheduleInt,1);
            structuredCouponSchedule= struct;
            structuredCouponSchedule.scheduleSize = structuredCouponScheduleSize;
            structuredCouponSchedule.eventType = steepenerParams.params.structuredCouponInfo.eventType;
            % coupon payment event happens at endDate
            % eventTime is endDate
            % payment date can be different from endDate
            
            % startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:structuredCouponSchedule.scheduleSize
%                 structuredCouponSchedule.payDate(i) = H_Date(structuredCouponScheduleInt(i,4));
                couponStartDate = H_Date(structuredCouponScheduleInt(i,2));
                couponEndDate = H_Date(structuredCouponScheduleInt(i,3));
                couponPayDate = H_Date(structuredCouponScheduleInt(i,4));
                
                structuredCouponSchedule.dayCount(i) = couponEndDate.DateDiff(couponStartDate);
                structuredCouponSchedule.dayCountFraction(i) = structuredCouponSchedule.dayCount(i)/365.0; 
                
                couponEndDateP = couponEndDate.AddDate( -1 ,'day');
                
                structuredCouponSchedule.startTime(i) = DateDiff(couponStartDate,valueDate);
                structuredCouponSchedule.endTimeP(i)  = DateDiff(couponEndDateP,valueDate);
                structuredCouponSchedule.endTime(i) = DateDiff(couponEndDate,valueDate);
                
                if strcmp(structuredCouponSchedule.eventType,'END')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                elseif strcmp(structuredCouponSchedule.eventType,'START')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponStartDate,valueDate);
                else % default is END : event happens at EndDate for StructuredCoupon
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                end
                structuredCouponSchedule.payTime(i) = DateDiff(couponPayDate,valueDate);
                
            end
            
            
            tenorL     = steepenerParams.params.structuredCouponInfo.tenorL;
            tenorS     = steepenerParams.params.structuredCouponInfo.tenorS;
            localFloor = steepenerParams.params.structuredCouponInfo.localFloor;
            localCap   = steepenerParams.params.structuredCouponInfo.localCap;
            multiplier = steepenerParams.params.structuredCouponInfo.multiplier;
            spread     = steepenerParams.params.structuredCouponInfo.spread;
            coupon     = steepenerParams.params.structuredCouponInfo.coupon;
           %% structuredCoupon schedule end
           
           %% call exercise event happens at the callExerciseDate
           %% callable schedule start
            callableScheduleInt = steepenerParams.params.callableInfo.callableScheduleInt;
            callableScheduleSize = size(callableScheduleInt,1);
            callableSchedule= struct;
            for i=1:callableScheduleSize
                callExerciseDate = H_Date(callableScheduleInt(i,1));
                callPayDate = H_Date(callableScheduleInt(i,2));
                callableSchedule.eventTime(i) = DateDiff(callExerciseDate,valueDate);
                callableSchedule.payTime(i) = DateDiff(callPayDate,valueDate);
                
                 callableSchedule.callPayment(i) = callableScheduleInt(i,3);
            end
            callableSchedule.scheduleSize = callableScheduleSize;
           
            isExercised = steepenerParams.params.callableInfo.isExercised;
           %% callable schedule end
           
          %% swapFloatingCouponSchedule start
            
            swapFloatingCouponScheduleInt = steepenerParams.params.swapFloatingCouponInfo.swapFloatingCouponScheduleInt;
            
            swapFloatingCouponSchedule= struct;
            swapFloatingCouponSchedule.scheduleSize = size(swapFloatingCouponScheduleInt,1);
            
            swapFloatingCouponSchedule.eventType = steepenerParams.params.swapFloatingCouponInfo.eventType;
            
            % floating coupon payment event happens at resetDate
            % eventTime is resetDate
            
            % resetTime,startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:swapFloatingCouponSchedule.scheduleSize
                flolatingCouponResetDate = H_Date(swapFloatingCouponScheduleInt(i,1));
                flolatingCouponStartDate = H_Date(swapFloatingCouponScheduleInt(i,2));
                flolatingCouponEndDate = H_Date(swapFloatingCouponScheduleInt(i,3));
                flolatingCouponPayDate = H_Date(swapFloatingCouponScheduleInt(i,4));
                
                swapFloatingCouponSchedule.dayCount(i) = flolatingCouponEndDate.DateDiff(flolatingCouponStartDate);
                swapFloatingCouponSchedule.dayCountFraction(i) = swapFloatingCouponSchedule.dayCount(i)/365.0; 
                
                flolatingCouponEndDateP = flolatingCouponEndDate.AddDate( -1 ,'day');
                
                swapFloatingCouponSchedule.resetTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                swapFloatingCouponSchedule.startTime(i) = DateDiff(flolatingCouponStartDate,valueDate);
                swapFloatingCouponSchedule.endTimeP(i)  = DateDiff(flolatingCouponEndDateP,valueDate);
                swapFloatingCouponSchedule.endTime(i) = DateDiff(flolatingCouponEndDate,valueDate);
                
                if strcmp(swapFloatingCouponSchedule.eventType,'RESET')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                elseif strcmp(swapFloatingCouponSchedule.eventType,'START')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponStartDate,valueDate);
                elseif strcmp(swapFloatingCouponSchedule.eventType,'END')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponEndDate,valueDate);
                else % default is RESET : for SwapFloatingCouponSchedule
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                end
                
%                 swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                
                swapFloatingCouponSchedule.payTime(i) = DateDiff(flolatingCouponPayDate,valueDate);
                
            end
            
            swapFloating_tenor  = steepenerParams.params.swapFloatingCouponInfo.tenorFloat;
            swapFloating_multiplier = steepenerParams.params.swapFloatingCouponInfo.multiplier;
            swapFloating_spread     = steepenerParams.params.swapFloatingCouponInfo.spread;
           
          %% SwapFloatingCouponSchedule end
            
            
          %% pricing time Step Generation start
            
            % eventTimeStep Generate
            mcOneTimeStep = numMethod.mcOneTimeStep;
            
          %% eventTime from dealStartTime and dealEndTime
           eventTimeStep = dealStartEventTime;
           eventTimeStep = union(eventTimeStep,dealEndEventTime);
           
          %% eventTime From structuredCouponSchedule:eventTime(endTime) +
          %% startTime,endTimeP
          %% daily weekly timeStep for couponSchedule
          
          %% eventTime From structuredCouponSchedule:eventTime
            eventTimeStep = union(eventTimeStep,structuredCouponSchedule.eventTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.startTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.endTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.endTimeP);
           %% eventTime From CallSchedule:eventTime
            eventTimeStep = union(eventTimeStep,callableSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
          %% eventTime From SwapFloatingCouponSchedule:eventTime
            eventTimeStep = union(eventTimeStep,swapFloatingCouponSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
            
            % alive Coupon : event has not happend Yet
            % from dealStartEventTime to 2nd AliveCoupon EventTime
            % dailyTimeStep should be added to eventTimeStep
            % for stable theta calculation
             
            aliveCouponIdxMax = max(find(structuredCouponSchedule.eventTime > 0));
            aliveCouponIdxMin = min(find(structuredCouponSchedule.eventTime > 0));
            
            aliveCouponNums = aliveCouponIdxMax - aliveCouponIdxMin +1;
            numOfAliveCouponDaily = min(aliveCouponNums,2);
            if numOfAliveCouponDaily > 0
              %% dailyTimeSteps for near couponEventTime, till dealStartEventTime and valueTime
                % for fixing handling and theta issue
                dailyTimeSteps = [structuredCouponSchedule.eventTime(aliveCouponIdxMin + numOfAliveCouponDaily - 1):-1:dealStartEventTime];
                dailyTimeSteps = union(dailyTimeSteps,0);
                eventTimeStep = union(eventTimeStep,dailyTimeSteps);
              %% weeklyTimeSteps for far couponEventTime
                weeklyTimeSteps = [eventTimeStep(end):-1*mcOneTimeStep:structuredCouponSchedule.eventTime(aliveCouponIdxMin + numOfAliveCouponDaily - 1)];
                eventTimeStep = union(eventTimeStep,weeklyTimeSteps);
            else
              %% if no aliveCouponEventTime( if last StructuredCouponEventTime is passed but not paid yet)
                % if dealStartEventTime is passed then we add dailyTimeStep
                % from valueTime till dealStartEventTime 
                % for fixing handling
                
                if dealStartEventTime < 0 
                    dailyTimeSteps = [0:-1:dealStartEventTime];
                    eventTimeStep = union(eventTimeStep,dailyTimeSteps);
                end
                
            end
           
            eventTimeStep = sort(eventTimeStep,'ascend');
            
%           %% debugging purpose 20201013 start
%             passedTimeStep = eventTimeStep(eventTimeStep<0);
%             load('cmsSteepnerTimeStep20201013.mat','cmsSteepnerTimeStep20201013');
%             aliveTimeStep = sort(cmsSteepnerTimeStep20201013,'ascend');
%             eventTimeStep = [];
%             eventTimeStep = union(passedTimeStep,cmsSteepnerTimeStep20201013);
%             eventTimeStep = sort(eventTimeStep,'ascend');
%           
%           %% debugging purpose 20201013 end
            
          %% aliveTimeStep for mc path generation
            aliveTimeStep = eventTimeStep(eventTimeStep>=0);
            aliveTimeStep = sort(aliveTimeStep,'ascend');
            
            totalAliveTimeStepSize = length(aliveTimeStep);
            
           %% eventTimeStep Generation End
           
           %% numeraire info
            numMatTime = eventTimeStep(end);
            dfNumMat = lgm2F.DFRisky(numMatTime/365.0);

            numeraire.numMatTime = numMatTime;
            numeraire.dfNumMat = dfNumMat;
              
            % should we use eventTimeIdx as future only time stes?
            % In this case we better use more generic approach
            % so that all times steps are included 
            
          %% pricing time Step Generation end
           
          %% MC init
            rng('default');
            rng(0);
            
            numOfFactors = lgm2F.numOfFactor;
            NMC = numMethod.pathSize; 
            MCTimeStep = totalAliveTimeStepSize -1; % nowDate (0) should be skipped
%             Z= randn(NMC/2,MCTimeStep);
            Z= randn(NMC/2,numOfFactors*MCTimeStep);
            Z=[Z;-Z];
            Z=Z';

            %TimeInversion
            sizeZ = size(Z,1);
            A = zeros(sizeZ,sizeZ);
            for i=1:sizeZ
                A(i,sizeZ + 1 -i) = 1;
            end
            Z = A*Z;

%                 U = Z';
            U = Z;

           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = zeros(numOfFactors,NMC);
           
           nextX = initX;
            
          %% payoff init
           nominal = steepenerParams.params.nominal;
                      
%            payoffInfo.nominal = nominal;
% 
%            payoffInfo.couponSchedule = couponSchedule;
%            payoffInfo.callableSchedule = callableSchedule;
% %            payoffInfo.refixingSchedule = refixingSchedule;
           
           % EODFlag : if true then pricing as EndOfDate(cashflow on
           % valueDate wil be not included(already paid)
           % if false then pricing as Intraday ( cashflow on valueDate will 
           % be included)
           
           %EODFlag_Cashflow: Cashflow EOD Flag
           %EODFlag_Exercise: Exercise(Option) EOD Flag
           EODFlag_Cashflow = steepenerParams.params.EODFlag_Cashflow;
           EODFlag_Exercise = steepenerParams.params.EODFlag_Exercise;
           
           
           modelStatesSize = NMC;
           
          %% Define Pricing Columns
           % pricingColumns : nonCall,Callable  
           % we need steepenerCoupon as state variables for storing 
           % CMS Steepener Coupon
           
           % for the cashflow generated pricing columns as pricerInfo
           % we do use pricerInfo Structure
           
         %% state variable initialize
           
          % avg steeper coupon
           steepenerCoupon = zeros(structuredCouponSchedule.scheduleSize,modelStatesSize);
           swapFloatingCoupon = zeros(swapFloatingCouponSchedule.scheduleSize,modelStatesSize);
           
           cachedModelStatesCoupon = zeros(structuredCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
  
          % for regression (2 modelState variables)
           cachedModelStatesCall = zeros(callableSchedule.scheduleSize*numOfFactors,modelStatesSize);
           
          % for swapFloatin 
           cachedModelStatesSwapFloating = zeros(swapFloatingCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
           
          %% PricingColumnGenericCashflow
          %% a structure that has cashflows with cashflowNames
          %% for CMS AVG structured we have nonCall, nonCallB, callable
           
            %nonCall
            cashflowNames = {'steepenerCoupon','swapFloatingCoupon'};
            cashflowSizeMap = containers.Map({'steepenerCoupon','swapFloatingCoupon'},{structuredCouponSchedule.scheduleSize, ...
                                            swapFloatingCouponSchedule.scheduleSize});
            
            cashflowInfo.cashflowNames = cashflowNames;
            cashflowInfo.cashflowSizeMap = cashflowSizeMap;
            cashflowInfo.pathSize = modelStatesSize;
            
           nonCall = PricingColumnInfoGenericCashflow(cashflowInfo);
           nonCallB = PricingColumnInfoGenericCashflow(cashflowInfo);
           callable = PricingColumnInfoGenericCashflow(cashflowInfo);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(1,modelStatesSize);
           payoffStateA.cashflow_npv = zeros(1,modelStatesSize);
           
           % variable to use accruedSumOfStockPrice for moving average calculation 
           
           cD = 0; % cumulated day count
           cP = 0; 
           dI = 0; % dummy idx for loop
           dcfC = 0;
           dcfN = 0;
           dD = 0;
      
           idxL = zeros(1,modelStatesSize);
           idxS = zeros(1,modelStatesSize);

           idxC = zeros(1,modelStatesSize);
           idxN = zeros(1,modelStatesSize);

           spreadC = zeros(1,modelStatesSize);
           spreadN = zeros(1,modelStatesSize);
%            probSpread = zeros(1,pathSize);
           
            % we use all path for regression
            % conversion ITM , puttable ITM ,  callable ITM
            % what's the use of Alive Path only Regression Sampling ?
           
          %% Path Generation Forward
          %% Store path dependent variables for coupon calculation 
          %% Store state variables to be used for AMC Regression
           
           %payoff descrption update index
           currentTimeIdx = 1;
           %MC numMethodState update index
           aliveTimeIdx = 1;
           % currentTimeStep is update in for statement
%            for currentTimeStep = eventTimeStep(1):1:eventTimeStep(end)
           for idxT = 1:1:length(eventTimeStep)
                currentTimeStep = eventTimeStep(idxT);
                
                if idxT < length(eventTimeStep)
                    nextTimeStep = eventTimeStep(idxT+1);
                end
                % evaluate scripts 
                % evaluate stateVariables(stores)
                % evaluate expression per each pricing columns
                % at the end of the loop we induct forward
                % from left to right
                % fictional payoff script columns are
                % cumCoupon,nonCall,Callable
                
                        
                % find the current
                % if find return errors then?
                try
                    
                    % cachedModelStateCoupon for modelState
                    idxCouponEvent = find(currentTimeStep == structuredCouponSchedule.eventTime);
                    if ~isempty(idxCouponEvent)
                        cachedModelStatesCoupon(numOfFactors*(idxCouponEvent-1)+1:numOfFactors*idxCouponEvent,:) = nextX;
                    end
                    
                    % cachedModelStateCall for modelState
                    idxCallEvent = find(currentTimeStep == callableSchedule.eventTime);
                    if ~isempty(idxCallEvent)
                        cachedModelStatesCall(numOfFactors*(idxCallEvent-1)+1:numOfFactors*idxCallEvent,:) = nextX;
                    end
                    
                    % cachedModelStateSwapFloatingEvent for modelState
                    idxSwapFloatingEvent = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                    if ~isempty(idxSwapFloatingEvent)
                        cachedModelStatesSwapFloating(numOfFactors*(idxSwapFloatingEvent-1)+1:numOfFactors*idxSwapFloatingEvent,:) = nextX;
                    end
                    
                    % find the corresponding nearest future coupon event
                  %% structuredCouponSchedule event start
                    idx = min(find(currentTimeStep <= structuredCouponSchedule.eventTime));
                    if ~isempty(idx)
                      %% stateVariable floatingCoupon
                        idxL = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorL,nextX);
                        idxS = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorS,nextX);
                        
                        dcfN = nextTimeStep;
                        dcfC = currentTimeStep;
                        % if coupon accrual condition is met
                        % period startDate is included but 
                        % endDate is not included
                        
                        if currentTimeStep >= structuredCouponSchedule.startTime(idx) ...
                           && currentTimeStep < structuredCouponSchedule.eventTime(idx)
                            
                            dD = dcfN - dcfC;
                            cD = cD + dD;
                            cP = cP + (idxL-idxS)*dD;
                            
                        end
                         
                        % now if we are at couponSchedule eventTime
                        % update floatingCoupon
                        if currentTimeStep == structuredCouponSchedule.eventTime(idx)

                            steepenerCoupon(idx,:) =  max(min(multiplier * cP/cD + spread,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idx) * nominal;
                            
                            % resetting local variables for averaging
                            cD = 0; % cumulated day count
                            cP = 0; % cumulated steepener sum
                            
                            if idx == structuredCouponSchedule.scheduleSize
                                payoffStateA.cashflow = nominal*0.0 + steepenerCoupon(idx,:);
                             else
                                payoffStateA.cashflow = steepenerCoupon(idx,:);
                            end
                            
                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             payTime = structuredCouponSchedule.payTime(idx);
                             
                             %process payoff cashflow 
                             nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                     payoffStateA.cashflow,'steepenerCoupon',idx);
     
                        end
                    end
                    
                 %% structuredCouponSchedule event end
                    
                 %% now if we are at swapFloatingCouponSchedule eventTime
                 
                 %% swapFloatingCouponSchedule event start
                 
                    idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                    if ~isempty(idxFloat)
                        
                        floatRate = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
                        swapFloatingCoupon(idxFloat,:) =  (swapFloating_multiplier * floatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxFloat) * nominal;

                        payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);
                        
                     %% since we are paying floating CMS Coupon
                         % we multiply -1 with floating coupon before we
                         % process payoff
                        payoffStateA.cashflow = payoffStateA.cashflow * -1.0;
                        
                        payTime = swapFloatingCouponSchedule.payTime(idxFloat);

                        %process payoff cashflow 
                        nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                    end
                    
                  %% swapFloatingCouponSchedule event end

                    % if currentTimeSteps is today or future then inductMCForward
                    if currentTimeStep >= 0 && currentTimeStep < eventTimeStep(end)
                        mcmcOut   = inductMCForwardNew(lgm2F,currentTimeStep,nextTimeStep,...
                                    nextX,U(numOfFactors*(aliveTimeIdx-1)+1:numOfFactors*aliveTimeIdx,:));
                        nextX = mcmcOut.nextX;
                        %MC numMethodState update
                        aliveTimeIdx = aliveTimeIdx + 1;
                    end
                    
                    currentTimeIdx = currentTimeIdx +1;
                catch exception
                    throw(exception)
                end
                
                
           end
           
          %% Forward Induction End
          
           % backward pricingColumn
           % nonCallB,
           % callable
           
           
          %% Backward induction start
           for idxT=length(eventTimeStep):-1:1
               
               currentTimeStep = eventTimeStep(idxT);
               try
                   
                   % swapFloatingCoupon event happens at resetDate so, in
                   % backward induction it should be positioned
                   % before call event
                   
                 %% swapFloatingCoupon event start  
                   idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                   % if currentTime  is floatingCouponSchedule.eventTime
                   if ~isempty(idxFloat)
                       
                         % we use cached(saved) floatingCoupon that
                         % was generated from Forward Induction
                         payTime = swapFloatingCouponSchedule.payTime(idxFloat);
                         payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);
                         
                         cachedModelStatesSwapFloatingV = cachedModelStatesSwapFloating(numOfFactors*(idxFloat-1)+1:numOfFactors*idxFloat,:);
                         
                      %% since we are paying floating CMS Coupon
                         % we multiply -1 with floating coupon before we
                         % process payoff
                         payoffStateA.cashflow = payoffStateA.cashflow *-1.0;

                         % at coupon EventDate AddFunctor
                         %process payoff cashflow 
                         nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                               payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                         callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                               payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);
                            
                   end
                   
                 %% swapFloatingCoupon event end  
                   
                 %% issuer callable event
                 %% AssignExercise Functor : applied only when eventTime >=0
                 %% if EODAdjust is YES : endOfDate conversion option has expired
                 %% if EODAdjust is NO : intraDay, and option is still alive
                   idxCall = find(currentTimeStep == callableSchedule.eventTime);
                   if ~isempty(idxCall)
                       if isExercised 
                           
                            payTime = callableSchedule.payTime(idxCall);
                            
                            % future payment <- is this necessary ?
                            if payTime >= 0
                                
                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);

                               % exercised by notional amount
                               exerciseValue = callPayment*nominal*ones(1,modelStatesSize);
                               
                               % discount exerciseValues from payTime to eventTime
                               df_ep = lgm2F.discountFactorRisky(currentTimeStep, payTime,numMatTime,cachedModelStatesCallV);
                               for k=1:modelStatesSize
                                   exerciseValue(k) = exerciseValue(k) * df_ep(k);
                               end
                               
                               outCallablePayoff = callable.ProcessAssignPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCallV, ...
                                     exerciseValue);
                                
                            end
                       else
                           
                           if currentTimeStep >=0 && EODAdjust(EODFlag_Exercise,currentTimeStep)

                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);

                               exerciseValue = zeros(1,modelStatesSize);
                               exerciseITMCondition = zeros(1,modelStatesSize);

                               for idxEx=1:modelStatesSize
                                   exerciseValue(idxEx) = callPayment*nominal; 
                                   exerciseITMCondition(idxEx) = 1;
                               end

                               payTime = callableSchedule.payTime(idxCall);

                               % discount exerciseValues from payTime to eventTime
                               df_ep = lgm2F.discountFactorRisky(currentTimeStep, payTime,numMatTime,cachedModelStatesCallV);
                               for k=1:modelStatesSize
                                   exerciseValue(k) = exerciseValue(k) * df_ep(k);
                               end

                               % callable is minimizing the payoff
                               minMaxFlag   = 'MIN';
                               decisionVar1 =  cachedModelStatesCallV;
                               outCallablePayoff = AssignExercise(lgm2F,currentTimeStep,callable,exerciseValue,exerciseITMCondition,...
                                          minMaxFlag,decisionVar1,nominal,numeraire,cachedModelStatesCallV);

                           end
                       
                       end
                   end
                   
                 %% coupon event  
                   idxCoupon = find(currentTimeStep == structuredCouponSchedule.eventTime);
                   % if currentTime  is couponSchedule.eventTime
                   if ~isempty(idxCoupon)
                       
                       if currentTimeStep == structuredCouponSchedule.eventTime(idxCoupon)
                             % we use cached(saved) floating Coupon that
                             % was generated from Forward Induction
                             payTime = structuredCouponSchedule.payTime(idxCoupon);
                             
                             if idxCoupon == structuredCouponSchedule.scheduleSize
                                payoffStateA.cashflow = nominal*0.0 + steepenerCoupon(idxCoupon,:);
                             else
                                payoffStateA.cashflow = steepenerCoupon(idxCoupon,:);
                             end
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
                            
                             cachedModelStatesCouponV = cachedModelStatesCoupon(numOfFactors*(idxCoupon-1)+1:numOfFactors*idxCoupon,:);
                             
                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,'steepenerCoupon',idxCoupon);

                             callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,'steepenerCoupon',idxCoupon);
                            
                       end
                   end
                 
                 
                   
               catch exception
                    throw(exception)
               end    
           end
           
           
          %% Finalize output
           nonCall.npv = mean(nonCall.payoff);
           nonCallB.npv = mean(nonCallB.payoff);
           callable.npv = mean(callable.payoff);
       
           %forward
           out.nonCall = nonCall; 
           %backward
           out.nonCallB = nonCallB; 
           out.callable = callable;
           
           out.eventTimeStep = eventTimeStep';
           
        end
        
        function out = computeAVGSteepenerSwapGeneric2(lgm2F,valueDate,steepenerParams,numMethod)
%           
            % model schedule generation
            dealStartDate = steepenerParams.startDate;
            dealStartEventTime = DateDiff(dealStartDate,valueDate);
            expiryDate = steepenerParams.endDate;
            dealEndEventTime = DateDiff(expiryDate,valueDate);
            %% structuredCouponSchedule start
            
            structuredCouponScheduleInt = steepenerParams.params.structuredCouponInfo.structuredCouponScheduleInt;
            
            % structuredCouponSchedule is a schedule of Structured Coupon
            % events
            
            % main backbone schedule
            structuredCouponScheduleSize = size(structuredCouponScheduleInt,1);
            structuredCouponSchedule= struct;
            structuredCouponSchedule.scheduleSize = structuredCouponScheduleSize;
            structuredCouponSchedule.eventType = steepenerParams.params.structuredCouponInfo.eventType;
            % coupon payment event happens at endDate
            % eventTime is endDate
            % payment date can be different from endDate
            
            % startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:structuredCouponSchedule.scheduleSize
%                 structuredCouponSchedule.payDate(i) = H_Date(structuredCouponScheduleInt(i,4));
                couponStartDate = H_Date(structuredCouponScheduleInt(i,2));
                couponEndDate = H_Date(structuredCouponScheduleInt(i,3));
                couponPayDate = H_Date(structuredCouponScheduleInt(i,4));
                
                structuredCouponSchedule.dayCount(i) = couponEndDate.DateDiff(couponStartDate);
                structuredCouponSchedule.dayCountFraction(i) = structuredCouponSchedule.dayCount(i)/365.0; 
                
                couponEndDateP = couponEndDate.AddDate( -1 ,'day');
                
                structuredCouponSchedule.startTime(i) = DateDiff(couponStartDate,valueDate);
                structuredCouponSchedule.endTimeP(i)  = DateDiff(couponEndDateP,valueDate);
                structuredCouponSchedule.endTime(i) = DateDiff(couponEndDate,valueDate);
                
                if strcmp(structuredCouponSchedule.eventType,'END')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                elseif strcmp(structuredCouponSchedule.eventType,'START')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponStartDate,valueDate);
                else % default is END : event happens at EndDate for StructuredCoupon
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                end
                structuredCouponSchedule.payTime(i) = DateDiff(couponPayDate,valueDate);
                
            end
            
            
            tenorL     = steepenerParams.params.structuredCouponInfo.tenorL;
            tenorS     = steepenerParams.params.structuredCouponInfo.tenorS;
            
           %% rateType for structuredCoupon : default CMS
            if isfield(steepenerParams.params.structuredCouponInfo, 'rateType_tenorL')
                rateType_tenorL = steepenerParams.params.structuredCouponInfo.rateType_tenorL;
            else
                rateType_tenorL = 'CMS';
            end
            
            if isfield(steepenerParams.params.structuredCouponInfo, 'rateType_tenorS')
                rateType_tenorS = steepenerParams.params.structuredCouponInfo.rateType_tenorS;
            else
                rateType_tenorS = 'CMS';
            end
            
            
            localFloor = steepenerParams.params.structuredCouponInfo.localFloor;
            localCap   = steepenerParams.params.structuredCouponInfo.localCap;
            multiplier = steepenerParams.params.structuredCouponInfo.multiplier;
            spread     = steepenerParams.params.structuredCouponInfo.spread;
%             coupon     = steepenerParams.params.structuredCouponInfo.coupon;
           %% structuredCoupon schedule end
           
           %% call exercise event happens at the callExerciseDate
           %% callable schedule start
            callableScheduleInt = steepenerParams.params.callableInfo.callableScheduleInt;
            callableScheduleSize = size(callableScheduleInt,1);
            callableSchedule= struct;
            for i=1:callableScheduleSize
                callExerciseDate = H_Date(callableScheduleInt(i,1));
                callPayDate = H_Date(callableScheduleInt(i,2));
                callableSchedule.eventTime(i) = DateDiff(callExerciseDate,valueDate);
                callableSchedule.payTime(i) = DateDiff(callPayDate,valueDate);
                
                 callableSchedule.callPayment(i) = callableScheduleInt(i,3);
            end
            callableSchedule.scheduleSize = callableScheduleSize;
           
            isExercised = steepenerParams.params.callableInfo.isExercised;
           %% callable schedule end
           
          %% swapFloatingCouponSchedule start
            
            swapFloatingCouponScheduleInt = steepenerParams.params.swapFloatingCouponInfo.swapFloatingCouponScheduleInt;
            
            swapFloatingCouponSchedule= struct;
            swapFloatingCouponSchedule.scheduleSize = size(swapFloatingCouponScheduleInt,1);
            
            swapFloatingCouponSchedule.eventType = steepenerParams.params.swapFloatingCouponInfo.eventType;
            
            % floating coupon payment event happens at resetDate
            % eventTime is resetDate
            
            % resetTime,startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:swapFloatingCouponSchedule.scheduleSize
                flolatingCouponResetDate = H_Date(swapFloatingCouponScheduleInt(i,1));
                flolatingCouponStartDate = H_Date(swapFloatingCouponScheduleInt(i,2));
                flolatingCouponEndDate = H_Date(swapFloatingCouponScheduleInt(i,3));
                flolatingCouponPayDate = H_Date(swapFloatingCouponScheduleInt(i,4));
                
                swapFloatingCouponSchedule.dayCount(i) = flolatingCouponEndDate.DateDiff(flolatingCouponStartDate);
                swapFloatingCouponSchedule.dayCountFraction(i) = swapFloatingCouponSchedule.dayCount(i)/365.0; 
                
                flolatingCouponEndDateP = flolatingCouponEndDate.AddDate( -1 ,'day');
                
                swapFloatingCouponSchedule.resetTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                swapFloatingCouponSchedule.startTime(i) = DateDiff(flolatingCouponStartDate,valueDate);
                swapFloatingCouponSchedule.endTimeP(i)  = DateDiff(flolatingCouponEndDateP,valueDate);
                swapFloatingCouponSchedule.endTime(i) = DateDiff(flolatingCouponEndDate,valueDate);
                
                if strcmp(swapFloatingCouponSchedule.eventType,'RESET')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                elseif strcmp(swapFloatingCouponSchedule.eventType,'START')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponStartDate,valueDate);
                elseif strcmp(swapFloatingCouponSchedule.eventType,'END')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponEndDate,valueDate);
                else % default is RESET : for SwapFloatingCouponSchedule
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                end
                
%                 swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                
                swapFloatingCouponSchedule.payTime(i) = DateDiff(flolatingCouponPayDate,valueDate);
                
            end
            
            swapFloating_tenor  = steepenerParams.params.swapFloatingCouponInfo.tenorFloat;
            
            %% rateType for swapFloatingCoupon : default LIBOR
            if isfield(steepenerParams.params.structuredCouponInfo, 'rateType_tenorFloat')
                rateType_tenorFloat = steepenerParams.params.structuredCouponInfo.rateType_tenorFloat;
            else
                rateType_tenorFloat = 'LIBOR';
            end
            
            swapFloating_multiplier = steepenerParams.params.swapFloatingCouponInfo.multiplier;
            swapFloating_spread     = steepenerParams.params.swapFloatingCouponInfo.spread;
           
          %% SwapFloatingCouponSchedule end
            
            
          %% pricing time Step Generation start
            
            % eventTimeStep Generate
            mcOneTimeStep = numMethod.mcOneTimeStep;
            
          %% eventTime from dealStartTime and dealEndTime
           eventTimeStep = dealStartEventTime;
           eventTimeStep = union(eventTimeStep,dealEndEventTime);
           
          %% eventTime From structuredCouponSchedule:eventTime(endTime) +
          %% startTime,endTimeP
          %% daily weekly timeStep for couponSchedule
          
          %% eventTime From structuredCouponSchedule:eventTime
            eventTimeStep = union(eventTimeStep,structuredCouponSchedule.eventTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.startTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.endTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.endTimeP);
           %% eventTime From CallSchedule:eventTime
            eventTimeStep = union(eventTimeStep,callableSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
          %% eventTime From SwapFloatingCouponSchedule:eventTime
            eventTimeStep = union(eventTimeStep,swapFloatingCouponSchedule.resetTime);  
            eventTimeStep = union(eventTimeStep,swapFloatingCouponSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
            
            % alive Coupon : event has not happend Yet
            % from dealStartEventTime to 2nd AliveCoupon EventTime
            % dailyTimeStep should be added to eventTimeStep
            % for stable theta calculation
             
            aliveCouponIdxMax = max(find(structuredCouponSchedule.eventTime > 0));
            aliveCouponIdxMin = min(find(structuredCouponSchedule.eventTime > 0));
            
            aliveCouponNums = aliveCouponIdxMax - aliveCouponIdxMin +1;
            numOfAliveCouponDaily = min(aliveCouponNums,2);
            if numOfAliveCouponDaily > 0
              %% dailyTimeSteps for near couponEventTime, till dealStartEventTime and valueTime
                % for fixing handling and theta issue
                dailyTimeSteps = [structuredCouponSchedule.eventTime(aliveCouponIdxMin + numOfAliveCouponDaily - 1):-1:dealStartEventTime];
                dailyTimeSteps = union(dailyTimeSteps,0);
                eventTimeStep = union(eventTimeStep,dailyTimeSteps);
              %% weeklyTimeSteps for far couponEventTime
                weeklyTimeSteps = [eventTimeStep(end):-1*mcOneTimeStep:structuredCouponSchedule.eventTime(aliveCouponIdxMin + numOfAliveCouponDaily - 1)];
                eventTimeStep = union(eventTimeStep,weeklyTimeSteps);
            else
              %% if no aliveCouponEventTime( if last StructuredCouponEventTime is passed but not paid yet)
                % if dealStartEventTime is passed then we add dailyTimeStep
                % from valueTime till dealStartEventTime 
                % for fixing handling
                
                if dealStartEventTime < 0 
                    dailyTimeSteps = [0:-1:dealStartEventTime];
                    eventTimeStep = union(eventTimeStep,dailyTimeSteps);
                end
                
            end
           
            eventTimeStep = sort(eventTimeStep,'ascend');
            
%           %% debugging purpose 20201013 start
%             passedTimeStep = eventTimeStep(eventTimeStep<0);
%             load('cmsSteepnerTimeStep20201013.mat','cmsSteepnerTimeStep20201013');
%             aliveTimeStep = sort(cmsSteepnerTimeStep20201013,'ascend');
%             eventTimeStep = [];
%             eventTimeStep = union(passedTimeStep,cmsSteepnerTimeStep20201013);
%             eventTimeStep = sort(eventTimeStep,'ascend');
%           
%           %% debugging purpose 20201013 end
            
          %% aliveTimeStep for mc path generation
            aliveTimeStep = eventTimeStep(eventTimeStep>=0);
            aliveTimeStep = sort(aliveTimeStep,'ascend');
            
            totalAliveTimeStepSize = length(aliveTimeStep);
            
           %% eventTimeStep Generation End
           
           %% numeraire info
            numMatTime = eventTimeStep(end);
            dfNumMat = lgm2F.DFRisky(numMatTime/365.0);

            numeraire.numMatTime = numMatTime;
            numeraire.dfNumMat = dfNumMat;
              
            % should we use eventTimeIdx as future only time stes?
            % In this case we better use more generic approach
            % so that all times steps are included 
            
          %% pricing time Step Generation end
           
          %% MC init
            rng('default');
            rng(0);
            
            numOfFactors = lgm2F.numOfFactor;
            NMC = numMethod.pathSize; 
            MCTimeStep = totalAliveTimeStepSize -1; % nowDate (0) should be skipped
%             Z= randn(NMC/2,MCTimeStep);
            Z= randn(NMC/2,numOfFactors*MCTimeStep);
            Z=[Z;-Z];
            Z=Z';

            %TimeInversion
            sizeZ = size(Z,1);
            A = zeros(sizeZ,sizeZ);
            for i=1:sizeZ
                A(i,sizeZ + 1 -i) = 1;
            end
            Z = A*Z;

%                 U = Z';
            U = Z;

           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = zeros(numOfFactors,NMC);
           
           nextX = initX;
            
          %% payoff init
           nominal = steepenerParams.params.nominal;
                      
%            payoffInfo.nominal = nominal;
% 
%            payoffInfo.couponSchedule = couponSchedule;
%            payoffInfo.callableSchedule = callableSchedule;
% %            payoffInfo.refixingSchedule = refixingSchedule;
           
           % EODFlag : if true then pricing as EndOfDate(cashflow on
           % valueDate wil be not included(already paid)
           % if false then pricing as Intraday ( cashflow on valueDate will 
           % be included)
           
           %EODFlag_Cashflow: Cashflow EOD Flag
           %EODFlag_Exercise: Exercise(Option) EOD Flag
           EODFlag_Cashflow = steepenerParams.params.EODFlag_Cashflow;
           EODFlag_Exercise = steepenerParams.params.EODFlag_Exercise;
           
           
           modelStatesSize = NMC;
           
          %% Define Pricing Columns
           % pricingColumns : nonCall,Callable  
           % we need steepenerCoupon as state variables for storing 
           % CMS Steepener Coupon
           
           % for the cashflow generated pricing columns as pricerInfo
           % we do use pricerInfo Structure
           
         %% (global) state variable initialize  : these correspond to ColumnVal[t] in generic pricing deal description
           
          % avg steeper coupon
           steepenerCoupon = zeros(structuredCouponSchedule.scheduleSize,modelStatesSize);
           swapFloatingCoupon = zeros(swapFloatingCouponSchedule.scheduleSize,modelStatesSize);
           
           prefixedSwapFloatingRate = zeros(swapFloatingCouponSchedule.scheduleSize,modelStatesSize);
           
           cachedModelStatesCoupon = zeros(structuredCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
  
          % for regression (2 modelState variables)
           cachedModelStatesCall = zeros(callableSchedule.scheduleSize*numOfFactors,modelStatesSize);
           
          % for swapFloatin 
           cachedModelStatesSwapFloating = zeros(swapFloatingCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
           
          %% PricingColumnGenericCashflow
          %% a structure that has cashflows with cashflowNames
          %% for CMS AVG structured we have nonCall, nonCallB, callable
           
            %nonCall
            cashflowNames = {'steepenerCoupon','swapFloatingCoupon'};
            cashflowSizeMap = containers.Map({'steepenerCoupon','swapFloatingCoupon'},{structuredCouponSchedule.scheduleSize, ...
                                            swapFloatingCouponSchedule.scheduleSize});
            
            cashflowInfo.cashflowNames = cashflowNames;
            cashflowInfo.cashflowSizeMap = cashflowSizeMap;
            cashflowInfo.pathSize = modelStatesSize;
            
           nonCall = PricingColumnInfoGenericCashflow(cashflowInfo);
           nonCallB = PricingColumnInfoGenericCashflow(cashflowInfo);
           callable = PricingColumnInfoGenericCashflow(cashflowInfo);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(1,modelStatesSize);
           payoffStateA.cashflow_npv = zeros(1,modelStatesSize);
           
           % variable to use accruedSumOfStockPrice for moving average calculation 
           
           cD = 0; % cumulated day count
           cP = 0; 
           dI = 0; % dummy idx for loop
           dcfC = 0;
           dcfN = 0;
           dD = 0;
      
           idxL = zeros(1,modelStatesSize);
           idxS = zeros(1,modelStatesSize);

           idxC = zeros(1,modelStatesSize);
           idxN = zeros(1,modelStatesSize);

           spreadC = zeros(1,modelStatesSize);
           spreadN = zeros(1,modelStatesSize);
%            probSpread = zeros(1,pathSize);
           
            % we use all path for regression
            % conversion ITM , puttable ITM ,  callable ITM
            % what's the use of Alive Path only Regression Sampling ?
           
          %% Path Generation Forward
          %% Store path dependent variables for coupon calculation 
          %% Store state variables to be used for AMC Regression
           
           %payoff descrption update index
           currentTimeIdx = 1;
           %MC numMethodState update index
           aliveTimeIdx = 1;
           % currentTimeStep is update in for statement
%            for currentTimeStep = eventTimeStep(1):1:eventTimeStep(end)
           for idxT = 1:1:length(eventTimeStep)
                currentTimeStep = eventTimeStep(idxT);
                
                if idxT < length(eventTimeStep)
                    nextTimeStep = eventTimeStep(idxT+1);
                end
                % evaluate scripts 
                % evaluate stateVariables(stores)
                % evaluate expression per each pricing columns
                % at the end of the loop we induct forward
                % from left to right
                % fictional payoff script columns are
                % cumCoupon,nonCall,Callable
                
                        
                % find the current
                % if find return errors then?
                try
                    
                    % cachedModelStateCoupon for modelState
                    idxCouponEvent = find(currentTimeStep == structuredCouponSchedule.eventTime);
                    if ~isempty(idxCouponEvent)
                        cachedModelStatesCoupon(numOfFactors*(idxCouponEvent-1)+1:numOfFactors*idxCouponEvent,:) = nextX;
                    end
                    
                    % cachedModelStateCall for modelState
                    idxCallEvent = find(currentTimeStep == callableSchedule.eventTime);
                    if ~isempty(idxCallEvent)
                        cachedModelStatesCall(numOfFactors*(idxCallEvent-1)+1:numOfFactors*idxCallEvent,:) = nextX;
                    end
                    
                    % cachedModelStateSwapFloatingEvent for modelState
                    idxSwapFloatingEvent = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                    if ~isempty(idxSwapFloatingEvent)
                        cachedModelStatesSwapFloating(numOfFactors*(idxSwapFloatingEvent-1)+1:numOfFactors*idxSwapFloatingEvent,:) = nextX;
                    end
                    
                    % find the corresponding nearest future coupon event
                  %% structuredCouponSchedule event start
                    idx = min(find(currentTimeStep <= structuredCouponSchedule.eventTime));
                    if ~isempty(idx)
                      %% stateVariable floatingCoupon
                        idxL = lgm2F.Rate_IndexGeneric(rateType_tenorL,valueDate,currentTimeStep,numMatTime, tenorL,nextX);
                        idxS = lgm2F.Rate_IndexGeneric(rateType_tenorS,valueDate,currentTimeStep,numMatTime, tenorS,nextX);
                        
%                         idxL = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorL,nextX);
%                         idxS = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorS,nextX);
                        
                        dcfN = nextTimeStep;
                        dcfC = currentTimeStep;
                        % if coupon accrual condition is met
                        % period startDate is included but 
                        % endDate is not included
                        
                        if currentTimeStep >= structuredCouponSchedule.startTime(idx) ...
                           && currentTimeStep < structuredCouponSchedule.eventTime(idx)
                            
                            dD = dcfN - dcfC;
                            cD = cD + dD;
                            cP = cP + (idxL-idxS)*dD;
                            
                        end
                         
                        % now if we are at couponSchedule eventTime
                        % update floatingCoupon
                        if currentTimeStep == structuredCouponSchedule.eventTime(idx)

                            steepenerCoupon(idx,:) =  max(min(multiplier * cP/cD + spread,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idx) * nominal;
                            
                            % resetting local variables for averaging
                            cD = 0; % cumulated day count
                            cP = 0; % cumulated steepener sum
                            
                            if idx == structuredCouponSchedule.scheduleSize
                                payoffStateA.cashflow = nominal*0.0 + steepenerCoupon(idx,:);
                             else
                                payoffStateA.cashflow = steepenerCoupon(idx,:);
                            end
                            
                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             payTime = structuredCouponSchedule.payTime(idx);
                             
                             %process payoff cashflow 
                             nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                     payoffStateA.cashflow,'steepenerCoupon',idx);
     
                        end
                    end
                    
                 %% structuredCouponSchedule event end
                    
                 %% now if we are at swapFloatingCouponSchedule eventTime
                    
                 %% swapFloatingCouponSchedule event start
                 %% if swapFloatingCoupon event happens at RESET Date
                 %% then we fix floatingCoupon at resetDate and we raise event at resetdate
                 %% if swapFloatingCoupon event happens at END date
                 %% then we fix floatingCoupon at resetDate and we raise event at endDate
                 
                   if strcmp(swapFloatingCouponSchedule.eventType,'RESET')
                     %% now if we are at swapFloatingCouponSchedule eventTime
                     %% swapFloatingCouponSchedule event start

                        idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                        if ~isempty(idxFloat)

                            floatRate = lgm2F.Rate_IndexGeneric(rateType_tenorFloat,valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
%                             floatRate = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
                            
                            swapFloatingCoupon(idxFloat,:) =  (swapFloating_multiplier * floatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxFloat) * nominal;

                            payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                         %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                            payoffStateA.cashflow = payoffStateA.cashflow * -1.0;

                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
                            
                            payTime = swapFloatingCouponSchedule.payTime(idxFloat);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                        end
                      %% swapFloatingCouponSchedule event end
                       
                   elseif strcmp(swapFloatingCouponSchedule.eventType,'END')
                     %% if we raise swapFloatingCouponEvent at EndDate then
                     %% we need to raise fixing event at resetDate and store fixed floatRate as stateVariables
                     %% and reference it at ENDDate
                        idxFloatReset = find(currentTimeStep == swapFloatingCouponSchedule.resetTime);
                        if ~isempty(idxFloatReset)
                            
                            floatRate = lgm2F.Rate_IndexGeneric(rateType_tenorFloat,valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
%                             floatRate = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
                            
                            prefixedSwapFloatingRate(idxFloatReset,:) = floatRate;
                        end
                        
                        idxEnd = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                        if ~isempty(idxEnd)
                            % now if we are at swapFloatingCouponSchedule
                            % eventTime(=endDate)
                            % then we update swapFloatingCoupon
                            prefixedFloatRate = prefixedSwapFloatingRate(idxEnd,:);
                            swapFloatingCoupon(idxEnd,:) =  (swapFloating_multiplier * prefixedFloatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxEnd) * nominal;

                            payoffStateA.cashflow = swapFloatingCoupon(idxEnd,:);

                         %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                            payoffStateA.cashflow = payoffStateA.cashflow * -1.0;
                            
                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                            payTime = swapFloatingCouponSchedule.payTime(idxEnd);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'swapFloatingCoupon',idxEnd);
                            
                        end
                 %% swapFloatingCouponSchedule event end
                   else
                       error('not supported eventType')
                   end
                    % if currentTimeSteps is today or future then inductMCForward
                    if currentTimeStep >= 0 && currentTimeStep < eventTimeStep(end)
                        mcmcOut   = inductMCForwardNew(lgm2F,currentTimeStep,nextTimeStep,...
                                    nextX,U(numOfFactors*(aliveTimeIdx-1)+1:numOfFactors*aliveTimeIdx,:));
                        nextX = mcmcOut.nextX;
                        %MC numMethodState update
                        aliveTimeIdx = aliveTimeIdx + 1;
                    end
                    
                    currentTimeIdx = currentTimeIdx +1;
                catch exception
                    throw(exception)
                end
                
                
           end
           
          %% Forward Induction End
          
           % backward pricingColumn
           % nonCallB,
           % callable
           
           
          %% Backward induction start
           for idxT=length(eventTimeStep):-1:1
               
               currentTimeStep = eventTimeStep(idxT);
               try
                  if strcmp(swapFloatingCouponSchedule.eventType,'RESET') 
                    %% if swapFloatingCoupon event happens at resetDate so, in
                    %% backward induction it should be positioned
                    %% before call event

                     %% swapFloatingCoupon event start  
                       idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                       % if currentTime  is floatingCouponSchedule.eventTime
                       if ~isempty(idxFloat)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = swapFloatingCouponSchedule.payTime(idxFloat);
                             payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                             cachedModelStatesSwapFloatingV = cachedModelStatesSwapFloating(numOfFactors*(idxFloat-1)+1:numOfFactors*idxFloat,:);

                          %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                             payoffStateA.cashflow = payoffStateA.cashflow *-1.0;
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                             callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                       end

                     %% swapFloatingCoupon event end  
                  end
                 %% issuer callable event
                 %% AssignExercise Functor : applied only when eventTime >=0
                 %% if EODAdjust is YES : endOfDate conversion option has expired
                 %% if EODAdjust is NO : intraDay, and option is still alive
                   idxCall = find(currentTimeStep == callableSchedule.eventTime);
                   if ~isempty(idxCall)
                       if isExercised 
                           
                            payTime = callableSchedule.payTime(idxCall);
                            
                            % future payment <- is this necessary ?
                            if payTime >= 0
                                
                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);

                               % exercised by notional amount
                               exerciseValue = callPayment*nominal*ones(1,modelStatesSize);
                               
                               % discount exerciseValues from payTime to eventTime
                               df_ep = lgm2F.discountFactorRisky(currentTimeStep, payTime,numMatTime,cachedModelStatesCallV);
                               for k=1:modelStatesSize
                                   exerciseValue(k) = exerciseValue(k) * df_ep(k);
                               end
                               
                               outCallablePayoff = callable.ProcessAssignPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCallV, ...
                                     exerciseValue);
                                
                            end
                       else
                           
                           if currentTimeStep >=0 && EODAdjust(EODFlag_Exercise,currentTimeStep)

                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);

                               exerciseValue = zeros(1,modelStatesSize);
                               exerciseITMCondition = zeros(1,modelStatesSize);

                               for idxEx=1:modelStatesSize
                                   exerciseValue(idxEx) = callPayment*nominal; 
                                   exerciseITMCondition(idxEx) = 1;
                               end

                               payTime = callableSchedule.payTime(idxCall);

                               % discount exerciseValues from payTime to eventTime
                               df_ep = lgm2F.discountFactorRisky(currentTimeStep, payTime,numMatTime,cachedModelStatesCallV);
                               for k=1:modelStatesSize
                                   exerciseValue(k) = exerciseValue(k) * df_ep(k);
                               end

                               % callable is minimizing the payoff
                               minMaxFlag   = 'MIN';
                               decisionVar1 =  cachedModelStatesCallV;
                               outCallablePayoff = AssignExercise(lgm2F,currentTimeStep,callable,exerciseValue,exerciseITMCondition,...
                                          minMaxFlag,decisionVar1,nominal,numeraire,cachedModelStatesCallV);

                           end
                       
                       end
                   end
                   
                 %% coupon event  
                   idxCoupon = find(currentTimeStep == structuredCouponSchedule.eventTime);
                   % if currentTime  is couponSchedule.eventTime
                   if ~isempty(idxCoupon)
                       
                       if currentTimeStep == structuredCouponSchedule.eventTime(idxCoupon)
                             % we use cached(saved) floating Coupon that
                             % was generated from Forward Induction
                             payTime = structuredCouponSchedule.payTime(idxCoupon);
                             
                             if idxCoupon == structuredCouponSchedule.scheduleSize
                                payoffStateA.cashflow = nominal*0.0 + steepenerCoupon(idxCoupon,:);
                             else
                                payoffStateA.cashflow = steepenerCoupon(idxCoupon,:);
                             end
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
                            
                             cachedModelStatesCouponV = cachedModelStatesCoupon(numOfFactors*(idxCoupon-1)+1:numOfFactors*idxCoupon,:);
                             
                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,'steepenerCoupon',idxCoupon);

                             callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,'steepenerCoupon',idxCoupon);
                            
                       end
                   end
                  
                  if strcmp(swapFloatingCouponSchedule.eventType,'END')  
                    %% if swapFloatingCoupon event happens at endDate 
                    %% backward induction it should be positioned
                    %% after call event

                     %% swapFloatingCoupon event start  
                       idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                       % if currentTime  is floatingCouponSchedule.eventTime
                       if ~isempty(idxFloat)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = swapFloatingCouponSchedule.payTime(idxFloat);
                             payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                             cachedModelStatesSwapFloatingV = cachedModelStatesSwapFloating(numOfFactors*(idxFloat-1)+1:numOfFactors*idxFloat,:);

                          %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                             payoffStateA.cashflow = payoffStateA.cashflow *-1.0;
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                             callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                       end

                     %% swapFloatingCoupon event end 
                  end
                 
                   
               catch exception
                    throw(exception)
               end    
           end
           
           
          %% Finalize output
           nonCall.npv = mean(nonCall.payoff);
           nonCallB.npv = mean(nonCallB.payoff);
           callable.npv = mean(callable.payoff);
       
           %forward
           out.nonCall = nonCall; 
           %backward
           out.nonCallB = nonCallB; 
           out.callable = callable;
           
           out.eventTimeStep = eventTimeStep';
           
        end
        
        function out = computeSteepenerSwapGeneric2(lgm2F,valueDate,steepenerParams,numMethod)
%           
            % model schedule generation
            dealStartDate = steepenerParams.startDate;
            dealStartEventTime = DateDiff(dealStartDate,valueDate);
            expiryDate = steepenerParams.endDate;
            dealEndEventTime = DateDiff(expiryDate,valueDate);
            %% structuredCouponSchedule start
            
            structuredCouponScheduleInt = steepenerParams.params.structuredCouponInfo.structuredCouponScheduleInt;
            
            % structuredCouponSchedule is a schedule of Structured Coupon
            % events
            
            % main backbone schedule
            structuredCouponScheduleSize = size(structuredCouponScheduleInt,1);
            structuredCouponSchedule= struct;
            structuredCouponSchedule.scheduleSize = structuredCouponScheduleSize;
            structuredCouponSchedule.eventType = steepenerParams.params.structuredCouponInfo.eventType;
            % coupon payment event happens at endDate
            % eventTime is endDate
            % payment date can be different from endDate
            
            % startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:structuredCouponSchedule.scheduleSize
%                 structuredCouponSchedule.payDate(i) = H_Date(structuredCouponScheduleInt(i,4));
                couponResetDate = H_Date(structuredCouponScheduleInt(i,1));
                couponStartDate = H_Date(structuredCouponScheduleInt(i,2));
                couponEndDate = H_Date(structuredCouponScheduleInt(i,3));
                couponPayDate = H_Date(structuredCouponScheduleInt(i,4));
                
                structuredCouponSchedule.dayCount(i) = couponEndDate.DateDiff(couponStartDate);
                structuredCouponSchedule.dayCountFraction(i) = structuredCouponSchedule.dayCount(i)/365.0; 
                
                couponEndDateP = couponEndDate.AddDate( -1 ,'day');
                
                structuredCouponSchedule.resetTime(i) = DateDiff(couponResetDate,valueDate);
                structuredCouponSchedule.startTime(i) = DateDiff(couponStartDate,valueDate);
                structuredCouponSchedule.endTimeP(i)  = DateDiff(couponEndDateP,valueDate);
                structuredCouponSchedule.endTime(i) = DateDiff(couponEndDate,valueDate);
                
                if strcmp(structuredCouponSchedule.eventType,'RESET')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponResetDate,valueDate);
                elseif strcmp(structuredCouponSchedule.eventType,'END')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                elseif strcmp(structuredCouponSchedule.eventType,'START')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponStartDate,valueDate);
                else % default is END : event happens at EndDate for StructuredCoupon
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                end
                structuredCouponSchedule.payTime(i) = DateDiff(couponPayDate,valueDate);
                
            end
            
            
            tenorL     = steepenerParams.params.structuredCouponInfo.tenorL;
            tenorS     = steepenerParams.params.structuredCouponInfo.tenorS;
            
           %% rateType for structuredCoupon : default CMS
            if isfield(steepenerParams.params.structuredCouponInfo, 'rateType_tenorL')
                rateType_tenorL = steepenerParams.params.structuredCouponInfo.rateType_tenorL;
            else
                rateType_tenorL = 'CMS';
            end
            
            if isfield(steepenerParams.params.structuredCouponInfo, 'rateType_tenorS')
                rateType_tenorS = steepenerParams.params.structuredCouponInfo.rateType_tenorS;
            else
                rateType_tenorS = 'CMS';
            end
            
            
            localFloor = steepenerParams.params.structuredCouponInfo.localFloor;
            localCap   = steepenerParams.params.structuredCouponInfo.localCap;
            multiplier = steepenerParams.params.structuredCouponInfo.multiplier;
            spread     = steepenerParams.params.structuredCouponInfo.spread;
%             coupon     = steepenerParams.params.structuredCouponInfo.coupon;
           %% structuredCoupon schedule end
           
           %% call exercise event happens at the callExerciseDate
           %% callable schedule start
            callableScheduleInt = steepenerParams.params.callableInfo.callableScheduleInt;
            callableScheduleSize = size(callableScheduleInt,1);
            callableSchedule= struct;
            for i=1:callableScheduleSize
                callExerciseDate = H_Date(callableScheduleInt(i,1));
                callPayDate = H_Date(callableScheduleInt(i,2));
                callableSchedule.eventTime(i) = DateDiff(callExerciseDate,valueDate);
                callableSchedule.payTime(i) = DateDiff(callPayDate,valueDate);
                
                 callableSchedule.callPayment(i) = callableScheduleInt(i,3);
            end
            callableSchedule.scheduleSize = callableScheduleSize;
           
            isExercised = steepenerParams.params.callableInfo.isExercised;
           %% callable schedule end
           
          %% swapFloatingCouponSchedule start
            
            swapFloatingCouponScheduleInt = steepenerParams.params.swapFloatingCouponInfo.swapFloatingCouponScheduleInt;
            
            swapFloatingCouponSchedule= struct;
            swapFloatingCouponSchedule.scheduleSize = size(swapFloatingCouponScheduleInt,1);
            
            swapFloatingCouponSchedule.eventType = steepenerParams.params.swapFloatingCouponInfo.eventType;
            
            % floating coupon payment event happens at resetDate
            % eventTime is resetDate
            
            % resetTime,startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:swapFloatingCouponSchedule.scheduleSize
                flolatingCouponResetDate = H_Date(swapFloatingCouponScheduleInt(i,1));
                flolatingCouponStartDate = H_Date(swapFloatingCouponScheduleInt(i,2));
                flolatingCouponEndDate = H_Date(swapFloatingCouponScheduleInt(i,3));
                flolatingCouponPayDate = H_Date(swapFloatingCouponScheduleInt(i,4));
                
                swapFloatingCouponSchedule.dayCount(i) = flolatingCouponEndDate.DateDiff(flolatingCouponStartDate);
                swapFloatingCouponSchedule.dayCountFraction(i) = swapFloatingCouponSchedule.dayCount(i)/365.0; 
                
                flolatingCouponEndDateP = flolatingCouponEndDate.AddDate( -1 ,'day');
                
                swapFloatingCouponSchedule.resetTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                swapFloatingCouponSchedule.startTime(i) = DateDiff(flolatingCouponStartDate,valueDate);
                swapFloatingCouponSchedule.endTimeP(i)  = DateDiff(flolatingCouponEndDateP,valueDate);
                swapFloatingCouponSchedule.endTime(i) = DateDiff(flolatingCouponEndDate,valueDate);
                
                if strcmp(swapFloatingCouponSchedule.eventType,'RESET')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                elseif strcmp(swapFloatingCouponSchedule.eventType,'START')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponStartDate,valueDate);
                elseif strcmp(swapFloatingCouponSchedule.eventType,'END')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponEndDate,valueDate);
                else % default is RESET : for SwapFloatingCouponSchedule
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                end
                
%                 swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                
                swapFloatingCouponSchedule.payTime(i) = DateDiff(flolatingCouponPayDate,valueDate);
                
            end
            
            swapFloating_tenor  = steepenerParams.params.swapFloatingCouponInfo.tenorFloat;
            
            %% rateType for swapFloatingCoupon : default LIBOR
            if isfield(steepenerParams.params.structuredCouponInfo, 'rateType_tenorFloat')
                rateType_tenorFloat = steepenerParams.params.structuredCouponInfo.rateType_tenorFloat;
            else
                rateType_tenorFloat = 'LIBOR';
            end
            
            swapFloating_multiplier = steepenerParams.params.swapFloatingCouponInfo.multiplier;
            swapFloating_spread     = steepenerParams.params.swapFloatingCouponInfo.spread;
           
          %% SwapFloatingCouponSchedule end
            
            
          %% pricing time Step Generation start
            
            % eventTimeStep Generate
            mcOneTimeStep = numMethod.mcOneTimeStep;
            
          %% eventTime from dealStartTime and dealEndTime
           eventTimeStep = dealStartEventTime;
           eventTimeStep = union(eventTimeStep,dealEndEventTime);
           
          %% eventTime From structuredCouponSchedule:eventTime(endTime) +
          %% startTime,endTimeP
          %% daily weekly timeStep for couponSchedule
          
          %% eventTime From structuredCouponSchedule:eventTime
            eventTimeStep = union(eventTimeStep,structuredCouponSchedule.eventTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.resetTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.startTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.endTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.endTimeP);
           %% eventTime From CallSchedule:eventTime
            eventTimeStep = union(eventTimeStep,callableSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
          %% eventTime From SwapFloatingCouponSchedule:eventTime
            eventTimeStep = union(eventTimeStep,swapFloatingCouponSchedule.resetTime);  
            eventTimeStep = union(eventTimeStep,swapFloatingCouponSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
            eventTimeStep = union(eventTimeStep,[0]);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
%             if dealStartEventTime < 0 
%                 dailyTimeSteps = [0:-1:dealStartEventTime];
%                 eventTimeStep = union(eventTimeStep,dailyTimeSteps);
%             end
            
%             % alive Coupon : event has not happend Yet
%             % from dealStartEventTime to 2nd AliveCoupon EventTime
%             % dailyTimeStep should be added to eventTimeStep
%             % for stable theta calculation
%              
%             aliveCouponIdxMax = max(find(structuredCouponSchedule.eventTime > 0));
%             aliveCouponIdxMin = min(find(structuredCouponSchedule.eventTime > 0));
%             
%             aliveCouponNums = aliveCouponIdxMax - aliveCouponIdxMin +1;
%             numOfAliveCouponDaily = min(aliveCouponNums,2);
%             if numOfAliveCouponDaily > 0
%               %% dailyTimeSteps for near couponEventTime, till dealStartEventTime and valueTime
%                 % for fixing handling and theta issue
%                 dailyTimeSteps = [structuredCouponSchedule.eventTime(aliveCouponIdxMin + numOfAliveCouponDaily - 1):-1:dealStartEventTime];
%                 dailyTimeSteps = union(dailyTimeSteps,0);
%                 eventTimeStep = union(eventTimeStep,dailyTimeSteps);
%               %% weeklyTimeSteps for far couponEventTime
%                 weeklyTimeSteps = [eventTimeStep(end):-1*mcOneTimeStep:structuredCouponSchedule.eventTime(aliveCouponIdxMin + numOfAliveCouponDaily - 1)];
%                 eventTimeStep = union(eventTimeStep,weeklyTimeSteps);
%             else
%               %% if no aliveCouponEventTime( if last StructuredCouponEventTime is passed but not paid yet)
%                 % if dealStartEventTime is passed then we add dailyTimeStep
%                 % from valueTime till dealStartEventTime 
%                 % for fixing handling
%                 
%                 if dealStartEventTime < 0 
%                     dailyTimeSteps = [0:-1:dealStartEventTime];
%                     eventTimeStep = union(eventTimeStep,dailyTimeSteps);
%                 end
%                 
%             end
           
%             eventTimeStep = sort(eventTimeStep,'ascend');
            
%           %% debugging purpose 20201013 start
%             passedTimeStep = eventTimeStep(eventTimeStep<0);
%             load('cmsSteepnerTimeStep20201013.mat','cmsSteepnerTimeStep20201013');
%             aliveTimeStep = sort(cmsSteepnerTimeStep20201013,'ascend');
%             eventTimeStep = [];
%             eventTimeStep = union(passedTimeStep,cmsSteepnerTimeStep20201013);
%             eventTimeStep = sort(eventTimeStep,'ascend');
%           
%           %% debugging purpose 20201013 end
            
          %% aliveTimeStep for mc path generation
            aliveTimeStep = eventTimeStep(eventTimeStep>=0);
            aliveTimeStep = sort(aliveTimeStep,'ascend');
            
            totalAliveTimeStepSize = length(aliveTimeStep);
            
           %% eventTimeStep Generation End
           
           %% numeraire info
            numMatTime = eventTimeStep(end);
            dfNumMat = lgm2F.DFRisky(numMatTime/365.0);

            numeraire.numMatTime = numMatTime;
            numeraire.dfNumMat = dfNumMat;
              
            % should we use eventTimeIdx as future only time stes?
            % In this case we better use more generic approach
            % so that all times steps are included 
            
          %% pricing time Step Generation end
           
          %% MC init
            rng('default');
            rng(0);
            
            numOfFactors = lgm2F.numOfFactor;
            NMC = numMethod.pathSize; 
            MCTimeStep = totalAliveTimeStepSize -1; % nowDate (0) should be skipped
%             Z= randn(NMC/2,MCTimeStep);
            Z= randn(NMC/2,numOfFactors*MCTimeStep);
            Z=[Z;-Z];
            Z=Z';

            %TimeInversion
            sizeZ = size(Z,1);
            A = zeros(sizeZ,sizeZ);
            for i=1:sizeZ
                A(i,sizeZ + 1 -i) = 1;
            end
            Z = A*Z;

%                 U = Z';
            U = Z;

           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = zeros(numOfFactors,NMC);
           
           nextX = initX;
            
          %% payoff init
           nominal = steepenerParams.params.nominal;
                      
%            payoffInfo.nominal = nominal;
% 
%            payoffInfo.couponSchedule = couponSchedule;
%            payoffInfo.callableSchedule = callableSchedule;
% %            payoffInfo.refixingSchedule = refixingSchedule;
           
           % EODFlag : if true then pricing as EndOfDate(cashflow on
           % valueDate wil be not included(already paid)
           % if false then pricing as Intraday ( cashflow on valueDate will 
           % be included)
           
           %EODFlag_Cashflow: Cashflow EOD Flag
           %EODFlag_Exercise: Exercise(Option) EOD Flag
           EODFlag_Cashflow = steepenerParams.params.EODFlag_Cashflow;
           EODFlag_Exercise = steepenerParams.params.EODFlag_Exercise;
           
           
           modelStatesSize = NMC;
           
          %% Define Pricing Columns
           % pricingColumns : nonCall,Callable  
           % we need steepenerCoupon as state variables for storing 
           % CMS Steepener Coupon
           
           % for the cashflow generated pricing columns as pricerInfo
           % we do use pricerInfo Structure
           
         %% (global) state variable initialize  : these correspond to ColumnVal[t] in generic pricing deal description
           
          % avg steeper coupon
           steepenerCoupon = zeros(structuredCouponSchedule.scheduleSize,modelStatesSize);
           prefixedCMSSpreadFloatingRate = zeros(structuredCouponSchedule.scheduleSize,modelStatesSize);
           
           swapFloatingCoupon = zeros(swapFloatingCouponSchedule.scheduleSize,modelStatesSize);
           prefixedSwapFloatingRate = zeros(swapFloatingCouponSchedule.scheduleSize,modelStatesSize);
           
           cachedModelStatesCoupon = zeros(structuredCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
  
          % for regression (2 modelState variables)
           cachedModelStatesCall = zeros(callableSchedule.scheduleSize*numOfFactors,modelStatesSize);
           
          % for swapFloatin 
           cachedModelStatesSwapFloating = zeros(swapFloatingCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
           
          %% PricingColumnGenericCashflow
          %% a structure that has cashflows with cashflowNames
          %% for CMS AVG structured we have nonCall, nonCallB, callable
           
            %nonCall
            cashflowNames = {'steepenerCoupon','swapFloatingCoupon'};
            cashflowSizeMap = containers.Map({'steepenerCoupon','swapFloatingCoupon'},{structuredCouponSchedule.scheduleSize, ...
                                            swapFloatingCouponSchedule.scheduleSize});
            
            cashflowInfo.cashflowNames = cashflowNames;
            cashflowInfo.cashflowSizeMap = cashflowSizeMap;
            cashflowInfo.pathSize = modelStatesSize;
            
           nonCall = PricingColumnInfoGenericCashflow(cashflowInfo);
           nonCallB = PricingColumnInfoGenericCashflow(cashflowInfo);
           callable = PricingColumnInfoGenericCashflow(cashflowInfo);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(1,modelStatesSize);
           payoffStateA.cashflow_npv = zeros(1,modelStatesSize);
           
%            % variable to use accruedSumOfStockPrice for moving average calculation 
%            
%            cD = 0; % cumulated day count
%            cP = 0; 
%            dI = 0; % dummy idx for loop
%            dcfC = 0;
%            dcfN = 0;
%            dD = 0;
%       
%            idxL = zeros(1,modelStatesSize);
%            idxS = zeros(1,modelStatesSize);
% 
%            idxC = zeros(1,modelStatesSize);
%            idxN = zeros(1,modelStatesSize);
% 
%            spreadC = zeros(1,modelStatesSize);
%            spreadN = zeros(1,modelStatesSize);
%            probSpread = zeros(1,pathSize);
           
            % we use all path for regression
            % conversion ITM , puttable ITM ,  callable ITM
            % what's the use of Alive Path only Regression Sampling ?
           
          %% Path Generation Forward
          %% Store path dependent variables for coupon calculation 
          %% Store state variables to be used for AMC Regression
           
           %payoff descrption update index
           currentTimeIdx = 1;
           %MC numMethodState update index
           aliveTimeIdx = 1;
           % currentTimeStep is update in for statement
%            for currentTimeStep = eventTimeStep(1):1:eventTimeStep(end)
           for idxT = 1:1:length(eventTimeStep)
                currentTimeStep = eventTimeStep(idxT);
                
                if idxT < length(eventTimeStep)
                    nextTimeStep = eventTimeStep(idxT+1);
                end
                % evaluate scripts 
                % evaluate stateVariables(stores)
                % evaluate expression per each pricing columns
                % at the end of the loop we induct forward
                % from left to right
                % fictional payoff script columns are
                % cumCoupon,nonCall,Callable
                
                        
                % find the current
                % if find return errors then?
                try
                    
                    % cachedModelStateCoupon for modelState
                    idxCouponEvent = find(currentTimeStep == structuredCouponSchedule.eventTime);
                    if ~isempty(idxCouponEvent)
                        cachedModelStatesCoupon(numOfFactors*(idxCouponEvent-1)+1:numOfFactors*idxCouponEvent,:) = nextX;
                    end
                    
                    % cachedModelStateCall for modelState
                    idxCallEvent = find(currentTimeStep == callableSchedule.eventTime);
                    if ~isempty(idxCallEvent)
                        cachedModelStatesCall(numOfFactors*(idxCallEvent-1)+1:numOfFactors*idxCallEvent,:) = nextX;
                    end
                    
                    % cachedModelStateSwapFloatingEvent for modelState
                    idxSwapFloatingEvent = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                    if ~isempty(idxSwapFloatingEvent)
                        cachedModelStatesSwapFloating(numOfFactors*(idxSwapFloatingEvent-1)+1:numOfFactors*idxSwapFloatingEvent,:) = nextX;
                    end
                    
                    % find the corresponding nearest future coupon event
%                   %% structuredCouponSchedule event start
%                     idx = min(find(currentTimeStep <= structuredCouponSchedule.eventTime));
%                     if ~isempty(idx)
%                       %% stateVariable floatingCoupon
%                         idxL = lgm2F.Rate_IndexGeneric(rateType_tenorL,valueDate,currentTimeStep,numMatTime, tenorL,nextX);
%                         idxS = lgm2F.Rate_IndexGeneric(rateType_tenorS,valueDate,currentTimeStep,numMatTime, tenorS,nextX);
%                         
% %                         idxL = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorL,nextX);
% %                         idxS = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorS,nextX);
%                         
%                         dcfN = nextTimeStep;
%                         dcfC = currentTimeStep;
%                         % if coupon accrual condition is met
%                         % period startDate is included but 
%                         % endDate is not included
%                         
%                         if currentTimeStep >= structuredCouponSchedule.startTime(idx) ...
%                            && currentTimeStep < structuredCouponSchedule.eventTime(idx)
%                             
%                             dD = dcfN - dcfC;
%                             cD = cD + dD;
%                             cP = cP + (idxL-idxS)*dD;
%                             
%                         end
%                          
%                         % now if we are at couponSchedule eventTime
%                         % update floatingCoupon
%                         if currentTimeStep == structuredCouponSchedule.eventTime(idx)
% 
%                             steepenerCoupon(idx,:) =  max(min(multiplier * cP/cD + spread,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idx) * nominal;
%                             
%                             % resetting local variables for averaging
%                             cD = 0; % cumulated day count
%                             cP = 0; % cumulated steepener sum
%                             
%                             if idx == structuredCouponSchedule.scheduleSize
%                                 payoffStateA.cashflow = nominal*0.0 + steepenerCoupon(idx,:);
%                              else
%                                 payoffStateA.cashflow = steepenerCoupon(idx,:);
%                             end
%                             
%                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
%                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
% 
%                              payTime = structuredCouponSchedule.payTime(idx);
%                              
%                              %process payoff cashflow 
%                              nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
%                                                      payoffStateA.cashflow,'steepenerCoupon',idx);
%      
%                         end
%                     end
%                     
%                  %% structuredCouponSchedule event end

                 %% structuredCouponSchedule event start
                 %% if steepenerCoupon event happens at RESET Date
                 %% then we fix steepenerCoupon at resetDate and we raise event at resetdate
                 %% if steepenerCoupon event happens at END date then we reset CMSSpreadFloatingRate first
                 %% at ResetDate and we refer this rate at EventDate(EndDate)
                    if strcmp(structuredCouponSchedule.eventType,'RESET')
                     %% now if we are at structuredCouponSchedule eventTime
                     %% structuredCouponSchedule event start

                        idx = find(currentTimeStep == structuredCouponSchedule.eventTime);
                        if ~isempty(idx)

                            idxL = lgm2F.Rate_IndexGeneric(rateType_tenorL,valueDate,currentTimeStep,numMatTime, tenorL,nextX);
                            idxS = lgm2F.Rate_IndexGeneric(rateType_tenorS,valueDate,currentTimeStep,numMatTime, tenorS,nextX);

%                             inverseFloatRate = lgm2F.Rate_IndexGeneric(rateType_tenorInverseFloater,valueDate,currentTimeStep,numMatTime,tenorInverseFloater,nextX);
%                             floatRate = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
%                             inverseFloaterCoupon(idx,:) =  (swapFloating_multiplier * floatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxFloat) * nominal;
                            
                        %% inverseFloaterCoupon
                            steepenerCoupon(idx,:) =  max(min(multiplier * (idxL - idxS) + spread,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idx) * nominal;
%                             steepenerCoupon(idx,:) =  max(min(coupon -  multiplier * inverseFloatRate,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idx) * nominal;
                            
                            payoffStateA.cashflow = steepenerCoupon(idx,:);

%                          %% since we are paying floating CMS Coupon
%                              % we multiply -1 with floating coupon before we
%                              % process payoff
%                             payoffStateA.cashflow = payoffStateA.cashflow * -1.0;

                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
                            payTime = structuredCouponSchedule.payTime(idx);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'steepenerCoupon',idx);

                        end
                      %% structuredCouponSchedule event end
                       
                   elseif strcmp(structuredCouponSchedule.eventType,'END')
                     %% if we raise inverseFloaterCouponEvent at EndDate then
                     %% we need to raise fixing event at resetDate and store fixed inverseFloatRate as stateVariables
                     %% and reference it at ENDDate
                        idxCouponReset = find(currentTimeStep == structuredCouponSchedule.resetTime);
                        if ~isempty(idxCouponReset)
                            
                            idxL = lgm2F.Rate_IndexGeneric(rateType_tenorL,valueDate,currentTimeStep,numMatTime, tenorL,nextX);
                            idxS = lgm2F.Rate_IndexGeneric(rateType_tenorS,valueDate,currentTimeStep,numMatTime, tenorS,nextX);
                            
%                             inverseFloatRate = lgm2F.Rate_IndexGeneric(rateType_tenorInverseFloater,valueDate,currentTimeStep,numMatTime,tenorInverseFloater,nextX);
%                             floatRate = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
                            
                            prefixedCMSSpreadFloatingRate(idxCouponReset,:) = idxL - idxS;
                        end
                        
                        idxEnd = find(currentTimeStep == structuredCouponSchedule.eventTime);
                        if ~isempty(idxEnd)
                            % now if we are at swapFloatingCouponSchedule
                            % eventTime(=endDate)
                            % then we update swapFloatingCoupon
                            prefixedFloatRate = prefixedCMSSpreadFloatingRate(idxEnd,:);
%                             inverseFloaterCoupon(idxEnd,:) =  (swapFloating_multiplier * prefixedFloatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxEnd) * nominal;
                            
                            steepenerCoupon(idxEnd,:) =  max(min(multiplier * (prefixedFloatRate) + spread,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idx) * nominal;
                            
%                             inverseFloaterCoupon(idxEnd,:) =  max(min(coupon -  multiplier * prefixedFloatRate,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idxEnd) * nominal;

                            payoffStateA.cashflow = steepenerCoupon(idxEnd,:);

%                          %% since we are paying floating CMS Coupon
%                              % we multiply -1 with floating coupon before we
%                              % process payoff
%                             payoffStateA.cashflow = payoffStateA.cashflow * -1.0;
                            
                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                            payTime = structuredCouponSchedule.payTime(idxEnd);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'steepenerCoupon',idxEnd);
                            
                        end
                 %% swapFloatingCouponSchedule event end
                   else
                       error('not supported eventType')
                    end
                    
                 %% structuredCouponSchedule event end
                    
                 %% now if we are at swapFloatingCouponSchedule eventTime
                    
                 %% swapFloatingCouponSchedule event start
                 %% if swapFloatingCoupon event happens at RESET Date
                 %% then we fix floatingCoupon at resetDate and we raise event at resetdate
                 %% if swapFloatingCoupon event happens at END date
                 %% then we fix floatingCoupon at resetDate and we raise event at endDate
                 
                   if strcmp(swapFloatingCouponSchedule.eventType,'RESET')
                     %% now if we are at swapFloatingCouponSchedule eventTime
                     %% swapFloatingCouponSchedule event start

                        idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                        if ~isempty(idxFloat)

                            floatRate = lgm2F.Rate_IndexGeneric(rateType_tenorFloat,valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
%                             floatRate = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
                            
                            swapFloatingCoupon(idxFloat,:) =  (swapFloating_multiplier * floatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxFloat) * nominal;

                            payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                         %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                            payoffStateA.cashflow = payoffStateA.cashflow * -1.0;

                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
                            
                            payTime = swapFloatingCouponSchedule.payTime(idxFloat);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                        end
                      %% swapFloatingCouponSchedule event end
                       
                   elseif strcmp(swapFloatingCouponSchedule.eventType,'END')
                     %% if we raise swapFloatingCouponEvent at EndDate then
                     %% we need to raise fixing event at resetDate and store fixed floatRate as stateVariables
                     %% and reference it at ENDDate
                        idxFloatReset = find(currentTimeStep == swapFloatingCouponSchedule.resetTime);
                        if ~isempty(idxFloatReset)
                            
                            floatRate = lgm2F.Rate_IndexGeneric(rateType_tenorFloat,valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
%                             floatRate = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
                            
                            prefixedSwapFloatingRate(idxFloatReset,:) = floatRate;
                        end
                        
                        idxEnd = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                        if ~isempty(idxEnd)
                            % now if we are at swapFloatingCouponSchedule
                            % eventTime(=endDate)
                            % then we update swapFloatingCoupon
                            prefixedFloatRate = prefixedSwapFloatingRate(idxEnd,:);
                            swapFloatingCoupon(idxEnd,:) =  (swapFloating_multiplier * prefixedFloatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxEnd) * nominal;

                            payoffStateA.cashflow = swapFloatingCoupon(idxEnd,:);

                         %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                            payoffStateA.cashflow = payoffStateA.cashflow * -1.0;
                            
                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                            payTime = swapFloatingCouponSchedule.payTime(idxEnd);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'swapFloatingCoupon',idxEnd);
                            
                        end
                 %% swapFloatingCouponSchedule event end
                   else
                       error('not supported eventType')
                   end
                    % if currentTimeSteps is today or future then inductMCForward
                    if currentTimeStep >= 0 && currentTimeStep < eventTimeStep(end)
                        mcmcOut   = inductMCForwardNew(lgm2F,currentTimeStep,nextTimeStep,...
                                    nextX,U(numOfFactors*(aliveTimeIdx-1)+1:numOfFactors*aliveTimeIdx,:));
                        nextX = mcmcOut.nextX;
                        %MC numMethodState update
                        aliveTimeIdx = aliveTimeIdx + 1;
                    end
                    
                    currentTimeIdx = currentTimeIdx +1;
                catch exception
                    throw(exception)
                end
                
                
           end
           
          %% Forward Induction End
          
           % backward pricingColumn
           % nonCallB,
           % callable
           
           
          %% Backward induction start
           for idxT=length(eventTimeStep):-1:1
               
               currentTimeStep = eventTimeStep(idxT);
               try
                  
                  if strcmp(structuredCouponSchedule.eventType,'RESET') 
                    %% if structuredCoupon event happens at resetDate so, in
                    %% backward induction it should be positioned
                    %% before call event

                     %% structuredCouponSchedule event start  
                       idxCoupon = find(currentTimeStep == structuredCouponSchedule.eventTime);
                       % if currentTime  is structuredCouponSchedule.eventTime
                       if ~isempty(idxCoupon)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = structuredCouponSchedule.payTime(idxCoupon);
                             
                             if idxCoupon == structuredCouponSchedule.scheduleSize
                                    payoffStateA.cashflow = nominal*0.0 + steepenerCoupon(idxCoupon,:);
                                 else
                                    payoffStateA.cashflow = steepenerCoupon(idxCoupon,:);
                             end
                                 
%                              payoffStateA.cashflow = steepenerCoupon(idxCoupon,:);

%                              cachedModelStatesSwapFloatingV = cachedModelStatesSwapFloating(numOfFactors*(idxFloat-1)+1:numOfFactors*idxFloat,:);
                             cachedModelStatesCouponV = cachedModelStatesCoupon(numOfFactors*(idxCoupon-1)+1:numOfFactors*idxCoupon,:);
                             
%                           %% since we are paying floating CMS Coupon
%                              % we multiply -1 with floating coupon before we
%                              % process payoff
%                              payoffStateA.cashflow = payoffStateA.cashflow *-1.0;
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,'steepenerCoupon',idxCoupon);

                             callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,'steepenerCoupon',idxCoupon);

                       end

                     %% structuredCouponSchedule event end  
                  end
                  
                  if strcmp(swapFloatingCouponSchedule.eventType,'RESET') 
                    %% if swapFloatingCoupon event happens at resetDate so, in
                    %% backward induction it should be positioned
                    %% before call event

                     %% swapFloatingCoupon event start  
                       idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                       % if currentTime  is floatingCouponSchedule.eventTime
                       if ~isempty(idxFloat)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = swapFloatingCouponSchedule.payTime(idxFloat);
                             payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                             cachedModelStatesSwapFloatingV = cachedModelStatesSwapFloating(numOfFactors*(idxFloat-1)+1:numOfFactors*idxFloat,:);

                          %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                             payoffStateA.cashflow = payoffStateA.cashflow *-1.0;
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                             callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                       end

                     %% swapFloatingCoupon event end  
                  end
                 %% issuer callable event
                 %% AssignExercise Functor : applied only when eventTime >=0
                 %% if EODAdjust is YES : endOfDate conversion option has expired
                 %% if EODAdjust is NO : intraDay, and option is still alive
                   idxCall = find(currentTimeStep == callableSchedule.eventTime);
                   if ~isempty(idxCall)
                       if isExercised 
                           
                            payTime = callableSchedule.payTime(idxCall);
                            
                            % future payment <- is this necessary ?
                            if payTime >= 0
                                
                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);

                               % exercised by notional amount
                               exerciseValue = callPayment*nominal*ones(1,modelStatesSize);
                               
                               % discount exerciseValues from payTime to eventTime
                               df_ep = lgm2F.discountFactorRisky(currentTimeStep, payTime,numMatTime,cachedModelStatesCallV);
                               for k=1:modelStatesSize
                                   exerciseValue(k) = exerciseValue(k) * df_ep(k);
                               end
                               
                               outCallablePayoff = callable.ProcessAssignPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCallV, ...
                                     exerciseValue);
                                
                            end
                       else
                           
                           if currentTimeStep >=0 && EODAdjust(EODFlag_Exercise,currentTimeStep)

                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);

                               exerciseValue = zeros(1,modelStatesSize);
                               exerciseITMCondition = zeros(1,modelStatesSize);

                               for idxEx=1:modelStatesSize
                                   exerciseValue(idxEx) = callPayment*nominal; 
                                   exerciseITMCondition(idxEx) = 1;
                               end

                               payTime = callableSchedule.payTime(idxCall);

                               % discount exerciseValues from payTime to eventTime
                               df_ep = lgm2F.discountFactorRisky(currentTimeStep, payTime,numMatTime,cachedModelStatesCallV);
                               for k=1:modelStatesSize
                                   exerciseValue(k) = exerciseValue(k) * df_ep(k);
                               end

                               % callable is minimizing the payoff
                               minMaxFlag   = 'MIN';
                               decisionVar1 =  cachedModelStatesCallV;
                               outCallablePayoff = AssignExercise(lgm2F,currentTimeStep,callable,exerciseValue,exerciseITMCondition,...
                                          minMaxFlag,decisionVar1,nominal,numeraire,cachedModelStatesCallV);

                           end
                       
                       end
                   end
                   
                   if strcmp(structuredCouponSchedule.eventType,'END')
                   
                     %% coupon event  
                       idxCoupon = find(currentTimeStep == structuredCouponSchedule.eventTime);
                       % if currentTime  is couponSchedule.eventTime
                       if ~isempty(idxCoupon)

                           if currentTimeStep == structuredCouponSchedule.eventTime(idxCoupon)
                                 % we use cached(saved) floating Coupon that
                                 % was generated from Forward Induction
                                 payTime = structuredCouponSchedule.payTime(idxCoupon);

                                 if idxCoupon == structuredCouponSchedule.scheduleSize
                                    payoffStateA.cashflow = nominal*0.0 + steepenerCoupon(idxCoupon,:);
                                 else
                                    payoffStateA.cashflow = steepenerCoupon(idxCoupon,:);
                                 end

                                 eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                                 payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                                 cachedModelStatesCouponV = cachedModelStatesCoupon(numOfFactors*(idxCoupon-1)+1:numOfFactors*idxCoupon,:);

                                 % at coupon EventDate AddFunctor
                                 %process payoff cashflow 
                                 nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                       payoffStateA.cashflow,'steepenerCoupon',idxCoupon);

                                 callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                       payoffStateA.cashflow,'steepenerCoupon',idxCoupon);

                           end
                       end
                       
                   end
                   
                  if strcmp(swapFloatingCouponSchedule.eventType,'END')  
                    %% if swapFloatingCoupon event happens at endDate 
                    %% backward induction it should be positioned
                    %% after call event

                     %% swapFloatingCoupon event start  
                       idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                       % if currentTime  is floatingCouponSchedule.eventTime
                       if ~isempty(idxFloat)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = swapFloatingCouponSchedule.payTime(idxFloat);
                             payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                             cachedModelStatesSwapFloatingV = cachedModelStatesSwapFloating(numOfFactors*(idxFloat-1)+1:numOfFactors*idxFloat,:);

                          %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                             payoffStateA.cashflow = payoffStateA.cashflow *-1.0;
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                             callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                       end

                     %% swapFloatingCoupon event end 
                  end
                 
                   
               catch exception
                    throw(exception)
               end    
           end
           
           
          %% Finalize output
           nonCall.npv = mean(nonCall.payoff);
           nonCallB.npv = mean(nonCallB.payoff);
           callable.npv = mean(callable.payoff);
       
           %forward
           out.nonCall = nonCall; 
           %backward
           out.nonCallB = nonCallB; 
           out.callable = callable;
           
           out.eventTimeStep = eventTimeStep';
           
        end
        
        function out = computeInverseFloaterSwapGeneric2(lgm2F,valueDate,steepenerParams,numMethod)
%           
            % model schedule generation
            dealStartDate = steepenerParams.startDate;
            dealStartEventTime = DateDiff(dealStartDate,valueDate);
            expiryDate = steepenerParams.endDate;
            dealEndEventTime = DateDiff(expiryDate,valueDate);
            %% structuredCouponSchedule start
            
            structuredCouponScheduleInt = steepenerParams.params.structuredCouponInfo.structuredCouponScheduleInt;
            
            % structuredCouponSchedule is a schedule of Structured Coupon
            % events
            
            % main backbone schedule
            structuredCouponScheduleSize = size(structuredCouponScheduleInt,1);
            structuredCouponSchedule= struct;
            structuredCouponSchedule.scheduleSize = structuredCouponScheduleSize;
            structuredCouponSchedule.eventType = steepenerParams.params.structuredCouponInfo.eventType;
            % coupon payment event happens at endDate
            % eventTime is endDate
            % payment date can be different from endDate
            
            % startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:structuredCouponSchedule.scheduleSize
                couponResetDate = H_Date(structuredCouponScheduleInt(i,1));
                couponStartDate = H_Date(structuredCouponScheduleInt(i,2));
                couponEndDate = H_Date(structuredCouponScheduleInt(i,3));
                couponPayDate = H_Date(structuredCouponScheduleInt(i,4));
                
                structuredCouponSchedule.dayCount(i) = couponEndDate.DateDiff(couponStartDate);
                structuredCouponSchedule.dayCountFraction(i) = structuredCouponSchedule.dayCount(i)/365.0; 
                
                couponEndDateP = couponEndDate.AddDate( -1 ,'day');
                
                structuredCouponSchedule.resetTime(i) = DateDiff(couponResetDate,valueDate);
                structuredCouponSchedule.startTime(i) = DateDiff(couponStartDate,valueDate);
                structuredCouponSchedule.endTimeP(i)  = DateDiff(couponEndDateP,valueDate);
                structuredCouponSchedule.endTime(i) = DateDiff(couponEndDate,valueDate);
                
                if strcmp(structuredCouponSchedule.eventType,'RESET')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponResetDate,valueDate);
                elseif strcmp(structuredCouponSchedule.eventType,'START')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponStartDate,valueDate);
                elseif strcmp(structuredCouponSchedule.eventType,'END')
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                else % default is END : event happens at EndDate for StructuredCoupon
                    structuredCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                end
                structuredCouponSchedule.payTime(i) = DateDiff(couponPayDate,valueDate);
                
            end
            
            
            tenorInverseFloater     = steepenerParams.params.structuredCouponInfo.tenorInverseFloater;
            
           %% rateType for structuredCoupon : default CMS
            if isfield(steepenerParams.params.structuredCouponInfo, 'rateType_tenorInverseFloater')
                rateType_tenorInverseFloater = steepenerParams.params.structuredCouponInfo.rateType_tenorInverseFloater;
            else
                rateType_tenorInverseFloater = 'CMS';
            end
            
            
            localFloor = steepenerParams.params.structuredCouponInfo.localFloor;
            localCap   = steepenerParams.params.structuredCouponInfo.localCap;
            multiplier = steepenerParams.params.structuredCouponInfo.multiplier;
            spread     = steepenerParams.params.structuredCouponInfo.spread;
            coupon     = steepenerParams.params.structuredCouponInfo.coupon;
           %% structuredCoupon schedule end
           
           %% call exercise event happens at the callExerciseDate
           %% callable schedule start
            callableScheduleInt = steepenerParams.params.callableInfo.callableScheduleInt;
            callableScheduleSize = size(callableScheduleInt,1);
            callableSchedule= struct;
            for i=1:callableScheduleSize
                callExerciseDate = H_Date(callableScheduleInt(i,1));
                callPayDate = H_Date(callableScheduleInt(i,2));
                callableSchedule.eventTime(i) = DateDiff(callExerciseDate,valueDate);
                callableSchedule.payTime(i) = DateDiff(callPayDate,valueDate);
                
                 callableSchedule.callPayment(i) = callableScheduleInt(i,3);
            end
            callableSchedule.scheduleSize = callableScheduleSize;
           
            isExercised = steepenerParams.params.callableInfo.isExercised;
           %% callable schedule end
           
          %% swapFloatingCouponSchedule start
            
            swapFloatingCouponScheduleInt = steepenerParams.params.swapFloatingCouponInfo.swapFloatingCouponScheduleInt;
            
            swapFloatingCouponSchedule= struct;
            swapFloatingCouponSchedule.scheduleSize = size(swapFloatingCouponScheduleInt,1);
            
            swapFloatingCouponSchedule.eventType = steepenerParams.params.swapFloatingCouponInfo.eventType;
            
            % floating coupon payment event happens at resetDate
            % eventTime is resetDate
            
            % resetTime,startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:swapFloatingCouponSchedule.scheduleSize
                flolatingCouponResetDate = H_Date(swapFloatingCouponScheduleInt(i,1));
                flolatingCouponStartDate = H_Date(swapFloatingCouponScheduleInt(i,2));
                flolatingCouponEndDate = H_Date(swapFloatingCouponScheduleInt(i,3));
                flolatingCouponPayDate = H_Date(swapFloatingCouponScheduleInt(i,4));
                
                swapFloatingCouponSchedule.dayCount(i) = flolatingCouponEndDate.DateDiff(flolatingCouponStartDate);
                swapFloatingCouponSchedule.dayCountFraction(i) = swapFloatingCouponSchedule.dayCount(i)/365.0; 
                
                flolatingCouponEndDateP = flolatingCouponEndDate.AddDate( -1 ,'day');
                
                swapFloatingCouponSchedule.resetTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                swapFloatingCouponSchedule.startTime(i) = DateDiff(flolatingCouponStartDate,valueDate);
                swapFloatingCouponSchedule.endTimeP(i)  = DateDiff(flolatingCouponEndDateP,valueDate);
                swapFloatingCouponSchedule.endTime(i) = DateDiff(flolatingCouponEndDate,valueDate);
                
                if strcmp(swapFloatingCouponSchedule.eventType,'RESET')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                elseif strcmp(swapFloatingCouponSchedule.eventType,'START')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponStartDate,valueDate);
                elseif strcmp(swapFloatingCouponSchedule.eventType,'END')
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponEndDate,valueDate);
                else % default is RESET : for SwapFloatingCouponSchedule
                    swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                end
                
%                 swapFloatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                
                swapFloatingCouponSchedule.payTime(i) = DateDiff(flolatingCouponPayDate,valueDate);
                
            end
            
            swapFloating_tenor  = steepenerParams.params.swapFloatingCouponInfo.tenorFloat;
            
            %% rateType for swapFloatingCoupon : default LIBOR
            if isfield(steepenerParams.params.swapFloatingCouponInfo, 'rateType_tenorFloat')
                rateType_tenorFloat = steepenerParams.params.swapFloatingCouponInfo.rateType_tenorFloat;
            else
                rateType_tenorFloat = 'LIBOR';
            end
            
            swapFloating_multiplier = steepenerParams.params.swapFloatingCouponInfo.multiplier;
            swapFloating_spread     = steepenerParams.params.swapFloatingCouponInfo.spread;
           
          %% SwapFloatingCouponSchedule end
            
            
          %% pricing time Step Generation start
            
            % eventTimeStep Generate
            mcOneTimeStep = numMethod.mcOneTimeStep;
            
          %% eventTime from dealStartTime and dealEndTime
           eventTimeStep = dealStartEventTime;
           eventTimeStep = union(eventTimeStep,dealEndEventTime);
           
          %% eventTime From structuredCouponSchedule:eventTime(endTime) +
          %% startTime,endTimeP
          %% daily weekly timeStep for couponSchedule
          
          %% InverseFloater
          %% eventTime From structuredCouponSchedule:eventTime
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.resetTime);
            eventTimeStep = union(eventTimeStep,structuredCouponSchedule.eventTime);
%             eventTimeStep = union(eventTimeStep, structuredCouponSchedule.startTime);
            eventTimeStep = union(eventTimeStep, structuredCouponSchedule.endTime);
%             eventTimeStep = union(eventTimeStep, structuredCouponSchedule.endTimeP);
           %% eventTime From CallSchedule:eventTime
            eventTimeStep = union(eventTimeStep,callableSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
          %% eventTime From SwapFloatingCouponSchedule:eventTime
            eventTimeStep = union(eventTimeStep,swapFloatingCouponSchedule.resetTime);  
            eventTimeStep = union(eventTimeStep,swapFloatingCouponSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
            
            if dealStartEventTime < 0 
                dailyTimeSteps = [0:-1:dealStartEventTime];
                eventTimeStep = union(eventTimeStep,dailyTimeSteps);
            end
                
%           %% Daily Weekly Time Step Generation for AllTheTime ScheduleType, FromTo Schedule start
%             
%             % alive Coupon : event has not happend Yet
%             % from dealStartEventTime to 2nd AliveCoupon EventTime
%             % dailyTimeStep should be added to eventTimeStep
%             % for stable theta calculation
%              
%             aliveCouponIdxMax = max(find(structuredCouponSchedule.eventTime > 0));
%             aliveCouponIdxMin = min(find(structuredCouponSchedule.eventTime > 0));
%             
%             aliveCouponNums = aliveCouponIdxMax - aliveCouponIdxMin +1;
%             numOfAliveCouponDaily = min(aliveCouponNums,2);
%             if numOfAliveCouponDaily > 0
%               %% dailyTimeSteps for near couponEventTime, till dealStartEventTime and valueTime
%                 % for fixing handling and theta issue
%                 dailyTimeSteps = [structuredCouponSchedule.eventTime(aliveCouponIdxMin + numOfAliveCouponDaily - 1):-1:dealStartEventTime];
%                 dailyTimeSteps = union(dailyTimeSteps,0);
%                 eventTimeStep = union(eventTimeStep,dailyTimeSteps);
%               %% weeklyTimeSteps for far couponEventTime
%                 weeklyTimeSteps = [eventTimeStep(end):-1*mcOneTimeStep:structuredCouponSchedule.eventTime(aliveCouponIdxMin + numOfAliveCouponDaily - 1)];
%                 eventTimeStep = union(eventTimeStep,weeklyTimeSteps);
%             else
%               %% if no aliveCouponEventTime( if last StructuredCouponEventTime is passed but not paid yet)
%                 % if dealStartEventTime is passed then we add dailyTimeStep
%                 % from valueTime till dealStartEventTime 
%                 % for fixing handling
%                 
%                 if dealStartEventTime < 0 
%                     dailyTimeSteps = [0:-1:dealStartEventTime];
%                     eventTimeStep = union(eventTimeStep,dailyTimeSteps);
%                 end
%                 
%             end
           
          %% Daily Weekly Time Step Generation for AllTheTime ScheduleType, FromTo Schedule end
            
            eventTimeStep = sort(eventTimeStep,'ascend');
            
%           %% debugging purpose 20201013 start
%             passedTimeStep = eventTimeStep(eventTimeStep<0);
%             load('cmsSteepnerTimeStep20201013.mat','cmsSteepnerTimeStep20201013');
%             aliveTimeStep = sort(cmsSteepnerTimeStep20201013,'ascend');
%             eventTimeStep = [];
%             eventTimeStep = union(passedTimeStep,cmsSteepnerTimeStep20201013);
%             eventTimeStep = sort(eventTimeStep,'ascend');
%           
%           %% debugging purpose 20201013 end
            
          %% aliveTimeStep for mc path generation
            aliveTimeStep = eventTimeStep(eventTimeStep>=0);
            aliveTimeStep = sort(aliveTimeStep,'ascend');
            
            totalAliveTimeStepSize = length(aliveTimeStep);
            
           %% eventTimeStep Generation End
           
           %% numeraire info
            numMatTime = eventTimeStep(end);
            dfNumMat = lgm2F.DFRisky(numMatTime/365.0);

            numeraire.numMatTime = numMatTime;
            numeraire.dfNumMat = dfNumMat;
              
            % should we use eventTimeIdx as future only time stes?
            % In this case we better use more generic approach
            % so that all times steps are included 
            
          %% pricing time Step Generation end
           
          %% MC init
            rng('default');
            rng(0);
            
            numOfFactors = lgm2F.numOfFactor;
            NMC = numMethod.pathSize; 
            MCTimeStep = totalAliveTimeStepSize -1; % nowDate (0) should be skipped
%             Z= randn(NMC/2,MCTimeStep);
            Z= randn(NMC/2,numOfFactors*MCTimeStep);
            Z=[Z;-Z];
            Z=Z';

            %TimeInversion
            sizeZ = size(Z,1);
            A = zeros(sizeZ,sizeZ);
            for i=1:sizeZ
                A(i,sizeZ + 1 -i) = 1;
            end
            Z = A*Z;

%                 U = Z';
            U = Z;

           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = zeros(numOfFactors,NMC);
           
           nextX = initX;
            
          %% payoff init
           nominal = steepenerParams.params.nominal;
                      
%            payoffInfo.nominal = nominal;
% 
%            payoffInfo.couponSchedule = couponSchedule;
%            payoffInfo.callableSchedule = callableSchedule;
% %            payoffInfo.refixingSchedule = refixingSchedule;
           
           % EODFlag : if true then pricing as EndOfDate(cashflow on
           % valueDate wil be not included(already paid)
           % if false then pricing as Intraday ( cashflow on valueDate will 
           % be included)
           
           %EODFlag_Cashflow: Cashflow EOD Flag
           %EODFlag_Exercise: Exercise(Option) EOD Flag
           EODFlag_Cashflow = steepenerParams.params.EODFlag_Cashflow;
           EODFlag_Exercise = steepenerParams.params.EODFlag_Exercise;
           
           
           modelStatesSize = NMC;
           
          %% Define Pricing Columns
           % pricingColumns : nonCall,Callable  
           % we need steepenerCoupon as state variables for storing 
           % CMS Steepener Coupon
           
           % for the cashflow generated pricing columns as pricerInfo
           % we do use pricerInfo Structure
           
         %% (global) state variable initialize  : these correspond to ColumnVal[t] in generic pricing deal description
           
          % avg steeper coupon
%            steepenerCoupon = zeros(structuredCouponSchedule.scheduleSize,modelStatesSize);
           inverseFloaterCoupon = zeros(structuredCouponSchedule.scheduleSize,modelStatesSize);
           prefixedInverseFloatRate = zeros(structuredCouponSchedule.scheduleSize,modelStatesSize);
           
           swapFloatingCoupon = zeros(swapFloatingCouponSchedule.scheduleSize,modelStatesSize);
           prefixedSwapFloatingRate = zeros(swapFloatingCouponSchedule.scheduleSize,modelStatesSize);
           
           cachedModelStatesCoupon = zeros(structuredCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
  
          % for regression (2 modelState variables)
           cachedModelStatesCall = zeros(callableSchedule.scheduleSize*numOfFactors,modelStatesSize);
           
          % for swapFloatin 
           cachedModelStatesSwapFloating = zeros(swapFloatingCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
           
          %% PricingColumnGenericCashflow
          %% a structure that has cashflows with cashflowNames
          %% for CMS AVG structured we have nonCall, nonCallB, callable
           
            %nonCall
            cashflowNames = {'inverseFloaterCoupon','swapFloatingCoupon'};
            cashflowSizeMap = containers.Map({'inverseFloaterCoupon','swapFloatingCoupon'},{structuredCouponSchedule.scheduleSize, ...
                                            swapFloatingCouponSchedule.scheduleSize});
            
            cashflowInfo.cashflowNames = cashflowNames;
            cashflowInfo.cashflowSizeMap = cashflowSizeMap;
            cashflowInfo.pathSize = modelStatesSize;
            
           nonCall = PricingColumnInfoGenericCashflow(cashflowInfo);
           nonCallB = PricingColumnInfoGenericCashflow(cashflowInfo);
           callable = PricingColumnInfoGenericCashflow(cashflowInfo);
           
           
           % intermediate variables
           payoffStateA.cashflow = zeros(1,modelStatesSize);
           payoffStateA.cashflow_npv = zeros(1,modelStatesSize);
           
           % we don't need local variable for inverse floater
           
%            % variable to use accruedSumOfStockPrice for moving average calculation 
%            
%            cD = 0; % cumulated day count
%            cP = 0; 
%            dI = 0; % dummy idx for loop
%            dcfC = 0;
%            dcfN = 0;
%            dD = 0;
%       
%            idxL = zeros(1,modelStatesSize);
%            idxS = zeros(1,modelStatesSize);
% 
%            idxC = zeros(1,modelStatesSize);
%            idxN = zeros(1,modelStatesSize);
% 
%            spreadC = zeros(1,modelStatesSize);
%            spreadN = zeros(1,modelStatesSize);
%            probSpread = zeros(1,pathSize);
           
            % we use all path for regression
            % conversion ITM , puttable ITM ,  callable ITM
            % what's the use of Alive Path only Regression Sampling ?
           
          %% Path Generation Forward
          %% Store path dependent variables for coupon calculation 
          %% Store state variables to be used for AMC Regression
           
           %payoff descrption update index
           currentTimeIdx = 1;
           %MC numMethodState update index
           aliveTimeIdx = 1;
           % currentTimeStep is update in for statement
%            for currentTimeStep = eventTimeStep(1):1:eventTimeStep(end)
           for idxT = 1:1:length(eventTimeStep)
                currentTimeStep = eventTimeStep(idxT);
                
                if idxT < length(eventTimeStep)
                    nextTimeStep = eventTimeStep(idxT+1);
                end
                % evaluate scripts 
                % evaluate stateVariables(stores)
                % evaluate expression per each pricing columns
                % at the end of the loop we induct forward
                % from left to right
                % fictional payoff script columns are
                % cumCoupon,nonCall,Callable
                
                        
                % find the current
                % if find return errors then?
                try
                    
                    % cachedModelStateCoupon for modelState
                    idxCouponEvent = find(currentTimeStep == structuredCouponSchedule.eventTime);
                    if ~isempty(idxCouponEvent)
                        cachedModelStatesCoupon(numOfFactors*(idxCouponEvent-1)+1:numOfFactors*idxCouponEvent,:) = nextX;
                    end
                    
                    % cachedModelStateCall for modelState
                    idxCallEvent = find(currentTimeStep == callableSchedule.eventTime);
                    if ~isempty(idxCallEvent)
                        cachedModelStatesCall(numOfFactors*(idxCallEvent-1)+1:numOfFactors*idxCallEvent,:) = nextX;
                    end
                    
                    % cachedModelStateSwapFloatingEvent for modelState
                    idxSwapFloatingEvent = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                    if ~isempty(idxSwapFloatingEvent)
                        cachedModelStatesSwapFloating(numOfFactors*(idxSwapFloatingEvent-1)+1:numOfFactors*idxSwapFloatingEvent,:) = nextX;
                    end
                    
%                     % find the corresponding nearest future coupon event
%                   %% structuredCouponSchedule event start
%                     idx = min(find(currentTimeStep <= structuredCouponSchedule.eventTime));
%                     if ~isempty(idx)
%                       %% stateVariable floatingCoupon
%                         idxL = lgm2F.Rate_IndexGeneric(rateType_tenorL,valueDate,currentTimeStep,numMatTime, tenorL,nextX);
%                         idxS = lgm2F.Rate_IndexGeneric(rateType_tenorS,valueDate,currentTimeStep,numMatTime, tenorS,nextX);
%                         
% %                         idxL = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorL,nextX);
% %                         idxS = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorS,nextX);
%                         
%                         dcfN = nextTimeStep;
%                         dcfC = currentTimeStep;
%                         % if coupon accrual condition is met
%                         % period startDate is included but 
%                         % endDate is not included
%                         
%                         if currentTimeStep >= structuredCouponSchedule.startTime(idx) ...
%                            && currentTimeStep < structuredCouponSchedule.eventTime(idx)
%                             
%                             dD = dcfN - dcfC;
%                             cD = cD + dD;
%                             cP = cP + (idxL-idxS)*dD;
%                             
%                         end
%                          
%                         % now if we are at couponSchedule eventTime
%                         % update floatingCoupon
%                         if currentTimeStep == structuredCouponSchedule.eventTime(idx)
% 
%                             steepenerCoupon(idx,:) =  max(min(multiplier * cP/cD + spread,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idx) * nominal;
%                             
%                             % resetting local variables for averaging
%                             cD = 0; % cumulated day count
%                             cP = 0; % cumulated steepener sum
%                             
%                             if idx == structuredCouponSchedule.scheduleSize
%                                 payoffStateA.cashflow = nominal*0.0 + steepenerCoupon(idx,:);
%                              else
%                                 payoffStateA.cashflow = steepenerCoupon(idx,:);
%                             end
%                             
%                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
%                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
% 
%                              payTime = structuredCouponSchedule.payTime(idx);
%                              
%                              %process payoff cashflow 
%                              nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
%                                                      payoffStateA.cashflow,'steepenerCoupon',idx);
%      
%                         end
%                     end
                    
                 %% structuredCouponSchedule event end
                 
                 %% now if we are at structuredCouponSchedule eventTime
                   
                 %% structuredCouponSchedule event start
                 %% if inverseFloaterCoupon event happens at RESET Date
                 %% then we fix floatingCoupon at resetDate and we raise event at resetdate
                 %% if inverseFloaterCoupon event happens at END date then we reset inverseFloatRate first
                 %% at ResetDate and we refer this rate at EventDate(EndDate)
                    if strcmp(structuredCouponSchedule.eventType,'RESET')
                     %% now if we are at structuredCouponSchedule eventTime
                     %% structuredCouponSchedule event start

                        idx = find(currentTimeStep == structuredCouponSchedule.eventTime);
                        if ~isempty(idx)

                            inverseFloatRate = lgm2F.Rate_IndexGeneric(rateType_tenorInverseFloater,valueDate,currentTimeStep,numMatTime,tenorInverseFloater,nextX);
%                             floatRate = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
%                             inverseFloaterCoupon(idx,:) =  (swapFloating_multiplier * floatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxFloat) * nominal;
                            
                        %% inverseFloaterCoupon
                            
                            inverseFloaterCoupon(idx,:) =  max(min(coupon -  multiplier * inverseFloatRate,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idx) * nominal;
                            
                            payoffStateA.cashflow = inverseFloaterCoupon(idx,:);

%                          %% since we are paying floating CMS Coupon
%                              % we multiply -1 with floating coupon before we
%                              % process payoff
%                             payoffStateA.cashflow = payoffStateA.cashflow * -1.0;

                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
                            payTime = structuredCouponSchedule.payTime(idx);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'inverseFloaterCoupon',idx);

                        end
                      %% structuredCouponSchedule event end
                       
                   elseif strcmp(structuredCouponSchedule.eventType,'END')
                     %% if we raise inverseFloaterCouponEvent at EndDate then
                     %% we need to raise fixing event at resetDate and store fixed inverseFloatRate as stateVariables
                     %% and reference it at ENDDate
                        idxInverseFloatReset = find(currentTimeStep == structuredCouponSchedule.resetTime);
                        if ~isempty(idxInverseFloatReset)
                            
                            inverseFloatRate = lgm2F.Rate_IndexGeneric(rateType_tenorInverseFloater,valueDate,currentTimeStep,numMatTime,tenorInverseFloater,nextX);
%                             floatRate = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
                            
                            prefixedInverseFloatRate(idxInverseFloatReset,:) = inverseFloatRate;
                        end
                        
                        idxEnd = find(currentTimeStep == structuredCouponSchedule.eventTime);
                        if ~isempty(idxEnd)
                            % now if we are at swapFloatingCouponSchedule
                            % eventTime(=endDate)
                            % then we update swapFloatingCoupon
                            prefixedFloatRate = prefixedInverseFloatRate(idxEnd,:);
%                             inverseFloaterCoupon(idxEnd,:) =  (swapFloating_multiplier * prefixedFloatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxEnd) * nominal;
                            inverseFloaterCoupon(idxEnd,:) =  max(min(coupon -  multiplier * prefixedFloatRate,localCap),localFloor) * structuredCouponSchedule.dayCountFraction(idxEnd) * nominal;

                            payoffStateA.cashflow = inverseFloaterCoupon(idxEnd,:);

%                          %% since we are paying floating CMS Coupon
%                              % we multiply -1 with floating coupon before we
%                              % process payoff
%                             payoffStateA.cashflow = payoffStateA.cashflow * -1.0;
                            
                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                            payTime = structuredCouponSchedule.payTime(idxEnd);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'inverseFloaterCoupon',idxEnd);
                            
                        end
                 %% swapFloatingCouponSchedule event end
                   else
                       error('not supported eventType')
                    end
                    
                 %% structuredCouponSchedule event end
                    
                 %% now if we are at swapFloatingCouponSchedule eventTime
                    
                 %% swapFloatingCouponSchedule event start
                 %% if swapFloatingCoupon event happens at RESET Date
                 %% then we fix floatingCoupon at resetDate and we raise event at resetdate
                 %% if swapFloatingCoupon event happens at END date
                 %% then we fix floatingCoupon at resetDate and we raise event at endDate
                 
                   if strcmp(swapFloatingCouponSchedule.eventType,'RESET')
                     %% now if we are at swapFloatingCouponSchedule eventTime
                     %% swapFloatingCouponSchedule event start

                        idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                        if ~isempty(idxFloat)

                            floatRate = lgm2F.Rate_IndexGeneric(rateType_tenorFloat,valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
%                             floatRate = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
                            
                            swapFloatingCoupon(idxFloat,:) =  (swapFloating_multiplier * floatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxFloat) * nominal;

                            payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                         %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                            payoffStateA.cashflow = payoffStateA.cashflow * -1.0;

                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;
                            
                            payTime = swapFloatingCouponSchedule.payTime(idxFloat);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                        end
                      %% swapFloatingCouponSchedule event end
                       
                   elseif strcmp(swapFloatingCouponSchedule.eventType,'END')
                     %% if we raise swapFloatingCouponEvent at EndDate then
                     %% we need to raise fixing event at resetDate and store fixed floatRate as stateVariables
                     %% and reference it at ENDDate
                        idxFloatReset = find(currentTimeStep == swapFloatingCouponSchedule.resetTime);
                        if ~isempty(idxFloatReset)
                            
                            floatRate = lgm2F.Rate_IndexGeneric(rateType_tenorFloat,valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
%                             floatRate = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime,swapFloating_tenor,nextX);
                            
                            prefixedSwapFloatingRate(idxFloatReset,:) = floatRate;
                        end
                        
                        idxEnd = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                        if ~isempty(idxEnd)
                            % now if we are at swapFloatingCouponSchedule
                            % eventTime(=endDate)
                            % then we update swapFloatingCoupon
                            prefixedFloatRate = prefixedSwapFloatingRate(idxEnd,:);
                            swapFloatingCoupon(idxEnd,:) =  (swapFloating_multiplier * prefixedFloatRate + swapFloating_spread) * swapFloatingCouponSchedule.dayCountFraction(idxEnd) * nominal;

                            payoffStateA.cashflow = swapFloatingCoupon(idxEnd,:);

                         %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                            payoffStateA.cashflow = payoffStateA.cashflow * -1.0;
                            
                            eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                            payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                            payTime = swapFloatingCouponSchedule.payTime(idxEnd);

                            %process payoff cashflow 
                            nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                    payoffStateA.cashflow,'swapFloatingCoupon',idxEnd);
                            
                        end
                 %% swapFloatingCouponSchedule event end
                   else
                       error('not supported eventType')
                   end
                    % if currentTimeSteps is today or future then inductMCForward
                    if currentTimeStep >= 0 && currentTimeStep < eventTimeStep(end)
                        mcmcOut   = inductMCForwardNew(lgm2F,currentTimeStep,nextTimeStep,...
                                    nextX,U(numOfFactors*(aliveTimeIdx-1)+1:numOfFactors*aliveTimeIdx,:));
                        nextX = mcmcOut.nextX;
                        %MC numMethodState update
                        aliveTimeIdx = aliveTimeIdx + 1;
                    end
                    
                    currentTimeIdx = currentTimeIdx +1;
                catch exception
                    throw(exception)
                end
                
                
           end
           
          %% Forward Induction End
          
           % backward pricingColumn
           % nonCallB,
           % callable
           
           
          %% Backward induction start
           for idxT=length(eventTimeStep):-1:1
               
               currentTimeStep = eventTimeStep(idxT);
               try
                  if strcmp(structuredCouponSchedule.eventType,'RESET') 
                    %% if swapFloatingCoupon event happens at resetDate so, in
                    %% backward induction it should be positioned
                    %% before call event

                     %% structuredCouponSchedule event start  
                       idxCoupon = find(currentTimeStep == structuredCouponSchedule.eventTime);
                       % if currentTime  is structuredCouponSchedule.eventTime
                       if ~isempty(idxCoupon)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = structuredCouponSchedule.payTime(idxCoupon);
                             payoffStateA.cashflow = inverseFloaterCoupon(idxCoupon,:);

%                              cachedModelStatesSwapFloatingV = cachedModelStatesSwapFloating(numOfFactors*(idxFloat-1)+1:numOfFactors*idxFloat,:);
                             cachedModelStatesCouponV = cachedModelStatesCoupon(numOfFactors*(idxCoupon-1)+1:numOfFactors*idxCoupon,:);
                             
%                           %% since we are paying floating CMS Coupon
%                              % we multiply -1 with floating coupon before we
%                              % process payoff
%                              payoffStateA.cashflow = payoffStateA.cashflow *-1.0;
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,'inverseFloaterCoupon',idxCoupon);

                             callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                   payoffStateA.cashflow,'inverseFloaterCoupon',idxCoupon);

                       end

                     %% structuredCouponSchedule event end  
                  end
                  
                  if strcmp(swapFloatingCouponSchedule.eventType,'RESET') 
                    %% if swapFloatingCoupon event happens at resetDate so, in
                    %% backward induction it should be positioned
                    %% before call event

                     %% swapFloatingCoupon event start  
                       idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                       % if currentTime  is floatingCouponSchedule.eventTime
                       if ~isempty(idxFloat)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = swapFloatingCouponSchedule.payTime(idxFloat);
                             payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                             cachedModelStatesSwapFloatingV = cachedModelStatesSwapFloating(numOfFactors*(idxFloat-1)+1:numOfFactors*idxFloat,:);
                             
                          %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                             payoffStateA.cashflow = payoffStateA.cashflow *-1.0;
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                             callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                       end

                     %% swapFloatingCoupon event end  
                  end
                 %% issuer callable event
                 %% AssignExercise Functor : applied only when eventTime >=0
                 %% if EODAdjust is YES : endOfDate conversion option has expired
                 %% if EODAdjust is NO : intraDay, and option is still alive
                   idxCall = find(currentTimeStep == callableSchedule.eventTime);
                   if ~isempty(idxCall)
                       if isExercised 
                           
                            payTime = callableSchedule.payTime(idxCall);
                            
                            % future payment <- is this necessary ?
                            if payTime >= 0
                                
                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);

                               % exercised by notional amount
                               exerciseValue = callPayment*nominal*ones(1,modelStatesSize);
                               
                               % discount exerciseValues from payTime to eventTime
                               df_ep = lgm2F.discountFactorRisky(currentTimeStep, payTime,numMatTime,cachedModelStatesCallV);
                               for k=1:modelStatesSize
                                   exerciseValue(k) = exerciseValue(k) * df_ep(k);
                               end
                               
                               outCallablePayoff = callable.ProcessAssignPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCallV, ...
                                     exerciseValue);
                                
                            end
                       else
                           
                           if currentTimeStep >=0 && EODAdjust(EODFlag_Exercise,currentTimeStep)

                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);

                               exerciseValue = zeros(1,modelStatesSize);
                               exerciseITMCondition = zeros(1,modelStatesSize);

                               for idxEx=1:modelStatesSize
                                   exerciseValue(idxEx) = callPayment*nominal; 
                                   exerciseITMCondition(idxEx) = 1;
                               end

                               payTime = callableSchedule.payTime(idxCall);

                               % discount exerciseValues from payTime to eventTime
                               df_ep = lgm2F.discountFactorRisky(currentTimeStep, payTime,numMatTime,cachedModelStatesCallV);
                               for k=1:modelStatesSize
                                   exerciseValue(k) = exerciseValue(k) * df_ep(k);
                               end

                               % callable is minimizing the payoff
                               minMaxFlag   = 'MIN';
                               decisionVar1 =  cachedModelStatesCallV;
                               outCallablePayoff = AssignExercise(lgm2F,currentTimeStep,callable,exerciseValue,exerciseITMCondition,...
                                          minMaxFlag,decisionVar1,nominal,numeraire,cachedModelStatesCallV);

                           end
                       
                       end
                   end
                   
                 %% coupon event
                  if strcmp(structuredCouponSchedule.eventType,'END')
                       idxCoupon = find(currentTimeStep == structuredCouponSchedule.eventTime);
                       % if currentTime  is couponSchedule.eventTime
                       if ~isempty(idxCoupon)

                           if currentTimeStep == structuredCouponSchedule.eventTime(idxCoupon)
                                 % we use cached(saved) floating Coupon that
                                 % was generated from Forward Induction
                                 payTime = structuredCouponSchedule.payTime(idxCoupon);

                                 if idxCoupon == structuredCouponSchedule.scheduleSize
                                    payoffStateA.cashflow = nominal*0.0 + inverseFloaterCoupon(idxCoupon,:);
                                 else
                                    payoffStateA.cashflow = inverseFloaterCoupon(idxCoupon,:);
                                 end

                                 eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                                 payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                                 cachedModelStatesCouponV = cachedModelStatesCoupon(numOfFactors*(idxCoupon-1)+1:numOfFactors*idxCoupon,:);

                                 % at coupon EventDate AddFunctor
                                 %process payoff cashflow 
                                 nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                       payoffStateA.cashflow,'inverseFloaterCoupon',idxCoupon);

                                 callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                                       payoffStateA.cashflow,'inverseFloaterCoupon',idxCoupon);

                           end
                       end
                  end
                 
                  if strcmp(swapFloatingCouponSchedule.eventType,'END')  
                    %% if swapFloatingCoupon event happens at endDate 
                    %% backward induction it should be positioned
                    %% after call event

                     %% swapFloatingCoupon event start  
                       idxFloat = find(currentTimeStep == swapFloatingCouponSchedule.eventTime);
                       % if currentTime  is floatingCouponSchedule.eventTime
                       if ~isempty(idxFloat)

                             % we use cached(saved) floatingCoupon that
                             % was generated from Forward Induction
                             payTime = swapFloatingCouponSchedule.payTime(idxFloat);
                             payoffStateA.cashflow = swapFloatingCoupon(idxFloat,:);

                             cachedModelStatesSwapFloatingV = cachedModelStatesSwapFloating(numOfFactors*(idxFloat-1)+1:numOfFactors*idxFloat,:);

                          %% since we are paying floating CMS Coupon
                             % we multiply -1 with floating coupon before we
                             % process payoff
                             payoffStateA.cashflow = payoffStateA.cashflow *-1.0;
                             
                             eodFlag = EODAdjust(EODFlag_Cashflow,currentTimeStep);
                             payoffStateA.cashflow = payoffStateA.cashflow * eodFlag;

                             % at coupon EventDate AddFunctor
                             %process payoff cashflow 
                             nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                             callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesSwapFloatingV, ...
                                                   payoffStateA.cashflow,'swapFloatingCoupon',idxFloat);

                       end

                     %% swapFloatingCoupon event end 
                  end
                 
                   
               catch exception
                    throw(exception)
               end    
           end
           
           
          %% Finalize output
           nonCall.npv = mean(nonCall.payoff);
           nonCallB.npv = mean(nonCallB.payoff);
           callable.npv = mean(callable.payoff);
       
           %forward
           out.nonCall = nonCall; 
           %backward
           out.nonCallB = nonCallB; 
           out.callable = callable;
           
           out.eventTimeStep = eventTimeStep';
           
        end
        
        function out = computeCallableCMSSwap(lgm2F,valueDate,cmsSwapParams,numMethod)
%           
            % model schedule generation
            dealStartDate = cmsSwapParams.startDate;
            dealStartEventTime = DateDiff(dealStartDate,valueDate);
            expiryDate = cmsSwapParams.endDate;
            dealEndEventTime = DateDiff(expiryDate,valueDate);
            %% fixedCouponSchedule start
            
            fixedCouponScheduleInt = cmsSwapParams.params.fixedCouponInfo.fixedCouponScheduleInt;
            
            % schedule is fixedCouponSchedule
            % main backbone schedule
            scheduleSize = size(fixedCouponScheduleInt,1);
            fixedCouponSchedule= struct;
            fixedCouponSchedule.scheduleSize = scheduleSize;
            % coupon payment event happens at endDate
            % eventTime is endDate
            % payment date can be different from endDate
            
            % startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:fixedCouponSchedule.scheduleSize
%                 fixedCouponSchedule.payDate(i) = H_Date(fixedCouponScheduleInt(i,4));
                couponStartDate = H_Date(fixedCouponScheduleInt(i,2));
                couponEndDate = H_Date(fixedCouponScheduleInt(i,3));
                couponPayDate = H_Date(fixedCouponScheduleInt(i,4));
                
                fixedCouponSchedule.dayCount(i) = couponEndDate.DateDiff(couponStartDate);
                fixedCouponSchedule.dayCountFraction(i) = fixedCouponSchedule.dayCount(i)/365.0; 
                
                couponEndDateP = couponEndDate.AddDate( -1 ,'day');
                
                fixedCouponSchedule.startTime(i) = DateDiff(couponStartDate,valueDate);
                fixedCouponSchedule.endTimeP(i)  = DateDiff(couponEndDateP,valueDate);
                fixedCouponSchedule.endTime(i) = DateDiff(couponEndDate,valueDate);
                
                fixedCouponSchedule.eventTime(i) = DateDiff(couponEndDate,valueDate);
                fixedCouponSchedule.payTime(i) = DateDiff(couponPayDate,valueDate);
                
            end
            
            fixedCoupon = cmsSwapParams.params.fixedCouponInfo.fixedCoupon;
%             tenorL = cmsSwapParams.params.couponInfo.tenorL;
%             tenorS = cmsSwapParams.params.couponInfo.tenorS;
%             localFloor = cmsSwapParams.params.couponInfo.localFloor;
%             localCap = cmsSwapParams.params.couponInfo.localCap;
%             multiplier = cmsSwapParams.params.couponInfo.multiplier;
%             spread     = cmsSwapParams.params.couponInfo.spread;
%             expiryDate = fixedCouponSchedule.payDate(scheduleSize);
           
            %% fixedCouponSchedule end
           
            %% floatingCouponSchedule start
            
            floatingCouponScheduleInt = cmsSwapParams.params.floatingCouponInfo.floatingCouponScheduleInt;
            
            floatingCouponSchedule= struct;
            floatingCouponSchedule.scheduleSize = size(floatingCouponScheduleInt,1);
            % floating coupon payment event happens at resetDate
            % eventTime is resetDate
            
            % resetTime,startTime,endTimeP should be included in timeStep Generation
            % with eventTime
            
            for i=1:floatingCouponSchedule.scheduleSize
                flolatingCouponResetDate = H_Date(floatingCouponScheduleInt(i,1));
                flolatingCouponStartDate = H_Date(floatingCouponScheduleInt(i,2));
                flolatingCouponEndDate = H_Date(floatingCouponScheduleInt(i,3));
                flolatingCouponPayDate = H_Date(floatingCouponScheduleInt(i,4));
                
                floatingCouponSchedule.dayCount(i) = flolatingCouponEndDate.DateDiff(flolatingCouponStartDate);
                floatingCouponSchedule.dayCountFraction(i) = floatingCouponSchedule.dayCount(i)/365.0; 
                
                flolatingCouponEndDateP = flolatingCouponEndDate.AddDate( -1 ,'day');
                
                floatingCouponSchedule.resetTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                floatingCouponSchedule.startTime(i) = DateDiff(flolatingCouponStartDate,valueDate);
                floatingCouponSchedule.endTimeP(i)  = DateDiff(flolatingCouponEndDateP,valueDate);
                floatingCouponSchedule.endTime(i) = DateDiff(flolatingCouponEndDate,valueDate);
                
                floatingCouponSchedule.eventTime(i) = DateDiff(flolatingCouponResetDate,valueDate);
                floatingCouponSchedule.payTime(i) = DateDiff(flolatingCouponPayDate,valueDate);
                
            end
            
            tenorFloat  = cmsSwapParams.params.floatingCouponInfo.tenorFloat;
             multiplier = cmsSwapParams.params.floatingCouponInfo.multiplier;
             spread     = cmsSwapParams.params.floatingCouponInfo.spread;
           
            %% floatingCouponSchedule end
            
            %% call exercise event happens the same date of payment
            %% callable schedule start
            callableScheduleInt = cmsSwapParams.params.callableInfo.callableScheduleInt;
            callableScheduleSize = size(callableScheduleInt,1);
            callableSchedule= struct;
            for i=1:callableScheduleSize
                callExerciseDate = H_Date(callableScheduleInt(i,1));
                callPayDate = H_Date(callableScheduleInt(i,2));
                callableSchedule.eventTime(i) = DateDiff(callExerciseDate,valueDate);
                callableSchedule.payTime(i) = DateDiff(callPayDate,valueDate);
                
                 callableSchedule.callPayment(i) = callableScheduleInt(i,3);
            end
            callableSchedule.scheduleSize = callableScheduleSize;
           
            
           %% callable schedule end
            
          %% pricing time Step Generation start
%             % if valueDate is passed expiryDate then price is 0
%             maturity = DateDiff(expiryDate,valueDate);
%             if maturity <=0
%                 out = 0;
%                 return;
%             end
            
            % eventTimeStep Generate
            mcOneTimeStep = numMethod.mcOneTimeStep;
            
          %% eventTime from dealStartTime and dealEndTime
           eventTimeStep = dealStartEventTime;
           eventTimeStep = union(eventTimeStep,dealEndEventTime);
           
          %% eventTime From fixedCouponSchedule:eventTime(endTime) only
            eventTimeStep = union(eventTimeStep,fixedCouponSchedule.eventTime);
          %% eventTime From floatingCouponSchedule:eventTime(resetTime) only
            eventTimeStep = union(eventTimeStep,floatingCouponSchedule.eventTime);
          %% eventTime From CallSchedule:eventTime(endTime) +
            eventTimeStep = union(eventTimeStep,callableSchedule.eventTime);
            eventTimeStep = sort(eventTimeStep,'ascend');
            
            % we do not need FromTo dailyWeekly Schedule generation !
            % one step generation is enough
            
          %%
            % if dealStartEventTime is passed then we add dailyTimeStep
            % from valueTime till dealStartEventTime 
            % for fixing handling
            % future timeStep is generated by mcOneTimeStep    
          
          %% mcTimeSteps for futures
            weeklyTimeSteps = [eventTimeStep(end):-1*mcOneTimeStep:0];
            eventTimeStep = union(eventTimeStep,weeklyTimeSteps);
                
            if dealStartEventTime < 0 
                dailyTimeSteps = [0:-1:dealStartEventTime];
                eventTimeStep = union(eventTimeStep,dailyTimeSteps);
            end
            
                
%             
%             % alive Coupon PayTime
%             % from dealStartEventTime to 2nd AliveCoupon EventTime
%             % dailyTimeStep should be added to eventTimeStep
%             % for stable theta calculation
%             
%             aliveCouponNums = max(find(fixedCouponSchedule.eventTime >=0));
%             aliveCouponNear = min(aliveCouponNums,2);
%             if aliveCouponNear > 0
%               %% dailyTimeSteps for near couponEventTime, till dealStartEventTime and valueTime
%                 % for fixing handling and theta issue
%                 dailyTimeSteps = [fixedCouponSchedule.eventTime(aliveCouponNear):-1:dealStartEventTime];
%                 dailyTimeSteps = union(dailyTimeSteps,0);
%                 eventTimeStep = union(eventTimeStep,dailyTimeSteps);
%               %% weeklyTimeSteps for far couponEventTime
%                 weeklyTimeSteps = [eventTimeStep(end):-1*mcOneTimeStep:fixedCouponSchedule.eventTime(aliveCouponNear)];
%                 eventTimeStep = union(eventTimeStep,weeklyTimeSteps);
%             else
%               %% if no aliveCouponEventTime( if last CouponEventTime is passed but not paid yet)
%                 % if dealStartEventTime is passed then we add dailyTimeStep
%                 % from valueTime till dealStartEventTime 
%                 % for fixing handling
%                 
%                 if dealStartEventTime < 0 
%                     dailyTimeSteps = [0:-1:dealStartEventTime];
%                 end
%                 eventTimeStep = union(eventTimeStep,dailyTimeSteps);
%             end
           
            eventTimeStep = sort(eventTimeStep,'ascend');
            
%           %% debugging purpose 20201013 start
%             passedTimeStep = eventTimeStep(eventTimeStep<0);
%             load('cmsSteepnerTimeStep20201013.mat','cmsSteepnerTimeStep20201013');
%             aliveTimeStep = sort(cmsSteepnerTimeStep20201013,'ascend');
%             eventTimeStep = [];
%             eventTimeStep = union(passedTimeStep,cmsSteepnerTimeStep20201013);
%             eventTimeStep = sort(eventTimeStep,'ascend');
%           
%           %% debugging purpose 20201013 end
            
          %% aliveTimeStep for mc path generation
            aliveTimeStep = eventTimeStep(eventTimeStep>=0);
            aliveTimeStep = sort(aliveTimeStep,'ascend');
            
            totalAliveTimeStepSize = length(aliveTimeStep);
            
           %% eventTimeStep Generation End
           
           %% numeraire info
            numMatTime = eventTimeStep(end);
            dfNumMat = lgm2F.DFRisky(numMatTime/365.0);

            numeraire.numMatTime = numMatTime;
            numeraire.dfNumMat = dfNumMat;
              
            % should we use eventTimeIdx as future only time stes?
            % In this case we better use more generic approach
            % such time all times steps are included 
            % and when payoff description is decoded, time should be
            % checked later
            

          %% pricing time Step Generation end
           
          %% MC init
            rng('default');
            rng(0);
            
            numOfFactors = lgm2F.numOfFactor;
            NMC = numMethod.pathSize; 
            MCTimeStep = totalAliveTimeStepSize -1; % nowDate (0) should be skipped
%             Z= randn(NMC/2,MCTimeStep);
            Z= randn(NMC/2,numOfFactors*MCTimeStep);
            Z=[Z;-Z];
            Z=Z';

            %TimeInversion
            sizeZ = size(Z,1);
            A = zeros(sizeZ,sizeZ);
            for i=1:sizeZ
                A(i,sizeZ + 1 -i) = 1;
            end
            Z = A*Z;

%                 U = Z';
            U = Z;

           currentTime = 0; %NowDate
           currentTimeIdx = 1; % 1st Time Steps
           currentNodeIdx = 1; % no eventDate for nowDate
           lastTimeIdx = currentTimeIdx;
           
           % initial modelState
           initX = zeros(numOfFactors,NMC);
           
           nextX = initX;
            
          %% payoff init
           nominal = cmsSwapParams.params.nominal;
           
           % EODFlag : if YES then pricing as EndOfDate(cashflow on
           % valueDate wil be not included(already paid)
           % if NO then pricing as Intraday ( cashflow on valueDate will 
           % be included)
           
           %EODFlag_Cashflow: Cashflow EOD Flag
           %EODFlag_Exercise: Exercise(Option) EOD Flag
           EODFlag_Cashflow = cmsSwapParams.params.EODFlag_Cashflow;
           EODFlag_Exercise = cmsSwapParams.params.EODFlag_Exercise;
           
           
           modelStatesSize = NMC;
           
          %% Define Pricing Columns
           % pricingColumns : fixedCoupon floatingCoupon nonCall,Callable  
           % we need floatingCoupon as state variables for storing floating
           % CMS Rate Coupon
           
           % for the cashflow generated pricing columns as pricerInfo
           % we do use pricerInfo Structure
           
         %% state variable initialize
           
          %fixedCoupon & floatingCoupon
          fixedCouponVar    = zeros(fixedCouponSchedule.scheduleSize,modelStatesSize);
          floatingCouponVar = zeros(floatingCouponSchedule.scheduleSize,modelStatesSize);
            
          cachedModelStatesFixedCoupon = zeros(fixedCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
  
          cachedModelStatesFloatingCoupon = zeros(floatingCouponSchedule.scheduleSize*numOfFactors,modelStatesSize);
  
          % for regression (2 modelState variables)
           cachedModelStatesCall = zeros(callableSchedule.scheduleSize*numOfFactors,modelStatesSize);
           
          %% PricingColumns that has cashflows
           % we have nonCall, callable
            
            %nonCall
            nonCall = PricingColumnInfoGeneric(fixedCouponSchedule.scheduleSize,modelStatesSize);
            
%             nonCall = PricingColumnInfoGeneric(scheduleSize,modelStatesSize);
            
%             %callable
%             callable = PricingColumnInfoGeneric(scheduleSize,modelStatesSize);
           
           % intermediate variables
           payoffStateA.cashflow = zeros(1,modelStatesSize);
           payoffStateA.cashflow_npv = zeros(1,modelStatesSize);
           
           payoffStateB.cashflow = zeros(1,modelStatesSize);
           payoffStateB.cashflow_npv = zeros(1,modelStatesSize);
           
           payoffStateC.cashflow = zeros(1,modelStatesSize);
           payoffStateC.cashflow_npv = zeros(1,modelStatesSize);
           
           % variable to use accruedSumOfStockPrice for moving average calculation 
           
           cD = 0; % cumulated day count
           cP = 0; 
           dI = 0; % dummy idx for loop
           dcfC = 0;
           dcfN = 0;
           dD = 0;
      
           idxL = zeros(1,modelStatesSize);
           idxS = zeros(1,modelStatesSize);

           idxC = zeros(1,modelStatesSize);
           idxN = zeros(1,modelStatesSize);

           spreadC = zeros(1,modelStatesSize);
           spreadN = zeros(1,modelStatesSize);
%            probSpread = zeros(1,pathSize);
           
           currentPeriodCoupon = zeros(1,modelStatesSize);
           currentPeriodCouponNPV = zeros(1,modelStatesSize);
            % we use all path for regression
            % conversion ITM , puttable ITM ,  callable ITM
            % what's the use of Alive Path only Regression Sampling ?
           
          %% Path Generation Forward
          %% Store path dependent variables for coupon calculation 
          %% Store state variables to be used for AMC Regression
           currentCouponPeriod = 1;
           
           %payoff descrption update index
           currentTimeIdx = 1;
           %MC numMethodState update index
           aliveTimeIdx = 1;
           % currentTimeStep is update in for statement
%            for currentTimeStep = eventTimeStep(1):1:eventTimeStep(end)
           for idxT = 1:1:length(eventTimeStep)
                currentTimeStep = eventTimeStep(idxT);
                
                if idxT < length(eventTimeStep)
                    nextTimeStep = eventTimeStep(idxT+1);
                end
                % evaluate scripts 
                % evaluate stateVariables(stores)
                % evaluate expression per each pricing columns
                % at the end of the loop we induct forward
                % from left to right
                % fictional payoff script columns are
                % (fixedCoupon,floatingCoupon),nonCall,Callable
                
                try
                    
                 %% modelState Caching Start
                    % cachedModelStateFixedCoupon for fixedCoupon Discounting in backward induction
                    idxFixedCouponEvent = find(currentTimeStep == fixedCouponSchedule.eventTime);
                    if ~isempty(idxFixedCouponEvent)
                        cachedModelStatesFixedCoupon(numOfFactors*(idxFixedCouponEvent-1)+1:numOfFactors*idxFixedCouponEvent,:) = nextX;
                    end
                    
                    % cachedModelStateFloatingCoupon for floatingCoupon Calculation modelState
                    idxFloatingCouponEvent = find(currentTimeStep == floatingCouponSchedule.eventTime);
                    if ~isempty(idxFloatingCouponEvent)
                        cachedModelStatesFloatingCoupon(numOfFactors*(idxFloatingCouponEvent-1)+1:numOfFactors*idxFloatingCouponEvent,:) = nextX;
                    end
                    
                    % cachedModelState for modelState
                    idxCallEvent = find(currentTimeStep == callableSchedule.eventTime);
                    if ~isempty(idxCallEvent)
                        cachedModelStatesCall(numOfFactors*(idxCallEvent-1)+1:numOfFactors*idxCallEvent,:) = nextX;
                    end
                    
                 %% modelState Caching End
                    
                 %% now if we are at fixedCouponSchedule eventTime
                    % update fixedCoupon
                    idx = find(currentTimeStep == fixedCouponSchedule.eventTime);
                    if ~isempty(idx)
                        
                        fixedCouponVar(idx,:) =  (fixedCoupon) * fixedCouponSchedule.dayCountFraction(idx) * nominal;

                        payoffStateA.cashflow = fixedCouponVar(idx,:);
                        
                        payTime = fixedCouponSchedule.payTime(idx);

                        %process payoff cashflow 
                        nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                payoffStateA.cashflow,idx);

                    end
                    
                    %% now if we are at floatingCouponSchedule eventTime
                    % update floatingCoupon
                    idx = find(currentTimeStep == floatingCouponSchedule.eventTime);
                    if ~isempty(idx)
                        
                        idxL = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorFloat,nextX);
                        floatingCouponVar(idx,:) =  (multiplier * idxL + spread) * floatingCouponSchedule.dayCountFraction(idx) * nominal;

                        payoffStateA.cashflow = floatingCouponVar(idx,:);
                        
                     %% since we are paying floating CMS Coupon
                         % we multiply -1 with floating coupon before we
                         % process payoff
                        payoffStateA.cashflow = payoffStateA.cashflow * -1.0;
                        
                        payTime = floatingCouponSchedule.payTime(idx);

                        %process payoff cashflow 
                        nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
                                                payoffStateA.cashflow,idx);

                    end
                        
%                     % find the corresponding nearest future floating coupon event
%                     idx = min(find(currentTimeStep <= floatingCouponSchedule.eventTime));
%                     if ~isempty(idx)
%                       %% stateVariable floatingCoupon
%                         idxL = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorFloat,nextX);
% %                         idxS = lgm2F.CMS_PSA_SimpleDateGeneric(valueDate,currentTimeStep,numMatTime, tenorS,nextX);
%                         
% %                         dcfN = nextTimeStep;
% %                         dcfC = currentTimeStep;
% %                         % if coupon accrual condition is met
% %                         % period startDate is included but 
% %                         % endDate is not included
% %                         
% %                         if currentTimeStep >= fixedCouponSchedule.startTime(idx) ...
% %                            && currentTimeStep < fixedCouponSchedule.eventTime(idx)
% %                             
% %                             dD = dcfN - dcfC;
% %                             cD = cD + dD;
% %                             cP = cP + (idxL-idxS)*dD;
% %                             
% %                         end
%                          
%                         % now if we are at fixedCouponSchedule eventTime
%                         % update floatingCoupon
%                         if currentTimeStep == floatingCouponSchedule.eventTime(idx)
% 
%                             floatingCoupon(idx,:) =  max(min(multiplier * cP/cD + spread,localCap),localFloor) * fixedCouponSchedule.dayCountFraction(idx) * nominal;
%                             
%                             % resetting local variables for averaging
%                             cD = 0; % cumulated day count
%                             cP = 0; % cumulated steepener sum
%                             
%                             if idx == fixedCouponSchedule.scheduleSize
%                                 payoffStateA.cashflow = nominal + floatingCoupon(idx,:);
%                              else
%                                 payoffStateA.cashflow = floatingCoupon(idx,:);
%                              end
% 
%                              payTime = fixedCouponSchedule.payTime(idx);
%                              
%                              %process payoff cashflow 
%                              nonCall.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,nextX, ...
%                                                      payoffStateA.cashflow,idx);
%      
%                         end
%                     end

                    % if currentTimeSteps is today or future then inductMCForward
                    if currentTimeStep >= 0 && currentTimeStep < eventTimeStep(end)
                        mcmcOut   = inductMCForwardNew(lgm2F,currentTimeStep,nextTimeStep,...
                                    nextX,U(numOfFactors*(aliveTimeIdx-1)+1:numOfFactors*aliveTimeIdx,:));
                        nextX = mcmcOut.nextX;
                        %MC numMethodState update
                        aliveTimeIdx = aliveTimeIdx + 1;
                    end
                    
                    currentTimeIdx = currentTimeIdx +1;
                catch exception
                    throw(exception)
                end
                
                
           end
           
          %% Forward Induction End
          
           % backward pricingColumn
           % nonCallB,
           % callable
           
           %nonCallB  
           nonCallB = PricingColumnInfoGeneric(scheduleSize,modelStatesSize);
           callable = PricingColumnInfoGeneric(scheduleSize,modelStatesSize);
           
           % variables initialize to be used in backward induction
           cachedModelStatesCouponV= zeros(numOfFactors,modelStatesSize);
           cachedModelStatesCallV = zeros(numOfFactors,modelStatesSize);
           %% Backward induction start
           for idxT=length(eventTimeStep):-1:1
               
               currentTimeStep = eventTimeStep(idxT);
               try 
                 %% issuer callable event
                 %% AssignExercise Functor : applied only when eventTime >=0
                 %% if EODAdjust is YES : endOfDate conversion option has expired
                 %% if EODAdjust is NO : intraDay, and option is still alive
                   idxCall = find(currentTimeStep == callableSchedule.eventTime);
                   if ~isempty(idxCall)
                       if currentTimeStep == callableSchedule.eventTime(idxCall) && currentTimeStep >=0
                           if EODAdjust(EODFlag_Exercise,currentTimeStep)
                               %find the corresponding callPayment;
                               callPayment = callableSchedule.callPayment(idxCall);
                               %look up the corresponding cachedModelStatesCall for
                               %call decision variables
                               cachedModelStatesCallV = cachedModelStatesCall(numOfFactors*(idxCall-1)+1:numOfFactors*idxCall,:);
                        
                               exerciseValue = zeros(1,modelStatesSize);
                               exerciseITMCondition = zeros(1,modelStatesSize);
                               
%                                idxCouponE = find(currentTimeStep <= fixedCouponSchedule.eventTime);
%                                % find the nearest floating coupon
%                                if ~isempty(idxCouponE)
%                                    idxCouponE = min(idxCouponE);
%                                else
%                                    error('wrong schedule')
%                                end

                               for idxEx=1:modelStatesSize
%                                    exerciseValue(idxEx) = callPayment*nominal + floatingCoupon(idxCouponE,idxEx);
                                   exerciseValue(idxEx) = callPayment*nominal;

                                   exerciseITMCondition(idxEx) = 1;
                                   
%                                %% callable condition activated 
%                                    if savedStockPriceV(idxEx) >= callableConditionalStrike
%                                         exerciseITMCondition(idxEx) = 1;
%                                    end
                               end

                               % callable is minimizing the payoff
                               minMaxFlag   = 'MIN';
                               decisionVar1 =  cachedModelStatesCallV;
                               outCallablePayoff = AssignExercise(lgm2F,currentTimeStep,callable,exerciseValue,exerciseITMCondition,...
                                          minMaxFlag,decisionVar1,nominal,numeraire,cachedModelStatesCallV);

                           end
                       end
                       
                   end
                   
                 %% fixed coupon event  
                   idxFixedCoupon = find(currentTimeStep == fixedCouponSchedule.eventTime);
                   % if currentTime  is fixedCouponSchedule.eventTime
                   if ~isempty(idxFixedCoupon)
                       
                         % we use cached(saved) fixedCoupon that
                         % was generated from Forward Induction
                         payTime = fixedCouponSchedule.payTime(idxFixedCoupon);
                         payoffStateA.cashflow = fixedCouponVar(idxFixedCoupon,:);
                         
                         cachedModelStatesCouponV = cachedModelStatesFixedCoupon(numOfFactors*(idxFixedCoupon-1)+1:numOfFactors*idxFixedCoupon,:);

                         % at coupon EventDate AddFunctor
                         %process payoff cashflow 
                         nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                               payoffStateA.cashflow,idxFixedCoupon);

                         callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                               payoffStateA.cashflow,idxFixedCoupon);
                            
                   end
                 
                 %% floating coupon event  
                   idxFloatingdCoupon = find(currentTimeStep == floatingCouponSchedule.eventTime);
                   % if currentTime  is floatingCouponSchedule.eventTime
                   if ~isempty(idxFloatingdCoupon)
                       
                         % we use cached(saved) floatingCoupon that
                         % was generated from Forward Induction
                         payTime = floatingCouponSchedule.payTime(idxFloatingdCoupon);
                         payoffStateA.cashflow = floatingCouponVar(idxFloatingdCoupon,:);
                         
                         cachedModelStatesCouponV = cachedModelStatesFloatingCoupon(numOfFactors*(idxFloatingdCoupon-1)+1:numOfFactors*idxFloatingdCoupon,:);
                         
                      %% since we are paying floating CMS Coupon
                         % we multiply -1 with floating coupon before we
                         % process payoff
                         payoffStateA.cashflow = payoffStateA.cashflow *-1.0;

                         % at coupon EventDate AddFunctor
                         %process payoff cashflow 
                         nonCallB.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                               payoffStateA.cashflow,idxFloatingdCoupon);

                         callable.ProcessAddPayoff(lgm2F,currentTimeStep,payTime,numeraire,cachedModelStatesCouponV, ...
                                               payoffStateA.cashflow,idxFloatingdCoupon);
                            
                   end
                   
                 
                   
               catch exception
                    throw(exception)
               end    
           end
           
           
          %% Finalize output
           nonCall.npv = mean(nonCall.payoff);
           nonCallB.npv = mean(nonCallB.payoff);
           callable.npv = mean(callable.payoff);
       
           %forward
           out.nonCall = nonCall; 
           %backward
           out.nonCallB = nonCallB; 
           out.callable = callable;
           
        end
        
        function atmSwaption = ATMSwaption(lgm2F,tx,tnr)
            tau = 0.25;
            pv01 = lgm2F.PV01(tx,tx+tnr,tau);
            fsr  = lgm2F.Fsr(tx,tx+tnr,tau);
            c_nN = lgm2F.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = lgm2F.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:2
                for j=1:2
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        
        function spreadInfo = SpreadInfo(ck2F,tx,tnr1,tnr2)
            tau = 0.25;
            cmsSpreadDigitalArg =  ck2F.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
       
        function out = CalibrateToATMSwaptionSurface(lgm2F,black,calibrationFlag,numOfIter)
            % Market Data Object , Vanilla Object
            % targetSwaption
            % targetCap
            
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% target instruments set up
            params.swaption_expiry_size = expirySize;
            params.swaption_tenor_size = tenorSize;
            
            params.swaption_calibration_yn = ones(expirySize,tenorSize);
            
            params.swaptionSurface.expiry = swaptionVolSurface.expiry;
            params.swaptionSurface.tenor  = swaptionVolSurface.tenor;
            params.swaptionSurface.quote  = zeros(expirySize,tenorSize);
            params.swaptionSurface.strike = zeros(expirySize,tenorSize);
            params.swaptionSurface.blackPrice = zeros(expirySize,tenorSize);
            
            % setting target diagonal swaption
            for i=1:expirySize
                for j=1:tenorSize
                    if params.swaption_calibration_yn(i,j) == 1
                        Ti = params.swaptionSurface.expiry(i);
                        Tj = params.swaptionSurface.tenor(j);
                        blackVolQuote = swaptionVolSurface.swaptionVol(expiry(i),tenor(j));
                        params.swaptionSurface.quote(i,j) = blackVolQuote;
                        params.swaptionSurface.strike(i,j) = black.Fsr(Ti,Ti + Tj,0.25);
                        params.swaptionSurface.blackPrice(i,j) = black.BlackSwaptionWithVol(Ti,Tj,params.swaptionSurface.strike(i,j),blackVolQuote);
                    end
                end

            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            params.dH1 = lgm2F.dH1;
            params.dH2 = lgm2F.dH2;
            
            dH1_size = length(lgm2F.dH1.quote);
            Alpha1_size = length(lgm2F.Alpha1.quote);
            
            params.dH1_size = dH1_size;
            params.Alpha1_size = Alpha1_size;
            
            params.Alpha1 = lgm2F.Alpha1;
            params.Alpha2 = lgm2F.Alpha2;
            
            lgm2F.dH1    = params.dH1;
            lgm2F.dH2    = params.dH2;
            
            lgm2F.Alpha1 = params.Alpha1;
            lgm2F.Alpha2 = params.Alpha2;
            
            params.dH1Orig = lgm2F.dH1Orig;
            params.dH2Orig = lgm2F.dH2Orig;
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            out.dH1 = params.dH1;
            out.dH2 = params.dH2;
            
            out.Alpha1 = params.Alpha1;
            out.Alpha2 = params.Alpha2;
            
            out.blackSwaptionPrice = params.swaptionSurface.blackPrice;
            out.lgm2FSwaptionPrice = zeros(expirySize,tenorSize);
            out.swaptionPriceRelDiff = zeros(expirySize,tenorSize);
            out.swaptionPriceRmseTotal = 0.0;
           
            out.market_swaption = zeros(expirySize,tenorSize);
            out.model_swaption = zeros(expirySize,tenorSize);
            
            tvar = [];
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
                
            case 'global'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH    
                                              2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    x=  zeros(optParams.numOfEquation,1);
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobal',tvar,x, numOfIter, options,'unc',lgm2F,optParams);
                    toc
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                        lgm2F.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        lgm2F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
                    
            otherwise
                    disp('unImplemented')
            end
        end
        
        
    end
    
end



In [ ]:
classdef LGM2F_Freq < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        Alpha1
        Alpha2
        dH1
        dH2
        correl
        dH1Orig
        dH2Orig
        smoothdH
        smoothAlpha
        smoothTermCorr
        targetTermCorr
        numOfFactor
        tau
    end
    
    methods
        function lgm2F = LGM2F_Freq(params)
            if nargin > 0
                if isKey(params.modelParams,'freq')
                    lgm2F.tau = 1.0/params.modelParams('freq');
                else
                    lgm2F.tau = 0.25; % default
                end
                

                lgm2F.zeroCurve = params.zeroCurve;
                lgm2F.Alpha1 = params.modelParams('Alpha1');
                lgm2F.Alpha2 = params.modelParams('Alpha2');
                lgm2F.dH1 =  params.modelParams('dH1');
                lgm2F.dH2 =  params.modelParams('dH2');
                
                lgm2F.dH1Orig =  params.modelParams('dH1Orig');
                lgm2F.dH2Orig =  params.modelParams('dH2Orig');
                lgm2F.correl =  params.modelParams('correl');
                lgm2F.smoothdH =  params.modelParams('smoothdH');
                lgm2F.smoothAlpha =  params.modelParams('smoothAlpha');
                lgm2F.smoothTermCorr = params.modelParams('smoothTermCorr');
                lgm2F.targetTermCorr = params.modelParams('targetTermCorr');
                
                lgm2F.numOfFactor = 2;
            end
        end
        
        function out = SimpleSRPrOverHedge(lgm2F,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(lgm2F,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(lgm2F,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(lgm2F,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
             elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = lgm2F.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out = SimpleDRPr(lgm2F,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS)
            outVec = 0.0;
            if cmsC >= ralower && cmsC <= raupper && spreadC >= ralowerS && spreadC <= raupperS
                outVec = 1.0;
            end
            out = outVec;
        end
        
        function out = DRPr(lgm2F,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
%                     outVec(i) = lgm2F.SimpleDRPrOverHedge(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            else
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            end
            
            out = outVec;
        end
        
        function out =  Libor_SimpleDate(lgm2F,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           %CMS
           fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
           fwdEndDate = fwdStartDate.AddDate(tenor,'year');
           fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
           deltaT = (fwdEndTime - fwdStartTime)/365.0;
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           
           stateSize = size(modelStatesT,2);
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           df_Tn = lgm2F.discountFactor(evalTime,fwdStartTime,numMatTime,modelStatesT);
           df_TN = lgm2F.discountFactor(evalTime,fwdEndTime,numMatTime,modelStatesT);
           
           liborOut = zeros(1,stateSize);
           
           for i=1:stateSize
                liborOut(i) = (df_Tn(i)/df_TN(i) - 1.0)/deltaT;
           end
           
           out = liborOut;
            
        end
        
        function out =  CMS_PSA_SimpleDate(lgm2F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
%            tau = 0.25;
           tau = lgm2F.tau;
           
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           if tenor < 1
               tenorMonth = round(tenor*12);
               tNDate = toDate.AddDate(tenorMonth,'month');
           else
               tNDate = toDate.AddDate(tenor,'year');
           end
           
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = lgm2F.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = lgm2F.DF(to);
           df_TN = lgm2F.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = lgm2F.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = lgm2F.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = lgm2F.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = lgm2F.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:2
               for j=1:2
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + c_nN(1)*modelStatesT(1,i) + c_nN(2) * modelStatesT(2,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
        end
        
        function out = HtT_r(lgm2F,t,T)
            out =  lgm2F.H(T,lgm2F.dH1) - lgm2F.H(t,lgm2F.dH1); 
        end
        
        function out = HtT_s(lgm2F,t,T)
            out =  lgm2F.H(T,lgm2F.dH2) - lgm2F.H(t,lgm2F.dH2); 
        end
        
        function out = fwdZcVariance(lgm2F, a, b, T1, T2)
            varMatrix = lgm2F.LocalVariance(a,b);
            
            localVariance = zeros(4,1);
            measureChange = zeros(4,1);
            
            localVariance(1) = (lgm2F.HtT_r(a,T1)*lgm2F.HtT_r(a,T1)-lgm2F.HtT_r(a,b)*lgm2F.HtT_r(a,b))*varMatrix(1,1);
            localVariance(2) = (lgm2F.HtT_r(a,T1)*lgm2F.HtT_s(a,T1)-lgm2F.HtT_r(a,b)*lgm2F.HtT_s(a,b))*varMatrix(1,2);
            localVariance(3) = (lgm2F.HtT_s(a,T1)*lgm2F.HtT_r(a,T1)-lgm2F.HtT_s(a,b)*lgm2F.HtT_r(a,b))*varMatrix(2,1);
            localVariance(4) = (lgm2F.HtT_s(a,T1)*lgm2F.HtT_s(a,T1)-lgm2F.HtT_s(a,b)*lgm2F.HtT_s(a,b))*varMatrix(2,2);
            
            measureChange(1) = (lgm2F.HtT_r(a,T1)-lgm2F.HtT_r(a,b))*lgm2F.HtT_r(a,T2)*varMatrix(1,1);
            measureChange(2) = (lgm2F.HtT_r(a,T1)-lgm2F.HtT_r(a,b))*lgm2F.HtT_s(a,T2)*varMatrix(1,2);
            measureChange(3) = (lgm2F.HtT_s(a,T1)-lgm2F.HtT_s(a,b))*lgm2F.HtT_r(a,T2)*varMatrix(2,1);
            measureChange(4) = (lgm2F.HtT_s(a,T1)-lgm2F.HtT_s(a,b))*lgm2F.HtT_s(a,T2)*varMatrix(2,2);
            
            value = localVariance(1) + localVariance(2);
            value = value + localVariance(3) + localVariance(4);
            
            measureChangeTerm = measureChange(1) + measureChange(2);
            measureChangeTerm = measureChangeTerm + measureChange(3) + measureChange(4);
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
        end
        
        function out = discountFactor(lgm2F, startTime, endTime,numMatTime,modelStatesT)
           zcT = lgm2F.DF(endTime/365.0);
           
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           % deterministic discount factor
           if startTime <= 1e-8
                for i=1:stateSize
                    discountFactorOut(i) = zcT;
                end
                out = discountFactorOut;
                return;
           end
           
           % deterministic discount factor or maturity in the past
           if startTime >= endTime
                out = ones(1,stateSize);
                return;
           end
           
           zct = lgm2F.DF(startTime/365.0);
           beta = lgm2F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = lgm2F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i) ...
                   - beta(2) * modelStatesT(2,i)); 
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoff(lgm2F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = lgm2F.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = lgm2F.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(lgm2F,fromTime, toTime)
            
            out = eye(2,2);
        end
        
        function out = stateLocalVariance(lgm2F,fromTime, toTime)
            out = lgm2F.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(lgm2F,from,to)

            sig(1) = lgm2F.Alpha1;
            sig(2) = lgm2F.Alpha2;
            corr_rs = lgm2F.correl;
            corr(1,1) = 1.0;
            corr(2,1) = corr_rs.quote(1);
            corr(1,2) = corr(2,1);
            corr(2,2) = 1.0;
            
            alphaSize = length(lgm2F.Alpha1.quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           variances = zeros(2,2);
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end;
                correl(1,1) =1.0;
                correl(2,2) =1.0;
                if (i< alphaSize) correl(2,1) = corr_rs.quote(i);else correl(2,1) = corr_rs.quote(alphaSize);end;
                correl(1,2) = correl(2,1);
                for j=1:2
                    for k=1:2
                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end
        
        function out = H(lgm2F,t,dH)
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    break;
                end
            end
            tidx = i;
            sum = 0.0;
            prev_t = 0.0;
            curr_t = 0.0;
            for i=1:tidx
                if i< tidx
                    curr_t = dH.tenor(i);
                else
                    curr_t = t;
                end
                sum = sum + dH.quote(i)*(curr_t-prev_t);

                prev_t = dH.tenor(i);
            end
            out = sum;
        end
        
        function out = Alpha_Test(lgm2F,t,Alpha)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(Alpha.tenor)
                alpha = Alpha.quote(i);
                if t<= Alpha.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end
        
        function out = dH_Test(lgm2F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = dH.quote(i);
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = dH.quote(i);
            end
            out = sum;
        end

        function out = H_Test(lgm2F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = sum + dH.quote(i)*(t-prev_t);
                    inRangeYN = true;
                    break;
                else
                    sum = sum + dH.quote(i)*(dH.tenor(i)-prev_t);
                end
                prev_t = dH.tenor(i);
            end
            
            if ~inRangeYN 
                sum = sum + dH.quote(i) * (t - dH.tenor(i));
            end
            out = sum;
        end
        
        function c_Test = C_Test(lgm2F,tx, tnr,tau)
            pv01_1 = lgm2F.PV01(tx,tx+tnr,tau);
            fsr_1  = lgm2F.Fsr(tx,tx+tnr,tau);
            c_nN_1 = lgm2F.C_nN(tx,tx+tnr,tau,pv01_1,fsr_1);
            c_Test = c_nN_1;
        end

        function beta = Beta(lgm2F,t,T)
            beta =[];
            
            beta(1) = lgm2F.H(T,lgm2F.dH1)-lgm2F.H(t,lgm2F.dH1);
            beta(2) = lgm2F.H(T,lgm2F.dH2)-lgm2F.H(t,lgm2F.dH2);
            
        end
        
        function beta = BetaDate(lgm2F,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            beta =[];
            
            beta(1) = lgm2F.H(T,lgm2F.dH1)-lgm2F.H(t,lgm2F.dH1);
            beta(2) = lgm2F.H(T,lgm2F.dH2)-lgm2F.H(t,lgm2F.dH2);
            
        end
        
        function c_nN = C_nN(lgm2F,tn,tN,tau,pnN,ynN)
            p = @lgm2F.DF;
            c_nN=[0.0 0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm2F.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
        end
        
        function c_nN = C_nNDate(lgm2F,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @lgm2F.DF;
            c_nN=[0.0 0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = lgm2F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
        end
        
        function g_nN = G_nN(lgm2F,tn,tN,tau,pnN)
            p = @lgm2F.DF;
            g_nN=[0.0 0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm2F.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
        end
        
        function g_nN = G_nNDate(lgm2F,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @lgm2F.DF;
            g_nN=[0.0 0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = lgm2F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
        end
        
        function h_nTerm = H_nTerm(lgm2F,tn,tTerm)
           h_nTerm =[];
           beta = lgm2F.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
        end
        
        function h_nTerm = H_nTermDate(lgm2F,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = lgm2F.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
        end
        
        function out = CMSDigitalArg(lgm2F,to,tp,tenor,tau)
           % probability that CMS rate > strike
           p_nN = lgm2F.PV01(to,to+tenor,tau);
           df_Tn = lgm2F.DF(to);
           df_TN = lgm2F.DF(to + tenor);
           y_nN = (df_Tn - df_TN)/p_nN;
           c_nN = lgm2F.C_nN(to,to + tenor,tau,p_nN,y_nN);
           g_nN = lgm2F.G_nN(to,to + tenor,tau,p_nN);
           h_nTerm = lgm2F.H_nTerm(to,tp);
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:2
               for j=1:2
                    driftTerm = driftTerm + c_nN(i) * (h_nTerm(j) + g_nN(j)) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:2
               for j=1:2
                    variance = variance + c_nN(i) * c_nN(j) * varCovar(i,j);
               end
           end
           
           out.mu = y_nN + driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN;
           out.varCovar = varCovar;
        end
        
        function out = CDDigitalArgDate(lgm2F,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           toTime =  toDate.DateDiff(valueDate);
           to      = toTime/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1Time = tN1Date.DateDiff(valueDate);
           tN1   =  tN1Time/365.0;
           
           deltaT = (tN1Time - toTime)/365.0;
           
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(tN1);
           
           c_nN1 = lgm2F.BetaDate(valueDate,toDate,tN1Date);
           c_nTerm = lgm2F.BetaDate(valueDate,toDate,tpDate);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:2
               for j=1:2
                    driftTerm = driftTerm + c_nN1(i) * c_nTerm(j) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:2
               for j=1:2
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
           out.mu = 0.5*variance  -1.0*log(df_TN1/df_Tn1) - driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
           out.deltaT = deltaT;
        end
        
        function out = CMSDigitalArgDate(lgm2F,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = lgm2F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = lgm2F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = lgm2F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = lgm2F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:2
               for j=1:2
                    driftTerm = driftTerm + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:2
               for j=1:2
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
           out.mu = y_nN1 + driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
        end
        
        function out = CMSDigital(lgm2F,to,tp,tenor,tau,strike)
           cmsDigitalArg= lgm2F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(lgm2F,to,tp,tenor,tau,lower,upper)
            lower_value = lgm2F.CMSDigital(to,tp,tenor,tau,lower);
            upper_value = lgm2F.CMSDigital(to,tp,tenor,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalArg(lgm2F,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           p_nN1 = lgm2F.PV01(to,to+tenor1,tau);
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = lgm2F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = lgm2F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = lgm2F.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = lgm2F.PV01(to,to+tenor2,tau);
           df_Tn2 = lgm2F.DF(to);
           df_TN2 = lgm2F.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = lgm2F.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = lgm2F.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = lgm2F.H_nTerm(to,tp);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:2
               for j=1:2
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1);
           out.sig2 = sqrt(variance2);
           
           out.pmu = driftTerm1 - driftTerm2;
           out.pmu1 = driftTerm1;
           out.pmu2 = driftTerm2;
           
           out.y1 = y_nN1;
           out.y2 = y_nN2;
           
           
        end
        
        function out = CMSSpreadDigitalArgDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = lgm2F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = lgm2F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = lgm2F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = lgm2F.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = lgm2F.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = lgm2F.DF(to);
           df_TN2 = lgm2F.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = lgm2F.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = lgm2F.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = lgm2F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:2
               for j=1:2
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigital(lgm2F,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSCMSSpreadDigitalDate(lgm2F,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,strike,strikeS)
           
           % probability that CMS tenor1 - CMS tenor 2 > strike
           % CMS 1
           cmsDigitalArg = lgm2F.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           c_nN = cmsDigitalArg.c_nN;
           varCovar = cmsDigitalArg.varCovar;
           
           %CMS Spread
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           
           c_nN1 = cmsSpreadDigitalArg.c_nN1;
           c_nN2 = cmsSpreadDigitalArg.c_nN2;
           
           muS = cmsSpreadDigitalArg.mu;
           sigmaS = cmsSpreadDigitalArg.sigma;
           
           % covariance between CMS Rate & CMS Spread
           covar = 0.0;
           for i=1:2
               for j=1:2
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           corr = covar/(sigma*sigmaS);
           
           z1 = (mu - strike)/sigma;
           z2 = (muS - strikeS)/sigmaS;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CDCMSSpreadDigitalDate(lgm2F,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,strike,strikeS)
           
           % probability that CD > strikeCD 
           % probability that CMS tenor1 - CMS tenor 2 > strike
           
           % CD
           cdDigitalArg = lgm2F.CDDigitalArgDate(valueDate,toDate,tpDate,tenor,tau);
           mu = cdDigitalArg.mu;
           sigma = cdDigitalArg.sigma;
           c_nN = cdDigitalArg.c_nN;
           varCovar = cdDigitalArg.varCovar;
           deltaT = cdDigitalArg.deltaT;
           
           %CMS Spread
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           
           c_nN1 = cmsSpreadDigitalArg.c_nN1;
           c_nN2 = cmsSpreadDigitalArg.c_nN2;
           
           muS = cmsSpreadDigitalArg.mu;
           sigmaS = cmsSpreadDigitalArg.sigma;
           
           % covariance between CMS Rate & CMS Spread
           covar = 0.0;
           for i=1:2
               for j=1:2
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           corr = covar/(sigma*sigmaS);
           
           z1 = (mu - log(1.0 + deltaT*strike))/sigma;
           z2 = (muS - strikeS)/sigmaS;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CMSSpreadDigitalRange(lgm2F,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm2F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = lgm2F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        
        function out = CMSSpreadDigitalRangeDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm2F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = lgm2F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSCMSSpreadDigitalRangeDate(lgm2F,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = lgm2F.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = lgm2F.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = lgm2F.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = lgm2F.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,upperS);
            
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function out = CDCMSSpreadDigitalRangeDate(lgm2F,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = lgm2F.CDCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = lgm2F.CDCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = lgm2F.CDCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = lgm2F.CDCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,upperS);
            
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function atmSwaption = ATMSwaption(lgm2F,tx,tnr)
%             tau = 0.25;
            tau = lgm2F.tau;
            
            pv01 = lgm2F.PV01(tx,tx+tnr,tau);
            fsr  = lgm2F.Fsr(tx,tx+tnr,tau);
            c_nN = lgm2F.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = lgm2F.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:2
                for j=1:2
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        
        function spreadInfo = SpreadInfo(lgm2F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = lgm2F.tau;
            cmsSpreadDigitalArg =  lgm2F.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function impVariance = ImpVariance(lgm2F,tx,tnr1)
%             tau = 0.25;
            tau = lgm2F.tau;
            pv01_1 = lgm2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            variances = lgm2F.LocalVariance(0,tx);
            var1 = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end
        
        function impCorrel = ImpCorrel(lgm2F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = lgm2F.tau;
            pv01_1 = lgm2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm2F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm2F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm2F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm2F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impCovar = ImpCovar(lgm2F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = lgm2F.tau;
            pv01_1 = lgm2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm2F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm2F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm2F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm2F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCovar = coVar;
        end
        
        function instFwdCorr = InsFwdCorr(lgm2F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = lgm2F.tau;
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = (lgm2F.H_Test(tx + tnr1 +0.001,lgm2F.dH1)-lgm2F.H_Test(tx + tnr1,lgm2F.dH1))/0.001;
            dH_tT1(2) = (lgm2F.H_Test(tx + tnr1 +0.001,lgm2F.dH2)-lgm2F.H_Test(tx + tnr1,lgm2F.dH2))/0.001;
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = (lgm2F.H_Test(tx + tnr2 +0.001,lgm2F.dH1)-lgm2F.H_Test(tx + tnr2,lgm2F.dH1))/0.001;
            dH_tT2(2) = (lgm2F.H_Test(tx + tnr2 +0.001,lgm2F.dH2)-lgm2F.H_Test(tx + tnr2,lgm2F.dH2))/0.001;
            
            instVariances = (lgm2F.LocalVariance(0,tx+0.001)-lgm2F.LocalVariance(0,tx))/0.001;
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + dH_tT1(j)*dH_tT1(k)*instVariances(j,k);
                    var2 = var2 + dH_tT2(j)*dH_tT2(k)*instVariances(j,k);
                    covar = covar + dH_tT1(j)*dH_tT2(k)*instVariances(j,k);
                end
            end
            instFwdCorr = covar/sqrt(var1*var2);
        end
        
        function instFwdCorrTest = InsFwdCorrTest(lgm2F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = lgm2F.tau;
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = lgm2F.dH_Test(tx + tnr1,lgm2F.dH1);
            dH_tT1(2) = lgm2F.dH_Test(tx + tnr1,lgm2F.dH2);
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = lgm2F.dH_Test(tx + tnr2,lgm2F.dH1);
            dH_tT2(2) = lgm2F.dH_Test(tx + tnr2,lgm2F.dH2);
            
            Alpha_t = zeros(2,1);
            Alpha_t(1) = lgm2F.Alpha_Test(tx,lgm2F.Alpha1);
            Alpha_t(2) = lgm2F.Alpha_Test(tx,lgm2F.Alpha2);
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:2
                var1 = var1 + dH_tT1(j)*dH_tT1(j)*Alpha_t(j)*Alpha_t(j);
                var2 = var2 + dH_tT2(j)*dH_tT2(j)*Alpha_t(j)*Alpha_t(j);
                covar = covar + dH_tT1(j)*dH_tT2(j)*Alpha_t(j)*Alpha_t(j);
            end
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function instFwdSwapCorrTest = InsSwapFwdCorrTest(lgm2F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = lgm2F.tau;
            C_tT1 = zeros(2,1);
            pv01_1 = lgm2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
            C_tT1(2) = c_nN_1(2);
            
            C_tT2 = zeros(2,1);
            pv01_2 = lgm2F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm2F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm2F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
            C_tT2(2) = c_nN_2(2);
            
            Alpha_t = zeros(2,1);
            Alpha_t(1) = lgm2F.Alpha_Test(tx,lgm2F.Alpha1);
            Alpha_t(2) = lgm2F.Alpha_Test(tx,lgm2F.Alpha2);
            
            covar = 0.0;
            var1 = 0.0;
            var2 = 0.0;
            for j=1:2
                var1 = var1 + C_tT1(j)*C_tT1(j)*Alpha_t(j)*Alpha_t(j);
                var2 = var2 + C_tT2(j)*C_tT2(j)*Alpha_t(j)*Alpha_t(j);
                covar = covar + C_tT1(j)*C_tT2(j)*Alpha_t(j)*Alpha_t(j);
            end
            instFwdSwapCorrTest = covar/sqrt(var1*var2);
        end
        
        function instFwdCovarTest = InsFwdCovarTest(lgm2F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = lgm2F.tau;
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = lgm2F.dH_Test(tx + tnr1,lgm2F.dH1);
            dH_tT1(2) = lgm2F.dH_Test(tx + tnr1,lgm2F.dH2);
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = lgm2F.dH_Test(tx + tnr2,lgm2F.dH1);
            dH_tT2(2) = lgm2F.dH_Test(tx + tnr2,lgm2F.dH2);
            
            Alpha_t = zeros(2,1);
            Alpha_t(1) = lgm2F.Alpha_Test(tx,lgm2F.Alpha1);
            Alpha_t(2) = lgm2F.Alpha_Test(tx,lgm2F.Alpha2);
            
            covar = 0.0;
            for j=1:2
                covar = covar + dH_tT1(j)*dH_tT2(j)*Alpha_t(j)*Alpha_t(j);
            end
            instFwdCovarTest = covar;
        end
        
        function instFwdCovarTestMatrix = InsFwdCovarTest2(lgm2F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCovarTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),2);
                eta =  zeros(2,2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm2F.H_Test(newtnr(i) + tau,lgm2F.dH1) - lgm2F.H_Test(newtnr(i),lgm2F.dH1))/tau;
                    dH_tT(i,2) = (lgm2F.H_Test(newtnr(i) + tau,lgm2F.dH2) - lgm2F.H_Test(newtnr(i),lgm2F.dH2))/tau;
                end

                eta(1,1) = lgm2F.Alpha_Test(tx,lgm2F.Alpha1);
                eta(2,2) = lgm2F.Alpha_Test(tx,lgm2F.Alpha2);
                eta(1,2) = 0;
                eta(2,1) = 0;

                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
            end
            
        end
        
        function instFwdCorrTestMatrix = InsFwdCorrTest2(lgm2F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCorrTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),2);
                eta =  zeros(2,2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm2F.H_Test(newtnr(i) + tau,lgm2F.dH1) - lgm2F.H_Test(newtnr(i),lgm2F.dH1))/tau;
                    dH_tT(i,2) = (lgm2F.H_Test(newtnr(i) + tau,lgm2F.dH2) - lgm2F.H_Test(newtnr(i),lgm2F.dH2))/tau;
                end

                eta(1,1) = lgm2F.Alpha_Test(tx,lgm2F.Alpha1);
                eta(2,2) = lgm2F.Alpha_Test(tx,lgm2F.Alpha2);
                eta(1,2) = 0;
                eta(2,1) = 0;
                
                testLoadingD = dH_tT*eta;
                
                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
                %normalize the correlation matrix and calculate the loading matrix D
                rInv_m = zeros(length(newtnr),length(newtnr));
                for i=1:length(newtnr)
                    rInv_m(i,i) = 1.0/sqrt(coVar(i,i));
                end
                instFwdCorrTestMatrix = zeros(length(newtnr),length(newtnr));
                instFwdCorrTestMatrix = rInv_m*instFwdCovarTestMatrix*rInv_m';
                
            end
            
        end
        
        function deltaHMatrix = DeltaHTest2(lgm2F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                deltaHMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                
                dH_tT = zeros(length(newtnr),2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm2F.H_Test(newtnr(i) + tau,lgm2F.dH1) - lgm2F.H_Test(newtnr(i),lgm2F.dH1))/tau;
                    dH_tT(i,2) = (lgm2F.H_Test(newtnr(i) + tau,lgm2F.dH2) - lgm2F.H_Test(newtnr(i),lgm2F.dH2))/tau;
                end

                deltaHMatrix = dH_tT;
            end
            
        end
        
        function etaMatrix = EtaTest2(lgm2F,tx)
            eta =  zeros(2,2);
            eta(1,1) = lgm2F.Alpha_Test(tx,lgm2F.Alpha1);
            eta(2,2) = lgm2F.Alpha_Test(tx,lgm2F.Alpha2);
            eta(1,2) = 0;
            eta(2,1) = 0;
            
            etaMatrix = eta;
            
        end
        
        function instFwdSwapCovarTest = InsSwapFwdCovarTest(lgm2F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = lgm2F.tau;
            C_tT1 = zeros(2,1);
            pv01_1 = lgm2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
            C_tT1(2) = c_nN_1(2);
            
            C_tT2 = zeros(2,1);
            pv01_2 = lgm2F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm2F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm2F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
            C_tT2(2) = c_nN_2(2);
            
            Alpha_t = zeros(2,1);
            Alpha_t(1) = lgm2F.Alpha_Test(tx,lgm2F.Alpha1);
            Alpha_t(2) = lgm2F.Alpha_Test(tx,lgm2F.Alpha2);
            
            covar = 0.0;
            for j=1:2
                covar = covar + C_tT1(j)*C_tT2(j)*Alpha_t(j)*Alpha_t(j);
            end
            instFwdSwapCovarTest = covar;
        end
        
        function out = RebonatoFwdCorr(lgm2F,tx,tnr1,tnr2)
           rho_inf = 0.4;
           rho_decay = 0.15;
           
%            rho_inf = 0.35;
%            rho_decay = 0.15;

%             rho_inf = 0.2;
%             rho_decay = 0.15; 
%             rho_decay = 0.15; 

%            rho_inf = 0.25;
%            rho_decay = 0.2;
           
%            rho_inf = 0.5847;
%            rho_decay = 0.021;
           
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function  out  = TargetFunctionLGM2FATMSwaptionSurfaceGlobalFmin(lgm2F,tvar,optParams)
        %     K_swaption = optParams.params.diagSwaption.strike;
        %     black_swaption = optParams.params.diagSwaption.blackPrice;
        %     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
        %     dt= 1.0/(optParams.params.freq);

            dH1 = optParams.params.dH1;
            dH2 = optParams.params.dH2;

            Alpha1 =optParams.params.Alpha1;
            Alpha2 =optParams.params.Alpha2;

            dH1_size = optParams.params.dH1_size;
            Alpha1_size = optParams.params.Alpha1_size;

            dH1Orig = optParams.params.dH1Orig;
            dH2Orig = optParams.params.dH2Orig;


        %     dH1 = optParams.params.dH1;
        %     Alpha1 =optParams.params.Alpha1;
        %     dH_size = optParams.dH_size;
        %     Alpha_size = optParams.Alpha_size;
        %     dH_compare = optParams.dH_compare;

        %     smooth_weight_Alpha = 1.0;
        %     smooth_weight_dH = 1.0;

            smooth_weight_Alpha = lgm2F.smoothAlpha;
            smooth_weight_dH = lgm2F.smoothdH;

            model_swaption = 0.0;
            market_swaption = 0.0;

            % assigning model parameters
            for j=1:dH1_size
                dH1.quote(j) = tvar(j);
                dH2.quote(j) = tvar(dH1_size + j);
            end

            for k=1:Alpha1_size
               Alpha1.quote(k) = tvar(2*dH1_size + k);
               Alpha2.quote(k) = tvar(2*dH1_size + Alpha1_size + k);
            end

            lgm2F.dH1.quote = dH1.quote; 
            lgm2F.dH2.quote = dH2.quote; 

            lgm2F.Alpha1.quote = Alpha1.quote;
            lgm2F.Alpha2.quote = Alpha2.quote; 


            % cost function for levenberg marquardt algorithm
%             out= zeros(optParams.numOfEquation,1);
            out= 0.0;
            
            idx =1;

            % constraint for dH
            for i=1:dH1_size
                out = out + smooth_weight_dH*(dH1.quote(i)/dH1Orig.quote(i) - 1.0)*smooth_weight_dH*(dH1.quote(i)/dH1Orig.quote(i) - 1.0);
            end

            for i=1:dH1_size
                out = out + smooth_weight_dH*(dH2.quote(i)/dH2Orig.quote(i) - 1.0)*smooth_weight_dH*(dH2.quote(i)/dH2Orig.quote(i) - 1.0);
            end

            % tikhonov constraint for Alpha
            for i=2:Alpha1_size-1
                out = out + smooth_weight_Alpha*(Alpha1.quote(i-1) + Alpha1.quote(i+1) - 2.0*Alpha1.quote(i))*smooth_weight_Alpha*(Alpha1.quote(i-1) + Alpha1.quote(i+1) - 2.0*Alpha1.quote(i));
            end

            for i=2:Alpha1_size-1
                out = out + smooth_weight_Alpha*(Alpha2.quote(i-1) + Alpha2.quote(i+1) - 2.0*Alpha2.quote(i))*(Alpha2.quote(i-1) + Alpha2.quote(i+1) - 2.0*Alpha2.quote(i));
            end

            for i=1:optParams.params.swaption_expiry_size
                for j=1:optParams.params.swaption_tenor_size
                    if optParams.params.swaption_calibration_yn(i,j) == 1
                        Ti = optParams.params.swaptionSurface.expiry(i);
                        Tj = optParams.params.swaptionSurface.tenor(j);
                        market_swaption = optParams.params.swaptionSurface.blackPrice(i,j);
                        model_swaption = lgm2F.ATMSwaption(Ti,Tj);
                        out = out + (model_swaption/market_swaption - 1.0)*(model_swaption/market_swaption - 1.0);
                    end
                end
            end

        end
        
        function  out  = TargetFunctionLGM2FATMSwaptionSurfaceGlobalFminDecorr(lgm2F,tvar,optParams)
        %     K_swaption = optParams.params.diagSwaption.strike;
        %     black_swaption = optParams.params.diagSwaption.blackPrice;
        %     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
        %     dt= 1.0/(optParams.params.freq);

            dH1 = optParams.params.dH1;
            dH2 = optParams.params.dH2;

            Alpha1 =optParams.params.Alpha1;
            Alpha2 =optParams.params.Alpha2;

            dH1_size = optParams.params.dH1_size;
            Alpha1_size = optParams.params.Alpha1_size;

            dH1Orig = optParams.params.dH1Orig;
            dH2Orig = optParams.params.dH2Orig;


        %     dH1 = optParams.params.dH1;
        %     Alpha1 =optParams.params.Alpha1;
        %     dH_size = optParams.dH_size;
        %     Alpha_size = optParams.Alpha_size;
        %     dH_compare = optParams.dH_compare;

        %     smooth_weight_Alpha = 1.0;
        %     smooth_weight_dH = 1.0;

            smooth_weight_Alpha = lgm2F.smoothAlpha;
            smooth_weight_dH = lgm2F.smoothdH;

            model_swaption = 0.0;
            market_swaption = 0.0;

            % assigning model parameters
            for j=1:dH1_size
                dH1.quote(j) = tvar(j);
                dH2.quote(j) = tvar(dH1_size + j);
            end

            for k=1:Alpha1_size
               Alpha1.quote(k) = tvar(2*dH1_size + k);
               Alpha2.quote(k) = tvar(2*dH1_size + Alpha1_size + k);
            end

            lgm2F.dH1.quote = dH1.quote; 
            lgm2F.dH2.quote = dH2.quote; 

            lgm2F.Alpha1.quote = Alpha1.quote;
            lgm2F.Alpha2.quote = Alpha2.quote; 


            % cost function for levenberg marquardt algorithm
%             out= zeros(optParams.numOfEquation,1);
            out= 0.0;
            
            idx =1;

            % constraint for dH
            for i=1:dH1_size
                out = out + smooth_weight_dH*(dH1.quote(i)/dH1Orig.quote(i) - 1.0)*(dH1.quote(i)/dH1Orig.quote(i) - 1.0);
            end

            for i=1:dH1_size
                out = out + smooth_weight_dH*(dH2.quote(i)/dH2Orig.quote(i) - 1.0)*(dH2.quote(i)/dH2Orig.quote(i) - 1.0);
            end

            % tikhonov constraint for Alpha
            for i=2:Alpha1_size-1
                out = out + smooth_weight_Alpha*(Alpha1.quote(i-1) + Alpha1.quote(i+1) - 2.0*Alpha1.quote(i))*(Alpha1.quote(i-1) + Alpha1.quote(i+1) - 2.0*Alpha1.quote(i));
            end

            for i=2:Alpha1_size-1
                out = out + smooth_weight_Alpha*(Alpha2.quote(i-1) + Alpha2.quote(i+1) - 2.0*Alpha2.quote(i))*(Alpha2.quote(i-1) + Alpha2.quote(i+1) - 2.0*Alpha2.quote(i));
            end

            for i=1:optParams.params.swaption_expiry_size
                for j=1:optParams.params.swaption_tenor_size
                    if optParams.params.swaption_calibration_yn(i,j) == 1
                        Ti = optParams.params.swaptionSurface.expiry(i);
                        Tj = optParams.params.swaptionSurface.tenor(j);
                        market_swaption = optParams.params.swaptionSurface.blackPrice(i,j);
                        model_swaption = lgm2F.ATMSwaption(Ti,Tj);
                        out = out + (model_swaption/market_swaption - 1.0)*(model_swaption/market_swaption - 1.0);
                    end
                end
            end
            
            for i=1:optParams.params.swaption_expiry_size
                Ti = optParams.params.swaptionSurface.expiry(i);
                for j=1:optParams.params.swaption_tenor_size-2
                    Tj = optParams.params.swaptionSurface.expiry(j);
                    for k=j+1:optParams.params.swaption_tenor_size-1
                        Tk = optParams.params.swaptionSurface.tenor(k);
                        Tk1 = optParams.params.swaptionSurface.tenor(k+1);
                        
                        corrS = lgm2F.InsFwdCorrTest(Ti,Tj,Tk);
                        corrL = lgm2F.InsFwdCorrTest(Ti,Tj,Tk1);
                        out = out + exp(max(corrL-corrS,0))-1;
                    end
                end
            end

        end
        
        function out = TargetFunctionFwdLiborCorrGlobal(lgm2F,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            
            for j=1:length(lgm2F.dH1.quote)
               lgm2F.dH1.quote(j) = tvar(j);
               lgm2F.dH2.quote(j) = tvar(j+length(lgm2F.dH1.quote));
            end
            
            dH1Orig = params.dH1Orig;
            dH2Orig = params.dH2Orig;
    
            smooth_weight_dH = lgm2F.smoothdH;
            
            out= 0;
            % constraint for dH
            dH1_size = length(lgm2F.dH1.quote);
            for i=1:dH1_size
                out = out + smooth_weight_dH*(lgm2F.dH1.quote(i)/dH1Orig.quote(i) - 1.0)*(lgm2F.dH1.quote(i)/dH1Orig.quote(i) - 1.0);
            end

            for i=1:dH1_size
                out = out + smooth_weight_dH*(lgm2F.dH2.quote(i)/dH2Orig.quote(i) - 1.0)*(lgm2F.dH2.quote(i)/dH2Orig.quote(i) - 1.0);
            end
            
            lgm2FFwdLiborCorr = zeros(length(expiry),length(expiry));
            rebonatoCorr = zeros(length(expiry),length(expiry));
           
            tx = 0;
            tnr1 = expiry;
%             tau = 0.25;
            tau = lgm2F.tau;
            lgm2FFwdLiborCorr = lgm2F.InsFwdCorrTest2(tx,tnr1,tau);
            for j=1:length(expiry)-1
                for k=j+1:length(expiry)
                    lgm2FFwdCorr(j,k) = lgm2FFwdLiborCorr(j,k);
                    rebonatoCorr(j,k) = lgm2F.RebonatoFwdCorr(tx,expiry(j),expiry(k));
                    out = out + (lgm2FFwdCorr(j,k)-rebonatoCorr(j,k))*(lgm2FFwdCorr(j,k)-rebonatoCorr(j,k));
                end
            end
        end
        
       
        function out = CalibrateToATMSwaptionSurface(lgm2F,black,calibrationFlag,numOfIter)
            % Market Data Object , Vanilla Object
            % targetSwaption
            % targetCap
            
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% target instruments set up
            params.swaption_expiry_size = expirySize;
            params.swaption_tenor_size = tenorSize;
            
            params.swaption_calibration_yn = ones(expirySize,tenorSize);
            
            params.swaptionSurface.expiry = swaptionVolSurface.expiry;
            params.swaptionSurface.tenor  = swaptionVolSurface.tenor;
            params.swaptionSurface.quote  = zeros(expirySize,tenorSize);
            params.swaptionSurface.strike = zeros(expirySize,tenorSize);
            params.swaptionSurface.blackPrice = zeros(expirySize,tenorSize);
            
            % setting target diagonal swaption
            tau1 = lgm2F.tau;
            for i=1:expirySize
                for j=1:tenorSize
                    if params.swaption_calibration_yn(i,j) == 1
                        Ti = params.swaptionSurface.expiry(i);
                        Tj = params.swaptionSurface.tenor(j);
                        blackVolQuote = swaptionVolSurface.swaptionVol(expiry(i),tenor(j));
                        params.swaptionSurface.quote(i,j) = blackVolQuote;
%                         params.swaptionSurface.strike(i,j) = black.Fsr(Ti,Ti + Tj,0.25);
                        params.swaptionSurface.strike(i,j) = black.Fsr(Ti,Ti + Tj,tau1);
                        params.swaptionSurface.blackPrice(i,j) = black.BlackSwaptionWithVol(Ti,Tj,params.swaptionSurface.strike(i,j),blackVolQuote);
                    end
                end

            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            params.dH1 = lgm2F.dH1;
            params.dH2 = lgm2F.dH2;
            
            dH1_size = length(lgm2F.dH1.quote);
            Alpha1_size = length(lgm2F.Alpha1.quote);
            
            params.dH1_size = dH1_size;
            params.Alpha1_size = Alpha1_size;
            
            params.Alpha1 = lgm2F.Alpha1;
            params.Alpha2 = lgm2F.Alpha2;
            
            lgm2F.dH1    = params.dH1;
            lgm2F.dH2    = params.dH2;
            
            lgm2F.Alpha1 = params.Alpha1;
            lgm2F.Alpha2 = params.Alpha2;
            
            params.dH1Orig = lgm2F.dH1Orig;
            params.dH2Orig = lgm2F.dH2Orig;
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            out.dH1 = params.dH1;
            out.dH2 = params.dH2;
            
            out.Alpha1 = params.Alpha1;
            out.Alpha2 = params.Alpha2;
            
            out.blackSwaptionPrice = params.swaptionSurface.blackPrice;
            out.lgm2FSwaptionPrice = zeros(expirySize,tenorSize);
            out.swaptionPriceRelDiff = zeros(expirySize,tenorSize);
            out.swaptionPriceRmseTotal = 0.0;
           
            out.market_swaption = zeros(expirySize,tenorSize);
            out.model_swaption = zeros(expirySize,tenorSize);
            
            tvar = [];
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
                
            case 'global'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH    
                                              2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobal',tvar,x, 1000, options,'unc',lgm2F,optParams);
                    % test hsw 20180313
%                     import targetFunctions.TargetFunctionLGM2FATMSwaptionSurfaceGlobal;
                    [ret, popt, info, covar]=levmar('targetFunctions.TargetFunctionLGM2FATMSwaptionSurfaceGlobal',tvar,x, numOfIter, options,'unc',lgm2F,optParams);
                    
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobalAlphaAbs',tvar,x, 200, options,'unc',lgm2F,optParams);
                    
                    
                    toc
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
%                     for j=1:Alpha1_size
%                         out.Alpha1.quote(j) = popt(j + 2*dH1_size);
%                         out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
%                         lgm2F.Alpha1.quote(j) = popt(j + 2*dH1_size);
%                         lgm2F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
%                     end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = sqrt(popt(j + 2*dH1_size)*popt(j + 2*dH1_size));
                        out.Alpha2.quote(j) = sqrt(popt(j + 2*dH1_size + Alpha1_size)*popt(j + 2*dH1_size + Alpha1_size));
                        lgm2F.Alpha1.quote(j) = sqrt(popt(j + 2*dH1_size)*popt(j + 2*dH1_size));
                        lgm2F.Alpha2.quote(j) = sqrt(popt(j + 2*dH1_size + Alpha1_size)*popt(j + 2*dH1_size + Alpha1_size));
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
            
            case 'globalAbs'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH    
                                              2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobal',tvar,x, 10000, options,'unc',lgm2F,optParams);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobalAbs',tvar,x, 10000, options,'unc',lgm2F,optParams);
                    
                    toc
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                        lgm2F.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        lgm2F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
                    
            case 'globalDecorr'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH    
                                              2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    degcons = (optParams.params.swaption_tenor_size-1)*(optParams.params.swaption_tenor_size-2)/2;
                    degcons = degcons * optParams.params.swaption_expiry_size;
                    x=  zeros(optParams.numOfEquation + degcons,1);
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobalDecorr',tvar,x, 10000, options,'unc',lgm2F,optParams);
                    toc
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                        lgm2F.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        lgm2F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
            
            case 'globalFmin'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH    
                                              2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
%                     
%                     x=  zeros(optParams.numOfEquation,1);
%                     tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobal',tvar,x, 500, options,'unc',lgm2F,optParams);
%                     toc
                    
                    popt=  zeros(optParams.numOfEquation,1);
                    targetfun = @(tvar) TargetFunctionLGM2FATMSwaptionSurfaceGlobalFmin(lgm2F,tvar,optParams);
                    options=optimset('Algorithm','interior-point');
%                     options=optimset('Algorithm','trust-region-reflective','MaxFunEvals',700);
%                     [popt, fval] = fmincon(targetfun,tvar,[],[],[],[],[],[],[],options);
                    [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],[],[],[],options);
                    
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                        lgm2F.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        lgm2F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
                    
            case 'globalFminDecorr'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH    
                                              2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
%                     
%                     x=  zeros(optParams.numOfEquation,1);
%                     tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobal',tvar,x, 500, options,'unc',lgm2F,optParams);
%                     toc
                    
                    popt=  zeros(optParams.numOfEquation,1);
                    targetfun = @(tvar) TargetFunctionLGM2FATMSwaptionSurfaceGlobalFminDecorr(lgm2F,tvar,optParams);
                    options=optimset('Algorithm','interior-point');
%                     options=optimset('Algorithm','trust-region-reflective','MaxFunEvals',700);
%                     [popt, fval] = fmincon(targetfun,tvar,[],[],[],[],[],[],[],options);
                    [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],[],[],[],options);
                    
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                        lgm2F.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        lgm2F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
            
            case 'globaldHLiborVol'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH
                                              2*(dH1_size -2);   % tikhonov regularization gamma constraint
                                              
%                                               2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    x=  zeros(optParams.numOfEquation,1);
                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobal',tvar,x, 1000, options,'unc',lgm2F,optParams);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobaldHLiborVol',tvar,x, 1000, options,'unc',lgm2F,optParams);
                    
                    toc
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                        lgm2F.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        lgm2F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
                    
                    out.LiborVol1 = zeros(dH1_size,1);
                    out.LiborVol2 = zeros(dH1_size,1);
                    out.LiborVol = zeros(dH1_size,1);
                    
                    for i=1:dH1_size
                        out.LiborVol1(i) = lgm2F.Alpha_Test(lgm2F.dH1.tenor(i),lgm2F.Alpha1)*lgm2F.dH1.tenor(i);
                        out.LiborVol2(i) = lgm2F.Alpha_Test(lgm2F.dH2.tenor(i),lgm2F.Alpha2)*lgm2F.dH2.tenor(i);
                        out.LiborVol(i) = sqrt(out.LiborVol1(i)*out.LiborVol1(i)+out.LiborVol2(i)*out.LiborVol2(i));
                    end

                    
            otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToFwdCorr(lgm2F,black,calibrationFlag)
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
%             expiry = [0,expiry];
            expiry = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];
            expirySize = size(expiry,1);
            tenorSize = size(tenor,2);
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            params.dH1Orig = lgm2F.dH1Orig;
            params.dH2Orig = lgm2F.dH2Orig;
            
            params.dH1 = lgm2F.dH1;
            params.dH2 = lgm2F.dH2;
            
            params.Alpha1 = lgm2F.Alpha1;
            params.Alpha2 = lgm2F.Alpha2;
            
            out.dH1 = params.dH1;
            out.dH2 = params.dH2;
            
            out.InsFwdCorr = zeros(expirySize, expirySize);
            out.Diff = zeros(expirySize,expirySize);
            out.DiffTotal=0.0;
            
            targetTenor = [2;5;10];
            targetMaturity = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15];
                    
            out.TermCorr = zeros(length(targetMaturity),length(targetTenor),length(targetTenor));
            out.TermCorrDiff = zeros(length(targetMaturity),length(targetTenor),length(targetTenor));
            out.TermCorrDiffTotal = 0;
            
            targetTermCorr = 0.92;
            
            tvar = [];
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
                
            case 'global'
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                        lb(j)= -10;
                        lb(length(lgm2F.dH1.quote)+j)= -10;
                        ub(j)= 10;
                        ub(length(lgm2F.dH1.quote)+j)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    
                    targetfun = @(tvar) TargetFunctionFwdCorrGlobal(lgm2F,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                    options=optimset('Algorithm','trust-region-reflective','MaxFunEvals',700);
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for j=1:length(lgm2F.dH1.quote)
                        out.dH1.quote(j) = x(j);
                        out.dH2.quote(j) = x(j+length(lgm2F.dH1.quote));
                        lgm2F.dH1.quote(j) = x(j);
                        lgm2F.dH2.quote(j) = x(j+length(lgm2F.dH1.quote));
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize-1
                            for l=k+1:tenorSize
                                out.InsFwdCorr(j,k,l) = lgm2F.InsFwdCorrTest(expiry(j),tenor(k),tenor(l));
                                out.Diff(j,k,l) = out.InsFwdCorr(j,k,l)-lgm2F.RebonatoFwdCorr(expiry(j),tenor(k),tenor(l));
                                out.DiffTotal = out.DiffTotal + out.Diff(j,k,l)*out.Diff(j,k,l);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.DiffTotal = sqrt(out.DiffTotal/N);
            case 'globalFwdLibor'
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                        lb(j)= 0.0001;
                        lb(length(lgm2F.dH1.quote)+j)= 0.0001;
                        ub(j)= 10;
                        ub(length(lgm2F.dH1.quote)+j)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    
                    targetfun = @(tvar) TargetFunctionFwdLiborCorrGlobal(lgm2F,tvar,params);
%                     options=optimset('Algorithm','interior-point');
%                     options=optimset('Algorithm','trust-region-reflective','MaxFunEvals',700);
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],[]);
                    
                    for j=1:length(lgm2F.dH1.quote)
                        out.dH1.quote(j) = x(j);
                        out.dH2.quote(j) = x(j+length(lgm2F.dH1.quote));
                        lgm2F.dH1.quote(j) = x(j);
                        lgm2F.dH2.quote(j) = x(j+length(lgm2F.dH1.quote));
                    end
                    
                    tx = 0;
                    tnr1 = expiry;
%                     tau = 0.25;
                    tau = lgm2F.tau;
                    lgm2FFwdLiborCorr = lgm2F.InsFwdCorrTest2(tx,tnr1,tau);
                    rebonatoCorr = zeros(length(expiry),length(expiry));
                    N=0;
                    for j=1:length(expiry)-1
                        for k=j+1:length(expiry)
                            out.lgm2FFwdCorr(j,k) = lgm2FFwdLiborCorr(j,k);
                            out.rebonatoCorr(j,k) = lgm2F.RebonatoFwdCorr(0,expiry(j),expiry(k));
                            out.Diff(j,k) = out.lgm2FFwdCorr(j,k)-out.rebonatoCorr(j,k);
                            out.DiffTotal = out.DiffTotal + out.Diff(j,k)*out.Diff(j,k);
                            N= N +1;
                        end
                    end
                   
                    out.DiffTotal = sqrt(out.DiffTotal/N);
            case 'globalFwdLiborLev'
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                        lb(j)= 0.0001;
                        lb(length(lgm2F.dH1.quote)+j)= 0.0001;
                        ub(j)= 10;
                        ub(length(lgm2F.dH1.quote)+j)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    
                    numOfEquation = 2*length(lgm2F.dH1.quote) + expirySize*(expirySize-1)/2;
                    x=  zeros(numOfEquation,1);
                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevGlobal',tvar,x, 1000, options,'unc',lgm2F,params);
%                     [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevGlobal',tvar,x, 1000, options,'unc',lgm2F,params);
                    [ret, popt, info, covar]=levmar('targetFunctions.TargetFunctionFwdLiborCorrLevGlobal',tvar,x, 200, options,'unc',lgm2F,params);

                    toc
                    
%                     targetfun = @(tvar) TargetFunctionFwdLiborCorrGlobal(lgm2F,tvar,params);
%                     options=optimset('Algorithm','interior-point');
%                     options=optimset('Algorithm','trust-region-reflective','MaxFunEvals',700);
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],[]);
                    
                    for j=1:length(lgm2F.dH1.quote)
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                    end
                    
                    tx = 0;
                    tnr1 = expiry;
%                     tau = 0.25;
                    tau = lgm2F.tau;
                    lgm2FFwdLiborCorr = lgm2F.InsFwdCorrTest2(tx,tnr1,tau);
                    rebonatoCorr = zeros(length(expiry),length(expiry));
                    N=0;
                    for j=1:length(expiry)-1
                        for k=j+1:length(expiry)
                            out.lgm2FFwdCorr(j,k) = lgm2FFwdLiborCorr(j,k);
                            out.rebonatoCorr(j,k) = lgm2F.RebonatoFwdCorr(0,expiry(j),expiry(k));
                            out.Diff(j,k) = out.lgm2FFwdCorr(j,k)-out.rebonatoCorr(j,k);
                            out.DiffTotal = out.DiffTotal + out.Diff(j,k)*out.Diff(j,k);
                            N = N+1;
                        end
                    end
                   
                    out.DiffTotal = sqrt(out.DiffTotal/N);
            case 'globalFwdLiborLevSimple'
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                        lb(j)= 0.0001;
                        lb(length(lgm2F.dH1.quote)+j)= 0.0001;
                        ub(j)= 10;
                        ub(length(lgm2F.dH1.quote)+j)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    
                    numOfEquation = expirySize*(expirySize-1)/2;
                    x=  zeros(numOfEquation,1);
                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevGlobal',tvar,x, 1000, options,'unc',lgm2F,params);
%                     [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevGlobal',tvar,x, 1000, options,'unc',lgm2F,params);
                    [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevGlobalSimple',tvar,x, 200, options,'unc',lgm2F,params);

                    toc
                    
%                     targetfun = @(tvar) TargetFunctionFwdLiborCorrGlobal(lgm2F,tvar,params);
%                     options=optimset('Algorithm','interior-point');
%                     options=optimset('Algorithm','trust-region-reflective','MaxFunEvals',700);
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],[]);
                    
                    for j=1:length(lgm2F.dH1.quote)
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                    end
                    
                    tx = 0;
                    tnr1 = expiry;
%                     tau = 0.25;
                    tau = lgm2F.tau;
                    lgm2FFwdLiborCorr = lgm2F.InsFwdCorrTest2(tx,tnr1,tau);
                    rebonatoCorr = zeros(length(expiry),length(expiry));
                    N=0;
                    for j=1:length(expiry)-1
                        for k=j+1:length(expiry)
                            out.lgm2FFwdCorr(j,k) = lgm2FFwdLiborCorr(j,k);
                            out.rebonatoCorr(j,k) = lgm2F.RebonatoFwdCorr(0,expiry(j),expiry(k));
                            out.Diff(j,k) = out.lgm2FFwdCorr(j,k)-out.rebonatoCorr(j,k);
                            out.DiffTotal = out.DiffTotal + out.Diff(j,k)*out.Diff(j,k);
                            N = N+1;
                        end
                    end
                   
                    out.DiffTotal = sqrt(out.DiffTotal/N);
            case 'globalTermCorr'
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                        lb(j)= 0.0001;
                        lb(length(lgm2F.dH1.quote)+j)= 0.0001;
                        ub(j)= 10;
                        ub(length(lgm2F.dH1.quote)+j)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    
                    numOfEquation = 2*length(lgm2F.dH1.quote) + length(targetMaturity)*length(targetTenor)*(length(targetTenor)-1)/2;
                    x=  zeros(numOfEquation,1);
                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevGlobal',tvar,x, 1000, options,'unc',lgm2F,params);
                    [ret, popt, info, covar]=levmar('TargetFunctionTermCorrLevGlobal',tvar,x, 1000, options,'unc',lgm2F,params);

                    toc
   
                    
                    for j=1:length(lgm2F.dH1.quote)
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                    end
                    
                    
                    N=0;
                    for i=1:length(targetMaturity)
                        for j=1:length(targetTenor)-1
                            for k=j+1:length(targetTenor)
                                out.TermCorr(i,j,k) = lgm2F.ImpCorrel(targetMaturity(i),targetTenor(j),targetTenor(k));
                                out.TermCorrDiff(i,j,k) = out.TermCorr(i,j,k)-targetTermCorr;
                                out.TermCorrDiffTotal = out.TermCorrDiffTotal + out.TermCorrDiff(i,j,k)*out.TermCorrDiff(i,j,k);
                                N = N+1;
                            end
                        end
                    end
                    out.TermCorrDiffTotal = sqrt(out.TermCorrDiffTotal/N);
                    
            case 'globalFwdLiborLevDetail'
%                     expiryDetail =  0.25:0.25:30;
                    tau1 = lgm2F.tau;
                    expiryDetail =  tau1:tau1:30;
                    
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                        lb(j)= 0.0001;
                        lb(length(lgm2F.dH1.quote)+j)= 0.0001;
                        ub(j)= 10;
                        ub(length(lgm2F.dH1.quote)+j)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    
                    
                    numOfEquation = 2*length(lgm2F.dH1.quote);
                    
                    numOfEquation = numOfEquation + length(expiryDetail)*(length(expiryDetail)-1)/2;
                    
                    x=  zeros(numOfEquation,1);
                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevGlobal',tvar,x, 1000, options,'unc',lgm2F,params);
                    [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevDetailGlobal',tvar,x, 1000, options,'unc',lgm2F,params);

                    toc
                    
%                     targetfun = @(tvar) TargetFunctionFwdLiborCorrGlobal(lgm2F,tvar,params);
%                     options=optimset('Algorithm','interior-point');
%                     options=optimset('Algorithm','trust-region-reflective','MaxFunEvals',700);
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],[]);
                    
                    for j=1:length(lgm2F.dH1.quote)
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                    end
                    
                    tx = 0;
                    tnr1 = expiry;
%                     tau = 0.25;
                    tau = lgm2F.tau;
                    lgm2FFwdLiborCorr = lgm2F.InsFwdCorrTest2(tx,tnr1,tau);
                    rebonatoCorr = zeros(length(expiry),length(expiry));
                    N=0;
                    for j=1:length(expiry)-1
                        for k=j+1:length(expiry)
                            out.lgm2FFwdCorr(j,k) = lgm2FFwdLiborCorr(j,k);
                            out.rebonatoCorr(j,k) = lgm2F.RebonatoFwdCorr(0,expiry(j),expiry(k));
                            out.Diff(j,k) = out.lgm2FFwdCorr(j,k)-out.rebonatoCorr(j,k);
                            out.DiffTotal = out.DiffTotal + out.Diff(j,k)*out.Diff(j,k);
                            N = N+1;
                        end
                    end
                   
                    out.DiffTotal = sqrt(out.DiffTotal/N);
            otherwise
                    disp('unImplemented')
            end
        end
        
        function out = FitToForwardCovariance(lgm2F,black,calibrationFlag)
            % Market Data Object , Vanilla Object
            % targetSwaption
            % targetCap
            
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% target instruments set up
            params.swaption_expiry_size = expirySize;
            params.swaption_tenor_size = tenorSize;
            
            params.swaption_calibration_yn = ones(expirySize,tenorSize);
            
            params.swaptionSurface.expiry = swaptionVolSurface.expiry;
            params.swaptionSurface.tenor  = swaptionVolSurface.tenor;
            params.swaptionSurface.quote  = zeros(expirySize,tenorSize);
            params.swaptionSurface.strike = zeros(expirySize,tenorSize);
            params.swaptionSurface.blackPrice = zeros(expirySize,tenorSize);
            
            % setting target diagonal swaption
            tau1 = lgm2F.tau;
            for i=1:expirySize
                for j=1:tenorSize
                    if params.swaption_calibration_yn(i,j) == 1
                        Ti = params.swaptionSurface.expiry(i);
                        Tj = params.swaptionSurface.tenor(j);
                        blackVolQuote = swaptionVolSurface.swaptionVol(expiry(i),tenor(j));
                        params.swaptionSurface.quote(i,j) = blackVolQuote;
%                         params.swaptionSurface.strike(i,j) = black.Fsr(Ti,Ti + Tj,0.25);
                        params.swaptionSurface.strike(i,j) = black.Fsr(Ti,Ti + Tj,tau1);
                        params.swaptionSurface.blackPrice(i,j) = black.BlackSwaptionWithVol(Ti,Tj,params.swaptionSurface.strike(i,j),blackVolQuote);
                    end
                end

            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            params.dH1 = lgm2F.dH1;
            params.dH2 = lgm2F.dH2;
            
            dH1_size = length(lgm2F.dH1.quote);
            Alpha1_size = length(lgm2F.Alpha1.quote);
            
            params.dH1_size = dH1_size;
            params.Alpha1_size = Alpha1_size;
            
            params.Alpha1 = lgm2F.Alpha1;
            params.Alpha2 = lgm2F.Alpha2;
            
            lgm2F.dH1    = params.dH1;
            lgm2F.dH2    = params.dH2;
            
            lgm2F.Alpha1 = params.Alpha1;
            lgm2F.Alpha2 = params.Alpha2;
            
            params.dH1Orig = lgm2F.dH1Orig;
            params.dH2Orig = lgm2F.dH2Orig;
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            out.dH1 = params.dH1;
            out.dH2 = params.dH2;
            
            out.Alpha1 = params.Alpha1;
            out.Alpha2 = params.Alpha2;
            
            out.blackSwaptionPrice = params.swaptionSurface.blackPrice;
            out.lgm2FSwaptionPrice = zeros(expirySize,tenorSize);
            out.swaptionPriceRelDiff = zeros(expirySize,tenorSize);
            out.swaptionPriceRmseTotal = 0.0;
           
            out.market_swaption = zeros(expirySize,tenorSize);
            out.model_swaption = zeros(expirySize,tenorSize);
            
            tvar = [];
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
                
            case 'global'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH    
                                              2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
                    
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobalAlphaAbs',tvar,x, 200, options,'unc',lgm2F,optParams);
                    
                    toc
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = sqrt(popt(j + 2*dH1_size)*popt(j + 2*dH1_size));
                        out.Alpha2.quote(j) = sqrt(popt(j + 2*dH1_size + Alpha1_size)*popt(j + 2*dH1_size + Alpha1_size));
                        lgm2F.Alpha1.quote(j) = sqrt(popt(j + 2*dH1_size)*popt(j + 2*dH1_size));
                        lgm2F.Alpha2.quote(j) = sqrt(popt(j + 2*dH1_size + Alpha1_size)*popt(j + 2*dH1_size + Alpha1_size));
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
            otherwise
                disp('unimplemented')  
            end
        end
        
    end
    
end



In [ ]:
classdef LGM2F < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        Alpha1
        Alpha2
        dH1
        dH2
        correl
        dH1Orig
        dH2Orig
        smoothdH
        smoothAlpha
        smoothTermCorr
        targetTermCorr
        numOfFactor
    end
    
    methods
        function lgm2F = LGM2F(params)
            if nargin > 0
                lgm2F.zeroCurve = params.zeroCurve;
                lgm2F.mktData = params.mktData;
                lgm2F.modelParams = params.modelParams;
                
                lgm2F.Alpha1 = params.modelParams('Alpha1');
                lgm2F.Alpha2 = params.modelParams('Alpha2');
                lgm2F.dH1 =  params.modelParams('dH1');
                lgm2F.dH2 =  params.modelParams('dH2');
                
                lgm2F.dH1Orig =  params.modelParams('dH1Orig');
                lgm2F.dH2Orig =  params.modelParams('dH2Orig');
                lgm2F.correl =  params.modelParams('correl');
                lgm2F.smoothdH =  params.modelParams('smoothdH');
                lgm2F.smoothAlpha =  params.modelParams('smoothAlpha');
                lgm2F.smoothTermCorr = params.modelParams('smoothTermCorr');
                lgm2F.targetTermCorr = params.modelParams('targetTermCorr');
                
                lgm2F.numOfFactor = 2;
            end
        end
        
        function out = SimpleSRPrOverHedge(lgm2F,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(lgm2F,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(lgm2F,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(lgm2F,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
             elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = lgm2F.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out = SimpleDRPr(lgm2F,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS)
            outVec = 0.0;
            if cmsC >= ralower && cmsC <= raupper && spreadC >= ralowerS && spreadC <= raupperS
                outVec = 1.0;
            end
            out = outVec;
        end
        
        function out = DRPr(lgm2F,cmsC,cmsN,ralower,raupper,spreadC,spreadN,ralowerS,raupperS,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN == 1
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
%                     outVec(i) = lgm2F.SimpleDRPrOverHedge(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            else
                for i =1:pathSize
                    outVec(i) = lgm2F.SimpleDRPr(cmsC(i),cmsN(i),ralower,raupper,spreadC(i),spreadN(i),ralowerS,raupperS);
                end
            end
            
            out = outVec;
        end
        
        function out =  Libor_SimpleDate(lgm2F,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           %CMS
           fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
           fwdEndDate = fwdStartDate.AddDate(tenor,'year');
           fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
           deltaT = (fwdEndTime - fwdStartTime)/365.0;
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           
           stateSize = size(modelStatesT,2);
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           df_Tn = lgm2F.discountFactor(evalTime,fwdStartTime,numMatTime,modelStatesT);
           df_TN = lgm2F.discountFactor(evalTime,fwdEndTime,numMatTime,modelStatesT);
           
           liborOut = zeros(1,stateSize);
           
           for i=1:stateSize
                liborOut(i) = (df_Tn(i)/df_TN(i) - 1.0)/deltaT;
           end
           
           out = liborOut;
            
        end
        
        function out =  CMS_PSA_SimpleDate(lgm2F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           tau = 0.25;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           if tenor < 1
               tenorMonth = round(tenor*12);
               tNDate = toDate.AddDate(tenorMonth,'month');
           else
               tNDate = toDate.AddDate(tenor,'year');
           end
           
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = lgm2F.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = lgm2F.DF(to);
           df_TN = lgm2F.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = lgm2F.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = lgm2F.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = lgm2F.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = lgm2F.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:2
               for j=1:2
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + c_nN(1)*modelStatesT(1,i) + c_nN(2) * modelStatesT(2,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
        end
        
        function out = HtT_r(lgm2F,t,T)
            out =  lgm2F.H(T,lgm2F.dH1) - lgm2F.H(t,lgm2F.dH1); 
        end
        
        function out = HtT_s(lgm2F,t,T)
            out =  lgm2F.H(T,lgm2F.dH2) - lgm2F.H(t,lgm2F.dH2); 
        end
        
        function out = fwdZcVariance(lgm2F, a, b, T1, T2)
            varMatrix = lgm2F.LocalVariance(a,b);
            
            localVariance = zeros(4,1);
            measureChange = zeros(4,1);
            
            localVariance(1) = (lgm2F.HtT_r(a,T1)*lgm2F.HtT_r(a,T1)-lgm2F.HtT_r(a,b)*lgm2F.HtT_r(a,b))*varMatrix(1,1);
            localVariance(2) = (lgm2F.HtT_r(a,T1)*lgm2F.HtT_s(a,T1)-lgm2F.HtT_r(a,b)*lgm2F.HtT_s(a,b))*varMatrix(1,2);
            localVariance(3) = (lgm2F.HtT_s(a,T1)*lgm2F.HtT_r(a,T1)-lgm2F.HtT_s(a,b)*lgm2F.HtT_r(a,b))*varMatrix(2,1);
            localVariance(4) = (lgm2F.HtT_s(a,T1)*lgm2F.HtT_s(a,T1)-lgm2F.HtT_s(a,b)*lgm2F.HtT_s(a,b))*varMatrix(2,2);
            
            measureChange(1) = (lgm2F.HtT_r(a,T1)-lgm2F.HtT_r(a,b))*lgm2F.HtT_r(a,T2)*varMatrix(1,1);
            measureChange(2) = (lgm2F.HtT_r(a,T1)-lgm2F.HtT_r(a,b))*lgm2F.HtT_s(a,T2)*varMatrix(1,2);
            measureChange(3) = (lgm2F.HtT_s(a,T1)-lgm2F.HtT_s(a,b))*lgm2F.HtT_r(a,T2)*varMatrix(2,1);
            measureChange(4) = (lgm2F.HtT_s(a,T1)-lgm2F.HtT_s(a,b))*lgm2F.HtT_s(a,T2)*varMatrix(2,2);
            
            value = localVariance(1) + localVariance(2);
            value = value + localVariance(3) + localVariance(4);
            
            measureChangeTerm = measureChange(1) + measureChange(2);
            measureChangeTerm = measureChangeTerm + measureChange(3) + measureChange(4);
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
        end
        
        function out = discountFactor(lgm2F, startTime, endTime,numMatTime,modelStatesT)
           zcT = lgm2F.DF(endTime/365.0);
           
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           % deterministic discount factor
           if startTime <= 1e-8
                for i=1:stateSize
                    discountFactorOut(i) = zcT;
                end
                out = discountFactorOut;
                return;
           end
           
           % deterministic discount factor or maturity in the past
           if startTime >= endTime
                out = ones(1,stateSize);
                return;
           end
           
           zct = lgm2F.DF(startTime/365.0);
           beta = lgm2F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = lgm2F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i) ...
                   - beta(2) * modelStatesT(2,i)); 
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoff(lgm2F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = lgm2F.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = lgm2F.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(lgm2F,fromTime, toTime)
            
            out = eye(2,2);
        end
        
        function out = stateLocalVariance(lgm2F,fromTime, toTime)
            out = lgm2F.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(lgm2F,from,to)

            sig(1) = lgm2F.Alpha1;
            sig(2) = lgm2F.Alpha2;
            corr_rs = lgm2F.correl;
            corr(1,1) = 1.0;
            corr(2,1) = corr_rs.quote(1);
            corr(1,2) = corr(2,1);
            corr(2,2) = 1.0;
            
            alphaSize = length(lgm2F.Alpha1.quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i)
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i)
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           variances = zeros(2,2);
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                if (i< alphaSize) vol(2) = sig(2).quote(i);else vol(2) = sig(2).quote(alphaSize);end;
                correl(1,1) =1.0;
                correl(2,2) =1.0;
                if (i< alphaSize) correl(2,1) = corr_rs.quote(i);else correl(2,1) = corr_rs.quote(alphaSize);end;
                correl(1,2) = correl(2,1);
                for j=1:2
                    for k=1:2
                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end
        
        function out = H(lgm2F,t,dH)
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    break;
                end
            end
            tidx = i;
            sum = 0.0;
            prev_t = 0.0;
            curr_t = 0.0;
            for i=1:tidx
                if i< tidx
                    curr_t = dH.tenor(i);
                else
                    curr_t = t;
                end
                sum = sum + dH.quote(i)*(curr_t-prev_t);

                prev_t = dH.tenor(i);
            end
            out = sum;
        end
        
        function out = Alpha_Test(lgm2F,t,Alpha)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(Alpha.tenor)
                alpha = Alpha.quote(i);
                if t<= Alpha.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end
        
        function out = dH_Test(lgm2F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = dH.quote(i);
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = dH.quote(i);
            end
            out = sum;
        end

        function out = H_Test(lgm2F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = sum + dH.quote(i)*(t-prev_t);
                    inRangeYN = true;
                    break;
                else
                    sum = sum + dH.quote(i)*(dH.tenor(i)-prev_t);
                end
                prev_t = dH.tenor(i);
            end
            
            if ~inRangeYN 
                sum = sum + dH.quote(i) * (t - dH.tenor(i));
            end
            out = sum;
        end
        
        function c_Test = C_Test(lgm2F,tx, tnr,tau)
            pv01_1 = lgm2F.PV01(tx,tx+tnr,tau);
            fsr_1  = lgm2F.Fsr(tx,tx+tnr,tau);
            c_nN_1 = lgm2F.C_nN(tx,tx+tnr,tau,pv01_1,fsr_1);
            c_Test = c_nN_1;
        end

        function beta = Beta(lgm2F,t,T)
            beta =[];
            
            beta(1) = lgm2F.H(T,lgm2F.dH1)-lgm2F.H(t,lgm2F.dH1);
            beta(2) = lgm2F.H(T,lgm2F.dH2)-lgm2F.H(t,lgm2F.dH2);
            
        end
        
        function beta = BetaDate(lgm2F,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            beta =[];
            
            beta(1) = lgm2F.H(T,lgm2F.dH1)-lgm2F.H(t,lgm2F.dH1);
            beta(2) = lgm2F.H(T,lgm2F.dH2)-lgm2F.H(t,lgm2F.dH2);
            
        end
        
        function c_nN = C_nN(lgm2F,tn,tN,tau,pnN,ynN)
            p = @lgm2F.DF;
            c_nN=[0.0 0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm2F.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
        end
        
        function c_nN = C_nNDate(lgm2F,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @lgm2F.DF;
            c_nN=[0.0 0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = lgm2F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(2) = c_nN(2) +tau * beta(2) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            c_nN(2) = c_nN(2)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            c_nN(2) = c_nN(2) + beta(2)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            c_nN(2) = c_nN(2)/pnN;
        end
        
        function g_nN = G_nN(lgm2F,tn,tN,tau,pnN)
            p = @lgm2F.DF;
            g_nN=[0.0 0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm2F.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
        end
        
        function g_nN = G_nNDate(lgm2F,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @lgm2F.DF;
            g_nN=[0.0 0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = lgm2F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(2) = g_nN(2) +tau * beta(2) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
            g_nN(2) = g_nN(2)/pnN;
        end
        
        function h_nTerm = H_nTerm(lgm2F,tn,tTerm)
           h_nTerm =[];
           beta = lgm2F.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
        end
        
        function h_nTerm = H_nTermDate(lgm2F,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = lgm2F.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
           h_nTerm(2) = -1.0*beta(2);
        end
        
        function out = CMSDigitalArg(lgm2F,to,tp,tenor,tau)
           % probability that CMS rate > strike
           p_nN = lgm2F.PV01(to,to+tenor,tau);
           df_Tn = lgm2F.DF(to);
           df_TN = lgm2F.DF(to + tenor);
           y_nN = (df_Tn - df_TN)/p_nN;
           c_nN = lgm2F.C_nN(to,to + tenor,tau,p_nN,y_nN);
           g_nN = lgm2F.G_nN(to,to + tenor,tau,p_nN);
           h_nTerm = lgm2F.H_nTerm(to,tp);
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:2
               for j=1:2
                    driftTerm = driftTerm + c_nN(i) * (h_nTerm(j) + g_nN(j)) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:2
               for j=1:2
                    variance = variance + c_nN(i) * c_nN(j) * varCovar(i,j);
               end
           end
           
           out.mu = y_nN + driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN;
           out.varCovar = varCovar;
        end
        
        function out = CDDigitalArgDate(lgm2F,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           toTime =  toDate.DateDiff(valueDate);
           to      = toTime/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1Time = tN1Date.DateDiff(valueDate);
           tN1   =  tN1Time/365.0;
           
           deltaT = (tN1Time - toTime)/365.0;
           
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(tN1);
           
           c_nN1 = lgm2F.BetaDate(valueDate,toDate,tN1Date);
           c_nTerm = lgm2F.BetaDate(valueDate,toDate,tpDate);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:2
               for j=1:2
                    driftTerm = driftTerm + c_nN1(i) * c_nTerm(j) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:2
               for j=1:2
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
           out.mu = 0.5*variance  -1.0*log(df_TN1/df_Tn1) - driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
           out.deltaT = deltaT;
        end
        
        function out = CMSDigitalArgDate(lgm2F,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = lgm2F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = lgm2F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = lgm2F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = lgm2F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:2
               for j=1:2
                    driftTerm = driftTerm + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:2
               for j=1:2
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
           out.mu = y_nN1 + driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
        end
        
        function out = CMSDigital(lgm2F,to,tp,tenor,tau,strike)
           cmsDigitalArg= lgm2F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(lgm2F,to,tp,tenor,tau,lower,upper)
            lower_value = lgm2F.CMSDigital(to,tp,tenor,tau,lower);
            upper_value = lgm2F.CMSDigital(to,tp,tenor,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalArg(lgm2F,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           p_nN1 = lgm2F.PV01(to,to+tenor1,tau);
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = lgm2F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = lgm2F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = lgm2F.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = lgm2F.PV01(to,to+tenor2,tau);
           df_Tn2 = lgm2F.DF(to);
           df_TN2 = lgm2F.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = lgm2F.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = lgm2F.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = lgm2F.H_nTerm(to,tp);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:2
               for j=1:2
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1);
           out.sig2 = sqrt(variance2);
           
           out.pmu = driftTerm1 - driftTerm2;
           out.pmu1 = driftTerm1;
           out.pmu2 = driftTerm2;
           
           out.y1 = y_nN1;
           out.y2 = y_nN2;
           
           
        end
        
        function out = CMSSpreadDigitalGCArg(lgm2F,to,tp,tenor1,tenor2,tau,termCorr)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           p_nN1 = lgm2F.PV01(to,to+tenor1,tau);
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = lgm2F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = lgm2F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = lgm2F.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = lgm2F.PV01(to,to+tenor2,tau);
           df_Tn2 = lgm2F.DF(to);
           df_TN2 = lgm2F.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = lgm2F.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = lgm2F.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = lgm2F.H_nTerm(to,tp);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           covarianceTerm = 0.0;
           for i=1:2
               for j=1:2
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           covarianceTerm = variance1 + variance2 - 2.0*termCorr*sqrt(variance1)*sqrt(variance2);
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           
           out.sigma = sqrt(covarianceTerm);
           out.sigmaOld = sqrt(spreadVariance);
           
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = 1.0/(-2.0)*(coVarianceTerm-variance1-variance2)/sqrt(variance1*variance2);
           out.corrOld = coVariance/sqrt(variance1*variance2);
           
           out.sig1 = sqrt(variance1);
           out.sig2 = sqrt(variance2);
           
           out.pmu = driftTerm1 - driftTerm2;
           out.pmu1 = driftTerm1;
           out.pmu2 = driftTerm2;
           
           out.y1 = y_nN1;
           out.y2 = y_nN2;
           
           
        end
        
        function out = CMSSpreadDigitalArgDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = lgm2F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = lgm2F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = lgm2F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = lgm2F.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = lgm2F.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = lgm2F.DF(to);
           df_TN2 = lgm2F.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = lgm2F.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = lgm2F.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = lgm2F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:2
               for j=1:2
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigitalGCArgDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,termCorr)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = lgm2F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = lgm2F.DF(to);
           df_TN1 = lgm2F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = lgm2F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = lgm2F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = lgm2F.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = lgm2F.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = lgm2F.DF(to);
           df_TN2 = lgm2F.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = lgm2F.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = lgm2F.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = lgm2F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = lgm2F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:2
               for j=1:2
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           coVarianceTerm = 0.0;
           for i=1:2
               for j=1:2
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           coVarianceTerm = variance1 + variance2 - 2.0*termCorr*sqrt(variance1)*sqrt(variance2);
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(coVarianceTerm);
           out.sigmaOld = sqrt(spreadVariance);
           
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = 1.0/(-2.0)*(coVarianceTerm-variance1-variance2)/sqrt(variance1*variance2);
           out.corrOld = coVariance/sqrt(variance1*variance2);
           
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigital(lgm2F,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalGCDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike,termCorr)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalGCArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,termCorr);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSCMSSpreadDigitalDate(lgm2F,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,strike,strikeS)
           
           % probability that CMS tenor1 - CMS tenor 2 > strike
           % CMS 1
           cmsDigitalArg = lgm2F.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           c_nN = cmsDigitalArg.c_nN;
           varCovar = cmsDigitalArg.varCovar;
           
           %CMS Spread
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           
           c_nN1 = cmsSpreadDigitalArg.c_nN1;
           c_nN2 = cmsSpreadDigitalArg.c_nN2;
           
           muS = cmsSpreadDigitalArg.mu;
           sigmaS = cmsSpreadDigitalArg.sigma;
           
           % covariance between CMS Rate & CMS Spread
           covar = 0.0;
           for i=1:2
               for j=1:2
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           corr = covar/(sigma*sigmaS);
           
           z1 = (mu - strike)/sigma;
           z2 = (muS - strikeS)/sigmaS;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CDCMSSpreadDigitalDate(lgm2F,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,strike,strikeS)
           
           % probability that CD > strikeCD 
           % probability that CMS tenor1 - CMS tenor 2 > strike
           
           % CD
           cdDigitalArg = lgm2F.CDDigitalArgDate(valueDate,toDate,tpDate,tenor,tau);
           mu = cdDigitalArg.mu;
           sigma = cdDigitalArg.sigma;
           c_nN = cdDigitalArg.c_nN;
           varCovar = cdDigitalArg.varCovar;
           deltaT = cdDigitalArg.deltaT;
           
           %CMS Spread
           cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           
           c_nN1 = cmsSpreadDigitalArg.c_nN1;
           c_nN2 = cmsSpreadDigitalArg.c_nN2;
           
           muS = cmsSpreadDigitalArg.mu;
           sigmaS = cmsSpreadDigitalArg.sigma;
           
           % covariance between CMS Rate & CMS Spread
           covar = 0.0;
           for i=1:2
               for j=1:2
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           corr = covar/(sigma*sigmaS);
           
           z1 = (mu - log(1.0 + deltaT*strike))/sigma;
           z2 = (muS - strikeS)/sigmaS;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CMSSpreadDigitalRange(lgm2F,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm2F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = lgm2F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        
        function out = CMSSpreadDigitalRangeDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm2F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = lgm2F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalRangeGCDate(lgm2F,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper,termCorr)
            lower_value = lgm2F.CMSSpreadDigitalGCDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,termCorr);
            upper_value = lgm2F.CMSSpreadDigitalGCDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper,termCOrr);
            out = lower_value - upper_value;
        end
        
        function out = CMSCMSSpreadDigitalRangeDate(lgm2F,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = lgm2F.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = lgm2F.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = lgm2F.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = lgm2F.CMSCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,upperS);
            
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function out = CDCMSSpreadDigitalRangeDate(lgm2F,valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = lgm2F.CDCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = lgm2F.CDCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = lgm2F.CDCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = lgm2F.CDCMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor,tenor1,tenor2,tau,upper,upperS);
            
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function atmSwaption = ATMSwaption(lgm2F,tx,tnr)
            tau = 0.25;
            pv01 = lgm2F.PV01(tx,tx+tnr,tau);
            fsr  = lgm2F.Fsr(tx,tx+tnr,tau);
            c_nN = lgm2F.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = lgm2F.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:2
                for j=1:2
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        
        function spreadInfo = SpreadInfo(lgm2F,tx,tnr1,tnr2)
            tau = 0.25;
            cmsSpreadDigitalArg =  lgm2F.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function spreadInfo = SpreadInfoGC(lgm2F,tx,tnr1,tnr2)
            tau = 0.25;
            cmsSpreadDigitalArg =  lgm2F.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function impVariance = ImpVariance(lgm2F,tx,tnr1)
            tau = 0.25;
            pv01_1 = lgm2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            variances = lgm2F.LocalVariance(0,tx);
            var1 = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end
        
        function impCorrel = ImpCorrel(lgm2F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = lgm2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm2F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm2F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm2F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm2F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impCovar = ImpCovar(lgm2F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = lgm2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm2F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm2F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm2F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm2F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCovar = coVar;
        end
        
        function instFwdCorr = InsFwdCorr(lgm2F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = (lgm2F.H_Test(tx + tnr1 +0.001,lgm2F.dH1)-lgm2F.H_Test(tx + tnr1,lgm2F.dH1))/0.001;
            dH_tT1(2) = (lgm2F.H_Test(tx + tnr1 +0.001,lgm2F.dH2)-lgm2F.H_Test(tx + tnr1,lgm2F.dH2))/0.001;
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = (lgm2F.H_Test(tx + tnr2 +0.001,lgm2F.dH1)-lgm2F.H_Test(tx + tnr2,lgm2F.dH1))/0.001;
            dH_tT2(2) = (lgm2F.H_Test(tx + tnr2 +0.001,lgm2F.dH2)-lgm2F.H_Test(tx + tnr2,lgm2F.dH2))/0.001;
            
            instVariances = (lgm2F.LocalVariance(0,tx+0.001)-lgm2F.LocalVariance(0,tx))/0.001;
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + dH_tT1(j)*dH_tT1(k)*instVariances(j,k);
                    var2 = var2 + dH_tT2(j)*dH_tT2(k)*instVariances(j,k);
                    covar = covar + dH_tT1(j)*dH_tT2(k)*instVariances(j,k);
                end
            end
            instFwdCorr = covar/sqrt(var1*var2);
        end
        
        function instFwdCorrTest = InsFwdCorrTest(lgm2F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = lgm2F.dH_Test(tx + tnr1,lgm2F.dH1);
            dH_tT1(2) = lgm2F.dH_Test(tx + tnr1,lgm2F.dH2);
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = lgm2F.dH_Test(tx + tnr2,lgm2F.dH1);
            dH_tT2(2) = lgm2F.dH_Test(tx + tnr2,lgm2F.dH2);
            
            Alpha_t = zeros(2,1);
            Alpha_t(1) = lgm2F.Alpha_Test(tx,lgm2F.Alpha1);
            Alpha_t(2) = lgm2F.Alpha_Test(tx,lgm2F.Alpha2);
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:2
                var1 = var1 + dH_tT1(j)*dH_tT1(j)*Alpha_t(j)*Alpha_t(j);
                var2 = var2 + dH_tT2(j)*dH_tT2(j)*Alpha_t(j)*Alpha_t(j);
                covar = covar + dH_tT1(j)*dH_tT2(j)*Alpha_t(j)*Alpha_t(j);
            end
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function instFwdSwapCorrTest = InsSwapFwdCorrTest(lgm2F,tx,tnr1,tnr2)
            tau = 0.25;
            C_tT1 = zeros(2,1);
            pv01_1 = lgm2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
            C_tT1(2) = c_nN_1(2);
            
            C_tT2 = zeros(2,1);
            pv01_2 = lgm2F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm2F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm2F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
            C_tT2(2) = c_nN_2(2);
            
            Alpha_t = zeros(2,1);
            Alpha_t(1) = lgm2F.Alpha_Test(tx,lgm2F.Alpha1);
            Alpha_t(2) = lgm2F.Alpha_Test(tx,lgm2F.Alpha2);
            
            covar = 0.0;
            var1 = 0.0;
            var2 = 0.0;
            for j=1:2
                var1 = var1 + C_tT1(j)*C_tT1(j)*Alpha_t(j)*Alpha_t(j);
                var2 = var2 + C_tT2(j)*C_tT2(j)*Alpha_t(j)*Alpha_t(j);
                covar = covar + C_tT1(j)*C_tT2(j)*Alpha_t(j)*Alpha_t(j);
            end
            instFwdSwapCorrTest = covar/sqrt(var1*var2);
        end
        
        function instFwdCovarTest = InsFwdCovarTest(lgm2F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = lgm2F.dH_Test(tx + tnr1,lgm2F.dH1);
            dH_tT1(2) = lgm2F.dH_Test(tx + tnr1,lgm2F.dH2);
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = lgm2F.dH_Test(tx + tnr2,lgm2F.dH1);
            dH_tT2(2) = lgm2F.dH_Test(tx + tnr2,lgm2F.dH2);
            
            Alpha_t = zeros(2,1);
            Alpha_t(1) = lgm2F.Alpha_Test(tx,lgm2F.Alpha1);
            Alpha_t(2) = lgm2F.Alpha_Test(tx,lgm2F.Alpha2);
            
            covar = 0.0;
            for j=1:2
                covar = covar + dH_tT1(j)*dH_tT2(j)*Alpha_t(j)*Alpha_t(j);
            end
            instFwdCovarTest = covar;
        end
        
        function instFwdCovarTestMatrix = InsFwdCovarTest2(lgm2F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCovarTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),2);
                eta =  zeros(2,2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm2F.H_Test(newtnr(i) + tau,lgm2F.dH1) - lgm2F.H_Test(newtnr(i),lgm2F.dH1))/tau;
                    dH_tT(i,2) = (lgm2F.H_Test(newtnr(i) + tau,lgm2F.dH2) - lgm2F.H_Test(newtnr(i),lgm2F.dH2))/tau;
                end

                eta(1,1) = lgm2F.Alpha_Test(tx,lgm2F.Alpha1);
                eta(2,2) = lgm2F.Alpha_Test(tx,lgm2F.Alpha2);
                eta(1,2) = 0;
                eta(2,1) = 0;

                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
            end
            
        end
        
        function instFwdCorrTestMatrix = InsFwdCorrTest2(lgm2F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCorrTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),2);
                eta =  zeros(2,2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm2F.H_Test(newtnr(i) + tau,lgm2F.dH1) - lgm2F.H_Test(newtnr(i),lgm2F.dH1))/tau;
                    dH_tT(i,2) = (lgm2F.H_Test(newtnr(i) + tau,lgm2F.dH2) - lgm2F.H_Test(newtnr(i),lgm2F.dH2))/tau;
                end

                eta(1,1) = lgm2F.Alpha_Test(tx,lgm2F.Alpha1);
                eta(2,2) = lgm2F.Alpha_Test(tx,lgm2F.Alpha2);
                eta(1,2) = 0;
                eta(2,1) = 0;
                
                testLoadingD = dH_tT*eta;
                
                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
                %normalize the correlation matrix and calculate the loading matrix D
                rInv_m = zeros(length(newtnr),length(newtnr));
                for i=1:length(newtnr)
                    rInv_m(i,i) = 1.0/sqrt(coVar(i,i));
                end
                instFwdCorrTestMatrix = zeros(length(newtnr),length(newtnr));
                instFwdCorrTestMatrix = rInv_m*instFwdCovarTestMatrix*rInv_m';
                
            end
            
        end
        
        function instFwdCorr = InsFwdCorrTest3(lgm2F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = (lgm2F.H_Test(tx + tnr1 + tau,lgm2F.dH1)-lgm2F.H_Test(tx + tnr1,lgm2F.dH1))/tau;
            dH_tT1(2) = (lgm2F.H_Test(tx + tnr1 + tau,lgm2F.dH2)-lgm2F.H_Test(tx + tnr1,lgm2F.dH2))/tau;
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = (lgm2F.H_Test(tx + tnr2 + tau,lgm2F.dH1)-lgm2F.H_Test(tx + tnr2,lgm2F.dH1))/tau;
            dH_tT2(2) = (lgm2F.H_Test(tx + tnr2 + tau,lgm2F.dH2)-lgm2F.H_Test(tx + tnr2,lgm2F.dH2))/tau;
            
            instVariances = lgm2F.LocalVariance(0,tx);
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:2
                for k=1:2
                    var1 = var1 + dH_tT1(j)*dH_tT1(k)*instVariances(j,k);
                    var2 = var2 + dH_tT2(j)*dH_tT2(k)*instVariances(j,k);
                    covar = covar + dH_tT1(j)*dH_tT2(k)*instVariances(j,k);
                end
            end
            
            instFwdCorr = covar/sqrt(var1*var2);
            
        end
        
        function deltaHMatrix = DeltaHTest2(lgm2F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                deltaHMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                
                dH_tT = zeros(length(newtnr),2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm2F.H_Test(newtnr(i) + tau,lgm2F.dH1) - lgm2F.H_Test(newtnr(i),lgm2F.dH1))/tau;
                    dH_tT(i,2) = (lgm2F.H_Test(newtnr(i) + tau,lgm2F.dH2) - lgm2F.H_Test(newtnr(i),lgm2F.dH2))/tau;
                end

                deltaHMatrix = dH_tT;
            end
            
        end
        
        function etaMatrix = EtaTest2(lgm2F,tx)
            eta =  zeros(2,2);
            eta(1,1) = lgm2F.Alpha_Test(tx,lgm2F.Alpha1);
            eta(2,2) = lgm2F.Alpha_Test(tx,lgm2F.Alpha2);
            eta(1,2) = 0;
            eta(2,1) = 0;
            
            etaMatrix = eta;
            
        end
        
        function instFwdSwapCovarTest = InsSwapFwdCovarTest(lgm2F,tx,tnr1,tnr2)
            tau = 0.25;
            C_tT1 = zeros(2,1);
            pv01_1 = lgm2F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm2F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm2F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
            C_tT1(2) = c_nN_1(2);
            
            C_tT2 = zeros(2,1);
            pv01_2 = lgm2F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm2F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm2F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
            C_tT2(2) = c_nN_2(2);
            
            Alpha_t = zeros(2,1);
            Alpha_t(1) = lgm2F.Alpha_Test(tx,lgm2F.Alpha1);
            Alpha_t(2) = lgm2F.Alpha_Test(tx,lgm2F.Alpha2);
            
            covar = 0.0;
            for j=1:2
                covar = covar + C_tT1(j)*C_tT2(j)*Alpha_t(j)*Alpha_t(j);
            end
            instFwdSwapCovarTest = covar;
        end
        
        function out = RebonatoFwdCorr(lgm2F,tx,tnr1,tnr2)
           rho_inf = 0.4;
           rho_decay = 0.15;
           
%            rho_inf = 0.35;
%            rho_decay = 0.15;

%             rho_inf = 0.2;
%             rho_decay = 0.15; 
%             rho_decay = 0.15; 

%            rho_inf = 0.25;
%            rho_decay = 0.2;
           
%            rho_inf = 0.5847;
%            rho_decay = 0.021;
           
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function out = RebonatoFwdCorr3(lgm2F,tx,tnr1,tnr2,rho_inf,beta_0,beta_1)
           
           beta = beta_0*exp(-beta_1*min(tnr1,tnr2));
           out =  rho_inf + (1-rho_inf)*exp(-beta*abs(tnr1-tnr2));
        end
        
        function  out  = TargetFunctionLGM2FATMSwaptionSurfaceGlobalFmin(lgm2F,tvar,optParams)
        %     K_swaption = optParams.params.diagSwaption.strike;
        %     black_swaption = optParams.params.diagSwaption.blackPrice;
        %     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
        %     dt= 1.0/(optParams.params.freq);

            dH1 = optParams.params.dH1;
            dH2 = optParams.params.dH2;

            Alpha1 =optParams.params.Alpha1;
            Alpha2 =optParams.params.Alpha2;

            dH1_size = optParams.params.dH1_size;
            Alpha1_size = optParams.params.Alpha1_size;

            dH1Orig = optParams.params.dH1Orig;
            dH2Orig = optParams.params.dH2Orig;


        %     dH1 = optParams.params.dH1;
        %     Alpha1 =optParams.params.Alpha1;
        %     dH_size = optParams.dH_size;
        %     Alpha_size = optParams.Alpha_size;
        %     dH_compare = optParams.dH_compare;

        %     smooth_weight_Alpha = 1.0;
        %     smooth_weight_dH = 1.0;

            smooth_weight_Alpha = lgm2F.smoothAlpha;
            smooth_weight_dH = lgm2F.smoothdH;

            model_swaption = 0.0;
            market_swaption = 0.0;

            % assigning model parameters
            for j=1:dH1_size
                dH1.quote(j) = tvar(j);
                dH2.quote(j) = tvar(dH1_size + j);
            end

            for k=1:Alpha1_size
               Alpha1.quote(k) = tvar(2*dH1_size + k);
               Alpha2.quote(k) = tvar(2*dH1_size + Alpha1_size + k);
            end

            lgm2F.dH1.quote = dH1.quote; 
            lgm2F.dH2.quote = dH2.quote; 

            lgm2F.Alpha1.quote = Alpha1.quote;
            lgm2F.Alpha2.quote = Alpha2.quote; 


            % cost function for levenberg marquardt algorithm
%             out= zeros(optParams.numOfEquation,1);
            out= 0.0;
            
            idx =1;

            % constraint for dH
            for i=1:dH1_size
                out = out + smooth_weight_dH*(dH1.quote(i)/dH1Orig.quote(i) - 1.0)*smooth_weight_dH*(dH1.quote(i)/dH1Orig.quote(i) - 1.0);
            end

            for i=1:dH1_size
                out = out + smooth_weight_dH*(dH2.quote(i)/dH2Orig.quote(i) - 1.0)*smooth_weight_dH*(dH2.quote(i)/dH2Orig.quote(i) - 1.0);
            end

            % tikhonov constraint for Alpha
            for i=2:Alpha1_size-1
                out = out + smooth_weight_Alpha*(Alpha1.quote(i-1) + Alpha1.quote(i+1) - 2.0*Alpha1.quote(i))*smooth_weight_Alpha*(Alpha1.quote(i-1) + Alpha1.quote(i+1) - 2.0*Alpha1.quote(i));
            end

            for i=2:Alpha1_size-1
                out = out + smooth_weight_Alpha*(Alpha2.quote(i-1) + Alpha2.quote(i+1) - 2.0*Alpha2.quote(i))*(Alpha2.quote(i-1) + Alpha2.quote(i+1) - 2.0*Alpha2.quote(i));
            end

            for i=1:optParams.params.swaption_expiry_size
                for j=1:optParams.params.swaption_tenor_size
                    if optParams.params.swaption_calibration_yn(i,j) == 1
                        Ti = optParams.params.swaptionSurface.expiry(i);
                        Tj = optParams.params.swaptionSurface.tenor(j);
                        market_swaption = optParams.params.swaptionSurface.blackPrice(i,j);
                        model_swaption = lgm2F.ATMSwaption(Ti,Tj);
                        out = out + (model_swaption/market_swaption - 1.0)*(model_swaption/market_swaption - 1.0);
                    end
                end
            end

        end
        
        function  out  = TargetFunctionLGM2FATMSwaptionSurfaceGlobalFminDecorr(lgm2F,tvar,optParams)
        %     K_swaption = optParams.params.diagSwaption.strike;
        %     black_swaption = optParams.params.diagSwaption.blackPrice;
        %     swaption_expiry_use_yn = optParams.params.swaption_expiry_use_yn;
        %     dt= 1.0/(optParams.params.freq);

            dH1 = optParams.params.dH1;
            dH2 = optParams.params.dH2;

            Alpha1 =optParams.params.Alpha1;
            Alpha2 =optParams.params.Alpha2;

            dH1_size = optParams.params.dH1_size;
            Alpha1_size = optParams.params.Alpha1_size;

            dH1Orig = optParams.params.dH1Orig;
            dH2Orig = optParams.params.dH2Orig;


        %     dH1 = optParams.params.dH1;
        %     Alpha1 =optParams.params.Alpha1;
        %     dH_size = optParams.dH_size;
        %     Alpha_size = optParams.Alpha_size;
        %     dH_compare = optParams.dH_compare;

        %     smooth_weight_Alpha = 1.0;
        %     smooth_weight_dH = 1.0;

            smooth_weight_Alpha = lgm2F.smoothAlpha;
            smooth_weight_dH = lgm2F.smoothdH;

            model_swaption = 0.0;
            market_swaption = 0.0;

            % assigning model parameters
            for j=1:dH1_size
                dH1.quote(j) = tvar(j);
                dH2.quote(j) = tvar(dH1_size + j);
            end

            for k=1:Alpha1_size
               Alpha1.quote(k) = tvar(2*dH1_size + k);
               Alpha2.quote(k) = tvar(2*dH1_size + Alpha1_size + k);
            end

            lgm2F.dH1.quote = dH1.quote; 
            lgm2F.dH2.quote = dH2.quote; 

            lgm2F.Alpha1.quote = Alpha1.quote;
            lgm2F.Alpha2.quote = Alpha2.quote; 


            % cost function for levenberg marquardt algorithm
%             out= zeros(optParams.numOfEquation,1);
            out= 0.0;
            
            idx =1;

            % constraint for dH
            for i=1:dH1_size
                out = out + smooth_weight_dH*(dH1.quote(i)/dH1Orig.quote(i) - 1.0)*(dH1.quote(i)/dH1Orig.quote(i) - 1.0);
            end

            for i=1:dH1_size
                out = out + smooth_weight_dH*(dH2.quote(i)/dH2Orig.quote(i) - 1.0)*(dH2.quote(i)/dH2Orig.quote(i) - 1.0);
            end

            % tikhonov constraint for Alpha
            for i=2:Alpha1_size-1
                out = out + smooth_weight_Alpha*(Alpha1.quote(i-1) + Alpha1.quote(i+1) - 2.0*Alpha1.quote(i))*(Alpha1.quote(i-1) + Alpha1.quote(i+1) - 2.0*Alpha1.quote(i));
            end

            for i=2:Alpha1_size-1
                out = out + smooth_weight_Alpha*(Alpha2.quote(i-1) + Alpha2.quote(i+1) - 2.0*Alpha2.quote(i))*(Alpha2.quote(i-1) + Alpha2.quote(i+1) - 2.0*Alpha2.quote(i));
            end

            for i=1:optParams.params.swaption_expiry_size
                for j=1:optParams.params.swaption_tenor_size
                    if optParams.params.swaption_calibration_yn(i,j) == 1
                        Ti = optParams.params.swaptionSurface.expiry(i);
                        Tj = optParams.params.swaptionSurface.tenor(j);
                        market_swaption = optParams.params.swaptionSurface.blackPrice(i,j);
                        model_swaption = lgm2F.ATMSwaption(Ti,Tj);
                        out = out + (model_swaption/market_swaption - 1.0)*(model_swaption/market_swaption - 1.0);
                    end
                end
            end
            
            for i=1:optParams.params.swaption_expiry_size
                Ti = optParams.params.swaptionSurface.expiry(i);
                for j=1:optParams.params.swaption_tenor_size-2
                    Tj = optParams.params.swaptionSurface.expiry(j);
                    for k=j+1:optParams.params.swaption_tenor_size-1
                        Tk = optParams.params.swaptionSurface.tenor(k);
                        Tk1 = optParams.params.swaptionSurface.tenor(k+1);
                        
                        corrS = lgm2F.InsFwdCorrTest(Ti,Tj,Tk);
                        corrL = lgm2F.InsFwdCorrTest(Ti,Tj,Tk1);
                        out = out + exp(max(corrL-corrS,0))-1;
                    end
                end
            end

        end
        
        function out = TargetFunctionFwdLiborCorrGlobal(lgm2F,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            
            for j=1:length(lgm2F.dH1.quote)
               lgm2F.dH1.quote(j) = tvar(j);
               lgm2F.dH2.quote(j) = tvar(j+length(lgm2F.dH1.quote));
            end
            
            dH1Orig = params.dH1Orig;
            dH2Orig = params.dH2Orig;
    
            smooth_weight_dH = lgm2F.smoothdH;
            
            out= 0;
            % constraint for dH
            dH1_size = length(lgm2F.dH1.quote);
            for i=1:dH1_size
                out = out + smooth_weight_dH*(lgm2F.dH1.quote(i)/dH1Orig.quote(i) - 1.0)*(lgm2F.dH1.quote(i)/dH1Orig.quote(i) - 1.0);
            end

            for i=1:dH1_size
                out = out + smooth_weight_dH*(lgm2F.dH2.quote(i)/dH2Orig.quote(i) - 1.0)*(lgm2F.dH2.quote(i)/dH2Orig.quote(i) - 1.0);
            end
            
            lgm2FFwdLiborCorr = zeros(length(expiry),length(expiry));
            rebonatoCorr = zeros(length(expiry),length(expiry));
           
            tx = 0;
            tnr1 = expiry;
            tau = 0.25;
            lgm2FFwdLiborCorr = lgm2F.InsFwdCorrTest2(tx,tnr1,tau);
            for j=1:length(expiry)-1
                for k=j+1:length(expiry)
                    lgm2FFwdCorr(j,k) = lgm2FFwdLiborCorr(j,k);
                    rebonatoCorr(j,k) = lgm2F.RebonatoFwdCorr(tx,expiry(j),expiry(k));
                    out = out + (lgm2FFwdCorr(j,k)-rebonatoCorr(j,k))*(lgm2FFwdCorr(j,k)-rebonatoCorr(j,k));
                end
            end
        end
        
        function out = CalibrateToATMSwaptionSurfaceNormal(lgm2F,black,calibrationFlag,numOfIter)
            % Market Data Object , Vanilla Object
            % targetSwaption
            % targetCap
            
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% target instruments set up
            params.swaption_expiry_size = expirySize;
            params.swaption_tenor_size = tenorSize;
            
            params.swaption_calibration_yn = ones(expirySize,tenorSize);
            
            params.swaptionSurface.expiry = swaptionVolSurface.expiry;
            params.swaptionSurface.tenor  = swaptionVolSurface.tenor;
            params.swaptionSurface.quote  = zeros(expirySize,tenorSize);
            params.swaptionSurface.strike = zeros(expirySize,tenorSize);
            params.swaptionSurface.blackPrice = zeros(expirySize,tenorSize);
            
            % setting target diagonal swaption
            for i=1:expirySize
                for j=1:tenorSize
                    if params.swaption_calibration_yn(i,j) == 1
                        Ti = params.swaptionSurface.expiry(i);
                        Tj = params.swaptionSurface.tenor(j);
                        blackVolQuote = swaptionVolSurface.swaptionVol(expiry(i),tenor(j));
                        params.swaptionSurface.quote(i,j) = blackVolQuote;
                        params.swaptionSurface.strike(i,j) = black.Fsr(Ti,Ti + Tj,0.25);
%                         params.swaptionSurface.blackPrice(i,j) = black.BlackSwaptionWithVol(Ti,Tj,params.swaptionSurface.strike(i,j),blackVolQuote);
                        params.swaptionSurface.blackPrice(i,j) = black.NormalSwaptionWithVol(Ti,Tj,params.swaptionSurface.strike(i,j),blackVolQuote);
                    end
                end

            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            params.dH1 = lgm2F.dH1;
            params.dH2 = lgm2F.dH2;
            
            dH1_size = length(lgm2F.dH1.quote);
            Alpha1_size = length(lgm2F.Alpha1.quote);
            
            params.dH1_size = dH1_size;
            params.Alpha1_size = Alpha1_size;
            
            params.Alpha1 = lgm2F.Alpha1;
            params.Alpha2 = lgm2F.Alpha2;
            
            lgm2F.dH1    = params.dH1;
            lgm2F.dH2    = params.dH2;
            
            lgm2F.Alpha1 = params.Alpha1;
            lgm2F.Alpha2 = params.Alpha2;
            
            params.dH1Orig = lgm2F.dH1Orig;
            params.dH2Orig = lgm2F.dH2Orig;
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            out.dH1 = params.dH1;
            out.dH2 = params.dH2;
            
            out.Alpha1 = params.Alpha1;
            out.Alpha2 = params.Alpha2;
            
            out.blackSwaptionPrice = params.swaptionSurface.blackPrice;
            out.lgm2FSwaptionPrice = zeros(expirySize,tenorSize);
            out.swaptionPriceRelDiff = zeros(expirySize,tenorSize);
            out.swaptionPriceRmseTotal = 0.0;
           
            out.market_swaption = zeros(expirySize,tenorSize);
            out.model_swaption = zeros(expirySize,tenorSize);
            
            tvar = [];
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
                
            case 'global'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH    
                                              2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobal',tvar,x, 1000, options,'unc',lgm2F,optParams);
                    % test hsw 20180313
%                     import targetFunctions.TargetFunctionLGM2FATMSwaptionSurfaceGlobal;
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobal',tvar,x, numOfIter, options,'unc',lgm2F,optParams);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionNormalSurfaceGlobal',tvar,x, numOfIter, options,'unc',lgm2F,optParams);
                    
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobalAlphaAbs',tvar,x, 200, options,'unc',lgm2F,optParams);
                    
                    
                    toc
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
%                     for j=1:Alpha1_size
%                         out.Alpha1.quote(j) = popt(j + 2*dH1_size);
%                         out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
%                         lgm2F.Alpha1.quote(j) = popt(j + 2*dH1_size);
%                         lgm2F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
%                     end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = sqrt(popt(j + 2*dH1_size)*popt(j + 2*dH1_size));
                        out.Alpha2.quote(j) = sqrt(popt(j + 2*dH1_size + Alpha1_size)*popt(j + 2*dH1_size + Alpha1_size));
                        lgm2F.Alpha1.quote(j) = sqrt(popt(j + 2*dH1_size)*popt(j + 2*dH1_size));
                        lgm2F.Alpha2.quote(j) = sqrt(popt(j + 2*dH1_size + Alpha1_size)*popt(j + 2*dH1_size + Alpha1_size));
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
            
            otherwise
                    disp('unImplemented')
            end
            
        end
       
        function out = CalibrateToATMSwaptionSurface(lgm2F,black,calibrationFlag,numOfIter)
            % Market Data Object , Vanilla Object
            % targetSwaption
            % targetCap
            
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% target instruments set up
            params.swaption_expiry_size = expirySize;
            params.swaption_tenor_size = tenorSize;
            
            params.swaption_calibration_yn = ones(expirySize,tenorSize);
            
            params.swaptionSurface.expiry = swaptionVolSurface.expiry;
            params.swaptionSurface.tenor  = swaptionVolSurface.tenor;
            params.swaptionSurface.quote  = zeros(expirySize,tenorSize);
            params.swaptionSurface.strike = zeros(expirySize,tenorSize);
            params.swaptionSurface.blackPrice = zeros(expirySize,tenorSize);
            
            % setting target diagonal swaption
            for i=1:expirySize
                for j=1:tenorSize
                    if params.swaption_calibration_yn(i,j) == 1
                        Ti = params.swaptionSurface.expiry(i);
                        Tj = params.swaptionSurface.tenor(j);
                        blackVolQuote = swaptionVolSurface.swaptionVol(expiry(i),tenor(j));
                        params.swaptionSurface.quote(i,j) = blackVolQuote;
                        params.swaptionSurface.strike(i,j) = black.Fsr(Ti,Ti + Tj,0.25);
                        params.swaptionSurface.blackPrice(i,j) = black.BlackSwaptionWithVol(Ti,Tj,params.swaptionSurface.strike(i,j),blackVolQuote);
                    end
                end

            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            params.dH1 = lgm2F.dH1;
            params.dH2 = lgm2F.dH2;
            
            dH1_size = length(lgm2F.dH1.quote);
            Alpha1_size = length(lgm2F.Alpha1.quote);
            
            params.dH1_size = dH1_size;
            params.Alpha1_size = Alpha1_size;
            
            params.Alpha1 = lgm2F.Alpha1;
            params.Alpha2 = lgm2F.Alpha2;
            
            lgm2F.dH1    = params.dH1;
            lgm2F.dH2    = params.dH2;
            
            lgm2F.Alpha1 = params.Alpha1;
            lgm2F.Alpha2 = params.Alpha2;
            
            params.dH1Orig = lgm2F.dH1Orig;
            params.dH2Orig = lgm2F.dH2Orig;
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            out.dH1 = params.dH1;
            out.dH2 = params.dH2;
            
            out.Alpha1 = params.Alpha1;
            out.Alpha2 = params.Alpha2;
            
            out.blackSwaptionPrice = params.swaptionSurface.blackPrice;
            out.lgm2FSwaptionPrice = zeros(expirySize,tenorSize);
            out.swaptionPriceRelDiff = zeros(expirySize,tenorSize);
            out.swaptionPriceRmseTotal = 0.0;
           
            out.market_swaption = zeros(expirySize,tenorSize);
            out.model_swaption = zeros(expirySize,tenorSize);
            
            tvar = [];
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
                
            case 'global'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH    
                                              2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobal',tvar,x, 1000, options,'unc',lgm2F,optParams);
                    % test hsw 20180313
%                     import targetFunctions.TargetFunctionLGM2FATMSwaptionSurfaceGlobal;
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobal',tvar,x, numOfIter, options,'unc',lgm2F,optParams);
                    
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobalAlphaAbs',tvar,x, 200, options,'unc',lgm2F,optParams);
                    
                    
                    toc
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
%                     for j=1:Alpha1_size
%                         out.Alpha1.quote(j) = popt(j + 2*dH1_size);
%                         out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
%                         lgm2F.Alpha1.quote(j) = popt(j + 2*dH1_size);
%                         lgm2F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
%                     end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = sqrt(popt(j + 2*dH1_size)*popt(j + 2*dH1_size));
                        out.Alpha2.quote(j) = sqrt(popt(j + 2*dH1_size + Alpha1_size)*popt(j + 2*dH1_size + Alpha1_size));
                        lgm2F.Alpha1.quote(j) = sqrt(popt(j + 2*dH1_size)*popt(j + 2*dH1_size));
                        lgm2F.Alpha2.quote(j) = sqrt(popt(j + 2*dH1_size + Alpha1_size)*popt(j + 2*dH1_size + Alpha1_size));
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
            
            case 'globalAbs'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH    
                                              2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobal',tvar,x, 10000, options,'unc',lgm2F,optParams);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobalAbs',tvar,x, 10000, options,'unc',lgm2F,optParams);
                    
                    toc
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                        lgm2F.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        lgm2F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
                    
            case 'globalDecorr'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH    
                                              2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    degcons = (optParams.params.swaption_tenor_size-1)*(optParams.params.swaption_tenor_size-2)/2;
                    degcons = degcons * optParams.params.swaption_expiry_size;
                    x=  zeros(optParams.numOfEquation + degcons,1);
                    tic
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobalDecorr',tvar,x, 10000, options,'unc',lgm2F,optParams);
                    toc
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                        lgm2F.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        lgm2F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
            
            case 'globalFmin'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH    
                                              2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
%                     
%                     x=  zeros(optParams.numOfEquation,1);
%                     tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobal',tvar,x, 500, options,'unc',lgm2F,optParams);
%                     toc
                    
                    popt=  zeros(optParams.numOfEquation,1);
                    targetfun = @(tvar) TargetFunctionLGM2FATMSwaptionSurfaceGlobalFmin(lgm2F,tvar,optParams);
                    options=optimset('Algorithm','interior-point');
%                     options=optimset('Algorithm','trust-region-reflective','MaxFunEvals',700);
%                     [popt, fval] = fmincon(targetfun,tvar,[],[],[],[],[],[],[],options);
                    [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],[],[],[],options);
                    
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                        lgm2F.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        lgm2F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
                    
            case 'globalFminDecorr'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH    
                                              2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
%                     
%                     x=  zeros(optParams.numOfEquation,1);
%                     tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobal',tvar,x, 500, options,'unc',lgm2F,optParams);
%                     toc
                    
                    popt=  zeros(optParams.numOfEquation,1);
                    targetfun = @(tvar) TargetFunctionLGM2FATMSwaptionSurfaceGlobalFminDecorr(lgm2F,tvar,optParams);
                    options=optimset('Algorithm','interior-point');
%                     options=optimset('Algorithm','trust-region-reflective','MaxFunEvals',700);
%                     [popt, fval] = fmincon(targetfun,tvar,[],[],[],[],[],[],[],options);
                    [popt,fval,exitflag,output] = fmincon(targetfun,tvar,[],[],[],[],[],[],[],options);
                    
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                        lgm2F.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        lgm2F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
            
            case 'globaldHLiborVol'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH
                                              2*(dH1_size -2);   % tikhonov regularization gamma constraint
                                              
%                                               2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    x=  zeros(optParams.numOfEquation,1);
                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobal',tvar,x, 1000, options,'unc',lgm2F,optParams);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobaldHLiborVol',tvar,x, 1000, options,'unc',lgm2F,optParams);
                    
                    toc
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        out.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                        lgm2F.Alpha1.quote(j) = popt(j + 2*dH1_size);
                        lgm2F.Alpha2.quote(j) = popt(j + 2*dH1_size + Alpha1_size);
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
                    
                    out.LiborVol1 = zeros(dH1_size,1);
                    out.LiborVol2 = zeros(dH1_size,1);
                    out.LiborVol = zeros(dH1_size,1);
                    
                    for i=1:dH1_size
                        out.LiborVol1(i) = lgm2F.Alpha_Test(lgm2F.dH1.tenor(i),lgm2F.Alpha1)*lgm2F.dH1.tenor(i);
                        out.LiborVol2(i) = lgm2F.Alpha_Test(lgm2F.dH2.tenor(i),lgm2F.Alpha2)*lgm2F.dH2.tenor(i);
                        out.LiborVol(i) = sqrt(out.LiborVol1(i)*out.LiborVol1(i)+out.LiborVol2(i)*out.LiborVol2(i));
                    end

                    
            otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToFwdCorr(lgm2F,black,calibrationFlag)
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
%             expiry = [0,expiry];
            expiry = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];
            expirySize = size(expiry,1);
            tenorSize = size(tenor,2);
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            params.dH1Orig = lgm2F.dH1Orig;
            params.dH2Orig = lgm2F.dH2Orig;
            
            params.dH1 = lgm2F.dH1;
            params.dH2 = lgm2F.dH2;
            
            params.Alpha1 = lgm2F.Alpha1;
            params.Alpha2 = lgm2F.Alpha2;
            
            out.dH1 = params.dH1;
            out.dH2 = params.dH2;
            
            out.InsFwdCorr = zeros(expirySize, expirySize);
            out.Diff = zeros(expirySize,expirySize);
            out.DiffTotal=0.0;
            
            targetTenor = [2;5;10];
            targetMaturity = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15];
                    
            out.TermCorr = zeros(length(targetMaturity),length(targetTenor),length(targetTenor));
            out.TermCorrDiff = zeros(length(targetMaturity),length(targetTenor),length(targetTenor));
            out.TermCorrDiffTotal = 0;
            
            targetTermCorr = 0.92;
            
            tvar = [];
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
                
            case 'global'
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                        lb(j)= -10;
                        lb(length(lgm2F.dH1.quote)+j)= -10;
                        ub(j)= 10;
                        ub(length(lgm2F.dH1.quote)+j)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    
                    targetfun = @(tvar) TargetFunctionFwdCorrGlobal(lgm2F,tvar,params);
%                     options=optimset('Algorithm','interior-point');
                    options=optimset('Algorithm','trust-region-reflective','MaxFunEvals',700);
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for j=1:length(lgm2F.dH1.quote)
                        out.dH1.quote(j) = x(j);
                        out.dH2.quote(j) = x(j+length(lgm2F.dH1.quote));
                        lgm2F.dH1.quote(j) = x(j);
                        lgm2F.dH2.quote(j) = x(j+length(lgm2F.dH1.quote));
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize-1
                            for l=k+1:tenorSize
                                out.InsFwdCorr(j,k,l) = lgm2F.InsFwdCorrTest(expiry(j),tenor(k),tenor(l));
                                out.Diff(j,k,l) = out.InsFwdCorr(j,k,l)-lgm2F.RebonatoFwdCorr(expiry(j),tenor(k),tenor(l));
                                out.DiffTotal = out.DiffTotal + out.Diff(j,k,l)*out.Diff(j,k,l);
                                N=N+1;
                            end
                        end
                    end
                   
                    out.DiffTotal = sqrt(out.DiffTotal/N);
            case 'globalFwdLibor'
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                        lb(j)= 0.0001;
                        lb(length(lgm2F.dH1.quote)+j)= 0.0001;
                        ub(j)= 10;
                        ub(length(lgm2F.dH1.quote)+j)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    
                    targetfun = @(tvar) TargetFunctionFwdLiborCorrGlobal(lgm2F,tvar,params);
%                     options=optimset('Algorithm','interior-point');
%                     options=optimset('Algorithm','trust-region-reflective','MaxFunEvals',700);
                    [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],[]);
                    
                    for j=1:length(lgm2F.dH1.quote)
                        out.dH1.quote(j) = x(j);
                        out.dH2.quote(j) = x(j+length(lgm2F.dH1.quote));
                        lgm2F.dH1.quote(j) = x(j);
                        lgm2F.dH2.quote(j) = x(j+length(lgm2F.dH1.quote));
                    end
                    
                    tx = 0;
                    tnr1 = expiry;
                    tau = 0.25;    
                    lgm2FFwdLiborCorr = lgm2F.InsFwdCorrTest2(tx,tnr1,tau);
                    rebonatoCorr = zeros(length(expiry),length(expiry));
                    N=0;
                    for j=1:length(expiry)-1
                        for k=j+1:length(expiry)
                            out.lgm2FFwdCorr(j,k) = lgm2FFwdLiborCorr(j,k);
                            out.rebonatoCorr(j,k) = lgm2F.RebonatoFwdCorr(0,expiry(j),expiry(k));
                            out.Diff(j,k) = out.lgm2FFwdCorr(j,k)-out.rebonatoCorr(j,k);
                            out.DiffTotal = out.DiffTotal + out.Diff(j,k)*out.Diff(j,k);
                            N= N +1;
                        end
                    end
                   
                    out.DiffTotal = sqrt(out.DiffTotal/N);
            case 'globalFwdLiborLev'
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                        lb(j)= 0.0001;
                        lb(length(lgm2F.dH1.quote)+j)= 0.0001;
                        ub(j)= 10;
                        ub(length(lgm2F.dH1.quote)+j)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    
                    numOfEquation = 2*length(lgm2F.dH1.quote) + expirySize*(expirySize-1)/2;
                    x=  zeros(numOfEquation,1);
                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevGlobal',tvar,x, 1000, options,'unc',lgm2F,params);
%                     [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevGlobal',tvar,x, 1000, options,'unc',lgm2F,params);
%                     [ret, popt, info, covar]=levmar('targetFunctions.TargetFunctionFwdLiborCorrLevGlobal',tvar,x, 200, options,'unc',lgm2F,params);
                    [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevGlobal',tvar,x, 200, options,'unc',lgm2F,params);

                    toc
                    
%                     targetfun = @(tvar) TargetFunctionFwdLiborCorrGlobal(lgm2F,tvar,params);
%                     options=optimset('Algorithm','interior-point');
%                     options=optimset('Algorithm','trust-region-reflective','MaxFunEvals',700);
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],[]);
                    
                    for j=1:length(lgm2F.dH1.quote)
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                    end
                    
                    tx = 0;
                    tnr1 = expiry;
                    tau = 0.25;    
                    lgm2FFwdLiborCorr = lgm2F.InsFwdCorrTest2(tx,tnr1,tau);
                    rebonatoCorr = zeros(length(expiry),length(expiry));
                    N=0;
                    for j=1:length(expiry)-1
                        for k=j+1:length(expiry)
                            out.lgm2FFwdCorr(j,k) = lgm2FFwdLiborCorr(j,k);
                            out.rebonatoCorr(j,k) = lgm2F.RebonatoFwdCorr(0,expiry(j),expiry(k));
                            out.Diff(j,k) = out.lgm2FFwdCorr(j,k)-out.rebonatoCorr(j,k);
                            out.DiffTotal = out.DiffTotal + out.Diff(j,k)*out.Diff(j,k);
                            N = N+1;
                        end
                    end
                   
                    out.DiffTotal = sqrt(out.DiffTotal/N);
            case 'globalFwdLiborLevSimple'
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                        lb(j)= 0.0001;
                        lb(length(lgm2F.dH1.quote)+j)= 0.0001;
                        ub(j)= 10;
                        ub(length(lgm2F.dH1.quote)+j)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    
                    numOfEquation = expirySize*(expirySize-1)/2;
                    x=  zeros(numOfEquation,1);
                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevGlobal',tvar,x, 1000, options,'unc',lgm2F,params);
%                     [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevGlobal',tvar,x, 1000, options,'unc',lgm2F,params);
                    [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevGlobalSimple',tvar,x, 200, options,'unc',lgm2F,params);

                    toc
                    
%                     targetfun = @(tvar) TargetFunctionFwdLiborCorrGlobal(lgm2F,tvar,params);
%                     options=optimset('Algorithm','interior-point');
%                     options=optimset('Algorithm','trust-region-reflective','MaxFunEvals',700);
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],[]);
                    
                    for j=1:length(lgm2F.dH1.quote)
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                    end
                    
                    tx = 0;
                    tnr1 = expiry;
                    tau = 0.25;    
                    lgm2FFwdLiborCorr = lgm2F.InsFwdCorrTest2(tx,tnr1,tau);
                    rebonatoCorr = zeros(length(expiry),length(expiry));
                    N=0;
                    for j=1:length(expiry)-1
                        for k=j+1:length(expiry)
                            out.lgm2FFwdCorr(j,k) = lgm2FFwdLiborCorr(j,k);
                            out.rebonatoCorr(j,k) = lgm2F.RebonatoFwdCorr(0,expiry(j),expiry(k));
                            out.Diff(j,k) = out.lgm2FFwdCorr(j,k)-out.rebonatoCorr(j,k);
                            out.DiffTotal = out.DiffTotal + out.Diff(j,k)*out.Diff(j,k);
                            N = N+1;
                        end
                    end
                   
                    out.DiffTotal = sqrt(out.DiffTotal/N);
            case 'globalTermCorr'
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                        lb(j)= 0.0001;
                        lb(length(lgm2F.dH1.quote)+j)= 0.0001;
                        ub(j)= 10;
                        ub(length(lgm2F.dH1.quote)+j)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    
                    numOfEquation = 2*length(lgm2F.dH1.quote) + length(targetMaturity)*length(targetTenor)*(length(targetTenor)-1)/2;
                    x=  zeros(numOfEquation,1);
                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevGlobal',tvar,x, 1000, options,'unc',lgm2F,params);
                    [ret, popt, info, covar]=levmar('TargetFunctionTermCorrLevGlobal',tvar,x, 1000, options,'unc',lgm2F,params);

                    toc
   
                    
                    for j=1:length(lgm2F.dH1.quote)
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                    end
                    
                    
                    N=0;
                    for i=1:length(targetMaturity)
                        for j=1:length(targetTenor)-1
                            for k=j+1:length(targetTenor)
                                out.TermCorr(i,j,k) = lgm2F.ImpCorrel(targetMaturity(i),targetTenor(j),targetTenor(k));
                                out.TermCorrDiff(i,j,k) = out.TermCorr(i,j,k)-targetTermCorr;
                                out.TermCorrDiffTotal = out.TermCorrDiffTotal + out.TermCorrDiff(i,j,k)*out.TermCorrDiff(i,j,k);
                                N = N+1;
                            end
                        end
                    end
                    out.TermCorrDiffTotal = sqrt(out.TermCorrDiffTotal/N);
                    
            case 'globalFwdLiborLevDetail'
                    expiryDetail =  0.25:0.25:30;
                    
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                        lb(j)= 0.0001;
                        lb(length(lgm2F.dH1.quote)+j)= 0.0001;
                        ub(j)= 10;
                        ub(length(lgm2F.dH1.quote)+j)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    
                    
                    numOfEquation = 2*length(lgm2F.dH1.quote);
                    
                    numOfEquation = numOfEquation + length(expiryDetail)*(length(expiryDetail)-1)/2;
                    
                    x=  zeros(numOfEquation,1);
                    tic
%                     [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevGlobal',tvar,x, 1000, options,'unc',lgm2F,params);
                    [ret, popt, info, covar]=levmar('TargetFunctionFwdLiborCorrLevDetailGlobal',tvar,x, 1000, options,'unc',lgm2F,params);

                    toc
                    
%                     targetfun = @(tvar) TargetFunctionFwdLiborCorrGlobal(lgm2F,tvar,params);
%                     options=optimset('Algorithm','interior-point');
%                     options=optimset('Algorithm','trust-region-reflective','MaxFunEvals',700);
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],[]);
                    
                    for j=1:length(lgm2F.dH1.quote)
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                    end
                    
                    tx = 0;
                    tnr1 = expiry;
                    tau = 0.25;    
                    lgm2FFwdLiborCorr = lgm2F.InsFwdCorrTest2(tx,tnr1,tau);
                    rebonatoCorr = zeros(length(expiry),length(expiry));
                    N=0;
                    for j=1:length(expiry)-1
                        for k=j+1:length(expiry)
                            out.lgm2FFwdCorr(j,k) = lgm2FFwdLiborCorr(j,k);
                            out.rebonatoCorr(j,k) = lgm2F.RebonatoFwdCorr(0,expiry(j),expiry(k));
                            out.Diff(j,k) = out.lgm2FFwdCorr(j,k)-out.rebonatoCorr(j,k);
                            out.DiffTotal = out.DiffTotal + out.Diff(j,k)*out.Diff(j,k);
                            N = N+1;
                        end
                    end
                   
                    out.DiffTotal = sqrt(out.DiffTotal/N);
            
            otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToFwdCorr3(lgm2F,black,calibrationFlag,rho_inf,beta_0,beta_1)
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
%             expiry = [0,expiry];
            expiry = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15;16;17;18;19;20;22;25;27;30];
            expirySize = size(expiry,1);
            tenorSize = size(tenor,2);
            
            params.rho_inf = rho_inf;
            params.beta_0 = beta_0;
            params.beta_1 = beta_1;
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            params.dH1Orig = lgm2F.dH1Orig;
            params.dH2Orig = lgm2F.dH2Orig;
            
            params.dH1 = lgm2F.dH1;
            params.dH2 = lgm2F.dH2;
            
            params.Alpha1 = lgm2F.Alpha1;
            params.Alpha2 = lgm2F.Alpha2;
            
            out.dH1 = params.dH1;
            out.dH2 = params.dH2;
            
            out.InsFwdCorr = zeros(expirySize, expirySize);
            out.Diff = zeros(expirySize,expirySize);
            out.DiffTotal=0.0;
            
            targetTenor = [2;5;10];
            targetMaturity = [1;2;3;4;5;6;7;8;9;10;11;12;13;14;15];
                    
            out.TermCorr = zeros(length(targetMaturity),length(targetTenor),length(targetTenor));
            out.TermCorrDiff = zeros(length(targetMaturity),length(targetTenor),length(targetTenor));
            out.TermCorrDiffTotal = 0;
            
            targetTermCorr = 0.92;
            
            tvar = [];
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
            
            case 'globalFwdLiborLevRebonato3'
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                        lb(j)= 0.0001;
                        lb(length(lgm2F.dH1.quote)+j)= 0.0001;
                        ub(j)= 10;
                        ub(length(lgm2F.dH1.quote)+j)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    
                    numOfEquation = 2*length(lgm2F.dH1.quote) + expirySize*(expirySize-1)/2;
                    x=  zeros(numOfEquation,1);
                    tic
                    [ret, popt, info, covar]=levmar('targetFunctions.TargetFunctionFwdLiborCorrLevGlobalRebonato3',tvar,x, 200, options,'unc',lgm2F,params);
                    toc

                    for j=1:length(lgm2F.dH1.quote)
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j+length(lgm2F.dH1.quote));
                    end
                    
                    tx = 0;
                    tnr1 = expiry;
                    tau = 0.25;    
                    lgm2FFwdLiborCorr = lgm2F.InsFwdCorrTest2(tx,tnr1,tau);
                    rebonatoCorr = zeros(length(expiry),length(expiry));
                    N=0;
                    for j=1:length(expiry)-1
                        for k=j+1:length(expiry)
                            out.lgm2FFwdCorr(j,k) = lgm2FFwdLiborCorr(j,k);
                            out.rebonatoCorr(j,k) = lgm2F.RebonatoFwdCorr3(0,expiry(j),expiry(k),rho_inf,beta_0,beta_1);
                            out.Diff(j,k) = out.lgm2FFwdCorr(j,k)-out.rebonatoCorr(j,k);
                            out.DiffTotal = out.DiffTotal + out.Diff(j,k)*out.Diff(j,k);
                            N = N+1;
                        end
                    end
                   
                    out.DiffTotal = sqrt(out.DiffTotal/N);
            otherwise
                    disp('unImplemented')
            end
        end
       
        
        function out = FitToForwardCovariance(lgm2F,black,calibrationFlag)
            % Market Data Object , Vanilla Object
            % targetSwaption
            % targetCap
            
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% target instruments set up
            params.swaption_expiry_size = expirySize;
            params.swaption_tenor_size = tenorSize;
            
            params.swaption_calibration_yn = ones(expirySize,tenorSize);
            
            params.swaptionSurface.expiry = swaptionVolSurface.expiry;
            params.swaptionSurface.tenor  = swaptionVolSurface.tenor;
            params.swaptionSurface.quote  = zeros(expirySize,tenorSize);
            params.swaptionSurface.strike = zeros(expirySize,tenorSize);
            params.swaptionSurface.blackPrice = zeros(expirySize,tenorSize);
            
            % setting target diagonal swaption
            for i=1:expirySize
                for j=1:tenorSize
                    if params.swaption_calibration_yn(i,j) == 1
                        Ti = params.swaptionSurface.expiry(i);
                        Tj = params.swaptionSurface.tenor(j);
                        blackVolQuote = swaptionVolSurface.swaptionVol(expiry(i),tenor(j));
                        params.swaptionSurface.quote(i,j) = blackVolQuote;
                        params.swaptionSurface.strike(i,j) = black.Fsr(Ti,Ti + Tj,0.25);
                        params.swaptionSurface.blackPrice(i,j) = black.BlackSwaptionWithVol(Ti,Tj,params.swaptionSurface.strike(i,j),blackVolQuote);
                    end
                end

            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            params.dH1 = lgm2F.dH1;
            params.dH2 = lgm2F.dH2;
            
            dH1_size = length(lgm2F.dH1.quote);
            Alpha1_size = length(lgm2F.Alpha1.quote);
            
            params.dH1_size = dH1_size;
            params.Alpha1_size = Alpha1_size;
            
            params.Alpha1 = lgm2F.Alpha1;
            params.Alpha2 = lgm2F.Alpha2;
            
            lgm2F.dH1    = params.dH1;
            lgm2F.dH2    = params.dH2;
            
            lgm2F.Alpha1 = params.Alpha1;
            lgm2F.Alpha2 = params.Alpha2;
            
            params.dH1Orig = lgm2F.dH1Orig;
            params.dH2Orig = lgm2F.dH2Orig;
            
            params.expiry = expiry;
            params.tenor = tenor;
            params.expirySize = expirySize;
            params.tenorSize =  tenorSize;
            
            out.dH1 = params.dH1;
            out.dH2 = params.dH2;
            
            out.Alpha1 = params.Alpha1;
            out.Alpha2 = params.Alpha2;
            
            out.blackSwaptionPrice = params.swaptionSurface.blackPrice;
            out.lgm2FSwaptionPrice = zeros(expirySize,tenorSize);
            out.swaptionPriceRelDiff = zeros(expirySize,tenorSize);
            out.swaptionPriceRmseTotal = 0.0;
           
            out.market_swaption = zeros(expirySize,tenorSize);
            out.model_swaption = zeros(expirySize,tenorSize);
            
            tvar = [];
            
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            switch calibrationType
                
            case 'global'
                    
                    optParams.params = params;
                    optParams.modelParamsSize = 2*dH1_size + 2*Alpha1_size;
                    optParams.numOfEquation = params.swaption_expiry_size*params.swaption_tenor_size + ...  % target cap & swaption
                                              2*dH1_size + ...;     % constraint on dH    
                                              2*(Alpha1_size -2);   % tikhonov regularization gamma constraint
                                              
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    for j=1:length(lgm2F.dH1.quote)
                        tvar(j) = lgm2F.dH1.quote(j);
                        tvar(length(lgm2F.dH1.quote)+j) = lgm2F.dH2.quote(j);
                    end
                    
                    for k=1:length(lgm2F.Alpha1.quote)
                       tvar(2*length(lgm2F.dH1.quote)+k) =lgm2F.Alpha1.quote(k);
                       tvar(2*length(lgm2F.dH1.quote)+length(lgm2F.Alpha1.quote)+k) = lgm2F.Alpha2.quote(k);
                    end
                    % initial parameters costval
                    
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
                    
                    x=  zeros(optParams.numOfEquation,1);
                    tic
                    
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM2FATMSwaptionSurfaceGlobalAlphaAbs',tvar,x, 200, options,'unc',lgm2F,optParams);
                    
                    toc
                    for j=1:dH1_size
                        out.dH1.quote(j) = popt(j);
                        out.dH2.quote(j) = popt(j + dH1_size);
                        lgm2F.dH1.quote(j) = popt(j);
                        lgm2F.dH2.quote(j) = popt(j + dH1_size);
                    end
                    
                    for j=1:Alpha1_size
                        out.Alpha1.quote(j) = sqrt(popt(j + 2*dH1_size)*popt(j + 2*dH1_size));
                        out.Alpha2.quote(j) = sqrt(popt(j + 2*dH1_size + Alpha1_size)*popt(j + 2*dH1_size + Alpha1_size));
                        lgm2F.Alpha1.quote(j) = sqrt(popt(j + 2*dH1_size)*popt(j + 2*dH1_size));
                        lgm2F.Alpha2.quote(j) = sqrt(popt(j + 2*dH1_size + Alpha1_size)*popt(j + 2*dH1_size + Alpha1_size));
                    end
                    
                    N=0;
                    for i=1:expirySize
                        for j=1:tenorSize
                            Ti = params.swaptionSurface.expiry(i);
                            Tj = params.swaptionSurface.tenor(j);
                            out.market_swaption(i,j) = params.swaptionSurface.blackPrice(i,j);
                            out.model_swaption(i,j)  = lgm2F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i,j) = out.model_swaption(i,j)/out.market_swaption(i,j) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i,j)*out.swaptionPriceRelDiff(i,j);
                            N=N+1;
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
            otherwise
                disp('unimplemented')  
            end
        end
        
    end
    
end



In [ ]:
classdef LGM1FRisky < IRModelRisky
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
        % vol functor
        Alpha1
        dH1
        dH1Prev
        smoothdH
        smoothAlpha
        matu
        joint_swap_tenor
        targetSize
        freq
        numOfFactor
        modelType
    end
    
    methods
        function lgm1F = LGM1FRisky(params)
            if nargin > 0
                lgm1F.zeroCurve = params.zeroCurve;
                
                lgm1F.discountCurve = params.discountCurve;
                lgm1F.mktData = params.mktData;
                lgm1F.modelParams = params.modelParams;
                
                lgm1F.Alpha1 = params.modelParams('Alpha1');
                lgm1F.dH1 =  params.modelParams('dH1');
                lgm1F.matu =  params.modelParams('matu');
                lgm1F.joint_swap_tenor =  params.modelParams('joint_swap_tenor');
                lgm1F.dH1Prev =  params.modelParams('dH1Prev');
                lgm1F.smoothdH =  params.modelParams('smoothdH');
                lgm1F.smoothAlpha =  params.modelParams('smoothAlpha');
                lgm1F.targetSize = params.modelParams('targetSize');
                lgm1F.freq  =params.modelParams('freq');
                
                lgm1F.numOfFactor = 1;
                lgm1F.modelType = 'LGM1F';
            end
        end
        
        function out = SimpleSRPrOverHedge(lgm1F,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(lgm1F,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(lgm1F,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(lgm1F,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN
                for i =1:pathSize
                    outVec(i) = lgm1F.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = lgm1F.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = lgm1F.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out =  Libor_SimpleDate(lgm1F,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           %CMS
           fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
           fwdEndDate = fwdStartDate.AddDate(tenor,'year');
           fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
           deltaT = (fwdEndTime - fwdStartTime)/365.0;
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           
           stateSize = size(modelStatesT,2);
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           df_Tn = lgm1F.discountFactor(evalTime,fwdStartTime,numMatTime,modelStatesT);
           df_TN = lgm1F.discountFactor(evalTime,fwdEndTime,numMatTime,modelStatesT);
           
           liborOut = zeros(1,stateSize);
           
           for i=1:stateSize
                liborOut(i) = (df_Tn(i)/df_TN(i) - 1.0)/deltaT;
           end
           
           out = liborOut;
            
        end
        
         function out =  CMS_PSA_SimpleDate(lgm1F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
%            tau = 0.25;
           tau = 1.0/lgm1F.freq;
           % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           
           if abs(tenor- 0.25) < 1e-8
                tau = tenor;
           end
           
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = lgm1F.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = lgm1F.DF(to);
           df_TN = lgm1F.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = lgm1F.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = lgm1F.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = lgm1F.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = lgm1F.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:1
               for j=1:1
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + c_nN(1)*modelStatesT(1,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
         end
         
         function out = HtT_r(lgm1F,t,T)
            out =  lgm1F.H(T,lgm1F.dH1) - lgm1F.H(t,lgm1F.dH1); 
        end
        
         function out = fwdZcVariance(lgm1F, a, b, T1, T2)
            varMatrix = lgm1F.LocalVariance(a,b);
            
            localVariance = zeros(1,1);
            measureChange = zeros(1,1);
            
            localVariance(1) = (lgm1F.HtT_r(a,T1)*lgm1F.HtT_r(a,T1)-lgm1F.HtT_r(a,b)*lgm1F.HtT_r(a,b))*varMatrix(1,1);

            measureChange(1) = (lgm1F.HtT_r(a,T1)-lgm1F.HtT_r(a,b))*lgm1F.HtT_r(a,T2)*varMatrix(1,1);
           
            value = localVariance(1);
            measureChangeTerm = measureChange(1);
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
         end
         
         function out = discountFactorRisky(lgm1F, startTime, endTime,numMatTime,modelStatesT)
           zcT = lgm1F.DFRisky(endTime/365.0);
           zct = lgm1F.DFRisky(startTime/365.0);
           beta = lgm1F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = lgm1F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i));
           end
           
           out = discountFactorOut;
        end
         
        function out = discountFactor(lgm1F, startTime, endTime,numMatTime,modelStatesT)
           zcT = lgm1F.DF(endTime/365.0);
           
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           % deterministic discount factor
           if startTime <= 1e-8
                for i=1:stateSize
                    discountFactorOut(i) = zcT;
                end
                out = discountFactorOut;
                return;
           end
           
           % deterministic discount factor or maturity in the past
           if startTime >= endTime
                out = ones(1,stateSize);
                return;
           end
           
           zct = lgm1F.DF(startTime/365.0);
           beta = lgm1F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = lgm1F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i));
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoffRisky(lgm1F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = lgm1F.DFRisky(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = lgm1F.discountFactorRisky(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = discountPayoff(lgm1F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = lgm1F.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = lgm1F.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(lgm1F,fromTime, toTime)
            
            out = eye(1,1);
        end
        
        function out = stateLocalVariance(lgm1F,fromTime, toTime)
            out = lgm1F.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(lgm1F,from,to)

            sig(1) = lgm1F.Alpha1;
            alphaSize = length(lgm1F.Alpha1.quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i);
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i);
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = zeros(1,1);
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                correl =ones(1,1);
                for j=1:1
                    for k=1:1
                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end
        
        function out = dAlpha_Test(lgm1F,t,Alpha1)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(Alpha1.tenor)
                alpha = Alpha1.quote(i);
                if t<= Alpha1.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end
        
        function out = Alpha_Test(lgm1F,t,Alpha1)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(Alpha1.tenor)
                alpha = Alpha1.quote(i) * Alpha1.quote(i);
                if t<= Alpha1.tenor(i)
                    sum = sum + alpha*(t-prev_t);
                    inRangeYN = true;
                    break;
                else
                    sum = sum + alpha*(Alpha1.tenor(i)-prev_t);
                end
                prev_t = Alpha1.tenor(i);
            end
            
            if ~inRangeYN 
                sum = sum + alpha * (t - Alpha1.tenor(i));
            end
            out = sum;
        end
        
        function out = dH_Test(lgm1F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = dH.quote(i);
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = dH.quote(i);
            end
            out = sum;
        end
        
        function out = H_Test(lgm1F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = sum + dH.quote(i)*(t-prev_t);
                    inRangeYN = true;
                    break;
                else
                    sum = sum + dH.quote(i)*(dH.tenor(i)-prev_t);
                end
                prev_t = dH.tenor(i);
            end
            
            if ~inRangeYN 
                sum = sum + dH.quote(i) * (t - dH.tenor(i));
            end
            out = sum;
        end
        
        function out = H(lgm1F,t,dH)
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    break;
                end
            end
            tidx = i;
            sum = 0.0;
            prev_t = 0.0;
            curr_t = 0.0;
            for i=1:tidx
                if i< tidx
                    curr_t = dH.tenor(i);
                else
                    curr_t = t;
                end
                sum = sum + dH.quote(i)*(curr_t-prev_t);

                prev_t = dH.tenor(i);
            end
            out = sum;
        end


        function beta = Beta(lgm1F,t,T)
            beta =[];
            beta(1) = lgm1F.H(T,lgm1F.dH1)-lgm1F.H(t,lgm1F.dH1);
        end
        
        function beta = BetaDate(lgm1F,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            beta =[];
            
            beta(1) = lgm1F.H(T,lgm1F.dH1)-lgm1F.H(t,lgm1F.dH1);
            
        end
        
        
        function c_Test = C_Test(lgm1F,tx, tnr,tau)
            pv01_1 = lgm1F.PV01(tx,tx+tnr,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr,tau,pv01_1,fsr_1);
            c_Test = c_nN_1;
        end
        
        function c_nN = C_nN(lgm1F,tn,tN,tau,pnN,ynN)
            p = @lgm1F.DF;
            c_nN=[0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm1F.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            
        end
        
        function c_nN = C_nNDate(lgm1F,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @lgm1F.DF;
            c_nN=[0.0];
            prevTime = tn;
            deltaT = 0.0;
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                deltaT = tp - prevTime;
                beta = lgm1F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
%                 c_nN(1) = c_nN(1) +tau * beta(1) * df;
                c_nN(1) = c_nN(1) + deltaT * beta(1) * df;
                prevTime = tp;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
        end
        
        function g_nN = G_nN(lgm1F,tn,tN,tau,pnN)
            p = @lgm1F.DF;
            g_nN=[0.0];
            
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm1F.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
        end
        
        function g_nN = G_nNDate(lgm1F,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @lgm1F.DF;
            g_nN=[0.0];
            
            prevTime = tn;
            deltaT = 0.0;
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                deltaT = tp - prevTime;
                beta = lgm1F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
%                 g_nN(1) = g_nN(1) +tau * beta(1) * df;
                g_nN(1) = g_nN(1) + deltaT * beta(1) * df;
                prevTime = tp;
            end
            
            g_nN(1) = g_nN(1)/pnN;
        end
        
        function h_nTerm = H_nTerm(lgm1F,tn,tTerm)
           h_nTerm =[];
           beta = lgm1F.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
        end
        
        function h_nTerm = H_nTermDate(lgm1F,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = lgm1F.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
        end
        
        function out = CDDigitalArgDate(lgm1F,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           toTime =  toDate.DateDiff(valueDate);
           to      = toTime/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1Time = tN1Date.DateDiff(valueDate);
           tN1   =  tN1Time/365.0;
           
           deltaT = (tN1Time - toTime)/365.0;
           
           df_Tn1 = lgm1F.DF(to);
           df_TN1 = lgm1F.DF(tN1);
           
           c_nN1 = lgm1F.BetaDate(valueDate,toDate,tN1Date);
           c_nTerm = lgm1F.BetaDate(valueDate,toDate,tpDate);
           
           varCovar = lgm1F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:1
               for j=1:1
                    driftTerm = driftTerm + c_nN1(i) * c_nTerm(j) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:1
               for j=1:1
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
%            out.mu = 0.5*variance  -1.0*log(df_TN1/df_Tn1) - driftTerm;
           out.mu = -0.5*variance  + 1.0*log(df_TN1/df_Tn1) + driftTerm;
           
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
           out.deltaT = deltaT;
        end
        
        function out = CMSDigitalArg(lgm1F,to,tp,tenor,tau)
           % probability that CMS rate > strike
           p_nN = lgm1F.PV01(to,to+tenor,tau);
           df_Tn = lgm1F.DF(to);
           df_TN = lgm1F.DF(to + tenor);
           y_nN = (df_Tn - df_TN)/p_nN;
           c_nN = lgm1F.C_nN(to,to + tenor,tau,p_nN,y_nN);
           g_nN = lgm1F.G_nN(to,to + tenor,tau,p_nN);
           h_nTerm = lgm1F.H_nTerm(to,tp);
           varCovar = lgm1F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:1
               for j=1:1
                    driftTerm = driftTerm + c_nN(i) * (h_nTerm(j) + g_nN(j)) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:1
               for j=1:1
                    variance = variance + c_nN(i) * c_nN(j) * varCovar(i,j);
               end
           end
           
           out.mu = y_nN + driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN;
           out.varCovar = varCovar;
        end
        
        function out = CMSDigitalArgDate(lgm1F,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
            % if tenor is 0.25(3M) then use tenor(0.25) instead
           % model(zeroCurve)'s freq
           tauIn = tau;
           if abs(tenor1- 0.25) < 1e-8
                tauIn = tenor1;
           end
           
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = lgm1F.PV01Date(valueDate,toDate,tN1Date,tauIn);
           
           df_Tn1 = lgm1F.DF(to);
           df_TN1 = lgm1F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = lgm1F.C_nNDate(valueDate,toDate,tN1Date,tauIn,p_nN1,y_nN1);
           g_nN1 = lgm1F.G_nNDate(valueDate,toDate,tN1Date,tauIn,p_nN1);
           h_nTerm1 = lgm1F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = lgm1F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:1
               for j=1:1
                    driftTerm = driftTerm + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:1
               for j=1:1
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
           out.fwdCMS = y_nN1;
           out.mu = y_nN1 + driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
        end
        
        function out = CMSDigital(lgm1F,to,tp,tenor,tau,strike)
           cmsDigitalArg= lgm1F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalDate(lgm1F,valueDate,toDate,tpDate,tenor1,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = lgm1F.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor1,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(lgm1F,to,tp,tenor,tau,lower,upper)
            lower_value = lgm1F.CMSDigital(to,tp,tenor,tau,lower);
            upper_value = lgm1F.CMSDigital(to,tp,tenor,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSDigitalRangeDate(lgm1F,valueDate,toDate,tpDate,tenor1,tau,lower,upper)
            lower_value = lgm1F.CMSDigitalDate(valueDate,toDate,tpDate,tenor1,tau,lower);
            upper_value = lgm1F.CMSDigitalDate(valueDate,toDate,tpDate,tenor1,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CDDigitalDate(lgm1F,valueDate,toDate,tpDate,tenor,tau,strike)
           
           % probability that CD > strikeCD 
           % probability that CMS tenor1 - CMS tenor 2 > strike
           
           % CD
           cdDigitalArg = lgm1F.CDDigitalArgDate(valueDate,toDate,tpDate,tenor,tau);
           mu = cdDigitalArg.mu;
           sigma = cdDigitalArg.sigma;
           c_nN = cdDigitalArg.c_nN;
           varCovar = cdDigitalArg.varCovar;
           deltaT = cdDigitalArg.deltaT;
           
%            %CMS Spread
%            cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
%            
%            c_nN1 = cmsSpreadDigitalArg.c_nN1;
%            c_nN2 = cmsSpreadDigitalArg.c_nN2;
%            
%            muS = cmsSpreadDigitalArg.mu;
%            sigmaS = cmsSpreadDigitalArg.sigma;
%            
           % covariance between CMS Rate & CMS Spread
%            covar = 0.0;
%            for i=1:2
%                for j=1:2
%                     covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
%                end
%            end
           
%            corr = covar/(sigma*sigmaS);
           
           z1 = (mu + log(1.0 + deltaT*strike))/sigma;
%            z2 = (muS - strikeS)/sigmaS;
           if ~isreal(z1) || ~isfinite(z1)
               out = 0.0;
           else
               out = H_ncdf(z1);
           end
             
%            mbar = [0 0];
%            corrBar = [1.0 corr;corr 1.0];
%            xbar = [z1 z2];
%            out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CDDigitalRangeDate(lgm1F,valueDate,toDate,tpDate,tenor,tau,lower,upper)
%             lower_value = lgm1F.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,lower);
%             upper_value = lgm1F.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,upper);
%             out = lower_value - upper_value;
%             
            upper_value = lgm1F.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,upper);
            lower_value = lgm1F.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,lower);
            out = upper_value - lower_value;
            
        end
        
        function out = CMSSpreadDigitalArg(lgm1F,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           p_nN1 = lgm1F.PV01(to,to+tenor1,tau);
           df_Tn1 = lgm1F.DF(to);
           df_TN1 = lgm1F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = lgm1F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = lgm1F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = lgm1F.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = lgm1F.PV01(to,to+tenor2,tau);
           df_Tn2 = lgm1F.DF(to);
           df_TN2 = lgm1F.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = lgm1F.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = lgm1F.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = lgm1F.H_nTerm(to,tp);
           
           varCovar = lgm1F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:1
               for j=1:1
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:1
               for j=1:1
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigitalArgDate(lgm1F,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = lgm1F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = lgm1F.DF(to);
           df_TN1 = lgm21.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = lgm1F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = lgm1F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = lgm1F.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = lgm1F.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = lgm1F.DF(to);
           df_TN2 = lgm1F.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = lgm1F.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = lgm1F.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = lgm1F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = lgm1F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:1
               for j=1:1
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:1
               for j=1:1
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigital(lgm1F,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm1F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(lgm1F,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm1F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalRange(lgm1F,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm1F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = lgm1F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalRangeDate(lgm1F,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm1F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = lgm1F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSCMSSpreadCovar(lgm1F,c_nN,c_nN1,c_nN2,varCovar,varCovarS)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           covar = 0.0;
           for i=1:1
               for j=1:1
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           out = covar;
           
        end
        
        function out = CMSCMSSpreadDualDigital(lgm1F,to,tp,tenor,tenor1,tenor2,tau,strike,strikeS)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = lgm1F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           c_nN = cmsDigitalArg.c_nN;
           varCovar = cmsDigitalArg.varCovar;
           
           cmsSpreadDigitalArg = lgm1F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           muS = cmsSpreadDigitalArg.mu;
           sigmaS = cmsSpreadDigitalArg.sigma;
           c_nN1 = cmsSpreadDigitalArg.c_nN1;
           c_nN2 = cmsSpreadDigitalArg.c_nN2;
           varCovarS = cmsSpreadDigitalArg.varCovarS;
           
           % covariance between CMS Rate & CMS Spread
           covar = 0.0;
           for i=1:1
               for j=1:1
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           corr = covar/(sigma*sigmaS);
           
           z1 = (mu - strike)/sigma;
           z2 = (muS - strikeS)/sigmaS;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CMSCMSSpreadDualDigitalRange(lgm1F,to,tp,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,upper,upperS);
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function out = CMSDigitalRangeAccrual(lgm1F,matu,tenor,tau,coupon,lower,upper,tbd)
           periodSum = @(Ta,Tb,tbd) sum(arrayfun(@(i) ...
               1/tbd*CMSDigitalRange(lgm1F,Ta+1/tbd*i,Tb,tenor,tau,lower,upper),...
               1:round((Tb-Ta)*tbd)));
           p = @lgm1F.DF;
           dcf = 1.0;
           out.cashflowNpv = zeros(1,round(matu));
           out.price = 0.0;
           for i=1:round(matu)
               out.cashflowNpv(i) = periodSum((i-1)*0.25,i*0.25,tbd)*coupon*p(i*0.25)*NA;
               out.price = out.price + out.cashflowNpv(i);
           end
           out.cashflowNpv(i+1) = p(matu)*NA;
           out.price = out.price + p(matu)*NA;
        end
        
        function atmSwaption = ATMSwaption(lgm1F,tx,tnr)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            pv01 = lgm1F.PV01(tx,tx+tnr,tau);
            fsr  = lgm1F.Fsr(tx,tx+tnr,tau);
            c_nN = lgm1F.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = lgm1F.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:1
                for j=1:1
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        function cap = Cap(lgm1F,cap_strike,dt,Ti)
            idx_mat = round(Ti/dt);
            
            HN = 0.0;
            Hn = 0.0;
            sAlpha = 0.0;
            vol = 0.0;
            d1 = 0.0;
            
            p = @lgm1F.DF;
            model_cap = 0;
            for i=1:idx_mat-1
                Hn = lgm1F.H(i*dt,lgm1F.dH1);
                HN = lgm1F.H((i+1)*dt,lgm1F.dH1);
                sAlpha = lgm1F.Alpha_Test(i*dt,lgm1F.Alpha1);
                vol = (HN-Hn)*sqrt(sAlpha);
                d1 = log(p(i*dt)/(p((i+1)*dt)*(1 + cap_strike*dt)))/vol + 0.5*vol;
                model_cap = model_cap + p(i*dt)*H_ncdf(d1)-p((i+1)*dt)*(1 + cap_strike*dt)*H_ncdf(d1-vol);
            end
            cap = model_cap;
        end
        
%         function atmSwaption = modelSwaptionExpiryTenor(lgm1F,tx,tnr)
%             tau = 0.25;
%             pv01 = lgm1F.PV01(tx,tx+tnr,tau);
%             fsr  = lgm1F.Fsr(tx,tx+tnr,tau);
%             c_nN = lgm1F.C_nN(tx,tx+tnr,tau,pv01,fsr);
%             variances = lgm1F.LocalVariance(0,tx);
%             totalVar = 0.0;
%             for i=1:1
%                 for j=1:1
%                     totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
%                 end
%             end
%             atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
%         end
        
        function impCorrel = ImpCorrel(lgm1F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm1F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impCovar = ImpCovar(lgm1F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm1F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCovar = coVar;
        end
        
        function spreadInfo = SpreadInfo(ck2F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            cmsSpreadDigitalArg =  ck2F.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function impVariance = ImpVariance(lgm1F,tx,tnr1)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            variances = lgm1F.LocalVariance(0,tx);
            var1 = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end
        
        function instFwdCorr = InsFwdCorr(lgm1F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = (lgm1F.H_Test(tx + tnr1 +0.001,lgm1F.dH1)-lgm1F.H_Test(tx + tnr1,lgm1F.dH1))/0.001;
            dH_tT1(2) = (lgm1F.H_Test(tx + tnr1 +0.001,lgm1F.dH2)-lgm1F.H_Test(tx + tnr1,lgm1F.dH2))/0.001;
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = (lgm1F.H_Test(tx + tnr2 +0.001,lgm1F.dH1)-lgm1F.H_Test(tx + tnr2,lgm1F.dH1))/0.001;
            dH_tT2(2) = (lgm1F.H_Test(tx + tnr2 +0.001,lgm1F.dH2)-lgm1F.H_Test(tx + tnr2,lgm1F.dH2))/0.001;
            
            instVariances = (lgm1F.LocalVariance(0,tx+0.001)-lgm1F.LocalVariance(0,tx))/0.001;
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + dH_tT1(j)*dH_tT1(k)*instVariances(j,k);
                    var2 = var2 + dH_tT2(j)*dH_tT2(k)*instVariances(j,k);
                    covar = covar + dH_tT1(j)*dH_tT2(k)*instVariances(j,k);
                end
            end
            instFwdCorr = covar/sqrt(var1*var2);
        end
        
        function instFwdCorrTest = InsFwdCorrTest(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH1);
            dH_tT1(2) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH2);
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH1);
            dH_tT2(2) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH2);
            
            dAlpha_t = zeros(2,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
            dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:1
                var1 = var1 + dH_tT1(j)*dH_tT1(j)*dAlpha_t(j)*dAlpha_t(j);
                var2 = var2 + dH_tT2(j)*dH_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
                covar = covar + dH_tT1(j)*dH_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function instFwdCovarTestMatrix = InsFwdCovarTest2(lgm1F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCovarTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),2);
                eta =  zeros(2,2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH1) - lgm1F.H_Test(newtnr(i),lgm1F.dH1))/tau;
                    dH_tT(i,2) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH2) - lgm1F.H_Test(newtnr(i),lgm1F.dH2))/tau;
                end

                eta(1,1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
                eta(2,2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
                eta(1,2) = 0;
                eta(2,1) = 0;

                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
            end
            
        end
        
        function instFwdCorrTestMatrix = InsFwdCorrTest2(lgm1F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCorrTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),2);
                eta =  zeros(2,2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH1) - lgm1F.H_Test(newtnr(i),lgm1F.dH1))/tau;
                    dH_tT(i,2) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH2) - lgm1F.H_Test(newtnr(i),lgm1F.dH2))/tau;
                end

                eta(1,1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
                eta(2,2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
                eta(1,2) = 0;
                eta(2,1) = 0;
                
                testLoadingD = dH_tT*eta;
                
                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
                %normalize the correlation matrix and calculate the loading matrix D
                rInv_m = zeros(length(newtnr),length(newtnr));
                for i=1:length(newtnr)
                    rInv_m(i,i) = 1.0/sqrt(coVar(i,i));
                end
                instFwdCorrTestMatrix = zeros(length(newtnr),length(newtnr));
                instFwdCorrTestMatrix = rInv_m*instFwdCovarTestMatrix*rInv_m';
                
            end
            
        end
        
        function deltaHMatrix = DeltaHTest2(lgm1F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                deltaHMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                
                dH_tT = zeros(length(newtnr),2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH1) - lgm1F.H_Test(newtnr(i),lgm1F.dH1))/tau;
                    dH_tT(i,2) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH2) - lgm1F.H_Test(newtnr(i),lgm1F.dH2))/tau;
                end

                deltaHMatrix = dH_tT;
            end
            
        end
        
        function etaMatrix = EtaTest2(lgm1F,tx)
            eta =  zeros(1,1);
            eta(1,1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             eta(2,2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
%             eta(1,2) = 0;
%             eta(2,1) = 0;
            
            etaMatrix = eta;
            
        end
        
        function instFwdCovarTest = InsFwdCovarTest(lgm1F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            dH_tT1 = zeros(1,1);
            dH_tT1(1) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH1);
%             dH_tT1(2) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH2);
            
            dH_tT2 = zeros(1,1);
            dH_tT2(1) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH1);
%             dH_tT2(2) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH2);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            for j=1:1
                covar = covar + dH_tT1(j)*dH_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdCovarTest = covar;
        end
        
        function instFwdSwapCovarTest = InsSwapFwdCovarTest(lgm1F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            C_tT1 = zeros(1,1);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
%             C_tT1(2) = c_nN_1(2);
            
            C_tT2 = zeros(1,1);
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
%             C_tT2(2) = c_nN_2(2);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            for j=1:1
                covar = covar + C_tT1(j)*C_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdSwapCovarTest = covar;
        end
        
        function instFwdSwapCorrTest = InsSwapFwdCorrTest(lgm1F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            C_tT1 = zeros(1,1);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
%             C_tT1(2) = c_nN_1(2);
            
            C_tT2 = zeros(1,1);
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
%             C_tT2(2) = c_nN_2(2);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            var1 = 0.0;
            var2 = 0.0;
            for j=1:1
                var1 = var1 + C_tT1(j)*C_tT1(j)*dAlpha_t(j)*dAlpha_t(j);
                var2 = var2 + C_tT2(j)*C_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
                covar = covar + C_tT1(j)*C_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdSwapCorrTest = covar/sqrt(var1*var2);
        end
        
        function out = RebonatoFwdCorr2(lgm1F,tx,tnr1,tnr2)
%            tau = 0.25;
           tau = 1.0/(lgm1F.freq); 
           rho_inf = 0.35;
           rho_decay = 0.15;
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function out = RebonatoFwdCorr(lgm1F,tx,tnr1,tnr2)
           rho_inf = 0.35;
           rho_decay = 0.15;
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function coVariance = CoVariance(lgm1F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm1F.LocalVariance(0,tx);
            totalVar = zeros(1,1);
            for j=1:1
                for k=1:1
                    totalVar(1,1) = totalVar(1,1) + c_nN_1(j)*c_nN_2(k)*variances(j,k);
%                     totalVar(1,2) = totalVar(1,2) + c_nN_1(j)*c_nN_2(k)*variances(j,k);
%                     totalVar(2,1) = totalVar(2,1) + c_nN_2(j)*c_nN_1(k)*variances(j,k);
%                     totalVar(2,2) = totalVar(2,2) + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                end
            end
            coVariance =  totalVar;
        end
        
        function out = TargetFunctionATMSwaptionGlobal(lgm1F,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            lgm1FSwaptionPrice = zeros(1,length(tenor));
            
            for j=1:length(lgm1F.dH1.quote)
               lgm1F.dH1.quote(j) = tvar(j);
            end

            for k=1:length(lgm1F.Alpha1.quote)
               lgm1F.Alpha1.quote(k) = tvar(length(lgm1F.dH1.quote)+k);
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                    out = out + (lgm1FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*(lgm1FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*10000;
                end
            end
        end
        
        
        function out = TargetFunctionATMSwaptionGlobalLevMar(lgm1F,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            lgm1FSwaptionPrice = zeros(1,length(tenor));
            % tracking calibration
            
            
%             for j=1:2*length(lgm1F.dH1.quote)+2*length(lgm1F.Alpha1.quote)
%                 lgm1F.debugInfo{1,1}.currentParameterVec(idx,j) = tvar(j);
%             end

            for j=1:length(lgm1F.dH1.quote)
               lgm1F.dH1.quote(j) = tvar(j);
            end

            for k=1:length(lgm1F.Alpha1.quote)
               lgm1F.Alpha1.quote(k) = tvar(length(lgm1F.dH1.quote)+k);
            end
            
             
            
            
            sizeOut = length(expiry) * length(tenor);
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = lgm1FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
        end
        
        
        function out = CalibrateToATMDiagSwaption(lgm1F,black,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);
%             blackSwaptionVega =  zeros(expirySize,tenorSize);
            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j) = black.BlackATMSwaption(expiry(k),tenor(j));
%                     blackSwaptionVega(k,j) = black.BlackATMSwaptionVega(expiry(k),tenor(j));
                end
            end
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
%             params.expiry = expiry;
%             params.tenor = tenor;
%             params.blackSwaptionPrice = blackSwaptionPrice;
%             params.blackSwaptionVega  = blackSwaptionVega;
%             params.expirySize = expirySize;
%             params.tenorSize =  tenorSize;
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            target_size = lgm1F.targetSize;
            params.target_size = target_size;
            params.matu = lgm1F.matu;
            params.freq = lgm1F.freq;
            params.swaption_expiry_use_yn = zeros(target_size,1);
            params.dH1.tenor = 1:1:target_size;
            params.dH1.tenor = params.dH1.tenor';
            params.dH1.quote = ones(target_size,1);
            
            params.Alpha1.tenor = 1:1:target_size;
            params.Alpha1.tenor = params.Alpha1.tenor';
            params.Alpha1.quote = 0.01*ones(target_size,1);
            
            params.diagSwaption.expiry = zeros(target_size,1);
            params.diagSwaption.tenor  = zeros(target_size,1);
            params.diagSwaption.quote  = zeros(target_size,target_size);
            params.diagSwaption.strike = zeros(target_size,1);
            params.diagSwaption.blackPrice = zeros(target_size,1);
            
            if lgm1F.matu <= 2.0
                params.swaption_expiry_use_yn(1) = 1;
                params.diagSwaption.expiry =  ones(1,1);
                params.diagSwaption.expiry(1) =  swaptionVolSurface.expiry(1);
                params.diagSwaption.tenor =  ones(1,1);
                params.diagSwaption.tenor(1) =  swaptionVolSurface.tenor(1);
                params.diagSwaption.quote = ones(1,1);
                params.diagSwaption.quote(1,1) = swaptionVolSurface.swaptionVol(expiry(1),tenor(1));
            else
                temp_tenor   = zeros(target_size,1);
                temp_expiry = zeros(target_size,1);
                for i=2:target_size
                    temp_tenor(i-1)  = params.Alpha1.tenor(target_size - (i-1));
                    temp_expiry(i-1) = params.matu - temp_tenor(i-1); 
                end
                
                upper_expiry = swaptionVolSurface.expiry(end);
                upper_tenor  = swaptionVolSurface.tenor(end);
                
                lower_expiry = swaptionVolSurface.expiry(1);
                
                temp_idx= 1;
                for i=1:target_size
                    for j=temp_idx:target_size
                        % only those swaption whose expiry date is away.
                        if temp_expiry(j) > 0
                            params.dH1.tenor(i)     = temp_expiry(j);
                            params.Alpha1.tenor(i) = temp_expiry(j);
                            params.diagSwaption.expiry(i) = temp_expiry(j);
                            params.diagSwaption.tenor(i)  = temp_tenor(j);
                            if(temp_expiry(j) > upper_expiry && temp_tenor(j) <= upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(upper_expiry,temp_tenor(j));
                            elseif(temp_expiry(j) <= upper_expiry && temp_tenor(j) > upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),upper_tenor);    
                            else
                                if temp_expiry(j) < lower_expiry
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(lower_expiry,temp_tenor(j));
                                else
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),temp_tenor(j));
                                end
                            end
                            
                            params.swaption_expiry_use_yn(i) = 1;
                            temp_idx = j+1;
                            break;
                        end
                    end
                    
                    if i>=2
                        params.dH1.tenor(i)     = params.dH1.tenor(i-1)     + 1;
                        params.Alpha1.tenor(i) = params.Alpha1.tenor(i-1) + 1;
                    end
                end
                
            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            lgm1F.dH1    = params.dH1;
            lgm1F.Alpha1 = params.Alpha1;
            
          %% target swaption price and forward swap rate
            tau = 1.0/(lgm1F.freq);
            for i=1:target_size
                if params.swaption_expiry_use_yn(i)
                    Ti = params.diagSwaption.expiry(i);
                    Tj = params.diagSwaption.tenor(i);
                    blackVolQuote = params.diagSwaption.quote(i,i);
                    params.diagSwaption.strike(i) = black.Fsr(Ti,Ti + Tj,tau);
%                     params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVol(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote);
                    params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVolTau(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote,tau);
                end
            end
            
           %% target swaption price and forward swap rate
            % tracking calibration
            
            out.blackSwaptionPrice = params.diagSwaption.blackPrice;
            out.lgm1FSwaptionPrice = zeros(target_size,1);
            out.priceRelDiff = zeros(target_size,1);
            out.priceRmseTotal = 0.0;
            out.dH1 = params.dH1;
            out.Alpha1 = params.Alpha1;
            
            out.market_swaption = zeros(target_size,1);
            out.model_swaption = zeros(target_size,1);
            out.re = zeros(target_size,1);
            out.rmseTotal = 0;
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            targetSwaptionSize = 0;
            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    
                    for j=1:length(lgm1F.Alpha1.quote)
                        if params.swaption_expiry_use_yn(j) == 1
                            targetSwaptionSize = targetSwaptionSize +1;
                        end
                    end
                    
                    optParams.dH_compare = params.dH1.quote(1);
                    
                    if lgm1F.matu <= 2.0
                        optParams.dHSize = targetSwaptionSize;
                    else
                        optParams.dHSize = targetSwaptionSize + 1;
                    end
                    
                    optParams.AlphaSize = targetSwaptionSize;
                    
                    optParams.modelParamsSize = optParams.dHSize + optParams.AlphaSize;
                    
                    if lgm1F.matu <= 2.0
                        optParams.numOfEquation = targetSwaptionSize + ... % # of target swaption instruments
                                                        0 + ...           % # of Alpha term structure gamma constraints
                                                  optParams.dHSize;    % # of dH absolute error relative to 1.0 
                    else
                        optParams.numOfEquation = targetSwaptionSize + ...         % # of target swaption instruments
                                                  optParams.AlphaSize -2 + ...     % # of Alpha term structure gamma constraints
                                                  optParams.dHSize;                % # of dH absolute error relative to 1.0 
                    end
                    
                    out.modelParamsSize = optParams.modelParamsSize;
                    out.numOfEquation = optParams.numOfEquation;
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=1:optParams.dHSize
                        tvar(j) = params.dH1.quote(j);
                        lb(j)= 0.0001;
                        ub(j)= 10;
                    end
                    
                    for j=1:optParams.AlphaSize
                        tvar(j + optParams.dHSize) = params.Alpha1.quote(j);
                        lb(j + optParams.dHSize)= 0.0001;
                        ub(j + optParams.dHSize)= 10;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionLGM1FATMSwaptionDiagGlobal(lgm1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM1FATMSwaptionDiagGlobal',tvar,x, 200, options,'unc',lgm1F,optParams);
                    
                    
                    % assigning model parameters
                    k=1;
                    for j=1:optParams.dHSize
                       lgm1F.dH1.quote(j) = max(popt(j),0.0001);
                       k=k+1;
                    end

                    for j=k:length(lgm1F.dH1.quote)
                       lgm1F.dH1.quote(j) = lgm1F.dH1.quote(optParams.dHSize);
                    end

                    k = 1;
                    for j=1:optParams.AlphaSize
                       lgm1F.Alpha1.quote(j) = max(popt(optParams.dHSize+j),0.0001);
                       k=k+1;
                    end

                    % flat extrapolation of Alpha
                    % AlphaSize != length(Alpha)
                    for j=k:length(lgm1F.Alpha1.quote)
                       lgm1F.Alpha1.quote(j) = lgm1F.Alpha1.quote(optParams.AlphaSize);
                    end
                    
                    out.dH1.quote = lgm1F.dH1.quote;
                    out.Alpha1.quote = lgm1F.Alpha1.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        if params.swaption_expiry_use_yn(i) == 1
                            Ti = params.diagSwaption.expiry(i);
                            Tj = params.diagSwaption.tenor(i);
                            out.market_swaption(i) = params.diagSwaption.blackPrice(i);
                            out.model_swaption(i)  = lgm1F.ATMSwaption(Ti,Tj);
                            out.re(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                            out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                            N=N+1; 
                        end
                    end
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    
                case 'globalLevMar'
                    for j=1:length(lgm1F.dH1.quote)
                        tvar(j) = lgm1F.dH1.quote(j);
                        lb(j)= -10;
                        ub(j)= 10;
                    end

                    for k=1:length(lgm1F.Alpha1.quote)
                        tvar(length(lgm1F.dH1.quote)+k) = lgm1F.Alpha1.quote(k);
                        lb(length(lgm1F.dH1.quote)+k)= -10;
                        ub(length(lgm1F.dH1.quote)+k)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.init_lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                            out.init_relDiff(j,k) = out.init_lgm1FSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.init_rmseTotal = out.init_rmseTotal + out.init_relDiff(j,k)*out.init_relDiff(j,k);
                            N=N+1;
                        end
                    end
                    out.init_rmseTotal = sqrt(out.init_rmseTotal/N);
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalLevMar(lgm1F,tvar,params);
                    options=optimset('Algorithm','levenberg-marquardt','MaxFunEvals',700);
                    [x, fval] = lsqnonlin(targetfun,tvar,[],[],options);
%                     [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
%                     options=optimset('Algorithm','trust-region-reflective');
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for j=1:length(lgm1F.dH1.quote)
                        out.dH1.quote(j) = x(j);
                        lgm1F.dH1.quote(j) = x(j);
                    end
            
                    for k=1:length(lgm1F.Alpha1.quote)
                        out.Alpha1.quote(k) = x(length(lgm1F.dH1.quote)+k);
                        lgm1F.Alpha1.quote(k) = x(length(lgm1F.dH1.quote)+k);
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.lgm1FSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
            
                    
                case 'dummy'
                    
                    for j=1:length(lgm1F.dH1.quote)
                        out.dH1.quote(j) = lgm1F.dH1.quote(j);
                        out.dH2.quote(j) = lgm1F.dH2.quote(j);
                    end
            
                    for k=1:length(lgm1F.Alpha1.quote)
                        out.Alpha1.quote(k) =  lgm1F.Alpha1.quote(k);
                        out.dAlpha2.quote(k) = lgm1F.dAlpha2.quote(k);
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.lgm1FSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMCapDiagSwaption(lgm1F,black,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
            capVol = black.capVol;
            capVolData = black.mktData('capVol');
            capVolrawData =  capVolData.params('rawData');
            
            capVolTermStructure.expiry = capVolrawData(:,1);
            capVolTermStructure.capVol = capVol; % capVol With interpolator
            
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            target_size = lgm1F.targetSize;
            params.target_size = target_size;
            params.matu = lgm1F.matu;
            params.freq = lgm1F.freq;
            
            params.cap_expiry_use_yn = zeros(target_size,1);
            params.swaption_expiry_use_yn = zeros(target_size,1);
            
            params.dH1.tenor = 1:1:target_size;
            params.dH1.tenor = params.dH1.tenor';
            params.dH1.quote = 0.1*ones(target_size,1);
            
            params.Alpha1.tenor = 1:1:target_size;
            params.Alpha1.tenor = params.Alpha1.tenor';
            params.Alpha1.quote = 0.01*ones(target_size,1);
            
            params.diagSwaption.expiry = zeros(target_size,1);
            params.diagSwaption.tenor  = zeros(target_size,1);
            params.diagSwaption.quote  = zeros(target_size,target_size);
            params.diagSwaption.strike = zeros(target_size,1);
            params.diagSwaption.blackPrice = zeros(target_size,1);
            
            params.cap.expiry = zeros(target_size,1);
            params.cap.quote = zeros(target_size,1);
            params.cap.strike = zeros(target_size,1);
            params.cap.blackPrice = zeros(target_size,1);
            
            % setting target diagonal swaption 
            if lgm1F.matu <= 2.0
                params.swaption_expiry_use_yn(1) = 1;
                params.diagSwaption.expiry =  ones(1,1);
                params.diagSwaption.expiry(1) =  swaptionVolSurface.expiry(1);
                params.diagSwaption.tenor =  ones(1,1);
                params.diagSwaption.tenor(1) =  swaptionVolSurface.tenor(1);
                params.diagSwaption.quote = ones(1,1);
                params.diagSwaption.quote(1,1) = swaptionVolSurface.swaptionVol(expiry(1),tenor(1));
            else
                temp_tenor   = zeros(target_size,1);
                temp_expiry = zeros(target_size,1);
                for i=2:target_size
                    temp_tenor(i-1)  = params.Alpha1.tenor(target_size - (i-1));
                    temp_expiry(i-1) = params.matu - temp_tenor(i-1); 
                end
                
                upper_expiry = swaptionVolSurface.expiry(end);
                upper_tenor  = swaptionVolSurface.tenor(end);
                
                lower_expiry = swaptionVolSurface.expiry(1);
                
                temp_idx= 1;
                for i=1:target_size
                    for j=temp_idx:target_size
                        % only those swaption whose expiry date is away.
                        if temp_expiry(j) > 0
                            params.Alpha1.tenor(i) = temp_expiry(j);
                            params.diagSwaption.expiry(i) = temp_expiry(j);
                            params.diagSwaption.tenor(i)  = temp_tenor(j);
                            if(temp_expiry(j) > upper_expiry && temp_tenor(j) <= upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(upper_expiry,temp_tenor(j));
                            elseif(temp_expiry(j) <= upper_expiry && temp_tenor(j) > upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),upper_tenor);    
                            else
                                if temp_expiry(j) < lower_expiry
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(lower_expiry,temp_tenor(j));
                                else
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),temp_tenor(j));
                                end
                            end
                            
                            params.swaption_expiry_use_yn(i) = 1;
                            temp_idx = j+1;
                            break;
                        end
                    end
                    
                    if i>=2
                        params.Alpha1.tenor(i) = params.Alpha1.tenor(i-1) + 1;
                    end
                end
                
            end
            
            % setting target cap
            params.cap.expiry(1) = 1;
            params.cap.quote(1) = capVolTermStructure.capVol(params.cap.expiry(1));
            params.cap_expiry_use_yn(1) = 1;
            
            for i=2:target_size
                params.cap.expiry(i) = i;
                params.cap.quote(i) = capVolTermStructure.capVol(params.cap.expiry(i));
                if params.cap.expiry(i-1) < params.matu + 0.25
                    params.cap_expiry_use_yn(i) = 1;
                end
                if i == 10  % exclude 12Y, 15Y cap vol from target     
                    break;
                end
            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            lgm1F.dH1    = params.dH1;
            lgm1F.Alpha1 = params.Alpha1;
            
            tau = 1.0/(lgm1F.freq);
          %% target swaption price and forward swap rate
            for i=1:target_size
                if params.swaption_expiry_use_yn(i)
                    Ti = params.diagSwaption.expiry(i);
                    Tj = params.diagSwaption.tenor(i);
                    blackVolQuote = params.diagSwaption.quote(i,i);
                    params.diagSwaption.strike(i) = black.Fsr(Ti,Ti + Tj,tau);
%                     params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVol(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote);
                    params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVolTau(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote,tau);
                end
            end
            
           %% target cap price and forward ATM strike
            for i=1:target_size
                if params.cap_expiry_use_yn(i)
                    Ti = params.cap.expiry(i);
                    params.cap.strike(i) = black.Fsr(0.25,Ti,0.25);
                    params.cap.blackPrice(i) = black.BlackCapWithVol(Ti,params.cap.strike(i),0.25,params.cap.quote(i));
                end
            end
            
           %% target swaption price and forward swap rate
            % tracking calibration
            params.cap_size = target_size;
            params.swaption_size = target_size;
            
            out.cap_size = target_size;
            out.swaption_size = target_size;
            
            out.blackSwaptionPrice = params.diagSwaption.blackPrice;
            out.lgm1FSwaptionPrice = zeros(target_size,1);
            out.swaptionPriceRelDiff = zeros(target_size,1);
            out.swaptionPriceRmseTotal = 0.0;
            
            out.blackCapPrice = params.cap.blackPrice;
            out.lgm1FCapPrice = zeros(target_size,1);
            out.capPriceRelDiff = zeros(target_size,1);
            out.capPriceRmseTotal = 0.0;
            
            out.dH1 = params.dH1;
            out.Alpha1 = params.Alpha1;
            
            out.market_swaption = zeros(target_size,1);
            out.model_swaption = zeros(target_size,1);
            
            out.market_cap = zeros(target_size,1);
            out.model_cap = zeros(target_size,1);
            
%             out.re = zeros(target_size,1);
%             out.rmseTotal = 0;
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
%             targetSwaptionSize = 0;
%             targetCapSize = 0;
            
%             dH_size = 0;
%             Alpha_size = 0;
            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    optParams.dH_size =0;
                    optParams.Alpha_size =0;
                    
                    for j=1:out.cap_size
                        if params.cap_expiry_use_yn(j) == 1
                            optParams.dH_size = optParams.dH_size +1;
                        end
                    end
                    
                    for j=1:out.swaption_size
                        if params.swaption_expiry_use_yn(j) == 1
                            optParams.Alpha_size = optParams.Alpha_size +1;
                        end
                    end
                    
                    if lgm1F.matu <= 10.0
                        optParams.dH_size = optParams.dH_size - 1;
                        optParams.Alpha_size = optParams.Alpha_size + 1;
                    end
                    
                    optParams.dH_1Y_fixed_value = params.dH1.quote(1);
                    
                    optParams.modelParamsSize = optParams.dH_size + optParams.Alpha_size;
                    
                    optParams.numOfEquation = optParams.dH_size + optParams.Alpha_size; 
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=2:optParams.dH_size+1
                        tvar(j-1) = params.dH1.quote(j);
                        lb(j-1)= -10.0;
                        ub(j-1)= 10.0;
                    end
                    
                    for j=1:optParams.Alpha_size
                        tvar(j + optParams.dH_size) = params.Alpha1.quote(j);
                        lb(j + optParams.dH_size)= -10.0;
                        ub(j + optParams.dH_size)= 10.0;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionLGM1FATMSwaptionDiagGlobal(lgm1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM1FATMCapSwaptionDiagGlobal',tvar,x, 200, options,'unc',lgm1F,optParams);
                    
                    
                    % assigning model parameters
                    k=2;
                    for j=2:optParams.dH_size+1
                       lgm1F.dH1.quote(j) = max(abs(popt(j-1)),0.0001);
                       k=k+1;
                    end

                    for j=k:length(lgm1F.dH1.quote)
                       lgm1F.dH1.quote(j) = lgm1F.dH1.quote(optParams.dH_size+1);
                    end

                    k = 1;
                    for j=1:optParams.Alpha_size
                       lgm1F.Alpha1.quote(j) = max(abs(popt(optParams.dH_size+j)),0.0001);
                       k=k+1;
                    end

                    % flat extrapolation of Alpha
                    % AlphaSize != length(Alpha)
                    for j=k:length(lgm1F.Alpha1.quote)
                       lgm1F.Alpha1.quote(j) = lgm1F.Alpha1.quote(optParams.Alpha_size);
                    end
                    
                    out.dH1.quote = lgm1F.dH1.quote;
                    out.Alpha1.quote = lgm1F.Alpha1.quote;
                    
                    N=0;
                    for i=1:params.swaption_size
                        if params.swaption_expiry_use_yn(i) == 1
                            Ti = params.diagSwaption.expiry(i);
                            Tj = params.diagSwaption.tenor(i);
                            out.market_swaption(i) = params.diagSwaption.blackPrice(i);
                            out.model_swaption(i)  = lgm1F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i)*out.swaptionPriceRelDiff(i);
                            N=N+1; 
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
                    
                    N=0;
                    for i=1:params.cap_size
                        if params.cap_expiry_use_yn(i) == 1
                            Ti = params.cap.expiry(i);
                            out.market_cap(i) = params.cap.blackPrice(i);
                            out.model_cap(i)  = lgm1F.Cap(params.cap.strike(i),0.25,Ti);
                            out.capPriceRelDiff(i) = out.model_cap(i)/out.market_cap(i) - 1.0;
                            out.capPriceRmseTotal = out.capPriceRmseTotal + out.capPriceRelDiff(i)*out.capPriceRelDiff(i);
                            N=N+1; 
                        end
                    end
                    out.capPriceRmseTotal = sqrt(out.capPriceRmseTotal/N);
                    
                
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMSwaptionTenorDiagSwaption(lgm1F,black,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            % dummy
%             target_size = lgm1F.targetSize;

            params.matu = lgm1F.matu;
            params.freq = lgm1F.freq;
            params.joint_swap_tenor = lgm1F.joint_swap_tenor;
            
            params.dh_initial = 1.0;
            params.alpha_initial = 0.01;
            params.smallNumber = 0.000001;
            
            % set the target swaption expiry & tenor
            remainingMatu = params.matu - floor(params.matu);
            max_swaption_expiry_size = ceil(params.matu);
            
            if remainingMatu == 0
                remainingMatu = 1.0;
            end
            
            if params.matu -1  < params.joint_swap_tenor
                max_swaption_tenor_size = floor(params.joint_swap_tenor);
            else
                max_swaption_tenor_size = floor(params.matu - params.smallNumber);
            end
            
            params.targetSwaption.expiry = zeros(max_swaption_expiry_size,1);
            params.targetSwaption.tenor  = zeros(max_swaption_tenor_size,1);
            params.targetSwaption.quote  = zeros(max_swaption_expiry_size,max_swaption_tenor_size);
            params.swaption_use_yn = zeros(max_swaption_expiry_size,max_swaption_tenor_size);
            
            for i=1:max_swaption_expiry_size
                params.targetSwaption.expiry(i) = remainingMatu + i-1;
            end
            
            for i=1:max_swaption_tenor_size
                params.targetSwaption.tenor(i) = i;
            end
            
            params.target_size = 0;
            for i=1:max_swaption_expiry_size
                swaptionExpiry = params.targetSwaption.expiry(i);
                for j=1:max_swaption_tenor_size
                    swaptionTenor = params.targetSwaption.tenor(j);
                    diag_yn = 0;
                    tenor_yn = 0;
                    
                    if abs(swaptionExpiry + swaptionTenor - params.matu) < params.smallNumber 
                        diag_yn = 1;
                    end
                    
                    if (swaptionTenor == params.joint_swap_tenor) && ((swaptionExpiry <= params.matu) || abs(swaptionExpiry - params.matu) < params.smallNumber)
                        tenor_yn = 1;
                    end
                    
                    if (diag_yn) || (tenor_yn)
                        params.swaption_use_yn(i,j) = 1;
                        params.target_size = params.target_size +1;
                    end
                end
            end
            
            params.target_expiry_idx = zeros(params.target_size,1);
            params.target_tenor_idx = zeros(params.target_size,1);
            params.targetSwaption.strike = zeros(params.target_size,1);
            params.targetSwaption.blackPrice = zeros(params.target_size,1);
            
            idx = 1;
            for i=1:max_swaption_expiry_size
                for j=1:max_swaption_tenor_size
                    if params.swaption_use_yn(i,j) == 1
                        params.target_expiry_idx(idx) = i;
                        params.target_tenor_idx(idx) = j;
                        idx = idx +1;
                    end
                end
            end
            
            dh_size = floor( params.matu) + 1 + params.joint_swap_tenor + 1;
            params.dH1.tenor = zeros(dh_size,1);
            params.dH1.quote = zeros(dh_size,1);
            
            for i=1:dh_size
                params.dH1.tenor(i) = remainingMatu + i-1;
                params.dH1.quote(i) = params.dh_initial;
            end
            
            alpha_size = floor(params.matu - params.smallNumber) + 2;
            params.Alpha1.tenor = zeros(alpha_size,1);
            params.Alpha1.quote = zeros(alpha_size,1);
            
            for i=1:alpha_size
                params.Alpha1.tenor(i) = remainingMatu + i-1;
                params.Alpha1.quote(i) = params.alpha_initial;
            end
            
            tau = 1.0/(lgm1F.freq);
            upper_expiry = swaptionVolSurface.expiry(end);
            upper_tenor  = swaptionVolSurface.tenor(end);
            lower_expiry = swaptionVolSurface.expiry(1);
            
            for i=1:params.target_size
                expiry_idx = params.target_expiry_idx(i);
                tenor_idx = params.target_tenor_idx(i);
                swaptionExpiry = params.targetSwaption.expiry(expiry_idx);
                swaptionTenor = params.targetSwaption.tenor(tenor_idx);
                
                if(swaptionExpiry > upper_expiry && swaptionTenor <= upper_tenor)
                    params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(upper_expiry,swaptionTenor);
                elseif(swaptionExpiry <= upper_expiry && swaptionTenor > upper_tenor)
                    params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(swaptionExpiry,upper_tenor);    
                else
                    if swaptionExpiry < lower_expiry
                        params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(lower_expiry,swaptionTenor);
                    else
                        params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(swaptionExpiry,swaptionTenor);
                    end
                end
                
                params.targetSwaption.strike(i) = black.Fsr(swaptionExpiry,swaptionExpiry + swaptionTenor,tau);
%                 annuity = black.PV01(swaptionExpiry,swaptionExpiry + swaptionTenor,0.25);
                blackVolQuote = params.targetSwaption.quote(expiry_idx,tenor_idx); 
%                 params.targetSwaption.blackPrice(i) = black.BlackSwaptionWithVol(swaptionExpiry,swaptionTenor,params.targetSwaption.strike(i),blackVolQuote);
                params.targetSwaption.blackPrice(i) = black.BlackSwaptionWithVolTau(swaptionExpiry,swaptionTenor,params.targetSwaption.strike(i),blackVolQuote,tau);
            end
            
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            lgm1F.dH1    = params.dH1;
            lgm1F.Alpha1 = params.Alpha1;
            
          
            
           %% target swaption price and forward swap rate
            % tracking calibration
            
            params.swaption_size = params.target_size;
            out.swaption_size = params.target_size;
            
            out.blackSwaptionPrice = params.targetSwaption.blackPrice;
            out.lgm1FSwaptionPrice = zeros(params.target_size,1);
            
%             out.swaptionPriceRelDiff = zeros(params.target_size,1);
%             out.swaptionPriceRmseTotal = 0.0;
            
            out.re = zeros(params.target_size,1);
            out.rmseTotal = 0.0;
            
            out.dH1 = params.dH1;
            out.Alpha1 = params.Alpha1;
            
            out.market_swaption = zeros(params.target_size,1);
            out.model_swaption = zeros(params.target_size,1);
            
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;

            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    
                    optParams.dH_size = length(params.dH1.quote);  
                    optParams.Alpha_size = length(params.Alpha1.quote);
                    
                    % dH1(end-1) = dH1(end)
                    optParams.modelParamsSize = optParams.dH_size-1 + optParams.Alpha_size;
                    
                    optParams.numOfEquation = params.target_size + ...  % target swaption
                                              optParams.Alpha_size -2 + ... % tikhonov regularization gamma constraint
                                              optParams.dH_size;     % constraint on dH  
                    
                    
                    out.modelParamsSize = optParams.modelParamsSize;
                    out.numOfEquation = optParams.numOfEquation;
                    out.target_size = params.target_size;
                    out.Alpha_size = optParams.Alpha_size;
                    out.dH_size = optParams.dH_size;
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=1:optParams.dH_size-1
                        tvar(j) = params.dH1.quote(j);
                        lb(j)= -10.0;
                        ub(j)= 10.0;
                    end
                    
                    for j=1:optParams.Alpha_size
                        tvar(j + optParams.dH_size-1) = params.Alpha1.quote(j);
                        lb(j + optParams.dH_size-1)= -10.0;
                        ub(j + optParams.dH_size-1)= 10.0;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionLGM1FATMSwaptionDiagGlobal(lgm1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM1FATMSwaptionTenorSwaptionDiagGlobal',tvar,x, 200, options,'unc',lgm1F,optParams);
                    
                    
                    % assigning model parameters
                    for j=1:optParams.dH_size-1
                       lgm1F.dH1.quote(j) = max(abs(popt(j)),0.0001);
                    end
                    lgm1F.dH1.quote(optParams.dH_size) = lgm1F.dH1.quote(optParams.dH_size-1);
                    
                    for j=1:optParams.Alpha_size
                       lgm1F.Alpha1.quote(j) = max(abs(popt(optParams.dH_size-1 + j)),0.0001);
                    end

                    out.dH1.quote = lgm1F.dH1.quote;
                    out.Alpha1.quote = lgm1F.Alpha1.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        expiry_idx = params.target_expiry_idx(i);
                        tenor_idx = params.target_tenor_idx(i);
                        swaptionExpiry = params.targetSwaption.expiry(expiry_idx);
                        swaptionTenor = params.targetSwaption.tenor(tenor_idx);
                        out.market_swaption(i) = params.targetSwaption.blackPrice(i);
                        out.model_swaption(i) = lgm1F.ATMSwaption(swaptionExpiry,swaptionTenor);
                        out.re(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                        out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                        N=N+1;
                    end
                    
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    
                
                otherwise
                    disp('unImplemented')
            end
        end 
        

    end
    
end



In [ ]:
classdef LGM1FOld < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
         % vol functor
        Alpha1
        dH1
        smoothdH
        smoothAlpha
        matu
        joint_swap_tenor
        targetSize
        freq
        numOfFactor
        
    end
    
    methods
        function lgm1F = LGM1FOld(params)
            if nargin > 0
                
                lgm1F.zeroCurve = params.zeroCurve;
                
                lgm1F.mktData = params.mktData;
                lgm1F.modelParams = params.modelParams;
                
                lgm1F.Alpha1 = params.modelParams('Alpha1');
                lgm1F.dH1 =  params.modelParams('dH1');
                lgm1F.matu =  params.modelParams('matu');
                lgm1F.joint_swap_tenor =  params.modelParams('joint_swap_tenor');
                lgm1F.smoothdH =  params.modelParams('smoothdH');
                lgm1F.smoothAlpha =  params.modelParams('smoothAlpha');
                lgm1F.targetSize = params.modelParams('targetSize');
                lgm1F.freq  =params.modelParams('freq');
                
                lgm1F.numOfFactor = 1;
                
            end
        end
        
        function out = SimpleSRPrOverHedge(lgm1F,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(lgm1F,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(lgm1F,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(lgm1F,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN
                for i =1:pathSize
                    outVec(i) = lgm1F.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = lgm1F.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out =  Libor_SimpleDate(lgm1F,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           %CMS
           fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
           fwdEndDate = fwdStartDate.AddDate(tenor,'year');
           fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
           deltaT = (fwdEndTime - fwdStartTime)/365.0;
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           
           stateSize = size(modelStatesT,2);
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           df_Tn = lgm1F.discountFactor(evalTime,fwdStartTime,numMatTime,modelStatesT);
           df_TN = lgm1F.discountFactor(evalTime,fwdEndTime,numMatTime,modelStatesT);
           
           liborOut = zeros(1,stateSize);
           
           for i=1:stateSize
                liborOut(i) = (df_Tn(i)/df_TN(i) - 1.0)/deltaT;
           end
           
           out = liborOut;
            
        end
        
         function out =  CMS_PSA_SimpleDate(lgm1F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
           tau = 0.25;
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = lgm1F.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = lgm1F.DF(to);
           df_TN = lgm1F.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = lgm1F.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = lgm1F.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = lgm1F.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = lgm1F.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:1
               for j=1:1
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + c_nN(1)*modelStatesT(1,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
         end
         
         function out = HtT_r(lgm1F,t,T)
            out =  lgm1F.H(T,lgm1F.dH1) - lgm1F.H(t,lgm1F.dH1); 
        end
        
         function out = fwdZcVariance(lgm1F, a, b, T1, T2)
            varMatrix = lgm1F.LocalVariance(a,b);
            
            localVariance = zeros(1,1);
            measureChange = zeros(1,1);
            
            localVariance(1) = (lgm1F.HtT_r(a,T1)*lgm1F.HtT_r(a,T1)-lgm1F.HtT_r(a,b)*lgm1F.HtT_r(a,b))*varMatrix(1,1);

            measureChange(1) = (lgm1F.HtT_r(a,T1)-lgm1F.HtT_r(a,b))*lgm1F.HtT_r(a,T2)*varMatrix(1,1);
           
            value = localVariance(1);
            measureChangeTerm = measureChange(1);
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
         end
         
        function out = discountFactor(lgm1F, startTime, endTime,numMatTime,modelStatesT)
           zcT = lgm1F.DF(endTime/365.0);
           
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           % deterministic discount factor
           if startTime <= 1e-8
                for i=1:stateSize
                    discountFactorOut(i) = zcT;
                end
                out = discountFactorOut;
                return;
           end
           
           % deterministic discount factor or maturity in the past
           if startTime >= endTime
                out = ones(1,stateSize);
                return;
           end
           
           zct = lgm1F.DF(startTime/365.0);
           beta = lgm1F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = lgm1F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i));
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoff(lgm1F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = lgm1F.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = lgm1F.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(lgm1F,fromTime, toTime)
            
            out = eye(1,1);
        end
        
        function out = stateLocalVariance(lgm1F,fromTime, toTime)
            out = lgm1F.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(lgm1F,from,to)

            sig(1) = lgm1F.Alpha1;
            alphaSize = length(lgm1F.Alpha1.quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i);
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i);
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = zeros(1,1);
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                correl =ones(1,1);
                for j=1:1
                    for k=1:1
                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end
        
        function out = dAlpha_Test(lgm1F,t,Alpha1)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(Alpha1.tenor)
                alpha = Alpha1.quote(i);
                if t<= Alpha1.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end
        
        function out = Alpha_Test(lgm1F,t,Alpha1)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(Alpha1.tenor)
                alpha = Alpha1.quote(i) * Alpha1.quote(i);
                if t<= Alpha1.tenor(i)
                    sum = sum + alpha*(t-prev_t);
                    inRangeYN = true;
                    break;
                else
                    sum = sum + alpha*(Alpha1.tenor(i)-prev_t);
                end
                prev_t = Alpha1.tenor(i);
            end
            
            if ~inRangeYN 
                sum = sum + alpha * (t - Alpha1.tenor(i));
            end
            out = sum;
        end
        
        function out = dH_Test(lgm1F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = dH.quote(i);
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = dH.quote(i);
            end
            out = sum;
        end
        
        function out = H_Test(lgm1F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = sum + dH.quote(i)*(t-prev_t);
                    inRangeYN = true;
                    break;
                else
                    sum = sum + dH.quote(i)*(dH.tenor(i)-prev_t);
                end
                prev_t = dH.tenor(i);
            end
            
            if ~inRangeYN 
                sum = sum + dH.quote(i) * (t - dH.tenor(i));
            end
            out = sum;
        end
        
        function out = H(lgm1F,t,dH)
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    break;
                end
            end
            tidx = i;
            sum = 0.0;
            prev_t = 0.0;
            curr_t = 0.0;
            for i=1:tidx
                if i< tidx
                    curr_t = dH.tenor(i);
                else
                    curr_t = t;
                end
                sum = sum + dH.quote(i)*(curr_t-prev_t);

                prev_t = dH.tenor(i);
            end
            out = sum;
        end


        function beta = Beta(lgm1F,t,T)
            beta =[];
            beta(1) = lgm1F.H(T,lgm1F.dH1)-lgm1F.H(t,lgm1F.dH1);
        end
        
        function c_Test = C_Test(lgm1F,tx, tnr,tau)
            pv01_1 = lgm1F.PV01(tx,tx+tnr,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr,tau,pv01_1,fsr_1);
            c_Test = c_nN_1;
        end
        
        function c_nN = C_nN(lgm1F,tn,tN,tau,pnN,ynN)
            p = @lgm1F.DF;
            c_nN=[0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm1F.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            
        end
        
        function g_nN = G_nN(lgm1F,tn,tN,tau,pnN)
            p = @lgm1F.DF;
            g_nN=[0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm1F.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
        end
        
        function h_nTerm = H_nTerm(lgm1F,tn,tTerm)
           h_nTerm =[];
           beta = lgm1F.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
        end
        
        function out = CMSDigitalArg(lgm1F,to,tp,tenor,tau)
           % probability that CMS rate > strike
           p_nN = lgm1F.PV01(to,to+tenor,tau);
           df_Tn = lgm1F.DF(to);
           df_TN = lgm1F.DF(to + tenor);
           y_nN = (df_Tn - df_TN)/p_nN;
           c_nN = lgm1F.C_nN(to,to + tenor,tau,p_nN,y_nN);
           g_nN = lgm1F.G_nN(to,to + tenor,tau,p_nN);
           h_nTerm = lgm1F.H_nTerm(to,tp);
           varCovar = lgm1F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:1
               for j=1:1
                    driftTerm = driftTerm + c_nN(i) * (h_nTerm(j) + g_nN(j)) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:1
               for j=1:1
                    variance = variance + c_nN(i) * c_nN(j) * varCovar(i,j);
               end
           end
           
           out.mu = y_nN + driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN;
           out.varCovar = varCovar;
        end
        
        function out = CMSDigital(lgm1F,to,tp,tenor,tau,strike)
           cmsDigitalArg= lgm1F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(lgm1F,to,tp,tenor,tau,lower,upper)
            lower_value = lgm1F.CMSDigital(to,tp,tenor,tau,lower);
            upper_value = lgm1F.CMSDigital(to,tp,tenor,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalArg(lgm1F,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           p_nN1 = lgm1F.PV01(to,to+tenor1,tau);
           df_Tn1 = lgm1F.DF(to);
           df_TN1 = lgm1F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = lgm1F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = lgm1F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = lgm1F.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = lgm1F.PV01(to,to+tenor2,tau);
           df_Tn2 = lgm1F.DF(to);
           df_TN2 = lgm1F.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = lgm1F.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = lgm1F.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = lgm1F.H_nTerm(to,tp);
           
           varCovar = lgm1F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:1
               for j=1:1
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:1
               for j=1:1
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigital(lgm1F,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm1F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalRange(lgm1F,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm1F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = lgm1F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSCMSSpreadCovar(lgm1F,c_nN,c_nN1,c_nN2,varCovar,varCovarS)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           covar = 0.0;
           for i=1:1
               for j=1:1
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           out = covar;
           
        end
        
        function out = CMSCMSSpreadDualDigital(lgm1F,to,tp,tenor,tenor1,tenor2,tau,strike,strikeS)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = lgm1F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           c_nN = cmsDigitalArg.c_nN;
           varCovar = cmsDigitalArg.varCovar;
           
           cmsSpreadDigitalArg = lgm1F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           muS = cmsSpreadDigitalArg.mu;
           sigmaS = cmsSpreadDigitalArg.sigma;
           c_nN1 = cmsSpreadDigitalArg.c_nN1;
           c_nN2 = cmsSpreadDigitalArg.c_nN2;
           varCovarS = cmsSpreadDigitalArg.varCovarS;
           
           % covariance between CMS Rate & CMS Spread
           covar = 0.0;
           for i=1:1
               for j=1:1
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           corr = covar/(sigma*sigmaS);
           
           z1 = (mu - strike)/sigma;
           z2 = (muS - strikeS)/sigmaS;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CMSCMSSpreadDualDigitalRange(lgm1F,to,tp,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,upper,upperS);
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function out = CMSDigitalRangeAccrual(lgm1F,matu,tenor,tau,coupon,lower,upper,tbd)
           periodSum = @(Ta,Tb,tbd) sum(arrayfun(@(i) ...
               1/tbd*CMSDigitalRange(lgm1F,Ta+1/tbd*i,Tb,tenor,tau,lower,upper),...
               1:round((Tb-Ta)*tbd)));
           p = @lgm1F.DF;
           dcf = 1.0;
           out.cashflowNpv = zeros(1,round(matu));
           out.price = 0.0;
           for i=1:round(matu)
               out.cashflowNpv(i) = periodSum((i-1)*0.25,i*0.25,tbd)*coupon*p(i*0.25)*NA;
               out.price = out.price + out.cashflowNpv(i);
           end
           out.cashflowNpv(i+1) = p(matu)*NA;
           out.price = out.price + p(matu)*NA;
        end
        
        function atmSwaption = ATMSwaption(lgm1F,tx,tnr)
            tau = 0.25;
            pv01 = lgm1F.PV01(tx,tx+tnr,tau);
            fsr  = lgm1F.Fsr(tx,tx+tnr,tau);
            c_nN = lgm1F.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = lgm1F.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:1
                for j=1:1
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        function cap = Cap(lgm1F,cap_strike,dt,Ti)
            idx_mat = round(Ti/dt);
            
            HN = 0.0;
            Hn = 0.0;
            sAlpha = 0.0;
            vol = 0.0;
            d1 = 0.0;
            
            p = @lgm1F.DF;
            model_cap = 0;
            for i=1:idx_mat-1
                Hn = lgm1F.H(i*dt,lgm1F.dH1);
                HN = lgm1F.H((i+1)*dt,lgm1F.dH1);
                sAlpha = lgm1F.Alpha_Test(i*dt,lgm1F.Alpha1);
                vol = (HN-Hn)*sqrt(sAlpha);
                d1 = log(p(i*dt)/(p((i+1)*dt)*(1 + cap_strike*dt)))/vol + 0.5*vol;
                model_cap = model_cap + p(i*dt)*H_ncdf(d1)-p((i+1)*dt)*(1 + cap_strike*dt)*H_ncdf(d1-vol);
            end
            cap = model_cap;
        end
        
%         function atmSwaption = modelSwaptionExpiryTenor(lgm1F,tx,tnr)
%             tau = 0.25;
%             pv01 = lgm1F.PV01(tx,tx+tnr,tau);
%             fsr  = lgm1F.Fsr(tx,tx+tnr,tau);
%             c_nN = lgm1F.C_nN(tx,tx+tnr,tau,pv01,fsr);
%             variances = lgm1F.LocalVariance(0,tx);
%             totalVar = 0.0;
%             for i=1:1
%                 for j=1:1
%                     totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
%                 end
%             end
%             atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
%         end
        
        function impCorrel = ImpCorrel(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm1F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impCovar = ImpCovar(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm1F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCovar = coVar;
        end
        
        function spreadInfo = SpreadInfo(ck2F,tx,tnr1,tnr2)
            tau = 0.25;
            cmsSpreadDigitalArg =  ck2F.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function impVariance = ImpVariance(lgm1F,tx,tnr1)
            tau = 0.25;
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            variances = lgm1F.LocalVariance(0,tx);
            var1 = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end
        
        function instFwdCorr = InsFwdCorr(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = (lgm1F.H_Test(tx + tnr1 +0.001,lgm1F.dH1)-lgm1F.H_Test(tx + tnr1,lgm1F.dH1))/0.001;
            dH_tT1(2) = (lgm1F.H_Test(tx + tnr1 +0.001,lgm1F.dH2)-lgm1F.H_Test(tx + tnr1,lgm1F.dH2))/0.001;
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = (lgm1F.H_Test(tx + tnr2 +0.001,lgm1F.dH1)-lgm1F.H_Test(tx + tnr2,lgm1F.dH1))/0.001;
            dH_tT2(2) = (lgm1F.H_Test(tx + tnr2 +0.001,lgm1F.dH2)-lgm1F.H_Test(tx + tnr2,lgm1F.dH2))/0.001;
            
            instVariances = (lgm1F.LocalVariance(0,tx+0.001)-lgm1F.LocalVariance(0,tx))/0.001;
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + dH_tT1(j)*dH_tT1(k)*instVariances(j,k);
                    var2 = var2 + dH_tT2(j)*dH_tT2(k)*instVariances(j,k);
                    covar = covar + dH_tT1(j)*dH_tT2(k)*instVariances(j,k);
                end
            end
            instFwdCorr = covar/sqrt(var1*var2);
        end
        
        function instFwdCorrTest = InsFwdCorrTest(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH1);
            dH_tT1(2) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH2);
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH1);
            dH_tT2(2) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH2);
            
            dAlpha_t = zeros(2,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
            dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:1
                var1 = var1 + dH_tT1(j)*dH_tT1(j)*dAlpha_t(j)*dAlpha_t(j);
                var2 = var2 + dH_tT2(j)*dH_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
                covar = covar + dH_tT1(j)*dH_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function instFwdCovarTestMatrix = InsFwdCovarTest2(lgm1F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCovarTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),2);
                eta =  zeros(2,2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH1) - lgm1F.H_Test(newtnr(i),lgm1F.dH1))/tau;
                    dH_tT(i,2) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH2) - lgm1F.H_Test(newtnr(i),lgm1F.dH2))/tau;
                end

                eta(1,1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
                eta(2,2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
                eta(1,2) = 0;
                eta(2,1) = 0;

                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
            end
            
        end
        
        function instFwdCorrTestMatrix = InsFwdCorrTest2(lgm1F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCorrTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),2);
                eta =  zeros(2,2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH1) - lgm1F.H_Test(newtnr(i),lgm1F.dH1))/tau;
                    dH_tT(i,2) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH2) - lgm1F.H_Test(newtnr(i),lgm1F.dH2))/tau;
                end

                eta(1,1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
                eta(2,2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
                eta(1,2) = 0;
                eta(2,1) = 0;
                
                testLoadingD = dH_tT*eta;
                
                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
                %normalize the correlation matrix and calculate the loading matrix D
                rInv_m = zeros(length(newtnr),length(newtnr));
                for i=1:length(newtnr)
                    rInv_m(i,i) = 1.0/sqrt(coVar(i,i));
                end
                instFwdCorrTestMatrix = zeros(length(newtnr),length(newtnr));
                instFwdCorrTestMatrix = rInv_m*instFwdCovarTestMatrix*rInv_m';
                
            end
            
        end
        
        function deltaHMatrix = DeltaHTest2(lgm1F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                deltaHMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                
                dH_tT = zeros(length(newtnr),2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH1) - lgm1F.H_Test(newtnr(i),lgm1F.dH1))/tau;
                    dH_tT(i,2) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH2) - lgm1F.H_Test(newtnr(i),lgm1F.dH2))/tau;
                end

                deltaHMatrix = dH_tT;
            end
            
        end
        
        function etaMatrix = EtaTest2(lgm1F,tx)
            eta =  zeros(1,1);
            eta(1,1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             eta(2,2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
%             eta(1,2) = 0;
%             eta(2,1) = 0;
            
            etaMatrix = eta;
            
        end
        
        function instFwdCovarTest = InsFwdCovarTest(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            dH_tT1 = zeros(1,1);
            dH_tT1(1) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH1);
%             dH_tT1(2) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH2);
            
            dH_tT2 = zeros(1,1);
            dH_tT2(1) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH1);
%             dH_tT2(2) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH2);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            for j=1:1
                covar = covar + dH_tT1(j)*dH_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdCovarTest = covar;
        end
        
        function instFwdSwapCovarTest = InsSwapFwdCovarTest(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            C_tT1 = zeros(1,1);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
%             C_tT1(2) = c_nN_1(2);
            
            C_tT2 = zeros(1,1);
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
%             C_tT2(2) = c_nN_2(2);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            for j=1:1
                covar = covar + C_tT1(j)*C_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdSwapCovarTest = covar;
        end
        
        function instFwdSwapCorrTest = InsSwapFwdCorrTest(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            C_tT1 = zeros(1,1);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
%             C_tT1(2) = c_nN_1(2);
            
            C_tT2 = zeros(1,1);
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
%             C_tT2(2) = c_nN_2(2);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            var1 = 0.0;
            var2 = 0.0;
            for j=1:1
                var1 = var1 + C_tT1(j)*C_tT1(j)*dAlpha_t(j)*dAlpha_t(j);
                var2 = var2 + C_tT2(j)*C_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
                covar = covar + C_tT1(j)*C_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdSwapCorrTest = covar/sqrt(var1*var2);
        end
        
        function out = RebonatoFwdCorr2(lgm1F,tx,tnr1,tnr2)
           tau = 0.25; 
           rho_inf = 0.35;
           rho_decay = 0.15;
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function out = RebonatoFwdCorr(lgm1F,tx,tnr1,tnr2)
           rho_inf = 0.35;
           rho_decay = 0.15;
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function coVariance = CoVariance(lgm1F,tx,tnr1,tnr2)
            tau = 0.25;
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm1F.LocalVariance(0,tx);
            totalVar = zeros(1,1);
            for j=1:1
                for k=1:1
                    totalVar(1,1) = totalVar(1,1) + c_nN_1(j)*c_nN_2(k)*variances(j,k);
%                     totalVar(1,2) = totalVar(1,2) + c_nN_1(j)*c_nN_2(k)*variances(j,k);
%                     totalVar(2,1) = totalVar(2,1) + c_nN_2(j)*c_nN_1(k)*variances(j,k);
%                     totalVar(2,2) = totalVar(2,2) + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                end
            end
            coVariance =  totalVar;
        end
        
        function out = TargetFunctionATMSwaptionGlobal(lgm1F,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            lgm1FSwaptionPrice = zeros(1,length(tenor));
            
            for j=1:length(lgm1F.dH1.quote)
               lgm1F.dH1.quote(j) = tvar(j);
            end

            for k=1:length(lgm1F.Alpha1.quote)
               lgm1F.Alpha1.quote(k) = tvar(length(lgm1F.dH1.quote)+k);
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                    out = out + (lgm1FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*(lgm1FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*10000;
                end
            end
        end
        
        
        function out = TargetFunctionATMSwaptionGlobalLevMar(lgm1F,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            lgm1FSwaptionPrice = zeros(1,length(tenor));
            % tracking calibration
            
            
%             for j=1:2*length(lgm1F.dH1.quote)+2*length(lgm1F.Alpha1.quote)
%                 lgm1F.debugInfo{1,1}.currentParameterVec(idx,j) = tvar(j);
%             end

            for j=1:length(lgm1F.dH1.quote)
               lgm1F.dH1.quote(j) = tvar(j);
            end

            for k=1:length(lgm1F.Alpha1.quote)
               lgm1F.Alpha1.quote(k) = tvar(length(lgm1F.dH1.quote)+k);
            end
            
             
            
            
            sizeOut = length(expiry) * length(tenor);
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = lgm1FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
        end
        
        
        function out = CalibrateToATMDiagSwaption(lgm1F,black,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);
%             blackSwaptionVega =  zeros(expirySize,tenorSize);
            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j) = black.BlackATMSwaption(expiry(k),tenor(j));
%                     blackSwaptionVega(k,j) = black.BlackATMSwaptionVega(expiry(k),tenor(j));
                end
            end
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
%             params.expiry = expiry;
%             params.tenor = tenor;
%             params.blackSwaptionPrice = blackSwaptionPrice;
%             params.blackSwaptionVega  = blackSwaptionVega;
%             params.expirySize = expirySize;
%             params.tenorSize =  tenorSize;
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            target_size = lgm1F.targetSize;
            params.target_size = target_size;
            params.matu = lgm1F.matu;
            params.freq = lgm1F.freq;
            params.swaption_expiry_use_yn = zeros(target_size,1);
            params.dH1.tenor = 1:1:target_size;
            params.dH1.tenor = params.dH1.tenor';
            params.dH1.quote = ones(target_size,1);
            
            params.Alpha1.tenor = 1:1:target_size;
            params.Alpha1.tenor = params.Alpha1.tenor';
            params.Alpha1.quote = 0.01*ones(target_size,1);
            
            params.diagSwaption.expiry = zeros(target_size,1);
            params.diagSwaption.tenor  = zeros(target_size,1);
            params.diagSwaption.quote  = zeros(target_size,target_size);
            params.diagSwaption.strike = zeros(target_size,1);
            params.diagSwaption.blackPrice = zeros(target_size,1);
            
            if lgm1F.matu <= 2.0
                params.swaption_expiry_use_yn(1) = 1;
                params.diagSwaption.expiry =  ones(1,1);
                params.diagSwaption.expiry(1) =  swaptionVolSurface.expiry(1);
                params.diagSwaption.tenor =  ones(1,1);
                params.diagSwaption.tenor(1) =  swaptionVolSurface.tenor(1);
                params.diagSwaption.quote = ones(1,1);
                params.diagSwaption.quote(1,1) = swaptionVolSurface.swaptionVol(expiry(1),tenor(1));
            else
                temp_tenor   = zeros(target_size,1);
                temp_expiry = zeros(target_size,1);
                for i=2:target_size
                    temp_tenor(i-1)  = params.Alpha1.tenor(target_size - (i-1));
                    temp_expiry(i-1) = params.matu - temp_tenor(i-1); 
                end
                
                upper_expiry = swaptionVolSurface.expiry(end);
                upper_tenor  = swaptionVolSurface.tenor(end);
                
                lower_expiry = swaptionVolSurface.expiry(1);
                
                temp_idx= 1;
                for i=1:target_size
                    for j=temp_idx:target_size
                        % only those swaption whose expiry date is away.
                        if temp_expiry(j) > 0
                            params.dH1.tenor(i)     = temp_expiry(j);
                            params.Alpha1.tenor(i) = temp_expiry(j);
                            params.diagSwaption.expiry(i) = temp_expiry(j);
                            params.diagSwaption.tenor(i)  = temp_tenor(j);
                            if(temp_expiry(j) > upper_expiry && temp_tenor(j) <= upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(upper_expiry,temp_tenor(j));
                            elseif(temp_expiry(j) <= upper_expiry && temp_tenor(j) > upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),upper_tenor);    
                            else
                                if temp_expiry(j) < lower_expiry
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(lower_expiry,temp_tenor(j));
                                else
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),temp_tenor(j));
                                end
                            end
                            
                            params.swaption_expiry_use_yn(i) = 1;
                            temp_idx = j+1;
                            break;
                        end
                    end
                    
                    if i>=2
                        params.dH1.tenor(i)     = params.dH1.tenor(i-1)     + 1;
                        params.Alpha1.tenor(i) = params.Alpha1.tenor(i-1) + 1;
                    end
                end
                
            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            lgm1F.dH1    = params.dH1;
            lgm1F.Alpha1 = params.Alpha1;
            
          %% target swaption price and forward swap rate
            for i=1:target_size
                if params.swaption_expiry_use_yn(i)
                    Ti = params.diagSwaption.expiry(i);
                    Tj = params.diagSwaption.tenor(i);
                    blackVolQuote = params.diagSwaption.quote(i,i);
                    params.diagSwaption.strike(i) = black.Fsr(Ti,Ti + Tj,0.25);
                    params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVol(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote);
                end
            end
            
           %% target swaption price and forward swap rate
            % tracking calibration
            
            out.blackSwaptionPrice = params.diagSwaption.blackPrice;
            out.lgm1FSwaptionPrice = zeros(target_size,1);
            out.priceRelDiff = zeros(target_size,1);
            out.priceRmseTotal = 0.0;
            out.dH1 = params.dH1;
            out.Alpha1 = params.Alpha1;
            
            out.market_swaption = zeros(target_size,1);
            out.model_swaption = zeros(target_size,1);
            out.re = zeros(target_size,1);
            out.rmseTotal = 0;
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            targetSwaptionSize = 0;
            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    
                    for j=1:length(lgm1F.Alpha1.quote)
                        if params.swaption_expiry_use_yn(j) == 1
                            targetSwaptionSize = targetSwaptionSize +1;
                        end
                    end
                    
                    optParams.dH_compare = params.dH1.quote(1);
                    
                    if lgm1F.matu <= 2.0
                        optParams.dHSize = targetSwaptionSize;
                    else
                        optParams.dHSize = targetSwaptionSize + 1;
                    end
                    
                    optParams.AlphaSize = targetSwaptionSize;
                    
                    optParams.modelParamsSize = optParams.dHSize + optParams.AlphaSize;
                    
                    if lgm1F.matu <= 2.0
                        optParams.numOfEquation = targetSwaptionSize + ... % # of target swaption instruments
                                                        0 + ...           % # of Alpha term structure gamma constraints
                                                  optParams.dHSize;    % # of dH absolute error relative to 1.0 
                    else
                        optParams.numOfEquation = targetSwaptionSize + ...         % # of target swaption instruments
                                                  optParams.AlphaSize -2 + ...     % # of Alpha term structure gamma constraints
                                                  optParams.dHSize;                % # of dH absolute error relative to 1.0 
                    end
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=1:optParams.dHSize
                        tvar(j) = params.dH1.quote(j);
                        lb(j)= 0.0001;
                        ub(j)= 10;
                    end
                    
                    for j=1:optParams.AlphaSize
                        tvar(j + optParams.dHSize) = params.Alpha1.quote(j);
                        lb(j + optParams.dHSize)= 0.0001;
                        ub(j + optParams.dHSize)= 10;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionLGM1FATMSwaptionDiagGlobal(lgm1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM1FATMSwaptionDiagGlobal',tvar,x, 200, options,'unc',lgm1F,optParams);
                    
                    
                    % assigning model parameters
                    k=1;
                    for j=1:optParams.dHSize
                       lgm1F.dH1.quote(j) = max(popt(j),0.0001);
                       k=k+1;
                    end

                    for j=k:length(lgm1F.dH1.quote)
                       lgm1F.dH1.quote(j) = lgm1F.dH1.quote(optParams.dHSize);
                    end

                    k = 1;
                    for j=1:optParams.AlphaSize
                       lgm1F.Alpha1.quote(j) = max(popt(optParams.dHSize+j),0.0001);
                       k=k+1;
                    end

                    % flat extrapolation of Alpha
                    % AlphaSize != length(Alpha)
                    for j=k:length(lgm1F.Alpha1.quote)
                       lgm1F.Alpha1.quote(j) = lgm1F.Alpha1.quote(optParams.AlphaSize);
                    end
                    
                    out.dH1.quote = lgm1F.dH1.quote;
                    out.Alpha1.quote = lgm1F.Alpha1.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        if params.swaption_expiry_use_yn(i) == 1
                            Ti = params.diagSwaption.expiry(i);
                            Tj = params.diagSwaption.tenor(i);
                            out.market_swaption(i) = params.diagSwaption.blackPrice(i);
                            out.model_swaption(i)  = lgm1F.ATMSwaption(Ti,Tj);
                            out.re(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                            out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                            N=N+1; 
                        end
                    end
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    
                case 'globalLevMar'
                    for j=1:length(lgm1F.dH1.quote)
                        tvar(j) = lgm1F.dH1.quote(j);
                        lb(j)= -10;
                        ub(j)= 10;
                    end

                    for k=1:length(lgm1F.Alpha1.quote)
                        tvar(length(lgm1F.dH1.quote)+k) = lgm1F.Alpha1.quote(k);
                        lb(length(lgm1F.dH1.quote)+k)= -10;
                        ub(length(lgm1F.dH1.quote)+k)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.init_lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                            out.init_relDiff(j,k) = out.init_lgm1FSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.init_rmseTotal = out.init_rmseTotal + out.init_relDiff(j,k)*out.init_relDiff(j,k);
                            N=N+1;
                        end
                    end
                    out.init_rmseTotal = sqrt(out.init_rmseTotal/N);
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalLevMar(lgm1F,tvar,params);
                    options=optimset('Algorithm','levenberg-marquardt','MaxFunEvals',700);
                    [x, fval] = lsqnonlin(targetfun,tvar,[],[],options);
%                     [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
%                     options=optimset('Algorithm','trust-region-reflective');
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for j=1:length(lgm1F.dH1.quote)
                        out.dH1.quote(j) = x(j);
                        lgm1F.dH1.quote(j) = x(j);
                    end
            
                    for k=1:length(lgm1F.Alpha1.quote)
                        out.Alpha1.quote(k) = x(length(lgm1F.dH1.quote)+k);
                        lgm1F.Alpha1.quote(k) = x(length(lgm1F.dH1.quote)+k);
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.lgm1FSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
            
                    
                case 'dummy'
                    
                    for j=1:length(lgm1F.dH1.quote)
                        out.dH1.quote(j) = lgm1F.dH1.quote(j);
                        out.dH2.quote(j) = lgm1F.dH2.quote(j);
                    end
            
                    for k=1:length(lgm1F.Alpha1.quote)
                        out.Alpha1.quote(k) =  lgm1F.Alpha1.quote(k);
                        out.dAlpha2.quote(k) = lgm1F.dAlpha2.quote(k);
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.lgm1FSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMCapDiagSwaption(lgm1F,black,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
            capVol = black.capVol;
            capVolData = black.mktData('capVol');
            capVolrawData =  capVolData.params('rawData');
            
            capVolTermStructure.expiry = capVolrawData(:,1);
            capVolTermStructure.capVol = capVol; % capVol With interpolator
            
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            target_size = lgm1F.targetSize;
            params.target_size = target_size;
            params.matu = lgm1F.matu;
            params.freq = lgm1F.freq;
            
            params.cap_expiry_use_yn = zeros(target_size,1);
            params.swaption_expiry_use_yn = zeros(target_size,1);
            
            params.dH1.tenor = 1:1:target_size;
            params.dH1.tenor = params.dH1.tenor';
            params.dH1.quote = 0.1*ones(target_size,1);
            
            params.Alpha1.tenor = 1:1:target_size;
            params.Alpha1.tenor = params.Alpha1.tenor';
            params.Alpha1.quote = 0.01*ones(target_size,1);
            
            params.diagSwaption.expiry = zeros(target_size,1);
            params.diagSwaption.tenor  = zeros(target_size,1);
            params.diagSwaption.quote  = zeros(target_size,target_size);
            params.diagSwaption.strike = zeros(target_size,1);
            params.diagSwaption.blackPrice = zeros(target_size,1);
            
            params.cap.expiry = zeros(target_size,1);
            params.cap.quote = zeros(target_size,1);
            params.cap.strike = zeros(target_size,1);
            params.cap.blackPrice = zeros(target_size,1);
            
            % setting target diagonal swaption 
            if lgm1F.matu <= 2.0
                params.swaption_expiry_use_yn(1) = 1;
                params.diagSwaption.expiry =  ones(1,1);
                params.diagSwaption.expiry(1) =  swaptionVolSurface.expiry(1);
                params.diagSwaption.tenor =  ones(1,1);
                params.diagSwaption.tenor(1) =  swaptionVolSurface.tenor(1);
                params.diagSwaption.quote = ones(1,1);
                params.diagSwaption.quote(1,1) = swaptionVolSurface.swaptionVol(expiry(1),tenor(1));
            else
                temp_tenor   = zeros(target_size,1);
                temp_expiry = zeros(target_size,1);
                for i=2:target_size
                    temp_tenor(i-1)  = params.Alpha1.tenor(target_size - (i-1));
                    temp_expiry(i-1) = params.matu - temp_tenor(i-1); 
                end
                
                upper_expiry = swaptionVolSurface.expiry(end);
                upper_tenor  = swaptionVolSurface.tenor(end);
                
                lower_expiry = swaptionVolSurface.expiry(1);
                
                temp_idx= 1;
                for i=1:target_size
                    for j=temp_idx:target_size
                        % only those swaption whose expiry date is away.
                        if temp_expiry(j) > 0
                            params.Alpha1.tenor(i) = temp_expiry(j);
                            params.diagSwaption.expiry(i) = temp_expiry(j);
                            params.diagSwaption.tenor(i)  = temp_tenor(j);
                            if(temp_expiry(j) > upper_expiry && temp_tenor(j) <= upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(upper_expiry,temp_tenor(j));
                            elseif(temp_expiry(j) <= upper_expiry && temp_tenor(j) > upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),upper_tenor);    
                            else
                                if temp_expiry(j) < lower_expiry
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(lower_expiry,temp_tenor(j));
                                else
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),temp_tenor(j));
                                end
                            end
                            
                            params.swaption_expiry_use_yn(i) = 1;
                            temp_idx = j+1;
                            break;
                        end
                    end
                    
                    if i>=2
                        params.Alpha1.tenor(i) = params.Alpha1.tenor(i-1) + 1;
                    end
                end
                
            end
            
            % setting target cap
            params.cap.expiry(1) = 1;
            params.cap.quote(1) = capVolTermStructure.capVol(params.cap.expiry(1));
            params.cap_expiry_use_yn(1) = 1;
            
            for i=2:target_size
                params.cap.expiry(i) = i;
                params.cap.quote(i) = capVolTermStructure.capVol(params.cap.expiry(i));
                if params.cap.expiry(i-1) < params.matu + 0.25
                    params.cap_expiry_use_yn(i) = 1;
                end
                if i == 10  % exclude 12Y, 15Y cap vol from target     
                    break;
                end
            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            lgm1F.dH1    = params.dH1;
            lgm1F.Alpha1 = params.Alpha1;
            
          %% target swaption price and forward swap rate
            for i=1:target_size
                if params.swaption_expiry_use_yn(i)
                    Ti = params.diagSwaption.expiry(i);
                    Tj = params.diagSwaption.tenor(i);
                    blackVolQuote = params.diagSwaption.quote(i,i);
                    params.diagSwaption.strike(i) = black.Fsr(Ti,Ti + Tj,0.25);
                    params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVol(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote);
                end
            end
            
           %% target cap price and forward ATM strike
            for i=1:target_size
                if params.cap_expiry_use_yn(i)
                    Ti = params.cap.expiry(i);
                    params.cap.strike(i) = black.Fsr(0.25,Ti,0.25);
                    params.cap.blackPrice(i) = black.BlackCapWithVol(Ti,params.cap.strike(i),0.25,params.cap.quote(i));
                end
            end
            
           %% target swaption price and forward swap rate
            % tracking calibration
            params.cap_size = target_size;
            params.swaption_size = target_size;
            
            out.cap_size = target_size;
            out.swaption_size = target_size;
            
            out.blackSwaptionPrice = params.diagSwaption.blackPrice;
            out.lgm1FSwaptionPrice = zeros(target_size,1);
            out.swaptionPriceRelDiff = zeros(target_size,1);
            out.swaptionPriceRmseTotal = 0.0;
            
            out.blackCapPrice = params.cap.blackPrice;
            out.lgm1FCapPrice = zeros(target_size,1);
            out.capPriceRelDiff = zeros(target_size,1);
            out.capPriceRmseTotal = 0.0;
            
            out.dH1 = params.dH1;
            out.Alpha1 = params.Alpha1;
            
            out.market_swaption = zeros(target_size,1);
            out.model_swaption = zeros(target_size,1);
            
            out.market_cap = zeros(target_size,1);
            out.model_cap = zeros(target_size,1);
            
%             out.re = zeros(target_size,1);
%             out.rmseTotal = 0;
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
%             targetSwaptionSize = 0;
%             targetCapSize = 0;
            
%             dH_size = 0;
%             Alpha_size = 0;
            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    optParams.dH_size =0;
                    optParams.Alpha_size =0;
                    
                    for j=1:out.cap_size
                        if params.cap_expiry_use_yn(j) == 1
                            optParams.dH_size = optParams.dH_size +1;
                        end
                    end
                    
                    for j=1:out.swaption_size
                        if params.swaption_expiry_use_yn(j) == 1
                            optParams.Alpha_size = optParams.Alpha_size +1;
                        end
                    end
                    
                    if lgm1F.matu <= 10.0
                        optParams.dH_size = optParams.dH_size - 1;
                        optParams.Alpha_size = optParams.Alpha_size + 1;
                    end
                    
                    optParams.dH_1Y_fixed_value = params.dH1.quote(1);
                    
                    optParams.modelParamsSize = optParams.dH_size + optParams.Alpha_size;
                    
                    optParams.numOfEquation = optParams.dH_size + optParams.Alpha_size; 
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=2:optParams.dH_size+1
                        tvar(j-1) = params.dH1.quote(j);
                        lb(j-1)= -10.0;
                        ub(j-1)= 10.0;
                    end
                    
                    for j=1:optParams.Alpha_size
                        tvar(j + optParams.dH_size) = params.Alpha1.quote(j);
                        lb(j + optParams.dH_size)= -10.0;
                        ub(j + optParams.dH_size)= 10.0;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionLGM1FATMSwaptionDiagGlobal(lgm1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM1FATMCapSwaptionDiagGlobal',tvar,x, 200, options,'unc',lgm1F,optParams);
                    
                    
                    % assigning model parameters
                    k=2;
                    for j=2:optParams.dH_size+1
                       lgm1F.dH1.quote(j) = max(abs(popt(j-1)),0.0001);
                       k=k+1;
                    end

                    for j=k:length(lgm1F.dH1.quote)
                       lgm1F.dH1.quote(j) = lgm1F.dH1.quote(optParams.dH_size+1);
                    end

                    k = 1;
                    for j=1:optParams.Alpha_size
                       lgm1F.Alpha1.quote(j) = max(abs(popt(optParams.dH_size+j)),0.0001);
                       k=k+1;
                    end

                    % flat extrapolation of Alpha
                    % AlphaSize != length(Alpha)
                    for j=k:length(lgm1F.Alpha1.quote)
                       lgm1F.Alpha1.quote(j) = lgm1F.Alpha1.quote(optParams.Alpha_size);
                    end
                    
                    out.dH1.quote = lgm1F.dH1.quote;
                    out.Alpha1.quote = lgm1F.Alpha1.quote;
                    
                    N=0;
                    for i=1:params.swaption_size
                        if params.swaption_expiry_use_yn(i) == 1
                            Ti = params.diagSwaption.expiry(i);
                            Tj = params.diagSwaption.tenor(i);
                            out.market_swaption(i) = params.diagSwaption.blackPrice(i);
                            out.model_swaption(i)  = lgm1F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i)*out.swaptionPriceRelDiff(i);
                            N=N+1; 
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
                    
                    N=0;
                    for i=1:params.cap_size
                        if params.cap_expiry_use_yn(i) == 1
                            Ti = params.cap.expiry(i);
                            out.market_cap(i) = params.cap.blackPrice(i);
                            out.model_cap(i)  = lgm1F.Cap(params.cap.strike(i),0.25,Ti);
                            out.capPriceRelDiff(i) = out.model_cap(i)/out.market_cap(i) - 1.0;
                            out.capPriceRmseTotal = out.capPriceRmseTotal + out.capPriceRelDiff(i)*out.capPriceRelDiff(i);
                            N=N+1; 
                        end
                    end
                    out.capPriceRmseTotal = sqrt(out.capPriceRmseTotal/N);
                    
                
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMSwaptionTenorDiagSwaption(lgm1F,black,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            % dummy
%             target_size = lgm1F.targetSize;

            params.matu = lgm1F.matu;
            params.freq = lgm1F.freq;
            params.joint_swap_tenor = lgm1F.joint_swap_tenor;
            
            params.dh_initial = 1.0;
            params.alpha_initial = 0.01;
            params.smallNumber = 0.000001;
            
            % set the target swaption expiry & tenor
            remainingMatu = params.matu - floor(params.matu);
            max_swaption_expiry_size = ceil(params.matu);
            
            if remainingMatu == 0
                remainingMatu = 1.0;
            end
            
            if params.matu -1  < params.joint_swap_tenor
                max_swaption_tenor_size = floor(params.joint_swap_tenor);
            else
                max_swaption_tenor_size = floor(params.matu - params.smallNumber);
            end
            
            params.targetSwaption.expiry = zeros(max_swaption_expiry_size,1);
            params.targetSwaption.tenor  = zeros(max_swaption_tenor_size,1);
            params.targetSwaption.quote  = zeros(max_swaption_expiry_size,max_swaption_tenor_size);
            params.swaption_use_yn = zeros(max_swaption_expiry_size,max_swaption_tenor_size);
            
            for i=1:max_swaption_expiry_size
                params.targetSwaption.expiry(i) = remainingMatu + i-1;
            end
            
            for i=1:max_swaption_tenor_size
                params.targetSwaption.tenor(i) = i;
            end
            
            params.target_size = 0;
            for i=1:max_swaption_expiry_size
                swaptionExpiry = params.targetSwaption.expiry(i);
                for j=1:max_swaption_tenor_size
                    swaptionTenor = params.targetSwaption.tenor(j);
                    diag_yn = 0;
                    tenor_yn = 0;
                    
                    if abs(swaptionExpiry + swaptionTenor - params.matu) < params.smallNumber 
                        diag_yn = 1;
                    end
                    
                    if (swaptionTenor == params.joint_swap_tenor) && ((swaptionExpiry <= params.matu) || abs(swaptionExpiry - params.matu) < params.smallNumber)
                        tenor_yn = 1;
                    end
                    
                    if (diag_yn) || (tenor_yn)
                        params.swaption_use_yn(i,j) = 1;
                        params.target_size = params.target_size +1;
                    end
                end
            end
            
            params.target_expiry_idx = zeros(params.target_size,1);
            params.target_tenor_idx = zeros(params.target_size,1);
            params.targetSwaption.strike = zeros(params.target_size,1);
            params.targetSwaption.blackPrice = zeros(params.target_size,1);
            
            idx = 1;
            for i=1:max_swaption_expiry_size
                for j=1:max_swaption_tenor_size
                    if params.swaption_use_yn(i,j) == 1
                        params.target_expiry_idx(idx) = i;
                        params.target_tenor_idx(idx) = j;
                        idx = idx +1;
                    end
                end
            end
            
            dh_size = floor( params.matu) + 1 + params.joint_swap_tenor + 1;
            params.dH1.tenor = zeros(dh_size,1);
            params.dH1.quote = zeros(dh_size,1);
            
            for i=1:dh_size
                params.dH1.tenor(i) = remainingMatu + i-1;
                params.dH1.quote(i) = params.dh_initial;
            end
            
            alpha_size = floor(params.matu - params.smallNumber) + 2;
            params.Alpha1.tenor = zeros(alpha_size,1);
            params.Alpha1.quote = zeros(alpha_size,1);
            
            for i=1:alpha_size
                params.Alpha1.tenor(i) = remainingMatu + i-1;
                params.Alpha1.quote(i) = params.alpha_initial;
            end
            
            upper_expiry = swaptionVolSurface.expiry(end);
            upper_tenor  = swaptionVolSurface.tenor(end);
            lower_expiry = swaptionVolSurface.expiry(1);
            
            for i=1:params.target_size
                expiry_idx = params.target_expiry_idx(i);
                tenor_idx = params.target_tenor_idx(i);
                swaptionExpiry = params.targetSwaption.expiry(expiry_idx);
                swaptionTenor = params.targetSwaption.tenor(tenor_idx);
                
                if(swaptionExpiry > upper_expiry && swaptionTenor <= upper_tenor)
                    params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(upper_expiry,swaptionTenor);
                elseif(swaptionExpiry <= upper_expiry && swaptionTenor > upper_tenor)
                    params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(swaptionExpiry,upper_tenor);    
                else
                    if swaptionExpiry < lower_expiry
                        params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(lower_expiry,swaptionTenor);
                    else
                        params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(swaptionExpiry,swaptionTenor);
                    end
                end
                
                params.targetSwaption.strike(i) = black.Fsr(swaptionExpiry,swaptionExpiry + swaptionTenor,0.25);
%                 annuity = black.PV01(swaptionExpiry,swaptionExpiry + swaptionTenor,0.25);
                blackVolQuote = params.targetSwaption.quote(expiry_idx,tenor_idx); 
                params.targetSwaption.blackPrice(i) = black.BlackSwaptionWithVol(swaptionExpiry,swaptionTenor,params.targetSwaption.strike(i),blackVolQuote);
            end
            
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            lgm1F.dH1    = params.dH1;
            lgm1F.Alpha1 = params.Alpha1;
            
          
            
           %% target swaption price and forward swap rate
            % tracking calibration
            
            params.swaption_size = params.target_size;
            out.swaption_size = params.target_size;
            
            out.blackSwaptionPrice = params.targetSwaption.blackPrice;
            out.lgm1FSwaptionPrice = zeros(params.target_size,1);
            out.swaptionPriceRelDiff = zeros(params.target_size,1);
            out.swaptionPriceRmseTotal = 0.0;
            
            out.dH1 = params.dH1;
            out.Alpha1 = params.Alpha1;
            
            out.market_swaption = zeros(params.target_size,1);
            out.model_swaption = zeros(params.target_size,1);
            
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;

            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    
                    optParams.dH_size = length(params.dH1.quote);  
                    optParams.Alpha_size = length(params.Alpha1.quote);
                    
                    % dH1(end-1) = dH1(end)
                    optParams.modelParamsSize = optParams.dH_size-1 + optParams.Alpha_size;
                    
                    optParams.numOfEquation = params.target_size + ...  % target swaption
                                              optParams.Alpha_size -2 + ... % tikhonov regularization gamma constraint
                                              optParams.dH_size;     % constraint on dH  
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=1:optParams.dH_size-1
                        tvar(j) = params.dH1.quote(j);
                        lb(j)= -10.0;
                        ub(j)= 10.0;
                    end
                    
                    for j=1:optParams.Alpha_size
                        tvar(j + optParams.dH_size-1) = params.Alpha1.quote(j);
                        lb(j + optParams.dH_size-1)= -10.0;
                        ub(j + optParams.dH_size-1)= 10.0;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionLGM1FATMSwaptionDiagGlobal(lgm1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM1FATMSwaptionTenorSwaptionDiagGlobal',tvar,x, 200, options,'unc',lgm1F,optParams);
                    
                    
                    % assigning model parameters
                    for j=1:optParams.dH_size-1
                       lgm1F.dH1.quote(j) = max(abs(popt(j)),0.0001);
                    end
                    lgm1F.dH1.quote(optParams.dH_size) = lgm1F.dH1.quote(optParams.dH_size-1);
                    
                    for j=1:optParams.Alpha_size
                       lgm1F.Alpha1.quote(j) = max(abs(popt(optParams.dH_size-1 + j)),0.0001);
                    end

                    out.dH1.quote = lgm1F.dH1.quote;
                    out.Alpha1.quote = lgm1F.Alpha1.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        expiry_idx = params.target_expiry_idx(i);
                        tenor_idx = params.target_tenor_idx(i);
                        swaptionExpiry = params.targetSwaption.expiry(expiry_idx);
                        swaptionTenor = params.targetSwaption.tenor(tenor_idx);
                        out.market_swaption(i) = params.targetSwaption.blackPrice(i);
                        out.model_swaption(i) = lgm1F.ATMSwaption(swaptionExpiry,swaptionTenor);
                        out.swaptionPriceRelDiff(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                        out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i)*out.swaptionPriceRelDiff(i);
                        N=N+1;
                    end
                    
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
                    
                
                otherwise
                    disp('unImplemented')
            end
        end 
        

    end
    
end



In [ ]:
classdef LGM1F < IRModel
    %UNTITLED Summary of this class goes here
    %   Detailed explanation goes here
    
    properties
         % vol functor
        Alpha1
        dH1
        smoothdH
        smoothAlpha
        matu
        joint_swap_tenor
        targetSize
        freq % 
        numOfFactor
        modelType
    end
    
    methods
        function lgm1F = LGM1F(params)
            if nargin > 0
                
                lgm1F.zeroCurve = params.zeroCurve;
                
                lgm1F.mktData = params.mktData;
                lgm1F.modelParams = params.modelParams;
                
                lgm1F.Alpha1 = params.modelParams('Alpha1');
                lgm1F.dH1 =  params.modelParams('dH1');
                lgm1F.matu =  params.modelParams('matu');
                lgm1F.joint_swap_tenor =  params.modelParams('joint_swap_tenor');
                lgm1F.smoothdH =  params.modelParams('smoothdH');
                lgm1F.smoothAlpha =  params.modelParams('smoothAlpha');
                lgm1F.targetSize = params.modelParams('targetSize');
                lgm1F.freq  =params.modelParams('freq');
                
                lgm1F.numOfFactor = 1;
                lgm1F.modelType = 'LGM1F';
                
            end
        end
        
        function out = SimpleSRPrOverHedge(lgm1F,spreadC,spreadN,ralower,raupper)
            buffer = 0.001;
            outVec = 0.0;
            if spreadC < ralower - buffer
                outVec = 0.0;
            elseif spreadC >= ralower - buffer && spreadC < ralower
                outVec = 1.0/buffer * (spreadC - (ralower - buffer));
            elseif spreadC >= ralower && spreadC < raupper
                outVec = 1.0;
            elseif spreadC >= raupper && spreadC < raupper + buffer
                outVec = -1.0/buffer *(spreadC - (raupper + buffer));
            else
                outVec = 0.0;
            end
            
            out = outVec;
        end
        
        function out = SimpleSRPr(lgm1F,spreadC,spreadN,ralower,raupper)
            outVec = 0.0;
            if spreadC >= ralower && spreadC <= raupper
                outVec = 1.0;
            end
            out = outVec;
        end
        
        
        function out = TRZ_SRPr(lgm1F,curr,next,L,U)
            length = 1.0;
            remain = 1.0;
            if (curr >= L && curr <= U && next >= L && next <= U)
                length = remain;
            elseif ( curr >= L && curr <= U && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr >= L && curr <= U && next < L ) 
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr > U && next >= L && next <= U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next < L )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next > U )
                length = ( U - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ...
                        -( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
            elseif ( curr < L && next >= L && next <= U )
                length = ( L - curr ) * ( remain - 0.0 ) / ( next - curr ) + 0.0 ;
                length = remain - length;
            elseif ( curr > U && next > U )
                length = 0.0;
            elseif ( curr < L && next < L)
                length = 0.0;
            else
                disp('error')
            end
            
            out = length;
        end
        
        function out = SRPr(lgm1F,spreadC,spreadN,ralower,raupper,simpleYN,pathSize)
            
            outVec = zeros(1,pathSize);
            
            if simpleYN
                for i =1:pathSize
                    outVec(i) = lgm1F.SimpleSRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            elseif simpleYN == 2
                for i =1:pathSize
                    outVec(i) = lgm1F.SimpleSRPrOverHedge(spreadC(i),spreadN(i),ralower,raupper);
                end
            else
                for i =1:pathSize
                    outVec(i) = lgm1F.TRZ_SRPr(spreadC(i),spreadN(i),ralower,raupper);
                end
            end
            
            out = outVec;
        end
        
        function out =  Libor_SimpleDate(lgm1F,valueDate,evalTime, fwdStartTime, numMatTime, tenor,modelStatesT)
           %CMS
           fwdStartDate = valueDate.AddDate(fwdStartTime,'day');
           fwdEndDate = fwdStartDate.AddDate(tenor,'year');
           fwdEndTime = fwdEndDate.DateDiff(valueDate);
           
           deltaT = (fwdEndTime - fwdStartTime)/365.0;
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           
           stateSize = size(modelStatesT,2);
           df_Tn = zeros(1,stateSize);
           df_TN = zeros(1,stateSize);
           
           df_Tn = lgm1F.discountFactor(evalTime,fwdStartTime,numMatTime,modelStatesT);
           df_TN = lgm1F.discountFactor(evalTime,fwdEndTime,numMatTime,modelStatesT);
           
           liborOut = zeros(1,stateSize);
           
           for i=1:stateSize
                liborOut(i) = (df_Tn(i)/df_TN(i) - 1.0)/deltaT;
           end
           
           out = liborOut;
            
        end
        
         function out =  CMS_PSA_SimpleDate(lgm1F,valueDate,observeTime,numMatTime, tenor,modelStatesT)
           %CMS
%            tau = 0.25;
           tau = 1.0/(lgm1F.freq);
           to      = observeTime/365.0;
           toDate = valueDate.AddDate(observeTime,'day');
           tNDate = toDate.AddDate(tenor,'year');
           tN   =  tNDate.DateDiff(valueDate)/365.0;
           
           numMatDate = valueDate.AddDate(numMatTime,'day');
           
           p_nN = lgm1F.PV01Date(valueDate,toDate,tNDate,tau);
           
           df_Tn = lgm1F.DF(to);
           df_TN = lgm1F.DF(tN);
           y_nN = (df_Tn - df_TN)/p_nN;
           
           c_nN = lgm1F.C_nNDate(valueDate,toDate,tNDate,tau,p_nN,y_nN);
           g_nN = lgm1F.G_nNDate(valueDate,toDate,tNDate,tau,p_nN);
           h_nTerm = lgm1F.H_nTermDate(valueDate,toDate,numMatDate);
           varCovar = lgm1F.LocalVariance(0,to);
           
           stateSize = size(modelStatesT,2);
           cms_PCA_SimpleOut = zeros(1,stateSize);
           
           driftTerm = 0;
           for i=1:1
               for j=1:1
                   driftTerm = driftTerm + (c_nN(i) * h_nTerm(j) + c_nN(i) * g_nN(j))*varCovar(i,j);
               end
           end
           
           for i=1:stateSize
               cms_PCA_SimpleOut(i) = y_nN + driftTerm + c_nN(1)*modelStatesT(1,i); 
           end
           
           out = cms_PCA_SimpleOut;
           
            
         end
         
         function out = HtT_r(lgm1F,t,T)
            out =  lgm1F.H(T,lgm1F.dH1) - lgm1F.H(t,lgm1F.dH1); 
        end
        
         function out = fwdZcVariance(lgm1F, a, b, T1, T2)
            varMatrix = lgm1F.LocalVariance(a,b);
            
            localVariance = zeros(1,1);
            measureChange = zeros(1,1);
            
            localVariance(1) = (lgm1F.HtT_r(a,T1)*lgm1F.HtT_r(a,T1)-lgm1F.HtT_r(a,b)*lgm1F.HtT_r(a,b))*varMatrix(1,1);

            measureChange(1) = (lgm1F.HtT_r(a,T1)-lgm1F.HtT_r(a,b))*lgm1F.HtT_r(a,T2)*varMatrix(1,1);
           
            value = localVariance(1);
            measureChangeTerm = measureChange(1);
            
            value = value + -2.0*measureChangeTerm;
            
            out = value;
            
         end
         
        function out = discountFactor(lgm1F, startTime, endTime,numMatTime,modelStatesT)
           zcT = lgm1F.DF(endTime/365.0);
           
           %numOfPath
           stateSize = size(modelStatesT,2);
           discountFactorOut = zeros(1,stateSize);
           
           % deterministic discount factor
           if startTime <= 1e-8
                for i=1:stateSize
                    discountFactorOut(i) = zcT;
                end
                out = discountFactorOut;
                return;
           end
           
           % deterministic discount factor or maturity in the past
           if startTime >= endTime
                out = ones(1,stateSize);
                return;
           end
           
           zct = lgm1F.DF(startTime/365.0);
           beta = lgm1F.Beta(startTime/365.0,endTime/365.0);
           zcDriftTerm = lgm1F.fwdZcVariance(0,startTime/365.0,endTime/365.0,numMatTime/365.0);
           
           
           for i=1:stateSize
               discountFactorOut(i) = zcT/zct * exp(-0.5*zcDriftTerm - beta(1)*modelStatesT(1,i));
           end
           
           out = discountFactorOut;
        end
        
        function out = discountPayoff(lgm1F,eventTime,numMatTime ...
                ,modelStatesT,cashflow)
            
            dfNumMat = lgm1F.DF(numMatTime/365.0);
            % for discountPayoff T=numMatTime
            % P(0,TnumMat)/P(Te,Tnummat)*Payoff
            %
            discountT = lgm1F.discountFactor(eventTime, numMatTime,numMatTime,modelStatesT);
            stateSize = size(modelStatesT,2);
            payoff = zeros(1,stateSize);
            for i=1:stateSize
                payoff(i)= dfNumMat/discountT(i)*cashflow(i);
            end    
            out = payoff;
        end
        
        function out = stateLocalDrift(lgm1F,fromTime, toTime)
            
            out = eye(1,1);
        end
        
        function out = stateLocalVariance(lgm1F,fromTime, toTime)
            out = lgm1F.LocalVariance(fromTime/365.0,toTime/365.0);
        end
        
        function variances = LocalVariance(lgm1F,from,to)

            sig(1) = lgm1F.Alpha1;
            alphaSize = length(lgm1F.Alpha1.quote);
            
            for i=1:length(sig(1).tenor)
                if from<= sig(1).tenor(i);
                    break;
                end
            end
            startIdx = i;
            
            for i=1:length(sig(1).tenor)
                if to<= sig(1).tenor(i);
                    break;
                end
            end
            endIdx = i;
            
  
           lastU  = from;
           
           variances = zeros(1,1);
           
           for i=startIdx:endIdx
                if (i< endIdx) U = sig(1).tenor(i);else U = to;end;
                if (i< alphaSize) vol(1) = sig(1).quote(i);else vol(1) = sig(1).quote(alphaSize);end;
                correl =ones(1,1);
                for j=1:1
                    for k=1:1
                        scale = U - lastU;
                        variances(j,k) = variances(j,k) + vol(j)*vol(k)*correl(j,k)...
                                       * scale;
                    end
                end
                lastU = U;
           end
           
        end
        
        function out = dAlpha_Test(lgm1F,t,Alpha1)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(Alpha1.tenor)
                alpha = Alpha1.quote(i);
                if t<= Alpha1.tenor(i)
                    sum = alpha;
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = alpha;
            end
            out = sum;
        end
        
        function out = Alpha_Test(lgm1F,t,Alpha1)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(Alpha1.tenor)
                alpha = Alpha1.quote(i) * Alpha1.quote(i);
                if t<= Alpha1.tenor(i)
                    sum = sum + alpha*(t-prev_t);
                    inRangeYN = true;
                    break;
                else
                    sum = sum + alpha*(Alpha1.tenor(i)-prev_t);
                end
                prev_t = Alpha1.tenor(i);
            end
            
            if ~inRangeYN 
                sum = sum + alpha * (t - Alpha1.tenor(i));
            end
            out = sum;
        end
        
        function out = dH_Test(lgm1F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = dH.quote(i);
                    inRangeYN = true;
                    break;
                end
            end
            
            if ~inRangeYN 
                sum = dH.quote(i);
            end
            out = sum;
        end
        
        function out = H_Test(lgm1F,t,dH)
            
            sum = 0.0;
            prev_t = 0.0;
            inRangeYN = false;
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    sum = sum + dH.quote(i)*(t-prev_t);
                    inRangeYN = true;
                    break;
                else
                    sum = sum + dH.quote(i)*(dH.tenor(i)-prev_t);
                end
                prev_t = dH.tenor(i);
            end
            
            if ~inRangeYN 
                sum = sum + dH.quote(i) * (t - dH.tenor(i));
            end
            out = sum;
        end
        
        function out = H(lgm1F,t,dH)
            for i=1:length(dH.tenor)
                if t<= dH.tenor(i)
                    break;
                end
            end
            tidx = i;
            sum = 0.0;
            prev_t = 0.0;
            curr_t = 0.0;
            for i=1:tidx
                if i< tidx
                    curr_t = dH.tenor(i);
                else
                    curr_t = t;
                end
                sum = sum + dH.quote(i)*(curr_t-prev_t);

                prev_t = dH.tenor(i);
            end
            out = sum;
        end


        function beta = Beta(lgm1F,t,T)
            beta =[];
            beta(1) = lgm1F.H(T,lgm1F.dH1)-lgm1F.H(t,lgm1F.dH1);
        end
        
        function beta = BetaDate(lgm1F,valueDate,tDate,TDate)
            t = tDate.DateDiff(valueDate)/365.0;
            T = TDate.DateDiff(valueDate)/365.0;
            
            beta =[];
            
            beta(1) = lgm1F.H(T,lgm1F.dH1)-lgm1F.H(t,lgm1F.dH1);
            
        end
        
        function c_Test = C_Test(lgm1F,tx, tnr,tau)
            pv01_1 = lgm1F.PV01(tx,tx+tnr,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr,tau,pv01_1,fsr_1);
            c_Test = c_nN_1;
        end
        
        function c_nN = C_nN(lgm1F,tn,tN,tau,pnN,ynN)
            p = @lgm1F.DF;
            c_nN=[0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm1F.Beta(tn,tp);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
            
        end
        
        function c_nN = C_nNDate(lgm1F,valueDate,tnDate,tNDate,tau,pnN,ynN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @lgm1F.DF;
            c_nN=[0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = lgm1F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                c_nN(1) = c_nN(1) +tau * beta(1) * df;
            end
            
            c_nN(1) = c_nN(1)* ynN;
            
            c_nN(1) = c_nN(1) + beta(1)*df;
            
            c_nN(1) = c_nN(1)/pnN;
        end
        
        function g_nN = G_nN(lgm1F,tn,tN,tau,pnN)
            p = @lgm1F.DF;
            g_nN=[0.0];
            for i=1:round((tN-tn)/tau)
                tp = tn+i*tau;
                beta = lgm1F.Beta(tn,tp);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
        end
        
        function g_nN = G_nNDate(lgm1F,valueDate,tnDate,tNDate,tau,pnN)
            tn = tnDate.DateDiff(valueDate)/365.0;
            tN = tNDate.DateDiff(valueDate)/365.0;
            addMonthUnit = round(12*tau);
            cnt = round((tN-tn)/tau);
            
            p = @lgm1F.DF;
            g_nN=[0.0];
            
            for i=1:cnt
                tpDate = tnDate.AddDate(i*addMonthUnit,'month');
                tp = tpDate.DateDiff(valueDate)/365.0;
                beta = lgm1F.BetaDate(valueDate,tnDate,tpDate);
                df = p(tp);
                g_nN(1) = g_nN(1) +tau * beta(1) * df;
            end
            
            g_nN(1) = g_nN(1)/pnN;
        end
        
        function h_nTerm = H_nTerm(lgm1F,tn,tTerm)
           h_nTerm =[];
           beta = lgm1F.Beta(tn,tTerm);
           h_nTerm(1) = -1.0*beta(1);
        end
        
        function h_nTerm = H_nTermDate(lgm1F,valueDate,tnDate,tTermDate)
           h_nTerm =[];
           beta = lgm1F.BetaDate(valueDate,tnDate,tTermDate);
           h_nTerm(1) = -1.0*beta(1);
        end
        
        function out = CDDigitalArgDate(lgm1F,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           toTime =  toDate.DateDiff(valueDate);
           to      = toTime/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1Time = tN1Date.DateDiff(valueDate);
           tN1   =  tN1Time/365.0;
           
           deltaT = (tN1Time - toTime)/365.0;
           
           df_Tn1 = lgm1F.DF(to);
           df_TN1 = lgm1F.DF(tN1);
           
           c_nN1 = lgm1F.BetaDate(valueDate,toDate,tN1Date);
           c_nTerm = lgm1F.BetaDate(valueDate,toDate,tpDate);
           
           varCovar = lgm1F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:1
               for j=1:1
                    driftTerm = driftTerm + c_nN1(i) * c_nTerm(j) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:1
               for j=1:1
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
%            out.mu = 0.5*variance  -1.0*log(df_TN1/df_Tn1) - driftTerm;
           out.mu = -0.5*variance  + 1.0*log(df_TN1/df_Tn1) + driftTerm;
           
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
           out.deltaT = deltaT;
        end
        
        function out = CMSDigitalArg(lgm1F,to,tp,tenor,tau)
           % probability that CMS rate > strike
           p_nN = lgm1F.PV01(to,to+tenor,tau);
           df_Tn = lgm1F.DF(to);
           df_TN = lgm1F.DF(to + tenor);
           y_nN = (df_Tn - df_TN)/p_nN;
           c_nN = lgm1F.C_nN(to,to + tenor,tau,p_nN,y_nN);
           g_nN = lgm1F.G_nN(to,to + tenor,tau,p_nN);
           h_nTerm = lgm1F.H_nTerm(to,tp);
           varCovar = lgm1F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:1
               for j=1:1
                    driftTerm = driftTerm + c_nN(i) * (h_nTerm(j) + g_nN(j)) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:1
               for j=1:1
                    variance = variance + c_nN(i) * c_nN(j) * varCovar(i,j);
               end
           end
           
           out.mu = y_nN + driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN;
           out.varCovar = varCovar;
        end
        
        function out = CMSDigitalArgDate(lgm1F,valueDate,toDate,tpDate,tenor1,tau)
           % probability that CMS rate > strike
            %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           if tenor1 < 1 && tenor1 >= 1.0/12.0
               tenorMonth = round(12*tenor1);
                tN1Date = toDate.AddDate(tenorMonth,'month');
           elseif tenor1 >= 1
               tN1Date = toDate.AddDate(tenor1,'year');
           else
               disp('unimplemented')
           end
           
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = lgm1F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = lgm1F.DF(to);
           df_TN1 = lgm1F.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = lgm1F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = lgm1F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = lgm1F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = lgm1F.LocalVariance(0,to);
           
           driftTerm=0.0;
           for i=1:1
               for j=1:1
                    driftTerm = driftTerm + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
               end
           end
           
           variance = 0.0;
           for i=1:1
               for j=1:1
                    variance = variance + c_nN1(i) * c_nN1(j) * varCovar(i,j);
               end
           end
           
           out.mu = y_nN1 + driftTerm;
           out.sigma = sqrt(variance);
           out.c_nN = c_nN1;
           out.varCovar = varCovar;
        end
        
        function out = CMSDigital(lgm1F,to,tp,tenor,tau,strike)
           cmsDigitalArg= lgm1F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalDate(lgm1F,valueDate,toDate,tpDate,tenor1,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = lgm1F.CMSDigitalArgDate(valueDate,toDate,tpDate,tenor1,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSDigitalRange(lgm1F,to,tp,tenor,tau,lower,upper)
            lower_value = lgm1F.CMSDigital(to,tp,tenor,tau,lower);
            upper_value = lgm1F.CMSDigital(to,tp,tenor,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSDigitalRangeDate(lgm1F,valueDate,toDate,tpDate,tenor1,tau,lower,upper)
            lower_value = lgm1F.CMSDigitalDate(valueDate,toDate,tpDate,tenor1,tau,lower);
            upper_value = lgm1F.CMSDigitalDate(valueDate,toDate,tpDate,tenor1,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CDDigitalDate(lgm1F,valueDate,toDate,tpDate,tenor,tau,strike)
           
           % probability that CD > strikeCD 
           % probability that CMS tenor1 - CMS tenor 2 > strike
           
           % CD
           cdDigitalArg = lgm1F.CDDigitalArgDate(valueDate,toDate,tpDate,tenor,tau);
           mu = cdDigitalArg.mu;
           sigma = cdDigitalArg.sigma;
           c_nN = cdDigitalArg.c_nN;
           varCovar = cdDigitalArg.varCovar;
           deltaT = cdDigitalArg.deltaT;
           
%            %CMS Spread
%            cmsSpreadDigitalArg = lgm2F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
%            
%            c_nN1 = cmsSpreadDigitalArg.c_nN1;
%            c_nN2 = cmsSpreadDigitalArg.c_nN2;
%            
%            muS = cmsSpreadDigitalArg.mu;
%            sigmaS = cmsSpreadDigitalArg.sigma;
%            
           % covariance between CMS Rate & CMS Spread
%            covar = 0.0;
%            for i=1:2
%                for j=1:2
%                     covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
%                end
%            end
           
%            corr = covar/(sigma*sigmaS);
           
           z1 = (mu + log(1.0 + deltaT*strike))/sigma;
%            z2 = (muS - strikeS)/sigmaS;
           if ~isreal(z1) || ~isfinite(z1)
               out = 0.0;
           else
               out = H_ncdf(z1);
           end
             
%            mbar = [0 0];
%            corrBar = [1.0 corr;corr 1.0];
%            xbar = [z1 z2];
%            out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CDDigitalRangeDate(lgm1F,valueDate,toDate,tpDate,tenor,tau,lower,upper)
%             lower_value = lgm1F.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,lower);
%             upper_value = lgm1F.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,upper);
%             out = lower_value - upper_value;
%             
            upper_value = lgm1F.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,upper);
            lower_value = lgm1F.CDDigitalDate(valueDate,toDate,tpDate,tenor,tau,lower);
            out = upper_value - lower_value;
            
        end
        
        function out = CMSSpreadDigitalArg(lgm1F,to,tp,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           p_nN1 = lgm1F.PV01(to,to+tenor1,tau);
           df_Tn1 = lgm1F.DF(to);
           df_TN1 = lgm1F.DF(to + tenor1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           c_nN1 = lgm1F.C_nN(to,to + tenor1,tau,p_nN1,y_nN1);
           g_nN1 = lgm1F.G_nN(to,to + tenor1,tau,p_nN1);
           h_nTerm1 = lgm1F.H_nTerm(to,tp);
           
           %CMS2
           p_nN2 = lgm1F.PV01(to,to+tenor2,tau);
           df_Tn2 = lgm1F.DF(to);
           df_TN2 = lgm1F.DF(to + tenor2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           c_nN2 = lgm1F.C_nN(to,to + tenor2,tau,p_nN2,y_nN2);
           g_nN2 = lgm1F.G_nN(to,to + tenor2,tau,p_nN2);
           h_nTerm2 = lgm1F.H_nTerm(to,tp);
           
           varCovar = lgm1F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:1
               for j=1:1
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:1
               for j=1:1
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
         function out = CMSSpreadDigitalArgDate(lgm1F,valueDate,toDate,tpDate,tenor1,tenor2,tau)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           to      = toDate.DateDiff(valueDate)/365.0;
           tN1Date = toDate.AddDate(tenor1,'year');
           tN1   =  tN1Date.DateDiff(valueDate)/365.0;
           
           p_nN1 = lgm1F.PV01Date(valueDate,toDate,tN1Date,tau);
           
           df_Tn1 = lgm1F.DF(to);
           df_TN1 = lgm21.DF(tN1);
           y_nN1 = (df_Tn1 - df_TN1)/p_nN1;
           
           c_nN1 = lgm1F.C_nNDate(valueDate,toDate,tN1Date,tau,p_nN1,y_nN1);
           g_nN1 = lgm1F.G_nNDate(valueDate,toDate,tN1Date,tau,p_nN1);
           h_nTerm1 = lgm1F.H_nTermDate(valueDate,toDate,tpDate);
           
           %CMS2
           tN2Date = toDate.AddDate(tenor2,'year');
           tN2   =  tN2Date.DateDiff(valueDate)/365.0;
           
           p_nN2 = lgm1F.PV01Date(valueDate,toDate,tN2Date,tau);
           
           df_Tn2 = lgm1F.DF(to);
           df_TN2 = lgm1F.DF(tN2);
           y_nN2 = (df_Tn2 - df_TN2)/p_nN2;
           
           c_nN2 = lgm1F.C_nNDate(valueDate,toDate,tN2Date,tau,p_nN2,y_nN2);
           g_nN2 = lgm1F.G_nNDate(valueDate,toDate,tN2Date,tau,p_nN2);
           h_nTerm2 = lgm1F.H_nTermDate(valueDate,toDate,tpDate);
           
           varCovar = lgm1F.LocalVariance(0,to);
           
           driftTerm1=0.0;
           driftTerm2=0.0;
           for i=1:1
               for j=1:1
                    driftTerm1 = driftTerm1 + c_nN1(i) * (h_nTerm1(j) + g_nN1(j)) * varCovar(i,j);
                    driftTerm2 = driftTerm2 + c_nN2(i) * (h_nTerm2(j) + g_nN2(j)) * varCovar(i,j);
               end
           end
           
           spreadVariance = 0.0;
           variance1 = 0.0;
           variance2 = 0.0;
           coVariance = 0.0;
           for i=1:1
               for j=1:1
                    spreadVariance = spreadVariance + (c_nN1(i)-c_nN2(i)) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
                    variance1 = variance1 + (c_nN1(i)) * (c_nN1(j)) * varCovar(i,j);
                    variance2 = variance2 + (c_nN2(i)) * (c_nN2(j)) * varCovar(i,j);
                    coVariance = coVariance + (c_nN1(i)) * (c_nN2(j)) * varCovar(i,j);
               end
           end
           
           out.fwdSpread = y_nN1 - y_nN2;
           out.mu = (y_nN1 + driftTerm1) - (y_nN2 + driftTerm2);
           out.sigma = sqrt(spreadVariance);
           out.c_nN1 = c_nN1;
           out.c_nN2 = c_nN2;
           out.varCovar = varCovar;
           out.mu1 = y_nN1 + driftTerm1;
           out.mu2 = y_nN2 + driftTerm2;
           out.corr = coVariance/sqrt(variance1*variance2);
           out.sig1 = sqrt(variance1/to);
           out.sig2 = sqrt(variance2/to);
           
        end
        
        function out = CMSSpreadDigital(lgm1F,to,tp,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm1F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalDate(lgm1F,valueDate,toDate,tpDate,tenor1,tenor2,tau,strike)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsSpreadDigitalArg = lgm1F.CMSSpreadDigitalArgDate(valueDate,toDate,tpDate,tenor1,tenor2,tau);
           mu = cmsSpreadDigitalArg.mu;
           sigma = cmsSpreadDigitalArg.sigma;
           z1= (mu - strike)/sigma;
           out = H_ncdf(z1);
           
        end
        
        function out = CMSSpreadDigitalRange(lgm1F,to,tp,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm1F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,lower);
            upper_value = lgm1F.CMSSpreadDigital(to,tp,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSSpreadDigitalRangeDate(lgm1F,valueDate,toDate,tpDate,tenor1,tenor2,tau,lower,upper)
            lower_value = lgm1F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,lower);
            upper_value = lgm1F.CMSSpreadDigitalDate(valueDate,toDate,tpDate,tenor1,tenor2,tau,upper);
            out = lower_value - upper_value;
        end
        
        function out = CMSCMSSpreadCovar(lgm1F,c_nN,c_nN1,c_nN2,varCovar,varCovarS)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           covar = 0.0;
           for i=1:1
               for j=1:1
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           out = covar;
           
        end
        
        function out = CMSCMSSpreadDualDigital(lgm1F,to,tp,tenor,tenor1,tenor2,tau,strike,strikeS)
           % probability that CMS tenor1 - CMS tenor 2 > strike
           %CMS 1
           cmsDigitalArg = lgm1F.CMSDigitalArg(to,tp,tenor,tau);
           mu = cmsDigitalArg.mu;
           sigma = cmsDigitalArg.sigma;
           c_nN = cmsDigitalArg.c_nN;
           varCovar = cmsDigitalArg.varCovar;
           
           cmsSpreadDigitalArg = lgm1F.CMSSpreadDigitalArg(to,tp,tenor1,tenor2,tau);
           muS = cmsSpreadDigitalArg.mu;
           sigmaS = cmsSpreadDigitalArg.sigma;
           c_nN1 = cmsSpreadDigitalArg.c_nN1;
           c_nN2 = cmsSpreadDigitalArg.c_nN2;
           varCovarS = cmsSpreadDigitalArg.varCovarS;
           
           % covariance between CMS Rate & CMS Spread
           covar = 0.0;
           for i=1:1
               for j=1:1
                    covar = covar + c_nN(i) * (c_nN1(j)-c_nN2(j)) * varCovar(i,j);
               end
           end
           
           corr = covar/(sigma*sigmaS);
           
           z1 = (mu - strike)/sigma;
           z2 = (muS - strikeS)/sigmaS;
           
           mbar = [0 0];
           corrBar = [1.0 corr;corr 1.0];
           xbar = [z1 z2];
           out = mvncdf(xbar, mbar, corrBar);
           
        end
        
        function out = CMSCMSSpreadDualDigitalRange(lgm1F,to,tp,tenor,tenor1,tenor2,tau,lower,upper,lowerS,upperS)
            value_ll = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,lower,lowerS);
            value_lu = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,lower,upperS);
            value_ul = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,upper,lowerS);
            value_uu = lgm1F.CMSCMSSpreadDualDigital(to,tp,tenor,tenor1,tenor2,tau,upper,upperS);
            out = value_ll - value_lu - value_ul + value_uu;
        end
        
        function out = CMSDigitalRangeAccrual(lgm1F,matu,tenor,tau,coupon,lower,upper,tbd)
           tau = 1.0/(lgm1F.freq);
           periodSum = @(Ta,Tb,tbd) sum(arrayfun(@(i) ...
               1/tbd*CMSDigitalRange(lgm1F,Ta+1/tbd*i,Tb,tenor,tau,lower,upper),...
               1:round((Tb-Ta)*tbd)));
           p = @lgm1F.DF;
           dcf = 1.0;
           out.cashflowNpv = zeros(1,round(matu));
           out.price = 0.0;
           for i=1:round(matu)
               out.cashflowNpv(i) = periodSum((i-1)*tau,i*tau,tbd)*coupon*p(i*tau)*NA;
               out.price = out.price + out.cashflowNpv(i);
           end
           out.cashflowNpv(i+1) = p(matu)*NA;
           out.price = out.price + p(matu)*NA;
        end
        
        function atmSwaption = ATMSwaption(lgm1F,tx,tnr)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            pv01 = lgm1F.PV01(tx,tx+tnr,tau);
            fsr  = lgm1F.Fsr(tx,tx+tnr,tau);
            c_nN = lgm1F.C_nN(tx,tx+tnr,tau,pv01,fsr);
            variances = lgm1F.LocalVariance(0,tx);
            totalVar = 0.0;
            for i=1:1
                for j=1:1
                    totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
                end
            end
            atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
        end
        
        function cap = Cap(lgm1F,cap_strike,dt,Ti)
            idx_mat = round(Ti/dt);
            
            HN = 0.0;
            Hn = 0.0;
            sAlpha = 0.0;
            vol = 0.0;
            d1 = 0.0;
            
            p = @lgm1F.DF;
            model_cap = 0;
            for i=1:idx_mat-1
                Hn = lgm1F.H(i*dt,lgm1F.dH1);
                HN = lgm1F.H((i+1)*dt,lgm1F.dH1);
                sAlpha = lgm1F.Alpha_Test(i*dt,lgm1F.Alpha1);
                vol = (HN-Hn)*sqrt(sAlpha);
                d1 = log(p(i*dt)/(p((i+1)*dt)*(1 + cap_strike*dt)))/vol + 0.5*vol;
                model_cap = model_cap + p(i*dt)*H_ncdf(d1)-p((i+1)*dt)*(1 + cap_strike*dt)*H_ncdf(d1-vol);
            end
            cap = model_cap;
        end
        
%         function atmSwaption = modelSwaptionExpiryTenor(lgm1F,tx,tnr)
%             tau = 0.25;
%             pv01 = lgm1F.PV01(tx,tx+tnr,tau);
%             fsr  = lgm1F.Fsr(tx,tx+tnr,tau);
%             c_nN = lgm1F.C_nN(tx,tx+tnr,tau,pv01,fsr);
%             variances = lgm1F.LocalVariance(0,tx);
%             totalVar = 0.0;
%             for i=1:1
%                 for j=1:1
%                     totalVar = totalVar + c_nN(i)*c_nN(j)*variances(i,j);
%                 end
%             end
%             atmSwaption =  pv01 * sqrt(totalVar)/sqrt(2.0*pi);
%         end
        
        function impCorrel = ImpCorrel(lgm1F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm1F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCorrel = coVar/sqrt(var1)/sqrt(var2);
        end
        
        function impCovar = ImpCovar(lgm1F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm1F.LocalVariance(0,tx);
            var1 = 0;
            var2 = 0;
            coVar = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                    var2 = var2 + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                    coVar = coVar + c_nN_1(j)*c_nN_2(k)*variances(j,k);
                end
            end
            impCovar = coVar;
        end
        
        function spreadInfo = SpreadInfo(ck2F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            cmsSpreadDigitalArg =  ck2F.CMSSpreadDigitalArg(tx,tx+1.0,tnr1,tnr2,tau);
            spreadInfo = cmsSpreadDigitalArg;
            spreadInfo.normSigma = cmsSpreadDigitalArg.sigma/sqrt(tx);
        end
        
        function impVariance = ImpVariance(lgm1F,tx,tnr1)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            variances = lgm1F.LocalVariance(0,tx);
            var1 = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + c_nN_1(j)*c_nN_1(k)*variances(j,k);
                end
            end
            impVariance = var1;
        end
        
        function instFwdCorr = InsFwdCorr(lgm1F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = (lgm1F.H_Test(tx + tnr1 +0.001,lgm1F.dH1)-lgm1F.H_Test(tx + tnr1,lgm1F.dH1))/0.001;
            dH_tT1(2) = (lgm1F.H_Test(tx + tnr1 +0.001,lgm1F.dH2)-lgm1F.H_Test(tx + tnr1,lgm1F.dH2))/0.001;
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = (lgm1F.H_Test(tx + tnr2 +0.001,lgm1F.dH1)-lgm1F.H_Test(tx + tnr2,lgm1F.dH1))/0.001;
            dH_tT2(2) = (lgm1F.H_Test(tx + tnr2 +0.001,lgm1F.dH2)-lgm1F.H_Test(tx + tnr2,lgm1F.dH2))/0.001;
            
            instVariances = (lgm1F.LocalVariance(0,tx+0.001)-lgm1F.LocalVariance(0,tx))/0.001;
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:1
                for k=1:1
                    var1 = var1 + dH_tT1(j)*dH_tT1(k)*instVariances(j,k);
                    var2 = var2 + dH_tT2(j)*dH_tT2(k)*instVariances(j,k);
                    covar = covar + dH_tT1(j)*dH_tT2(k)*instVariances(j,k);
                end
            end
            instFwdCorr = covar/sqrt(var1*var2);
        end
        
        function instFwdCorrTest = InsFwdCorrTest(lgm1F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            dH_tT1 = zeros(2,1);
            dH_tT1(1) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH1);
            dH_tT1(2) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH2);
            
            dH_tT2 = zeros(2,1);
            dH_tT2(1) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH1);
            dH_tT2(2) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH2);
            
            dAlpha_t = zeros(2,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
            dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            var1 = 0;
            var2 = 0;
            for j=1:1
                var1 = var1 + dH_tT1(j)*dH_tT1(j)*dAlpha_t(j)*dAlpha_t(j);
                var2 = var2 + dH_tT2(j)*dH_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
                covar = covar + dH_tT1(j)*dH_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdCorrTest = covar/sqrt(var1*var2);
        end
        
        function instFwdCovarTestMatrix = InsFwdCovarTest2(lgm1F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCovarTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),2);
                eta =  zeros(2,2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH1) - lgm1F.H_Test(newtnr(i),lgm1F.dH1))/tau;
                    dH_tT(i,2) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH2) - lgm1F.H_Test(newtnr(i),lgm1F.dH2))/tau;
                end

                eta(1,1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
                eta(2,2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
                eta(1,2) = 0;
                eta(2,1) = 0;

                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
            end
            
        end
        
        function instFwdCorrTestMatrix = InsFwdCorrTest2(lgm1F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                instFwdCorrTestMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                coVar = zeros(length(newtnr),length(newtnr));

                dH_tT = zeros(length(newtnr),2);
                eta =  zeros(2,2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH1) - lgm1F.H_Test(newtnr(i),lgm1F.dH1))/tau;
                    dH_tT(i,2) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH2) - lgm1F.H_Test(newtnr(i),lgm1F.dH2))/tau;
                end

                eta(1,1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
                eta(2,2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
                eta(1,2) = 0;
                eta(2,1) = 0;
                
                testLoadingD = dH_tT*eta;
                
                coVar =  dH_tT*eta*eta'*dH_tT';
                instFwdCovarTestMatrix = coVar;
                %normalize the correlation matrix and calculate the loading matrix D
                rInv_m = zeros(length(newtnr),length(newtnr));
                for i=1:length(newtnr)
                    rInv_m(i,i) = 1.0/sqrt(coVar(i,i));
                end
                instFwdCorrTestMatrix = zeros(length(newtnr),length(newtnr));
                instFwdCorrTestMatrix = rInv_m*instFwdCovarTestMatrix*rInv_m';
                
            end
            
        end
        
        function deltaHMatrix = DeltaHTest2(lgm1F,tx,tnr1,tau)
            if (tnr1(length(tnr1)) <= tx)
                disp('beyond the limit')
                deltaHMatrix = 0;
            else

                % find T0 such that T0 =min{T_i|tx <= T_i}
                ipos = find(tnr1-tx > 0);
                T0 = tnr1(ipos(1));
                idx = ipos(1);
                newtnr = tnr1(idx:end);
                % make a matrix that spans T0,...,TN
                
                dH_tT = zeros(length(newtnr),2);
                for i=1:length(newtnr)
                    dH_tT(i,1) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH1) - lgm1F.H_Test(newtnr(i),lgm1F.dH1))/tau;
                    dH_tT(i,2) = (lgm1F.H_Test(newtnr(i) + tau,lgm1F.dH2) - lgm1F.H_Test(newtnr(i),lgm1F.dH2))/tau;
                end

                deltaHMatrix = dH_tT;
            end
            
        end
        
        function etaMatrix = EtaTest2(lgm1F,tx)
            eta =  zeros(1,1);
            eta(1,1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             eta(2,2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
%             eta(1,2) = 0;
%             eta(2,1) = 0;
            
            etaMatrix = eta;
            
        end
        
        function instFwdCovarTest = InsFwdCovarTest(lgm1F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            dH_tT1 = zeros(1,1);
            dH_tT1(1) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH1);
%             dH_tT1(2) = lgm1F.dH_Test(tx + tnr1,lgm1F.dH2);
            
            dH_tT2 = zeros(1,1);
            dH_tT2(1) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH1);
%             dH_tT2(2) = lgm1F.dH_Test(tx + tnr2,lgm1F.dH2);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            for j=1:1
                covar = covar + dH_tT1(j)*dH_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdCovarTest = covar;
        end
        
        function instFwdSwapCovarTest = InsSwapFwdCovarTest(lgm1F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            C_tT1 = zeros(1,1);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
%             C_tT1(2) = c_nN_1(2);
            
            C_tT2 = zeros(1,1);
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
%             C_tT2(2) = c_nN_2(2);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            for j=1:1
                covar = covar + C_tT1(j)*C_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdSwapCovarTest = covar;
        end
        
        function instFwdSwapCorrTest = InsSwapFwdCorrTest(lgm1F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            C_tT1 = zeros(1,1);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            C_tT1(1) = c_nN_1(1);
%             C_tT1(2) = c_nN_1(2);
            
            C_tT2 = zeros(1,1);
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            C_tT2(1) = c_nN_2(1);
%             C_tT2(2) = c_nN_2(2);
            
            dAlpha_t = zeros(1,1);
            dAlpha_t(1) = lgm1F.dAlpha_Test(tx,lgm1F.Alpha1);
%             dAlpha_t(2) = lgm1F.dAlpha_Test(tx,lgm1F.dAlpha2);
            
            covar = 0.0;
            var1 = 0.0;
            var2 = 0.0;
            for j=1:1
                var1 = var1 + C_tT1(j)*C_tT1(j)*dAlpha_t(j)*dAlpha_t(j);
                var2 = var2 + C_tT2(j)*C_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
                covar = covar + C_tT1(j)*C_tT2(j)*dAlpha_t(j)*dAlpha_t(j);
            end
            instFwdSwapCorrTest = covar/sqrt(var1*var2);
        end
        
        function out = RebonatoFwdCorr2(lgm1F,tx,tnr1,tnr2)
           rho_inf = 0.4;
           rho_decay = 0.15;
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function out = RebonatoFwdCorr(lgm1F,tx,tnr1,tnr2)
           rho_inf = 0.4;
           rho_decay = 0.15;
           out =  rho_inf + (1-rho_inf)*exp(-rho_decay*abs(tnr1-tnr2));
        end
        
        function coVariance = CoVariance(lgm1F,tx,tnr1,tnr2)
%             tau = 0.25;
            tau = 1.0/(lgm1F.freq);
            pv01_1 = lgm1F.PV01(tx,tx+tnr1,tau);
            fsr_1  = lgm1F.Fsr(tx,tx+tnr1,tau);
            c_nN_1 = lgm1F.C_nN(tx,tx+tnr1,tau,pv01_1,fsr_1);
            
            pv01_2 = lgm1F.PV01(tx,tx+tnr2,tau);
            fsr_2  = lgm1F.Fsr(tx,tx+tnr2,tau);
            c_nN_2 = lgm1F.C_nN(tx,tx+tnr2,tau,pv01_2,fsr_2);
            
            variances = lgm1F.LocalVariance(0,tx);
            totalVar = zeros(1,1);
            for j=1:1
                for k=1:1
                    totalVar(1,1) = totalVar(1,1) + c_nN_1(j)*c_nN_2(k)*variances(j,k);
%                     totalVar(1,2) = totalVar(1,2) + c_nN_1(j)*c_nN_2(k)*variances(j,k);
%                     totalVar(2,1) = totalVar(2,1) + c_nN_2(j)*c_nN_1(k)*variances(j,k);
%                     totalVar(2,2) = totalVar(2,2) + c_nN_2(j)*c_nN_2(k)*variances(j,k);
                end
            end
            coVariance =  totalVar;
        end
        
        function out = TargetFunctionATMSwaptionGlobal(lgm1F,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            lgm1FSwaptionPrice = zeros(1,length(tenor));
            
            for j=1:length(lgm1F.dH1.quote)
               lgm1F.dH1.quote(j) = tvar(j);
            end

            for k=1:length(lgm1F.Alpha1.quote)
               lgm1F.Alpha1.quote(k) = tvar(length(lgm1F.dH1.quote)+k);
            end
            
            out= 0;
            for j=1:length(expiry)
                for k=1:length(tenor)
                    lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                    out = out + (lgm1FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*(lgm1FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0)*10000;
                end
            end
        end
        
        
        function out = TargetFunctionATMSwaptionGlobalLevMar(lgm1F,tvar,params)
            expiry = params.expiry;
            tenor = params.tenor;
            blackSwaptionPrice = params.blackSwaptionPrice;
            lgm1FSwaptionPrice = zeros(1,length(tenor));
            % tracking calibration
            
            
%             for j=1:2*length(lgm1F.dH1.quote)+2*length(lgm1F.Alpha1.quote)
%                 lgm1F.debugInfo{1,1}.currentParameterVec(idx,j) = tvar(j);
%             end

            for j=1:length(lgm1F.dH1.quote)
               lgm1F.dH1.quote(j) = tvar(j);
            end

            for k=1:length(lgm1F.Alpha1.quote)
               lgm1F.Alpha1.quote(k) = tvar(length(lgm1F.dH1.quote)+k);
            end
            
             
            
            
            sizeOut = length(expiry) * length(tenor);
            out= zeros(sizeOut,1);
            for j=1:length(expiry)
                for k=1:length(tenor)
                    lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                    out((j-1)*length(tenor) + k) = lgm1FSwaptionPrice(j,k)/blackSwaptionPrice(j,k)-1.0;
                end
            end
        end
        
        
        function out = CalibrateToATMDiagSwaption(lgm1F,black,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            blackSwaptionPrice = zeros(expirySize,tenorSize);
%             blackSwaptionVega =  zeros(expirySize,tenorSize);
            for k=1:expirySize
                for j=1:tenorSize
                    blackSwaptionPrice(k,j) = black.BlackATMSwaption(expiry(k),tenor(j));
%                     blackSwaptionVega(k,j) = black.BlackATMSwaptionVega(expiry(k),tenor(j));
                end
            end
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
%             params.expiry = expiry;
%             params.tenor = tenor;
%             params.blackSwaptionPrice = blackSwaptionPrice;
%             params.blackSwaptionVega  = blackSwaptionVega;
%             params.expirySize = expirySize;
%             params.tenorSize =  tenorSize;
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            target_size = lgm1F.targetSize;
            params.target_size = target_size;
            params.matu = lgm1F.matu;
            params.freq = lgm1F.freq;
            params.swaption_expiry_use_yn = zeros(target_size,1);
            params.dH1.tenor = 1:1:target_size;
            params.dH1.tenor = params.dH1.tenor';
            params.dH1.quote = ones(target_size,1);
            
            params.Alpha1.tenor = 1:1:target_size;
            params.Alpha1.tenor = params.Alpha1.tenor';
            params.Alpha1.quote = 0.01*ones(target_size,1);
            
            params.diagSwaption.expiry = zeros(target_size,1);
            params.diagSwaption.tenor  = zeros(target_size,1);
            params.diagSwaption.quote  = zeros(target_size,target_size);
            params.diagSwaption.strike = zeros(target_size,1);
            params.diagSwaption.blackPrice = zeros(target_size,1);
            
            if lgm1F.matu <= 2.0
                params.swaption_expiry_use_yn(1) = 1;
                params.diagSwaption.expiry =  ones(1,1);
                params.diagSwaption.expiry(1) =  swaptionVolSurface.expiry(1);
                params.diagSwaption.tenor =  ones(1,1);
                params.diagSwaption.tenor(1) =  swaptionVolSurface.tenor(1);
                params.diagSwaption.quote = ones(1,1);
                params.diagSwaption.quote(1,1) = swaptionVolSurface.swaptionVol(expiry(1),tenor(1));
            else
                temp_tenor   = zeros(target_size,1);
                temp_expiry = zeros(target_size,1);
                for i=2:target_size
                    temp_tenor(i-1)  = params.Alpha1.tenor(target_size - (i-1));
                    temp_expiry(i-1) = params.matu - temp_tenor(i-1); 
                end
                
                upper_expiry = swaptionVolSurface.expiry(end);
                upper_tenor  = swaptionVolSurface.tenor(end);
                
                lower_expiry = swaptionVolSurface.expiry(1);
                
                temp_idx= 1;
                for i=1:target_size
                    for j=temp_idx:target_size
                        % only those swaption whose expiry date is away.
                        if temp_expiry(j) > 0
                            params.dH1.tenor(i)     = temp_expiry(j);
                            params.Alpha1.tenor(i) = temp_expiry(j);
                            params.diagSwaption.expiry(i) = temp_expiry(j);
                            params.diagSwaption.tenor(i)  = temp_tenor(j);
                            if(temp_expiry(j) > upper_expiry && temp_tenor(j) <= upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(upper_expiry,temp_tenor(j));
                            elseif(temp_expiry(j) <= upper_expiry && temp_tenor(j) > upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),upper_tenor);    
                            else
                                if temp_expiry(j) < lower_expiry
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(lower_expiry,temp_tenor(j));
                                else
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),temp_tenor(j));
                                end
                            end
                            
                            params.swaption_expiry_use_yn(i) = 1;
                            temp_idx = j+1;
                            break;
                        end
                    end
                    
                    if i>=2
                        params.dH1.tenor(i)     = params.dH1.tenor(i-1)     + 1;
                        params.Alpha1.tenor(i) = params.Alpha1.tenor(i-1) + 1;
                    end
                end
                
            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            lgm1F.dH1    = params.dH1;
            lgm1F.Alpha1 = params.Alpha1;
            tau = 1.0/(lgm1F.freq);
          %% target swaption price and forward swap rate
            for i=1:target_size
                if params.swaption_expiry_use_yn(i)
                    Ti = params.diagSwaption.expiry(i);
                    Tj = params.diagSwaption.tenor(i);
                    blackVolQuote = params.diagSwaption.quote(i,i);
                    params.diagSwaption.strike(i) = black.Fsr(Ti,Ti + Tj,tau);
%                     params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVol(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote);
                    params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVolTau(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote,tau);
                    
                end
            end
            
           %% target swaption price and forward swap rate
            % tracking calibration
            
            out.blackSwaptionPrice = params.diagSwaption.blackPrice;
            out.lgm1FSwaptionPrice = zeros(target_size,1);
            out.priceRelDiff = zeros(target_size,1);
            out.priceRmseTotal = 0.0;
            out.dH1 = params.dH1;
            out.Alpha1 = params.Alpha1;
            
            out.market_swaption = zeros(target_size,1);
            out.model_swaption = zeros(target_size,1);
            out.re = zeros(target_size,1);
            out.rmseTotal = 0;
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
            targetSwaptionSize = 0;
            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    
                    for j=1:length(lgm1F.Alpha1.quote)
                        if params.swaption_expiry_use_yn(j) == 1
                            targetSwaptionSize = targetSwaptionSize +1;
                        end
                    end
                    
                    optParams.dH_compare = params.dH1.quote(1);
                    
                    if lgm1F.matu <= 2.0
                        optParams.dHSize = targetSwaptionSize;
                    else
                        optParams.dHSize = targetSwaptionSize + 1;
                    end
                    
                    optParams.AlphaSize = targetSwaptionSize;
                    
                    optParams.modelParamsSize = optParams.dHSize + optParams.AlphaSize;
                    
                    if lgm1F.matu <= 2.0
                        optParams.numOfEquation = targetSwaptionSize + ... % # of target swaption instruments
                                                        0 + ...           % # of Alpha term structure gamma constraints
                                                  optParams.dHSize;    % # of dH absolute error relative to 1.0 
                    else
                        optParams.numOfEquation = targetSwaptionSize + ...         % # of target swaption instruments
                                                  optParams.AlphaSize -2 + ...     % # of Alpha term structure gamma constraints
                                                  optParams.dHSize;                % # of dH absolute error relative to 1.0 
                    end
                    
                    out.modelParamsSize = optParams.modelParamsSize;
                    out.numOfEquation = optParams.numOfEquation;
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=1:optParams.dHSize
                        tvar(j) = params.dH1.quote(j);
                        lb(j)= 0.0001;
                        ub(j)= 10;
                    end
                    
                    for j=1:optParams.AlphaSize
                        tvar(j + optParams.dHSize) = params.Alpha1.quote(j);
                        lb(j + optParams.dHSize)= 0.0001;
                        ub(j + optParams.dHSize)= 10;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionLGM1FATMSwaptionDiagGlobal(lgm1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM1FATMSwaptionDiagGlobal',tvar,x, 200, options,'unc',lgm1F,optParams);
                    
                    
                    % assigning model parameters
                    k=1;
                    for j=1:optParams.dHSize
                       lgm1F.dH1.quote(j) = max(popt(j),0.0001);
                       k=k+1;
                    end

                    for j=k:length(lgm1F.dH1.quote)
                       lgm1F.dH1.quote(j) = lgm1F.dH1.quote(optParams.dHSize);
                    end

                    k = 1;
                    for j=1:optParams.AlphaSize
                       lgm1F.Alpha1.quote(j) = max(popt(optParams.dHSize+j),0.0001);
                       k=k+1;
                    end

                    % flat extrapolation of Alpha
                    % AlphaSize != length(Alpha)
                    for j=k:length(lgm1F.Alpha1.quote)
                       lgm1F.Alpha1.quote(j) = lgm1F.Alpha1.quote(optParams.AlphaSize);
                    end
                    
                    out.dH1.quote = lgm1F.dH1.quote;
                    out.Alpha1.quote = lgm1F.Alpha1.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        if params.swaption_expiry_use_yn(i) == 1
                            Ti = params.diagSwaption.expiry(i);
                            Tj = params.diagSwaption.tenor(i);
                            out.market_swaption(i) = params.diagSwaption.blackPrice(i);
                            out.model_swaption(i)  = lgm1F.ATMSwaption(Ti,Tj);
                            out.re(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                            out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                            N=N+1; 
                        end
                    end
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    
                    
                case 'globalLevMar'
                    for j=1:length(lgm1F.dH1.quote)
                        tvar(j) = lgm1F.dH1.quote(j);
                        lb(j)= -10;
                        ub(j)= 10;
                    end

                    for k=1:length(lgm1F.Alpha1.quote)
                        tvar(length(lgm1F.dH1.quote)+k) = lgm1F.Alpha1.quote(k);
                        lb(length(lgm1F.dH1.quote)+k)= -10;
                        ub(length(lgm1F.dH1.quote)+k)= 10;
                    end
                    
                    % initial parameters costval
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.init_lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                            out.init_relDiff(j,k) = out.init_lgm1FSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.init_rmseTotal = out.init_rmseTotal + out.init_relDiff(j,k)*out.init_relDiff(j,k);
                            N=N+1;
                        end
                    end
                    out.init_rmseTotal = sqrt(out.init_rmseTotal/N);
                       
                    targetfun = @(tvar) TargetFunctionATMSwaptionGlobalLevMar(lgm1F,tvar,params);
                    options=optimset('Algorithm','levenberg-marquardt','MaxFunEvals',700);
                    [x, fval] = lsqnonlin(targetfun,tvar,[],[],options);
%                     [x, fval] = lsqnonlin(targetfun,tvar,lb,ub,options);
%                     options=optimset('Algorithm','trust-region-reflective');
%                     [x, fval] = fmincon(targetfun,tvar,[],[],[],[],lb,ub,[],options);
                    
                    for j=1:length(lgm1F.dH1.quote)
                        out.dH1.quote(j) = x(j);
                        lgm1F.dH1.quote(j) = x(j);
                    end
            
                    for k=1:length(lgm1F.Alpha1.quote)
                        out.Alpha1.quote(k) = x(length(lgm1F.dH1.quote)+k);
                        lgm1F.Alpha1.quote(k) = x(length(lgm1F.dH1.quote)+k);
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.lgm1FSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
            
                    
                case 'dummy'
                    
                    for j=1:length(lgm1F.dH1.quote)
                        out.dH1.quote(j) = lgm1F.dH1.quote(j);
                        out.dH2.quote(j) = lgm1F.dH2.quote(j);
                    end
            
                    for k=1:length(lgm1F.Alpha1.quote)
                        out.Alpha1.quote(k) =  lgm1F.Alpha1.quote(k);
                        out.dAlpha2.quote(k) = lgm1F.dAlpha2.quote(k);
                    end
                    
                    N=0;
                    for j=1:expirySize
                        for k=1:tenorSize
                            out.blackATMSwaptionPrice(j,k) = blackSwaptionPrice(j,k);
                            out.lgm1FSwaptionPrice(j,k) = lgm1F.ATMSwaption(expiry(j),tenor(k));
                            out.relDiff(j,k) = out.lgm1FSwaptionPrice(j,k) /out.blackATMSwaptionPrice(j,k)-1;
                            out.rmseTotal = out.rmseTotal + out.relDiff(j,k)*out.relDiff(j,k);
                            N=N+1;
                        end
                    end
                   
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMCapDiagSwaption(lgm1F,black,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
            capVol = black.capVol;
            capVolData = black.mktData('capVol');
            capVolrawData =  capVolData.params('rawData');
            
            capVolTermStructure.expiry = capVolrawData(:,1);
            capVolTermStructure.capVol = capVol; % capVol With interpolator
            
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            target_size = lgm1F.targetSize;
            params.target_size = target_size;
            params.matu = lgm1F.matu;
            params.freq = lgm1F.freq;
            
            params.cap_expiry_use_yn = zeros(target_size,1);
            params.swaption_expiry_use_yn = zeros(target_size,1);
            
            params.dH1.tenor = 1:1:target_size;
            params.dH1.tenor = params.dH1.tenor';
            params.dH1.quote = 0.1*ones(target_size,1);
            
            params.Alpha1.tenor = 1:1:target_size;
            params.Alpha1.tenor = params.Alpha1.tenor';
            params.Alpha1.quote = 0.01*ones(target_size,1);
            
            params.diagSwaption.expiry = zeros(target_size,1);
            params.diagSwaption.tenor  = zeros(target_size,1);
            params.diagSwaption.quote  = zeros(target_size,target_size);
            params.diagSwaption.strike = zeros(target_size,1);
            params.diagSwaption.blackPrice = zeros(target_size,1);
            
            params.cap.expiry = zeros(target_size,1);
            params.cap.quote = zeros(target_size,1);
            params.cap.strike = zeros(target_size,1);
            params.cap.blackPrice = zeros(target_size,1);
            
            % setting target diagonal swaption 
            if lgm1F.matu <= 2.0
                params.swaption_expiry_use_yn(1) = 1;
                params.diagSwaption.expiry =  ones(1,1);
                params.diagSwaption.expiry(1) =  swaptionVolSurface.expiry(1);
                params.diagSwaption.tenor =  ones(1,1);
                params.diagSwaption.tenor(1) =  swaptionVolSurface.tenor(1);
                params.diagSwaption.quote = ones(1,1);
                params.diagSwaption.quote(1,1) = swaptionVolSurface.swaptionVol(expiry(1),tenor(1));
            else
                temp_tenor   = zeros(target_size,1);
                temp_expiry = zeros(target_size,1);
                for i=2:target_size
                    temp_tenor(i-1)  = params.Alpha1.tenor(target_size - (i-1));
                    temp_expiry(i-1) = params.matu - temp_tenor(i-1); 
                end
                
                upper_expiry = swaptionVolSurface.expiry(end);
                upper_tenor  = swaptionVolSurface.tenor(end);
                
                lower_expiry = swaptionVolSurface.expiry(1);
                
                temp_idx= 1;
                for i=1:target_size
                    for j=temp_idx:target_size
                        % only those swaption whose expiry date is away.
                        if temp_expiry(j) > 0
                            params.Alpha1.tenor(i) = temp_expiry(j);
                            params.diagSwaption.expiry(i) = temp_expiry(j);
                            params.diagSwaption.tenor(i)  = temp_tenor(j);
                            if(temp_expiry(j) > upper_expiry && temp_tenor(j) <= upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(upper_expiry,temp_tenor(j));
                            elseif(temp_expiry(j) <= upper_expiry && temp_tenor(j) > upper_tenor)
                                params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),upper_tenor);    
                            else
                                if temp_expiry(j) < lower_expiry
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(lower_expiry,temp_tenor(j));
                                else
                                    params.diagSwaption.quote(i,i) = swaptionVolSurface.swaptionVol(temp_expiry(j),temp_tenor(j));
                                end
                            end
                            
                            params.swaption_expiry_use_yn(i) = 1;
                            temp_idx = j+1;
                            break;
                        end
                    end
                    
                    if i>=2
                        params.Alpha1.tenor(i) = params.Alpha1.tenor(i-1) + 1;
                    end
                end
                
            end
            
            % setting target cap
            params.cap.expiry(1) = 1;
            params.cap.quote(1) = capVolTermStructure.capVol(params.cap.expiry(1));
            params.cap_expiry_use_yn(1) = 1;
            
            for i=2:target_size
                params.cap.expiry(i) = i;
                params.cap.quote(i) = capVolTermStructure.capVol(params.cap.expiry(i));
                if params.cap.expiry(i-1) < params.matu + 0.25
                    params.cap_expiry_use_yn(i) = 1;
                end
                if i == 10  % exclude 12Y, 15Y cap vol from target     
                    break;
                end
            end
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            lgm1F.dH1    = params.dH1;
            lgm1F.Alpha1 = params.Alpha1;
            
            tau = 1.0/(lgm1F.freq);
          %% target swaption price and forward swap rate
            for i=1:target_size
                if params.swaption_expiry_use_yn(i)
                    Ti = params.diagSwaption.expiry(i);
                    Tj = params.diagSwaption.tenor(i);
                    blackVolQuote = params.diagSwaption.quote(i,i);
                    params.diagSwaption.strike(i) = black.Fsr(Ti,Ti + Tj,tau);
%                     params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVol(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote);
                    params.diagSwaption.blackPrice(i) = black.BlackSwaptionWithVolTau(Ti,Tj,params.diagSwaption.strike(i),blackVolQuote,tau);
                end
            end
            
           %% target cap price and forward ATM strike
            for i=1:target_size
                if params.cap_expiry_use_yn(i)
                    Ti = params.cap.expiry(i);
                    params.cap.strike(i) = black.Fsr(0.25,Ti,0.25);
                    params.cap.blackPrice(i) = black.BlackCapWithVol(Ti,params.cap.strike(i),0.25,params.cap.quote(i));
                end
            end
            
           %% target swaption price and forward swap rate
            % tracking calibration
            params.cap_size = target_size;
            params.swaption_size = target_size;
            
            out.cap_size = target_size;
            out.swaption_size = target_size;
            
            out.blackSwaptionPrice = params.diagSwaption.blackPrice;
            out.lgm1FSwaptionPrice = zeros(target_size,1);
            out.swaptionPriceRelDiff = zeros(target_size,1);
            out.swaptionPriceRmseTotal = 0.0;
            
            out.blackCapPrice = params.cap.blackPrice;
            out.lgm1FCapPrice = zeros(target_size,1);
            out.capPriceRelDiff = zeros(target_size,1);
            out.capPriceRmseTotal = 0.0;
            
            out.dH1 = params.dH1;
            out.Alpha1 = params.Alpha1;
            
            out.market_swaption = zeros(target_size,1);
            out.model_swaption = zeros(target_size,1);
            
            out.market_cap = zeros(target_size,1);
            out.model_cap = zeros(target_size,1);
            
%             out.re = zeros(target_size,1);
%             out.rmseTotal = 0;
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;
%             targetSwaptionSize = 0;
%             targetCapSize = 0;
            
%             dH_size = 0;
%             Alpha_size = 0;
            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    optParams.dH_size =0;
                    optParams.Alpha_size =0;
                    
                    for j=1:out.cap_size
                        if params.cap_expiry_use_yn(j) == 1
                            optParams.dH_size = optParams.dH_size +1;
                        end
                    end
                    
                    for j=1:out.swaption_size
                        if params.swaption_expiry_use_yn(j) == 1
                            optParams.Alpha_size = optParams.Alpha_size +1;
                        end
                    end
                    
                    if lgm1F.matu <= 10.0
                        optParams.dH_size = optParams.dH_size - 1;
                        optParams.Alpha_size = optParams.Alpha_size + 1;
                    end
                    
                    optParams.dH_1Y_fixed_value = params.dH1.quote(1);
                    
                    optParams.modelParamsSize = optParams.dH_size + optParams.Alpha_size;
                    
                    optParams.numOfEquation = optParams.dH_size + optParams.Alpha_size; 
                    
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=2:optParams.dH_size+1
                        tvar(j-1) = params.dH1.quote(j);
                        lb(j-1)= -10.0;
                        ub(j-1)= 10.0;
                    end
                    
                    for j=1:optParams.Alpha_size
                        tvar(j + optParams.dH_size) = params.Alpha1.quote(j);
                        lb(j + optParams.dH_size)= -10.0;
                        ub(j + optParams.dH_size)= 10.0;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionLGM1FATMSwaptionDiagGlobal(lgm1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM1FATMCapSwaptionDiagGlobal',tvar,x, 200, options,'unc',lgm1F,optParams);
                    
                    
                    % assigning model parameters
                    k=2;
                    for j=2:optParams.dH_size+1
                       lgm1F.dH1.quote(j) = max(abs(popt(j-1)),0.0001);
                       k=k+1;
                    end

                    for j=k:length(lgm1F.dH1.quote)
                       lgm1F.dH1.quote(j) = lgm1F.dH1.quote(optParams.dH_size+1);
                    end

                    k = 1;
                    for j=1:optParams.Alpha_size
                       lgm1F.Alpha1.quote(j) = max(abs(popt(optParams.dH_size+j)),0.0001);
                       k=k+1;
                    end

                    % flat extrapolation of Alpha
                    % AlphaSize != length(Alpha)
                    for j=k:length(lgm1F.Alpha1.quote)
                       lgm1F.Alpha1.quote(j) = lgm1F.Alpha1.quote(optParams.Alpha_size);
                    end
                    
                    out.dH1.quote = lgm1F.dH1.quote;
                    out.Alpha1.quote = lgm1F.Alpha1.quote;
                    
                    N=0;
                    for i=1:params.swaption_size
                        if params.swaption_expiry_use_yn(i) == 1
                            Ti = params.diagSwaption.expiry(i);
                            Tj = params.diagSwaption.tenor(i);
                            out.market_swaption(i) = params.diagSwaption.blackPrice(i);
                            out.model_swaption(i)  = lgm1F.ATMSwaption(Ti,Tj);
                            out.swaptionPriceRelDiff(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                            out.swaptionPriceRmseTotal = out.swaptionPriceRmseTotal + out.swaptionPriceRelDiff(i)*out.swaptionPriceRelDiff(i);
                            N=N+1; 
                        end
                    end
                    out.swaptionPriceRmseTotal = sqrt(out.swaptionPriceRmseTotal/N);
                    
                    N=0;
                    for i=1:params.cap_size
                        if params.cap_expiry_use_yn(i) == 1
                            Ti = params.cap.expiry(i);
                            out.market_cap(i) = params.cap.blackPrice(i);
                            out.model_cap(i)  = lgm1F.Cap(params.cap.strike(i),0.25,Ti);
                            out.capPriceRelDiff(i) = out.model_cap(i)/out.market_cap(i) - 1.0;
                            out.capPriceRmseTotal = out.capPriceRmseTotal + out.capPriceRelDiff(i)*out.capPriceRelDiff(i);
                            N=N+1; 
                        end
                    end
                    out.capPriceRmseTotal = sqrt(out.capPriceRmseTotal/N);
                    
                
                otherwise
                    disp('unImplemented')
            end
        end
        
        function out = CalibrateToATMSwaptionTenorDiagSwaption(lgm1F,black,calibrationFlag)
          % Market Data Object , Vanilla Object
          
            swaptionVol = black.swaptionVol;
            swaptionVolData = black.mktData('swaptionVol');
            swaptionVolrawData =  swaptionVolData.params('rawData');
            
            expiry = swaptionVolrawData(2:size(swaptionVolrawData,1),1)';
            tenor  = swaptionVolrawData(1,2:size(swaptionVolrawData,2));
            
            expirySize = size(expiry,2);
            tenorSize = size(tenor,2);
            
            swaptionVolSurface.expiry = expiry;
            swaptionVolSurface.tenor = tenor;
            swaptionVolSurface.swaptionVol = swaptionVol; % volSurface with interpolator
            
          %% select the interpolated target diagonal swaption with expiry &
          %% tenor & linear interpolated vol
            % size for interpolated yearly call (swaption)
            
            % dummy
%             target_size = lgm1F.targetSize;

            params.matu = lgm1F.matu;
            params.freq = lgm1F.freq;
            params.joint_swap_tenor = lgm1F.joint_swap_tenor;
            
            params.dh_initial = 1.0;
            params.alpha_initial = 0.01;
            params.smallNumber = 0.000001;
            
            % set the target swaption expiry & tenor
            remainingMatu = params.matu - floor(params.matu);
            max_swaption_expiry_size = ceil(params.matu);
            
            if remainingMatu == 0
                remainingMatu = 1.0;
            end
            
            if params.matu -1  < params.joint_swap_tenor
                max_swaption_tenor_size = floor(params.joint_swap_tenor);
            else
                max_swaption_tenor_size = floor(params.matu - params.smallNumber);
            end
            
            params.targetSwaption.expiry = zeros(max_swaption_expiry_size,1);
            params.targetSwaption.tenor  = zeros(max_swaption_tenor_size,1);
            params.targetSwaption.quote  = zeros(max_swaption_expiry_size,max_swaption_tenor_size);
            params.swaption_use_yn = zeros(max_swaption_expiry_size,max_swaption_tenor_size);
            
            for i=1:max_swaption_expiry_size
                params.targetSwaption.expiry(i) = remainingMatu + i-1;
            end
            
            for i=1:max_swaption_tenor_size
                params.targetSwaption.tenor(i) = i;
            end
            
            params.target_size = 0;
            for i=1:max_swaption_expiry_size
                swaptionExpiry = params.targetSwaption.expiry(i);
                for j=1:max_swaption_tenor_size
                    swaptionTenor = params.targetSwaption.tenor(j);
                    diag_yn = 0;
                    tenor_yn = 0;
                    
                    if abs(swaptionExpiry + swaptionTenor - params.matu) < params.smallNumber 
                        diag_yn = 1;
                    end
                    
                    if (swaptionTenor == params.joint_swap_tenor) && ((swaptionExpiry <= params.matu) || abs(swaptionExpiry - params.matu) < params.smallNumber)
                        tenor_yn = 1;
                    end
                    
                    if (diag_yn) || (tenor_yn)
                        params.swaption_use_yn(i,j) = 1;
                        params.target_size = params.target_size +1;
                    end
                end
            end
            
            params.target_expiry_idx = zeros(params.target_size,1);
            params.target_tenor_idx = zeros(params.target_size,1);
            params.targetSwaption.strike = zeros(params.target_size,1);
            params.targetSwaption.blackPrice = zeros(params.target_size,1);
            
            idx = 1;
            for i=1:max_swaption_expiry_size
                for j=1:max_swaption_tenor_size
                    if params.swaption_use_yn(i,j) == 1
                        params.target_expiry_idx(idx) = i;
                        params.target_tenor_idx(idx) = j;
                        idx = idx +1;
                    end
                end
            end
            
            dh_size = floor( params.matu) + 1 + params.joint_swap_tenor + 1;
            params.dH1.tenor = zeros(dh_size,1);
            params.dH1.quote = zeros(dh_size,1);
            
            for i=1:dh_size
                params.dH1.tenor(i) = remainingMatu + i-1;
                params.dH1.quote(i) = params.dh_initial;
            end
            
            alpha_size = floor(params.matu - params.smallNumber) + 2;
            params.Alpha1.tenor = zeros(alpha_size,1);
            params.Alpha1.quote = zeros(alpha_size,1);
            
            for i=1:alpha_size
                params.Alpha1.tenor(i) = remainingMatu + i-1;
                params.Alpha1.quote(i) = params.alpha_initial;
            end
            
            upper_expiry = swaptionVolSurface.expiry(end);
            upper_tenor  = swaptionVolSurface.tenor(end);
            lower_expiry = swaptionVolSurface.expiry(1);
            
            tau = 1.0/(lgm1F.freq);
            for i=1:params.target_size
                expiry_idx = params.target_expiry_idx(i);
                tenor_idx = params.target_tenor_idx(i);
                swaptionExpiry = params.targetSwaption.expiry(expiry_idx);
                swaptionTenor = params.targetSwaption.tenor(tenor_idx);
                
                if(swaptionExpiry > upper_expiry && swaptionTenor <= upper_tenor)
                    params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(upper_expiry,swaptionTenor);
                elseif(swaptionExpiry <= upper_expiry && swaptionTenor > upper_tenor)
                    params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(swaptionExpiry,upper_tenor);    
                else
                    if swaptionExpiry < lower_expiry
                        params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(lower_expiry,swaptionTenor);
                    else
                        params.targetSwaption.quote(expiry_idx,tenor_idx) = swaptionVolSurface.swaptionVol(swaptionExpiry,swaptionTenor);
                    end
                end
                
                params.targetSwaption.strike(i) = black.Fsr(swaptionExpiry,swaptionExpiry + swaptionTenor,tau);
%                 annuity = black.PV01(swaptionExpiry,swaptionExpiry + swaptionTenor,0.25);
                blackVolQuote = params.targetSwaption.quote(expiry_idx,tenor_idx); 
%                 params.targetSwaption.blackPrice(i) = black.BlackSwaptionWithVol(swaptionExpiry,swaptionTenor,params.targetSwaption.strike(i),blackVolQuote);
                params.targetSwaption.blackPrice(i) = black.BlackSwaptionWithVolTau(swaptionExpiry,swaptionTenor,params.targetSwaption.strike(i),blackVolQuote,tau);
            end
            
            
            % set the new model parameters in accordance with the target
            % instruments maturities
            
            lgm1F.dH1    = params.dH1;
            lgm1F.Alpha1 = params.Alpha1;
            
          
            
           %% target swaption price and forward swap rate
            % tracking calibration
            
            params.swaption_size = params.target_size;
            out.swaption_size = params.target_size;
            
            out.blackSwaptionPrice = params.targetSwaption.blackPrice;
            out.lgm1FSwaptionPrice = zeros(params.target_size,1);
            
%             out.swaptionPriceRelDiff = zeros(params.target_size,1);
%             out.swaptionPriceRmseTotal = 0.0;
%             
            out.re = zeros(params.target_size,1);
            out.rmseTotal = 0.0;
            
            out.dH1 = params.dH1;
            out.Alpha1 = params.Alpha1;
            
            out.market_swaption = zeros(params.target_size,1);
            out.model_swaption = zeros(params.target_size,1);
            
            
            tvar = [];
            calibrationType = 'global';
            if nargin < 3
                calibrationType = 'global'; 
            else
                calibrationType = calibrationFlag;
            end
            N=0;

            switch calibrationType
                case 'global'
                    
                    optParams.params = params;
                    
                    optParams.dH_size = length(params.dH1.quote);  
                    optParams.Alpha_size = length(params.Alpha1.quote);
                    
                    % dH1(end-1) = dH1(end)
                    optParams.modelParamsSize = optParams.dH_size-1 + optParams.Alpha_size;
                    
                    optParams.numOfEquation = params.target_size + ...  % target swaption
                                              optParams.Alpha_size -2 + ... % tikhonov regularization gamma constraint
                                              optParams.dH_size;     % constraint on dH  
                    
                    out.modelParamsSize = optParams.modelParamsSize;
                    out.numOfEquation = optParams.numOfEquation;
                    out.target_size = params.target_size;
                    out.Alpha_size = optParams.Alpha_size;
                    out.dH_size = optParams.dH_size;
                    
                    tvar = zeros(optParams.modelParamsSize,1);
                    lb = zeros(optParams.modelParamsSize,1);
                    ub = zeros(optParams.modelParamsSize,1);
                    
                    for j=1:optParams.dH_size-1
                        tvar(j) = params.dH1.quote(j);
                        lb(j)= -10.0;
                        ub(j)= 10.0;
                    end
                    
                    for j=1:optParams.Alpha_size
                        tvar(j + optParams.dH_size-1) = params.Alpha1.quote(j);
                        lb(j + optParams.dH_size-1)= -10.0;
                        ub(j + optParams.dH_size-1)= 10.0;
                    end
                       
%                     targetfun = @(tvar) TargetFunctionLGM1FATMSwaptionDiagGlobal(lgm1F,tvar,optParams);
%                     options=[1E-03, 1E-25, 1E-25, 1E-25, 1E-25];
                    options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-11];
%                     options=[1E-03, 1E-17, 1E-17, 1E-17, 1E-6];
                    x=  zeros(optParams.numOfEquation,1);
                    [ret, popt, info, covar]=levmar('TargetFunctionLGM1FATMSwaptionTenorSwaptionDiagGlobal',tvar,x, 200, options,'unc',lgm1F,optParams);
%                     [ret, popt, info, covar]=levmar('TargetFunctionLGM1FATMSwaptionTenorSwaptionDiagGlobal',tvar,x, 1, options,'unc',lgm1F,optParams);
                    
                    
                    % assigning model parameters
                    for j=1:optParams.dH_size-1
                       lgm1F.dH1.quote(j) = max(abs(popt(j)),0.0001);
                    end
                    lgm1F.dH1.quote(optParams.dH_size) = lgm1F.dH1.quote(optParams.dH_size-1);
                    
                    for j=1:optParams.Alpha_size
                       lgm1F.Alpha1.quote(j) = max(abs(popt(optParams.dH_size-1 + j)),0.0001);
                    end

                    out.dH1.quote = lgm1F.dH1.quote;
                    out.Alpha1.quote = lgm1F.Alpha1.quote;
                    
                    N=0;
                    for i=1:params.target_size
                        expiry_idx = params.target_expiry_idx(i);
                        tenor_idx = params.target_tenor_idx(i);
                        swaptionExpiry = params.targetSwaption.expiry(expiry_idx);
                        swaptionTenor = params.targetSwaption.tenor(tenor_idx);
                        out.market_swaption(i) = params.targetSwaption.blackPrice(i);
                        out.model_swaption(i) = lgm1F.ATMSwaption(swaptionExpiry,swaptionTenor);
                        out.re(i) = out.model_swaption(i)/out.market_swaption(i) - 1.0;
                        out.rmseTotal = out.rmseTotal + out.re(i)*out.re(i);
                        N=N+1;
                    end
                    
                    out.rmseTotal = sqrt(out.rmseTotal/N);
                    
                
                otherwise
                    disp('unImplemented')
            end
        end 
        

    end
    
end



In [ ]:
function y = LewisPrice311(S,K,r,q,T,theta,kappa,sigma,rho,v0,trap,x,w)

% Implements the Heston model using the characteristic function described in
% the article by Alan Lewis (2001) "A Simple Option Formula for General 
% Jump Diffusion and Other Exponential Levy Processes"

% S = spot price
% K = strike price
% r = risk free rate
% q = dividend yield
% T = maturity
% theta, kappa, sigma, rho, v0 = Heston parameters
% trap = Little Trap (0) or Heston (1)
% x, w = abscissas and weights

lambda = 0;

% Compute the integral.
% The integration variable is complex k = kr - (1/2)*i
for j=1:length(x);
	u = x(j) - (1/2)*i;
    int(j) = w(j)*LewisIntegrand311(u,kappa,theta,lambda,rho,sigma,T,S,K,r,q,v0,trap);
end
Integral = sum(int);

% Equation (3.11) in Lewis (2011)
y = S*exp(-q*T) - (1/pi)*sqrt(K*S)*exp(-(r+q)*T/2)*Integral;


In [ ]:
function y = LewisIntegrand311(u,kappa,theta,lambda,rho,sigma,tau,S,K,r,q,v0,Trap)

% Returns the integrand for the formulation in Equation (3.11) of Lewis (2001).
% u = integration variable
% Heston parameters:
%    kappa  = volatility mean reversion speed parameter
%    theta  = volatility mean reversion level parameter
%    lambda = risk parameter
%    rho    = correlation between two Brownian motions
%    sigma  = volatility of variance
%    v0     = initial variance
% Option features.
%    tau = maturity
%    K = strike price
%    S = spot price
%    r = risk free rate
%    q = dividend yield
%    Trap = 1 "Little Trap" formulation 
%           0  Original Heston formulation

% Log of the stock price.
x = log(S);

% Heston a and b parameters
a = kappa*theta;
b = kappa + lambda;

d = sqrt((rho*sigma*i*u - b)^2 + sigma^2*(i*u + u^2));
g = (b - rho*sigma*i*u + d) / (b - rho*sigma*i*u - d);

if Trap==1
	% "Little Heston Trap" formulation
	c = 1/g;
	D = (b - rho*sigma*i*u - d)/sigma^2*((1-exp(-d*tau))/(1-c*exp(-d*tau)));
	G = (1 - c*exp(-d*tau))/(1-c);
	C = (r-q)*i*u*tau + a/sigma^2*((b - rho*sigma*i*u - d)*tau - 2*log(G));
elseif Trap==0
	% Original Heston formulation.
	G = (1 - g*exp(d*tau))/(1-g);
	C = (r-q)*i*u*tau + a/sigma^2*((b - rho*sigma*i*u + d)*tau - 2*log(G));
	D = (b - rho*sigma*i*u + d)/sigma^2*((1-exp(d*tau))/(1-g*exp(d*tau)));
end

% The Heston characteristic function f2 for ln S(T)
CFlnST = exp(C + D*v0 + i*u*x);

% The cf for the Levy process XT
Y = log(S) + (r-q)*tau;
CFXT = exp(-i*u*Y)*CFlnST;
 
% The integrand in Equation (3.11) of Lewis (2001)
u = u + i/2;
W = Y - log(K);
y = real(exp(i*u*W)*CFXT/(u^2 + 1/4));





In [ ]:
classdef Lewis < BlackFormula
    
    methods
        function out = optionOTMRaw(obj,phi,k,t)
            integrand = @(u) (real(exp(-1j.*u.*k).*phi(u-1j./2,t)./(u.^2+ 1/4)));
            k_minus = (k<0).*k;
            out = exp(k_minus) - exp(k./2)./pi .* integral(integrand,0,inf,'RelTol',1e-8);
%             out = exp(k_minus) - exp(k./2)./pi .* integral(integrand,0,inf);
            
        end
        
        function out = optionOTM(obj,phi,k,t)
            out = arrayfun(@(k) obj.optionOTMRaw(phi,k,t),k);
        end
        
        function out = impvolPhi(obj,phi)
            out = @(k,t) (obj.ivOTM(1,exp(k),t,obj.optionOTM(phi,k,t)));
        end
        
        function out = impvolPhiRaw(obj,phi)
            out = @(k,t) (obj.ivOTM(1,exp(k),t,obj.optionOTMRaw(phi,k,t)));
        end
    end
end



In [ ]:
classdef Leg < H_RootObject
    % FixedRateLeg
    % properties: fixedSchedule 
    
    properties(SetAccess = public)
        cashflows
    end
    
    methods
        % constructor
        function leg = Leg(cashflows)
            leg.cashflows = {};
            for i=1:length(cashflows)
                leg.cashflows{i} = cashflows{i};
            end
        end
        
        function out = NPV(leg,discountCurve)
            out = 0;
            for i=1:length(leg.cashflows)
                out = out + leg.cashflows{i}.NPV(discountCurve);
            end
        end
%         
%         function out = BPS(fixedRateLeg,discountCurve)
%             out = 0;
%             for i=1:fixedRateLeg.legSize
%                 out = out + fixedRateLeg.couponRates{i}.BPS(discountCurve);
%             end
%         end
%         
%         function out = StartDate(fixedRateLeg)
%             out = fixedRateLeg.couponRates{1}.accrualStartDate;
%         end
%         
%         function out = EndDate(fixedRateLeg)
%             out = fixedRateLeg.couponRates{fixedRateLeg.legSize}.accrualEndDate;
%         end
%         
    end
    
end



In [ ]:
function res = Lambda (H,N)
    M = 2*N - 2;
    C = zeros (1,M);
    G = 2*H;
    fbc = @(n)((n +1).^ G + abs(n -1).^ G - 2*n.^G )/2;
    C(1:N) = fbc (0:(N -1));
    C(N+1:M) = fliplr(C (2:(N -1)));
    res = real ( fft(C )).^0.5;
    
end

In [ ]:
import pandas as pd
import requests
from lxml import html
from tqdm import tqdm

In [ ]:
# 삼성전자
sample_code = '005930'

In [ ]:
# parsing URL
# 우리 컴퓨터 -> [접속] -> 에프앤가이드(Data Source) -> [크롤링/웹 스크래핑] -> 우리 컴퓨터
# client -> request -> [Server] -> response -> client

SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'


In [ ]:
# object -> 데이터 덩어리(추상화)

snap_url = SNAP_URL.format(sample_code)
snap_content = requests.get(snap_url).content # 문자열 binary
snap_tree = html.fromstring(snap_content) # 객체(object)
per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
per = float(per)

In [ ]:
per

8.15

In [ ]:
ratio_url = RATIO_URL.format(sample_code)
ratio_content = requests.get(ratio_url).content
ratio_tree = html.fromstring(ratio_content)
debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
debt_ratio = float(debt_ratio)

In [ ]:
# stockMkt => KOSPI
# kosdaqMkt => KOSDAQ
# konexMkt => KONEX

In [ ]:
def get_stock_list(market):
    market_code = ''
    if market == 'kospi':
        market_code = 'stockMkt'
    elif market == 'kosdaq':
        market_code = 'kosdaqMkt'
    elif market == 'konex':
        market_code = 'konexMkt'
    kind_url = 'https://kind.krx.co.kr/corpgeneral/corpList.do?method=download&pageIndex=1&currentPageSize=3000&comAbbrv=&beginIndex=&orderMode=3&orderStat=D&isurCd=&repIsuSrtCd=&searchCodeType=&marketType={}&searchType=13&industry=&fiscalYearEnd=all&comAbbrvTmp=&location=all'.format(market_code)                                                                 

    return pd.read_html(kind_url, converters={'종목코드':lambda x: str(x)})[0]


In [ ]:
def converter(x):
    return str(x)

# 람다 함수 == 무명(anonymous) 함수 == 일회용 함수
lambda x: str(x)

<function __main__.<lambda>(x)>

In [ ]:
kospi_df = get_stock_list('kospi')
print(kospi_df.shape)

(829, 9)


In [ ]:
kosdaq_df = get_stock_list('kosdaq')
print(kosdaq_df.shape)

(1632, 9)


In [ ]:
# merge -> SQL Join
# append | kospi_df.append([kosdaq_df])
# concatenate(합치다)
stock_list_df = pd.concat([kospi_df, kosdaq_df] )

In [ ]:
print(stock_list_df.shape)

(2461, 9)


In [ ]:
# stock_list_df['종목코드'].dropna()
stock_list_df = stock_list_df[stock_list_df['종목코드'].notnull()]

In [ ]:
stock_list_df = stock_list_df[~stock_list_df['회사명'].str.contains('스팩|리츠')]
print(stock_list_df.shape)

(2364, 9)


In [ ]:
# list comprehension
stock_list_df.index = [x for x in range(len(stock_list_df))]

In [ ]:
stock_list_df.to_csv('kospi_kosdaq_stock_list.csv', encoding='utf-8', index=True)

In [ ]:
code_list = stock_list_df['종목코드']
code_list

0       100090
1       453340
2       452260
3       450140
4       377740
         ...  
2359    013030
2360    019550
2361    019570
2362    019590
2363    006920
Name: 종목코드, Length: 2364, dtype: object

In [ ]:
sample_df = pd.DataFrame(
    {'005930':['삼성전자', 1, 2], '035720':['카카오', 1, 2], '015720':['카카오', 1, 2], '025720':['카카오', 1, 2]}
).transpose()



In [ ]:
sample_df.columns = ['name', 'PER', 'Debt_ratio']

In [ ]:
sample_df

,name,PER,Debt_ratio
005930,삼성전자,1,2
035720,카카오,1,2
015720,카카오,1,2
025720,카카오,1,2


In [ ]:
def FinanceInfoCrawler(li, df):
    result_dict = {}
    error_codes = []
    
    for code in tqdm(li):
        try:
            # Parsing URL setting
            SNAP_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_Main.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=101&stkGb=701'
            RATIO_URL = 'https://comp.fnguide.com/SVO2/ASP/SVD_FinanceRatio.asp?pGB=1&gicode=A{}&cID=&MenuYn=Y&ReportGB=&NewMenuID=104&stkGb=701'

            # company name
            company_name = df[df['종목코드'] == code]['회사명'].values[0]

            # Get PER
            snap_url = SNAP_URL.format(code)
            snap_content = requests.get(snap_url).content
            snap_tree = html.fromstring(snap_content)
            per = snap_tree.xpath('//*[@id="corp_group2"]/dl[1]/dd')[0].text
            per = float(per)

            # Get Debt ratio
            ratio_url = RATIO_URL.format(code)
            ratio_content = requests.get(ratio_url).content
            ratio_tree = html.fromstring(ratio_content)
            debt_ratio = ratio_tree.xpath('//*[@id="p_grid1_3"]/td[5]')[0].text
            debt_ratio = float(debt_ratio)
            
            result_dict[code] = [company_name, per, debt_ratio]
            
        except (TypeError, IndexError, AttributeError, ValueError):
            pass
#             print(code)
            error_codes.append(code)
    
    # convert dict to DataFrame
    result_df = pd.DataFrame(result_dict)
    
    # transpose DataFrame
    result_df = result_df.transpose()
    
    # Setting column names
    result_df.columns = ['Name', 'PER', 'Debt_ratio']
    
    return result_df, error_codes

In [ ]:
crawling_result_df = FinanceInfoCrawler(code_list[:50],stock_list_df)

100%|██████████| 50/50 [00:46<00:00,  1.07it/s]


In [ ]:
print(crawling_result_df[0].shape)
crawling_result_df[0].head()

(39, 3)


,Name,PER,Debt_ratio
100090,SK오션플랜트,38.11,132.0
377740,바이오노트,1.89,9.8
446070,유니드비티플러스,104.18,11.2
108320,LX세미콘,7.5,35.7
126720,수산인더스트리,6.3,24.7


In [ ]:
# original data
# crawling_result_df

# copy data
copy_df = crawling_result_df[0].copy()

In [ ]:
# PER 10 이하
# 부채비율 50 이하
# (상위) 20개 종목

final_result_df = copy_df[
    (copy_df['PER'] <= 10)&(copy_df['Debt_ratio'] <= 50)&(copy_df['PER'] > 0)
].sort_values(
    by='PER', ascending=True
).iloc[:20]


In [ ]:
import datetime

# 시간까지 포함한 날짜
now = datetime.datetime.now()

final_result_df.to_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

In [ ]:
pd.read_csv('LowPER_LowDR_{}.csv'.format(now.strftime('%Y%m%d')))

,Unnamed: 0,Name,PER,Debt_ratio
0,377740,바이오노트,1.89,9.8
1,137310,에스디바이오센서,2.39,10.7
2,383800,LX홀딩스,3.90,1.5
3,353200,대덕전자,6.16,39.4
4,363280,티와이홀딩스,6.17,47.6
5,126720,수산인더스트리,6.30,24.7
6,108320,LX세미콘,7.50,35.7
